In [1]:
from pprint import pprint # used for pretty printing
import requests # used to get data from the a URL
import pandas as pd # used to analyse the results
search_url = "https://www.ebi.ac.uk/pdbe/search/pdb/select?" # the rest of the URL used for PDBe's search API.

In [2]:
#Leer la base de datos (Las secuencias con aminoacidos inusuales (grampa.csv) fueron eliminados en bash)
datos_raw=pd.read_csv('grampa_AA.csv', index_col=0)
#Eliminar secuencias con modificaciones inusuales (README del repositorio de witten para más info)
datos_raw=datos_raw[datos_raw["has_unusual_modification"]==False]
#Eliminar columnas que no aportan información relevante 
datos_raw=datos_raw.drop(['url_source','datasource_has_modifications','database','has_unusual_modification'], axis='columns')

In [3]:
sec_raw=datos_raw["sequence"]
#Inicia una lista vacía donde se guardarán las secuencias únicas
sec_0=[]
[sec_0.append(x) for x in sec_raw if x not in sec_0]
len(sec_0)

6169

In [4]:
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

In [5]:
#Creamos una lista con las secuencias del DF para poder trabajarla en modlAMP 
sec_raw=pd.Series.tolist(datos_raw["sequence"])
#Calcular la longitud de las secuencias
globaldesc=GlobalDescriptor(sec_raw)
globaldesc.length()
lon=globaldesc.descriptor
datos_raw["sequence_length"]=lon

In [6]:
sec_prueba=datos_raw["sequence"]
#Inicia una lista vacía donde se guardarán las secuencias únicas
sec_1=[]
[sec_1.append(x) for x in sec_prueba if x not in sec_1]
len(sec_1)

6169

We will now define some functions which will use to get the data from PDBe's search API

In [7]:
def make_request_post(search_dict, number_of_rows=10):
    """
    makes a post request to the PDBe API
    :param dict search_dict: the terms used to search
    :param number_of_rows: number or rows to return - initially limited to 10
    :return dict: response JSON
    """
    # make sure we get the number of rows we need
    if 'rows' not in search_dict:
        search_dict['rows'] = number_of_rows
    # set the return type to JSON
    search_dict['wt'] = 'json'

    # do the query
    response = requests.post(search_url, data=search_dict)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))

    return {}

def format_sequence_search_terms(sequence, filter_terms=None):
    """
    Format parameters for a sequence search
    :param str sequence: one letter sequence
    :param lst filter_terms: Terms to filter the results by
    :return str: search string
    """
    # first we set the parameters which we will pass to PDBe's search
    params = {
        'json.nl': 'map',
        'start': '0',
        'sort': 'fasta(e_value) asc',
        'xjoin_fasta': 'true',
        'bf': 'fasta(percentIdentity)',
        'xjoin_fasta.external.expupperlim': '0.1',
        'xjoin_fasta.external.sequence': sequence,
        'q': '*:*',
        'fq': '{!xjoin}xjoin_fasta'
    }
    # we make sure that we add required filter terms if they aren't present
    if filter_terms:
        for term in ['pdb_id', 'entity_id', 'entry_entity', 'chain_id']:
            filter_terms.append(term)
        filter_terms = list(set(filter_terms))
        params['fl'] = ','.join(filter_terms)

    # returns the parameter dictionary
    return params


def run_sequence_search(sequence, filter_terms=None, number_of_rows=10):
    """
    Runs a sequence search and results the results
    :param str sequence: sequence in one letter code
    :param lst filter_terms: terms to filter the results by
    :param int number_of_rows: number of results to return
    :return lst: List of results
    """
    search_dict = format_sequence_search_terms(sequence=sequence, filter_terms=filter_terms)
    response = make_request_post(search_dict=search_dict, number_of_rows=number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results {}'.format(len(results)))

    # we now have to go through the FASTA results and join them with the main results

    raw_fasta_results = response.get('xjoin_fasta').get('external')
    fasta_results = {} # results from FASTA will be stored here - key'd by PDB ID and Chain ID

    # go through each FASTA result and get the E value, percentage identity and sequence from the result

    for fasta_row in raw_fasta_results:
        # join_id = fasta_row.get('joinId')
        fasta_doc = fasta_row.get('doc', {})
        percent_identity = fasta_doc.get('percent_identity')
        e_value = fasta_doc.get('e_value')
        return_sequence = fasta_row.get('return_sequence_string')
        pdb_id_chain = fasta_doc.get('pdb_id_chain').split('_')
        pdb_id = pdb_id_chain[0].lower()
        chain_id = pdb_id_chain[-1]
        join_id = '{}_{}'.format(pdb_id, chain_id)
        fasta_results[join_id] = {'e_value': e_value,
                                  'percentage_identity': percent_identity,
                                  'return_sequence': return_sequence}

    # now we go through the main results and add the FASTA results
    ret = [] # final results will be stored here.
    for row in results:
        pdb_id = row.get('pdb_id').lower()
        chain_ids = row.get('chain_id')
        for chain_id in chain_ids:
            search_id = '{}_{}'.format(pdb_id, chain_id)
            entry_fasta_results = fasta_results.get(search_id, {})
            # we will only keep results that match the search ID
            if entry_fasta_results:
                row['e_value'] = entry_fasta_results.get('e_value')
                row['percentage_identity'] = entry_fasta_results.get('percentage_identity')
                row['result_sequence'] = entry_fasta_results.get('return_sequence_string')

                ret.append(row)
    return ret

In [8]:
def multiple_sequence_search(sequences):
    """
    Performs sequence search for multiple entries
    :param lst sequences: a list coontaining all the query sequences
    """
    results=dict.fromkeys(sequences)
    i=0
    while i < len(sequences):
        for values in results:
            results[values]=run_sequence_search(sequences[i], filter_terms=['pdb_id', 'molecule_name'], number_of_rows=10000)
            i=i+1
            print(str(i)+" sequences of "+str(len(sequences)))
        
    return results

In [9]:
def results_to_dict(results):
    """
    Delete sequences with no matches inside results dictionary
    :param dict results: Dictionary containing all the results from multiple sequence search
    """
    new_dict = {k: v for k, v in results.items() if len(v) != 0 }
    return new_dict

## 4200 - 4500

In [10]:
sec_x1=sec_1[4200:4350]

In [11]:
r11_raw=multiple_sequence_search(sec_x1)

Number of results 0
1 sequences of 150
Number of results 0
2 sequences of 150
Number of results 4
3 sequences of 150
Number of results 0
4 sequences of 150
Number of results 0
5 sequences of 150
Number of results 1
6 sequences of 150
Number of results 25
7 sequences of 150
Number of results 1
8 sequences of 150
Number of results 0
9 sequences of 150
Number of results 0
10 sequences of 150
Number of results 0
11 sequences of 150
Number of results 0
12 sequences of 150
Number of results 0
13 sequences of 150
Number of results 0
14 sequences of 150
Number of results 0
15 sequences of 150
Number of results 0
16 sequences of 150
Number of results 0
17 sequences of 150
Number of results 7
18 sequences of 150
Number of results 111
19 sequences of 150
Number of results 0
20 sequences of 150
Number of results 0
21 sequences of 150
Number of results 0
22 sequences of 150
Number of results 0
23 sequences of 150
Number of results 0
24 sequences of 150
Number of results 2
25 sequences of 150
Number

In [12]:
r11=results_to_dict(r11_raw)

In [13]:
%store r11 r11_raw

Stored 'r11' (dict)
Stored 'r11_raw' (dict)


## 4350 - 4500

In [14]:
sec_12=sec_1[4350:4500]

In [15]:
r12_raw=multiple_sequence_search(sec_12)

Number of results 0
1 sequences of 150
Number of results 0
2 sequences of 150
Number of results 0
3 sequences of 150
Number of results 0
4 sequences of 150
Number of results 0
5 sequences of 150
Number of results 0
6 sequences of 150
Number of results 0
7 sequences of 150
Number of results 12
8 sequences of 150
Number of results 0
9 sequences of 150
Number of results 1
10 sequences of 150
Number of results 0
11 sequences of 150
Number of results 7
12 sequences of 150
Number of results 0
13 sequences of 150
Number of results 0
14 sequences of 150
Number of results 85
15 sequences of 150
Number of results 0
16 sequences of 150
Number of results 2
17 sequences of 150
Number of results 481
18 sequences of 150
Number of results 103
19 sequences of 150
Number of results 0
20 sequences of 150
Number of results 0
21 sequences of 150
Number of results 0
22 sequences of 150
Number of results 0
23 sequences of 150
Number of results 0
24 sequences of 150
Number of results 0
25 sequences of 150
Num

In [16]:
r12=results_to_dict(r12_raw)

In [17]:
%store r12 r11_raw

Stored 'r12' (dict)
Stored 'r11_raw' (dict)


## 4500 - 4750 10k

In [3]:
r13={'KWKLFKKIPKFLHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.019, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.019, 'percentage_identity': 80.0, 'result_sequence': None}], 'VFQFLGKIIHHVGNFVHGFSAVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 2.9e-11, 'percentage_identity': 95.7, 'result_sequence': None}], 'GSKKPVPIIYCQRRSGKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 2.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 2.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 2.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 2.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 2.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 1.4e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 1.4e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 1.4e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 2e-07, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 2e-07, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xol_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xol', 'e_value': 2.7e-07, 'percentage_identity': 81.0, 'result_sequence': None}], 'WRWKWRWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4r4h_1', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '4r4h', 'e_value': 0.099, 'percentage_identity': 85.7, 'result_sequence': None}], 'GLPVCGETCFGGTCNTPGCTCDPWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.3e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 6.4e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 6.4e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.7e-11, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 1.7e-11, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 2.6e-11, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 2.6e-11, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 3.1e-11, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 3.5e-10, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 1.7e-09, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 1.7e-09, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 1.7e-09, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 1.8e-09, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 2e-09, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 2e-09, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 9.2e-09, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 1.8e-08, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 2.2e-08, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 7.3e-08, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 7.3e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 1.3e-07, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 2.5e-07, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 3.6e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 3.6e-07, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 9.8e-07, 'percentage_identity': 63.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 2.2e-06, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 2.2e-06, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 4.8e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 5.2e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 5.4e-05, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 6.8e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.004, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.011, 'percentage_identity': 40.7, 'result_sequence': None}], 'VGPGGECGGRFGGCAGGQCCSRFGFCGSGPKYCAH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 2e-12, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 3.2e-07, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 1.1e-06, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 1.6e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 2.4e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 6.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 9.6e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 1.3e-05, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 2.2e-05, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 6.5e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.00014, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 0.00023, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 0.00023, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 0.00023, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 0.00086, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t0w_1', 'molecule_name': ['Hevein'], 'pdb_id': '1t0w', 'e_value': 0.0016, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.0018, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 0.0019, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.0019, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.0019, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 0.0019, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 0.0019, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.0019, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 0.002, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.0021, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.0021, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.0021, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.0021, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lnr_1', 'molecule_name': ['Class-1 chitinase'], 'pdb_id': '6lnr', 'e_value': 0.0023, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 0.0023, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 0.0028, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.0028, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 0.0028, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 0.018, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 0.018, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eis_1', 'molecule_name': ['Agglutinin isolectin VI'], 'pdb_id': '1eis', 'e_value': 0.03, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1en2_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1en2', 'e_value': 0.03, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 0.03, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1iqb_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1iqb', 'e_value': 0.03, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 0.03, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enm_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1enm', 'e_value': 0.03, 'percentage_identity': 48.3, 'result_sequence': None}], 'GLNALKKVFQPIHKAIKKINNHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 1.9e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 5.1e-08, 'percentage_identity': 87.5, 'result_sequence': None}], 'RWCVYARVRGVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.6e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.6e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 3.4e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 3.4e-06, 'percentage_identity': 76.2, 'result_sequence': None}], 'RWCVYARVRGVRYRRCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 8.7e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 8.7e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 0.00011, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 0.00011, 'percentage_identity': 75.0, 'result_sequence': None}], 'RKLPDAPGMHTWGGGGRCAVLSCLPKEQI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.01, 'percentage_identity': 92.9, 'result_sequence': None}], 'RVKRFKKFFRKIKKGFRKIFKKTKIFIG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.014, 'percentage_identity': 56.7, 'result_sequence': None}], 'AISYGNGVYCNKEKCWVNKAENKQAITGIVIGGWASSLAGMGH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.6e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.043, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 0.054, 'percentage_identity': 40.0, 'result_sequence': None}], 'MAKEFGIPAAVAGTVLNVVEAGGWVTTIVSILTAVGSGGLSLLAAAGRESIKSYLKKEIKKKGKRAVIAW': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 2.7e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 2.7e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 2.7e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 2.7e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 7.5e-26, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 0.00042, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 0.0038, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 0.029, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 0.029, 'percentage_identity': 64.0, 'result_sequence': None}], 'FKCWRWQWRWKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 7.9e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}], 'GFCWNVCVYRNGVRVCHRRCN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0i_1', 'molecule_name': ['NZ17074(N1)'], 'pdb_id': '5y0i', 'e_value': 5.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0j_1', 'molecule_name': ['N2'], 'pdb_id': '5y0j', 'e_value': 3.7e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5v0y_1', 'molecule_name': ['Arenicin-3'], 'pdb_id': '5v0y', 'e_value': 4.8e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0h_1', 'molecule_name': ['N6'], 'pdb_id': '5y0h', 'e_value': 8.6e-10, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4v_1', 'molecule_name': ['smart chimeric peptide G6'], 'pdb_id': '6k4v', 'e_value': 1.2e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4w_1', 'molecule_name': ['SCP-A6'], 'pdb_id': '6k4w', 'e_value': 1.2e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5v11_1', 'molecule_name': ['AA139'], 'pdb_id': '5v11', 'e_value': 7e-09, 'percentage_identity': 76.2, 'result_sequence': None}], 'GVVDILKGAAKDIAAHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 2.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 2.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 2.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 3.5e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 3.5e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 3.5e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'GFKEVLKADLGSLVKGIAAHVAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l5r_1', 'molecule_name': ['Antimicrobial peptide Alyteserin-1C'], 'pdb_id': '2l5r', 'e_value': 7.3e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'VGRIHRHLK': [{'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 0.073, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 0.074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 0.076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b31_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b31', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b32_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b32', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}], 'KTLTGKTITLE': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1gjz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1gjz', 'e_value': 0.0064, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5gvi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 0.0099, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2msg', 'e_value': 0.0099, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ons_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ons', 'e_value': 0.0099, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5ydr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydr', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rf1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4rf1', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3kw5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3kw5', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ig7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ig7', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hxd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4hxd', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4i6n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4i6n', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3prp_2', 'molecule_name': ['60S ribosomal protein L40'], 'pdb_id': '3prp', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3pt2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3pt2', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5l8w_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l8w', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rf0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4rf0', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3kvf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3kvf', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5khy_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5khy', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6h4h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6h4h', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ei1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ei1', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5cra_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cra', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djw', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '7bu0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bu0', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6o83_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6o83', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6pgv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6pgv', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gzs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6gzs', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1xd3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xd3', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3c0r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3c0r', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3ldz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ldz', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ibi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ibi', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3i3t_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3i3t', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ihp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ihp', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3by4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3by4', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2j7q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2j7q', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ifw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ifw', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3prm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3prm', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3phw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phw', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '7jms_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jms', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6t7f_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t7f', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'B', 'J', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '7mc9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7mc9', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 3, 'entry_entity': '7b5n_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5n', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 6, 'entry_entity': '7b5m_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5m', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 6, 'entry_entity': '7b5l_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5l', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5j26_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j26', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4uf6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4uf6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5eya_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5eya', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4q5e_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4q5e', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ohk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5wfi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wfi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ohm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5vey_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vey', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '4dhz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4bvu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bvu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vf0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vf0', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5fer_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5fer', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5dk8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dk8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xpk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xpk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xu8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xu8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5emz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5emz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4q5h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4q5h', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xve_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xve', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '5ohp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'I', 'K', 'M', 'J', 'N', 'O', 'L'], 'entity_id': 2, 'entry_entity': '5ohl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohl', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5e6j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5e6j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ohn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5oxi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5oxi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 3, 'entry_entity': '5edv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5edv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mm3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5w46_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5w46', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oxh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5oxh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ohv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'D', 'J', 'C'], 'entity_id': 3, 'entry_entity': '5ydk_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6i_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6i', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mjb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mjb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mur', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '2mbh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '4s1z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s1z', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4wur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wur', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xyz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xyz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5goj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goj', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5tut_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tut', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 1, 'entry_entity': '5h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5h7s', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gog_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gog', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4xkl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkl', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xof_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xof', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5goi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5go7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go7', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4wzp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wzp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gob', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '5kyd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3v', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mcn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1y_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1y', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m93_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5m93', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5lrx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrx', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xol_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xol', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4y1h_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4y1h', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5go8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5god_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5god', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'D', 'A', 'B', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4xkh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4r62_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4r62', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4s22_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s22', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5goc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4xok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xok', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5txk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5txk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['h', 'c', 'X', 'm'], 'entity_id': 10, 'entry_entity': '4zux_10', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zux', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5mn9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mn9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5u4p_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5u4p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gok', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5tr4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tr4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4i6l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4i6l', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7u', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ium_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ium', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xk5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3rul', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '4jio_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jio', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kox', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7w_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '4ldt_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ldt', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zlz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zlz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3wxg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '3wwq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wwq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'B', 'J', 'F'], 'entity_id': 2, 'entry_entity': '4dhj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhj', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7s', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ksk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 2, 'entry_entity': '3tbl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tbl', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5vzw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'D', 'M', 'Q', 'N', 'O', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lz6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lz6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '3vuy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuy', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vux_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vux', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5kye_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kye', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5lrw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3u_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3u', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6h', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2k_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n2k', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5l6j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mj5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5lrv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jp3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jp3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jbv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jbv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n13', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mre_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mre', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4un2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4un2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5l8h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l8h', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'O'], 'entity_id': 8, 'entry_entity': '5kgf_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kgf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 3, 'entry_entity': '5o44_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5o44', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mor_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mor', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3tmp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tmp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ugb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ugb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '4bos_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bos', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4jqw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jqw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4m0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4m0w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0t', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3vht_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vht', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4bbn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bbn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2qho_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2qho', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3m3j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3m3j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vnz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vnz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nvg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nvg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vo0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vo0', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vzm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mdk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '4boz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4boz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4uel_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4uel', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ifr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ifr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5j8p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j8p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5jby_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jby', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5jqs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jqs', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nmc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nmc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z9s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4z9s', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5ait_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ait', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5caw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5caw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3m', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wvo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['9'], 'entity_id': 9, 'entry_entity': '5a5b_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5a5b', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ulf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5bnb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bnb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5aiu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5aiu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jtv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mro_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mro', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6w9s_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9s', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'F'], 'entity_id': 2, 'entry_entity': '5yik_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yik', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6eqi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6eqi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6p5b_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6p5b', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7m', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6ums_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ums', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6fx4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fx4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '6ump_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ump', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq7', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6ulh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ulh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6dc6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6dc6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6tnf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tnf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 15, 'entry_entity': '6tbm_15', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tbm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 12, 'entry_entity': '6t9l_12', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t9l', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6v5d_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6v5d', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6asr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6asr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '6vae_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vae', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '6ttu_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ttu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 9, 'entry_entity': '6kiw_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k9p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'F'], 'entity_id': 3, 'entry_entity': '6a6i_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6a6i', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6txb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6txb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kox', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kow_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kow', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6e2b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6e2b', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6glc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6glc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6kl4_3', 'molecule_name': ['Ub'], 'pdb_id': '6kl4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 9, 'entry_entity': '6kiv_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6aqr_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6aqr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6vaf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vaf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 11, 'entry_entity': '6kiu_11', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '6ven_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ven', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msb_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5x3o_2', 'molecule_name': ['D-ubiquitin'], 'pdb_id': '5x3o', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 3, 'entry_entity': '5d0k_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0k', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujl', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xiu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xiu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3n', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 3, 'entry_entity': '5xit_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xit', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6xaa_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6xaa', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oam_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oam', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'L'], 'entity_id': 8, 'entry_entity': '7bwd_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bwd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 1, 'entry_entity': '6jwi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jwi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6fdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fdk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6cp2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6cp2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6k2u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k2u', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6q00_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q00', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oi4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oi4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'F'], 'entity_id': 1, 'entry_entity': '6wkr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wkr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6oq2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'W', 'R', 'U', 'P', 'M', 'Q', 'T', 'X', 'N', 'O', 'S'], 'entity_id': 1, 'entry_entity': '6w9r_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9r', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqs_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqs', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6s53_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6s53', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6fyh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fyh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'O'], 'entity_id': 8, 'entry_entity': '6ftx_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ftx', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djx', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q'], 'entity_id': 11, 'entry_entity': '6uh5_11', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uh5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5zbu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zbu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6jky_3', 'molecule_name': ['Ub'], 'pdb_id': '6jky', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4i_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k4i', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ud0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ud0', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F', 'L'], 'entity_id': 3, 'entry_entity': '6sqr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6lp2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6lp2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6fge_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fge', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 4, 'entry_entity': '6n13_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n13', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6xqc_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6xqc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '6nn6_8', 'molecule_name': ['Polyubiquitin-C'], 'pdb_id': '6nn6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6nxk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nxk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 7, 'entry_entity': '6nqa_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nqa', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6o82_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6o82', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6qml_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qml', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qf8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qf8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4zpz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zpz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1z_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1z', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xbo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5bz0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bz0', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5dfl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dfl', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 3, 'entry_entity': '5d0m_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0m', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5n38_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5n38', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6wtg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wtg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5n2w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5n2w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '7bxg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bxg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'L'], 'entity_id': 8, 'entry_entity': '6nj9_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nj9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '6o96_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6o96', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msd_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5xdp_1', 'molecule_name': ['D-ubiquitin'], 'pdb_id': '5xdp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5xis_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xis', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 5, 'entry_entity': '6nog_5', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6nog', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7cap_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7cap', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb7', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 7, 'entry_entity': '6jma_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jma', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ymy_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ymy', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6oq1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6q01_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q01', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '6pzv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6pzv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '5yij_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yij', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7o', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '6w9d_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9d', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6oaa_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oaa', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6tuv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tuv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyj', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6rya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6rya', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'J'], 'entity_id': 2, 'entry_entity': '6oa9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oa9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'K', 'D', 'B', 'C', 'J', 'N', 'O', 'F', 'G', 'L'], 'entity_id': 2, 'entry_entity': '6upu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6upu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6kg6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kg6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4wlr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wlr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5yt6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yt6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'C', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '6z7v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6z7v', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zqh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6zqh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6if1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6if1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wwz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7s', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q0w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ogw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ogw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '1wr6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wr1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1tbe', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v80_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v80', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4nqk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nqk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n32_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n32', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ru6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rr9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rr9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3n30_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n30', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4nql_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nql', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '3mhs_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3mhs', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3nhe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nhe', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rsu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rsu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jsv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jsv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wx1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'U', 'T', 'X', 'S'], 'entity_id': 3, 'entry_entity': '4p4h_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4p4h', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4nnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nnj', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3cmm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3cmm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1f9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1f9j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3a1q_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a1q', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2khw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2khw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2jt4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jt4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h1u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h1u', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2o6v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2o6v', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pea_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pea', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2dx5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ehv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ehv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2g45_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g45', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2k6d_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k6d', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2g3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g3q', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eec_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3eec', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2gmi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gmi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3efu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3efu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kde_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kde', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kdf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pe9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pe9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2oob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2oob', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3hm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3hm3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hth_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hth', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2jri_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jri', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jy6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jy6', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcs_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcs', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcq_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '2fcq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'X'], 'entity_id': 3, 'entry_entity': '3dvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvn', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2fif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fif', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8c_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8c', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kjh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kjh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a33_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a33', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k39_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k39', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8b', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jzz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jzz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 3, 'entry_entity': '3dvg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jwz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9k', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2z59_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2z59', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ayo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ayo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3alb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3alb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ofi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ofi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ns8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ns8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3oj3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2znv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2znv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 1, 'entry_entity': '1yd8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yd8', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3aul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3aul', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 2, 'entry_entity': '1p3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1p3q', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g6j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1g6j', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1uzx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1uzx', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1otr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1otr', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1zgu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zgu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yiw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '1yiw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wrd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wrd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yj1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yj1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v81_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v81', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3nob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nob', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3o65_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3o65', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '3olm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3olm', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3oj4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcc', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q5w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q5w', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1s1q', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fxt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1fxt', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1aar_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1aar', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cmx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1cmx', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1nbf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1nbf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '2c7n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7n', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7m', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3h7p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d3g_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2d3g', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2wdt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wdt', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lj5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wx0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx0', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2xbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xbb', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2bgf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2bgf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fid_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fid', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2fuh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fuh', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hd5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hd5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xqq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xqq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1d3z', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2jf5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jf5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2den_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2den', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0f_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0f', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vfk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vfk', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4pih_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pih', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4gu2_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gu2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l3z', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vuw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ld9', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2klg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2klg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '3phd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4ap4_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ap4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ptf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ptf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ktf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ktf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2l00_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l00', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4gsw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gsw', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kn5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kn5', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2xew_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xew', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hek_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hek', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6hei_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hei', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6isu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6isu', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6kfp_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kfp', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 7, 'entry_entity': '6j99_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6j99', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '7bbf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbf', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '7ay1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay1', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7mic_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7mic', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '7ay2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay2', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7jxv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jxv', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '7k6q_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k6q', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '7k6p_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k6p', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '7bbd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbd', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7keo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7keo', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4v3k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3k', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msm', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4v3l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3l', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '5mnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mnj', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4zfr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zfr', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4auq_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4auq', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msq', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4pqt_2', 'molecule_name': ['Protein UBBP4'], 'pdb_id': '4pqt', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4k1r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k1r', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '5ln1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ln1', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'L'], 'entity_id': 4, 'entry_entity': '3zni_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zni', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4zft_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zft', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hpr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hpr', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jvz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jvz', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jw0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jw0', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbr', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai1', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai0', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['III', 'LLL', 'CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ahz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ahz', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}], 'KFFRKLKKSVKKRAKKFFKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.6e-09, 'percentage_identity': 80.0, 'result_sequence': None}], 'KKAAAAAAAAAAAAWAAAAAAKKKK': [{'chain_id': ['Y2'], 'entity_id': 57, 'entry_entity': '7a5h_57', 'molecule_name': ['nascent chain'], 'pdb_id': '7a5h', 'e_value': 0.0013, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['R', 'U', 'I', 'C', 'O', 'F', 'L'], 'entity_id': 3, 'entry_entity': '7o85_3', 'molecule_name': ['Fab'], 'pdb_id': '7o85', 'e_value': 0.033, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['BX'], 'entity_id': 48, 'entry_entity': '4v7h_48', 'molecule_name': ['60S ribosomal protein L35'], 'pdb_id': '4v7h', 'e_value': 0.046, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 8, 'entry_entity': '5gao_8', 'molecule_name': ['66 kDa U4/U6.U5 small nuclear ribonucleoprotein component'], 'pdb_id': '5gao', 'e_value': 0.07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['4'], 'entity_id': 45, 'entry_entity': '5yzg_45', 'molecule_name': ['UNKNOW'], 'pdb_id': '5yzg', 'e_value': 0.08, 'percentage_identity': 94.7, 'result_sequence': None}], 'LCAIAKKKGKCKGPLKLVCKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mbd_1', 'molecule_name': ['Lasiocepsin'], 'pdb_id': '2mbd', 'e_value': 3.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GFGCPFNANECHAHCLSIGRKFGFCAGPLRATCTCGKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 9e-21, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.1e-09, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 3.4e-08, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 4.9e-07, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 9.9e-07, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 2.1e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.00078, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0023, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0023, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0023, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.0036, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0068, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.016, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.02, 'percentage_identity': 39.4, 'result_sequence': None}], 'LVKRFKKFFRKLKKSVKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.00091, 'percentage_identity': 87.5, 'result_sequence': None}], 'AQWFAIQHISLNPPRSTIAMRAINNYRWRSKNQNTFLR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 1.4e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 1.4e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 1.4e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 1.4e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 1.4e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 1.5e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 1.5e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 1.5e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 1.5e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 1.5e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 2.9e-18, 'percentage_identity': 92.1, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 8.3e-10, 'percentage_identity': 67.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 2.3e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 2.7e-08, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 3.1e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 3.2e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 3.2e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 3.2e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 3.2e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 8.3e-05, 'percentage_identity': 45.9, 'result_sequence': None}], 'ATCDLLSGTGVGHSACAAHCLLRGNRGGYCNGKGVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 2.7e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.5e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 4.3e-19, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 7.3e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 6.6e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 6.7e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.4e-11, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 5.5e-09, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 5.9e-08, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 3.4e-07, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 1.4e-05, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0016, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.075, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.075, 'percentage_identity': 36.4, 'result_sequence': None}], 'SPIHACRYQRGVCIPGPCRWPYYRVGSCGSGLKSCCVRNRWA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 6.3e-10, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.0015, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.046, 'percentage_identity': 40.5, 'result_sequence': None}], 'MTESLVLSPAPAKPKRVKASRRSASHPTYSEMIAAAIRAEKSRGGSSRQSIQKYIKSHYKVGHNADLQIKLSIRRLLAAGVLKQTKGVGASGSFRLAKSDKAKRSPGKKKKAVRRSTSPKKAARPRKARSPAKKPKATARKARKKSRASPKKAKKPKTVKAKSRKASKAKKVKRSKPRAKSGARKSPKKK': [{'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '6la2_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la2', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6n89_2', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6n89', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 7, 'entry_entity': '6lab_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6lab', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['S'], 'entity_id': 7, 'entry_entity': '6la8_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la8', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6n88_3', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6n88', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '6la9_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la9', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5x_8', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7k5x', 'e_value': 5.7e-41, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '7cow_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7cow', 'e_value': 1.5e-40, 'percentage_identity': 66.5, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 1, 'entry_entity': '7dbp_1', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7dbp', 'e_value': 1.6e-40, 'percentage_identity': 66.5, 'result_sequence': None}, {'chain_id': ['Z'], 'entity_id': 7, 'entry_entity': '5nl0_7', 'molecule_name': ['Histone H1.0-B'], 'pdb_id': '5nl0', 'e_value': 2.4e-38, 'percentage_identity': 62.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1hst_1', 'molecule_name': ['Histone H5'], 'pdb_id': '1hst', 'e_value': 1.5e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '4qlc_7', 'molecule_name': ['Histone H5'], 'pdb_id': '4qlc', 'e_value': 2.2e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 7, 'entry_entity': '5wcu_7', 'molecule_name': ['Histone H5'], 'pdb_id': '5wcu', 'e_value': 4.3e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hq1_1', 'molecule_name': ['Histone H1.0, N-terminally processed'], 'pdb_id': '6hq1', 'e_value': 5.3e-17, 'percentage_identity': 83.6, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5y_8', 'molecule_name': ['Histone H1.4'], 'pdb_id': '7k5y', 'e_value': 8e-15, 'percentage_identity': 39.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c0j_1', 'molecule_name': ['Double-stranded RNA-specific adenosine deaminase', 'Histone H5'], 'pdb_id': '7c0j', 'e_value': 2.9e-12, 'percentage_identity': 74.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k63_8', 'molecule_name': ['Histone H1.10', 'Histone H1.4'], 'pdb_id': '7k63', 'e_value': 3.1e-11, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['S'], 'entity_id': 7, 'entry_entity': '6l9z_7', 'molecule_name': ['Histone H1.10'], 'pdb_id': '6l9z', 'e_value': 4.9e-08, 'percentage_identity': 31.8, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k60_8', 'molecule_name': ['Histone H1.10'], 'pdb_id': '7k60', 'e_value': 4.9e-08, 'percentage_identity': 31.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ghc_1', 'molecule_name': ['Histone H1.11L'], 'pdb_id': '1ghc', 'e_value': 0.00034, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ust_1', 'molecule_name': ['Histone H1'], 'pdb_id': '1ust', 'e_value': 0.0015, 'percentage_identity': 37.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uhm_1', 'molecule_name': ['Histone H1'], 'pdb_id': '1uhm', 'e_value': 0.012, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 43, 'entry_entity': '6qx9_43', 'molecule_name': ['Serine/threonine-protein kinase PRP4 homolog'], 'pdb_id': '6qx9', 'e_value': 0.042, 'percentage_identity': 24.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rqp_1', 'molecule_name': ['Heterochromatin protein 1-binding protein 3'], 'pdb_id': '2rqp', 'e_value': 0.06, 'percentage_identity': 33.3, 'result_sequence': None}], 'MEKKSFAGLCFLFLVLFVAQECVLQTEAKTCENLADTFRGPCFATGNCDDHCKNKEHLLRGRCRDDFRCWCTRNC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 1.7e-22, 'percentage_identity': 89.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 1.9e-21, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 2.8e-21, 'percentage_identity': 85.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 1.7e-19, 'percentage_identity': 78.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 1.5e-17, 'percentage_identity': 72.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 2.3e-13, 'percentage_identity': 61.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 3.1e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 7.9e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 1.4e-05, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.00031, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 0.00084, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 0.0012, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6mry_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6mry', 'e_value': 0.0038, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 0.0081, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bk8_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1bk8', 'e_value': 0.01, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5kk4_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5kk4', 'e_value': 0.012, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '5vyp_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5vyp', 'e_value': 0.012, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 0.016, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '6b55_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '6b55', 'e_value': 0.024, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 0.061, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 0.061, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 0.061, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mr4_1', 'molecule_name': ['Defensin-like protein'], 'pdb_id': '1mr4', 'e_value': 0.061, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.069, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 0.095, 'percentage_identity': 32.0, 'result_sequence': None}], 'KWFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.0086, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.0086, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.0086, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.0086, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.0086, 'percentage_identity': 92.3, 'result_sequence': None}], 'GIGGALLSAGKSALKGLAKGLAEHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 6.7e-06, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.03, 'percentage_identity': 68.2, 'result_sequence': None}], 'GIGGALLSAGKSALKGLAKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 2e-06, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.021, 'percentage_identity': 60.0, 'result_sequence': None}], 'MPKWKVFKKIEKVGRNIRNGIVKAGPAIAVLGEAKALG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 7.7e-12, 'percentage_identity': 75.0, 'result_sequence': None}], 'GFIATLCTKVLDFGIDKLIQLIEDK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1of9_1', 'molecule_name': ['Pore-forming peptide ameobapore A'], 'pdb_id': '1of9', 'e_value': 2.7e-11, 'percentage_identity': 100.0, 'result_sequence': None}], 'AHCLAIGRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.036, 'percentage_identity': 88.9, 'result_sequence': None}], 'AKKVFKRLEKLFSKIFNFK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 5.7e-06, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 2.5e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 2.5e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 3.3e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.00014, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.0015, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.0037, 'percentage_identity': 84.2, 'result_sequence': None}], 'GLPRKILAAIAKKKGKCKGPLKLVAKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mbd_1', 'molecule_name': ['Lasiocepsin'], 'pdb_id': '2mbd', 'e_value': 4e-11, 'percentage_identity': 92.6, 'result_sequence': None}], 'TKPKGTKPKGTKPKGTKPKG': [{'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5y_8', 'molecule_name': ['Histone H1.4'], 'pdb_id': '7k5y', 'e_value': 0.058, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k63_8', 'molecule_name': ['Histone H1.10', 'Histone H1.4'], 'pdb_id': '7k63', 'e_value': 0.059, 'percentage_identity': 66.7, 'result_sequence': None}], 'GIMRVFKGVLKTAGKSVAKNVAGSFLDRLKCKISGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0001, 'percentage_identity': 54.1, 'result_sequence': None}], 'PPYNLNSHTQEYSKPEDTFD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 3e-07, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4bed_2', 'molecule_name': ['Hemocyanin 1'], 'pdb_id': '4bed', 'e_value': 1.3e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1lnl_1', 'molecule_name': ['Hemocyanin type 2 unit e'], 'pdb_id': '1lnl', 'e_value': 6.7e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['W', 'H', 'K', 'Q', 'c', 'B', 'T', 'N', 'Z', 'E'], 'entity_id': 2, 'entry_entity': '4yd9_2', 'molecule_name': ['hemocyanin'], 'pdb_id': '4yd9', 'e_value': 0.077, 'percentage_identity': 63.2, 'result_sequence': None}], 'RWKIFKKIEKVGRNVRDGIIKAGPAVAVVGQAATVVKG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 1.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGRKFLGGVKTTFRGGVKDFASKHLY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.012, 'percentage_identity': 48.1, 'result_sequence': None}], 'LLGDFFRKVKEKIGKEFKRIVQR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 2e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.6e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.6e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.6e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.6e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.6e-08, 'percentage_identity': 95.7, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 4e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'KRFKKFFRKLKKSVKKRKKEFKKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.7e-09, 'percentage_identity': 76.5, 'result_sequence': None}], 'GIMDTIKGAAKDLAGQLLDKLKCKITKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 3.9e-06, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 0.0028, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 0.0028, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 0.0028, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 0.0075, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 0.0075, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 0.0075, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 0.0078, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 0.0078, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 0.0078, 'percentage_identity': 57.1, 'result_sequence': None}], 'FKRIVQRIKDFLRGIGKFLHSAKKF': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.02, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.025, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.025, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.025, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.025, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.025, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.039, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.039, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.092, 'percentage_identity': 100.0, 'result_sequence': None}], 'LVKRFKKFFRKLKKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.012, 'percentage_identity': 84.6, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 6.1e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'DRCSQQCQHHRDPDRKQQCMRECRRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m6a_1', 'molecule_name': ['Predicted protein'], 'pdb_id': '2m6a', 'e_value': 0.00059, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lm0_1', 'molecule_name': ['L-2'], 'pdb_id': '5lm0', 'e_value': 0.013, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l2r_1', 'molecule_name': ['Antimicrobial peptide EcAMP1'], 'pdb_id': '2l2r', 'e_value': 0.063, 'percentage_identity': 44.0, 'result_sequence': None}], 'KRGKQGGKVRAKAKTRSS': [{'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 1.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 2.4e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 2.5e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6g0l_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b31_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b31', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b32_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b32', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 2.6e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '2f8n_7', 'molecule_name': ['Histone H2A type 1-H'], 'pdb_id': '2f8n', 'e_value': 2.8e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 2.9e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 6.2e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 6.2e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 6.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 6.4e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 0.00015, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 0.0002, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 0.00021, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 0.00021, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 0.00038, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 0.00074, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 0.00074, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 0.00074, 'percentage_identity': 82.4, 'result_sequence': None}], 'TAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 3.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 5.6e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 6.6e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 8.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 7.8e-08, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 4.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 4.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 4.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 4.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 4.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 5.4e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 6.7e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 6.8e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 7.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l60_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2l60', 'e_value': 7.5e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz5_1', 'molecule_name': ['Neuropeptide Y', 'Pancreatic hormone'], 'pdb_id': '1tz5', 'e_value': 3.7e-06, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3s_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3s', 'e_value': 0.00094, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3t_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3t', 'e_value': 0.00094, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2h4b_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h4b', 'e_value': 0.00094, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ppt_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ppt', 'e_value': 0.0012, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bba_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1bba', 'e_value': 0.0046, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljv_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ljv', 'e_value': 0.0047, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bf9_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2bf9', 'e_value': 0.01, 'percentage_identity': 52.4, 'result_sequence': None}], 'GSKKPVPIIYCQRRTGKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 1.2e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 1.2e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 1.2e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 1.2e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 1.2e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 7.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 7.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 7.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 1e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 1e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xol_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xol', 'e_value': 1.4e-07, 'percentage_identity': 85.7, 'result_sequence': None}], 'GLNALKKVFQGIHEAIKLINKHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 2.3e-10, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 3.9e-09, 'percentage_identity': 91.7, 'result_sequence': None}], 'GIGGALLSVGKSALKGLTKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.6e-05, 'percentage_identity': 72.0, 'result_sequence': None}], 'YRQSMNNFQGLRSFGCRFGTCTVQKLAHQIYQFTDKDKDNVAPRSKISPQGY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7s_1', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '2l7s', 'e_value': 2.5e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 5, 'entry_entity': '6uus_5', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '6uus', 'e_value': 2.5e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 6, 'entry_entity': '6uun_6', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '6uun', 'e_value': 2.5e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rwf_2', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '4rwf', 'e_value': 1.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 6, 'entry_entity': '6uva_6', 'molecule_name': ['Adrenomedullin-2'], 'pdb_id': '6uva', 'e_value': 0.0072, 'percentage_identity': 40.0, 'result_sequence': None}], 'SGSLSTFFRLFNRSFTQALGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 3.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GMWSKIKNAGKAAKAAAKAAGKAALGAVSEAM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jx6_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2jx6', 'e_value': 7.4e-09, 'percentage_identity': 84.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k9b_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2k9b', 'e_value': 7.4e-09, 'percentage_identity': 84.8, 'result_sequence': None}], 'AQCGAQGGGQTCPGGLCCSQWGWCGSTPKYCGAGCQSNCR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 1.5e-13, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 6.3e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 6.3e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 6.3e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 7.6e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 8.8e-12, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 8.8e-12, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 3.6e-11, 'percentage_identity': 71.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 1.4e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 1.4e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 1.4e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eis_1', 'molecule_name': ['Agglutinin isolectin VI'], 'pdb_id': '1eis', 'e_value': 1.5e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 1.5e-10, 'percentage_identity': 62.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1en2_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1en2', 'e_value': 1.5e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enm_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1enm', 'e_value': 1.5e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 1.8e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 1.8e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1iqb_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1iqb', 'e_value': 7.4e-10, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 1.6e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t0w_1', 'molecule_name': ['Hevein'], 'pdb_id': '1t0w', 'e_value': 2e-09, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 3.3e-09, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 3.4e-09, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 3.4e-09, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 3.4e-09, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 3.4e-09, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lnr_1', 'molecule_name': ['Class-1 chitinase'], 'pdb_id': '6lnr', 'e_value': 5.4e-09, 'percentage_identity': 65.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 9.5e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 1.4e-08, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 3.5e-08, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 2.7e-06, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 3.5e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 6e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 5.2e-05, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 6.3e-05, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 6.3e-05, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 7.4e-05, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 9e-05, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.00015, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.00026, 'percentage_identity': 71.4, 'result_sequence': None}], 'KKWGWLAWVDPAYEFIKGFGKGAIKEGNKDKWKNI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpk_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpk', 'e_value': 5.4e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpm_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpm', 'e_value': 5.4e-19, 'percentage_identity': 100.0, 'result_sequence': None}], 'VFHLLGKIIHHVGNFVYGFSHVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 5.1e-10, 'percentage_identity': 87.0, 'result_sequence': None}], 'DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVV': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4mvi_2', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4mvi', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mvl_2', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4mvl', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9r_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m9r', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9s_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m9s', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmp_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmp', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2m4j_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m4j', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmo_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmo', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 4, 'entry_entity': '4m1c_4', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4m1c', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['1', '3', '6', '5', '2', '4'], 'entity_id': 1, 'entry_entity': '6w0o_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '6w0o', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6shs_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '6shs', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6ti5_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '6ti5', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6xov_2', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '6xov', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6oc9_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '6oc9', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4nge_2', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4nge', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2otk_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2otk', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '3ifn_3', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '3ifn', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2g47_2', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2g47', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aml_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '1aml', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ba6_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '1ba6', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ba4_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '1ba4', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmq_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmq', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lfm_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lfm', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '4ong_3', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4ong', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmn_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmn', 'e_value': 2.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5oqv_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '5oqv', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '5kk3_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '5kk3', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2mxu_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2mxu', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2nao_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2nao', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'B', 'J', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6ti6_2', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '6ti6', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'B', 'O', 'F'], 'entity_id': 2, 'entry_entity': '6ti7_2', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '6ti7', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6rhy_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '6rhy', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6szf_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '6szf', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2wk3_2', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2wk3', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z0q_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '1z0q', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2beg_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2beg', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iyt_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '1iyt', 'e_value': 2.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7b3j_1', 'molecule_name': ['Gamma-secretase C-terminal fragment 57'], 'pdb_id': '7b3j', 'e_value': 3.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7b3k_1', 'molecule_name': ['Gamma-secretase C-terminal fragment 57'], 'pdb_id': '7b3k', 'e_value': 3.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lp1_1', 'molecule_name': ['C99'], 'pdb_id': '2lp1', 'e_value': 9.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2lnq_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lnq', 'e_value': 1e-19, 'percentage_identity': 97.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7o1q_1', 'molecule_name': ['Alpha-hemolysin', 'Amyloid-beta protein 42'], 'pdb_id': '7o1q', 'e_value': 2.6e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5buo_1', 'molecule_name': ['Soluble APP-beta'], 'pdb_id': '5buo', 'e_value': 9.9e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2mvx_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2mvx', 'e_value': 1.7e-18, 'percentage_identity': 97.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '4hix_3', 'molecule_name': ['P3(42)'], 'pdb_id': '4hix', 'e_value': 6.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '3bae_3', 'molecule_name': ['P3(42)'], 'pdb_id': '3bae', 'e_value': 6.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1amb_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1amb', 'e_value': 6.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1amc_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1amc', 'e_value': 6.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bjb_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1bjb', 'e_value': 2e-12, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bjc_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1bjc', 'e_value': 6.4e-12, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hz3_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1hz3', 'e_value': 1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nmj_1', 'molecule_name': ['P3(42)'], 'pdb_id': '1nmj', 'e_value': 1.7e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5aef_1', 'molecule_name': ['P3(42)'], 'pdb_id': '5aef', 'e_value': 7.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2loh_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2loh', 'e_value': 1.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2llm_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2llm', 'e_value': 1.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'a', 'D', 'M', 'Q', 'Y', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'Z', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '2mpz_1', 'molecule_name': ['P3(42)'], 'pdb_id': '2mpz', 'e_value': 2.7e-09, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3moq_1', 'molecule_name': ['P3(40)', 'Ig-like domain-containing protein'], 'pdb_id': '3moq', 'e_value': 7e-08, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 5, 'entry_entity': '6iyc_5', 'molecule_name': ['C83'], 'pdb_id': '6iyc', 'e_value': 2.9e-07, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'B', 'A', 'C', 'J', 'F', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6wxm_1', 'molecule_name': ['P3(40)'], 'pdb_id': '6wxm', 'e_value': 8.9e-05, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4f37_1', 'molecule_name': ['Colicin-E7 immunity protein'], 'pdb_id': '4f37', 'e_value': 0.00021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5hoy_1', 'molecule_name': ['P3(40)'], 'pdb_id': '5hoy', 'e_value': 0.00027, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5hox_1', 'molecule_name': ['P3(40)'], 'pdb_id': '5hox', 'e_value': 0.00027, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7jxn_1', 'molecule_name': ['P3(40)'], 'pdb_id': '7jxn', 'e_value': 0.0011, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5how_1', 'molecule_name': ['P3(40)'], 'pdb_id': '5how', 'e_value': 0.0018, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6yhx_1', 'molecule_name': ['P3(42)'], 'pdb_id': '6yhx', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6yhf_1', 'molecule_name': ['P3(42)'], 'pdb_id': '6yhf', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6yhp_1', 'molecule_name': ['P3(42)'], 'pdb_id': '6yhp', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7jxo_1', 'molecule_name': ['P3(40)'], 'pdb_id': '7jxo', 'e_value': 0.015, 'percentage_identity': 88.2, 'result_sequence': None}], 'GIMNTVKDVATGVATHLLNMVKCKITGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.00024, 'percentage_identity': 53.1, 'result_sequence': None}], 'KKLKLAPAKLALLWKALALKLKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 1.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 1.2e-07, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 1.2e-07, 'percentage_identity': 91.7, 'result_sequence': None}], 'KKLFKKILKYLGIGKFLHSAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}], 'FLPVIAGIAANVLPKLFCKLTKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 1e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 1e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 1e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.02, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.02, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.02, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.02, 'percentage_identity': 52.2, 'result_sequence': None}], 'AVLSWLPKEEQIGKWSTRGRKWWRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 8e-06, 'percentage_identity': 85.2, 'result_sequence': None}], 'VTCDILSVEAKGVKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 0.0021, 'percentage_identity': 76.9, 'result_sequence': None}], 'GIGKFLHSAKKFGKAFVGEAMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 7e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 7.2e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.8e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.5e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.6e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.6e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}], 'KGGPCAKKPCCGPLGHYKVDCSTIPDYPCCSKYGFCGSGPQYCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.00065, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.00094, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 0.0011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0014, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.0023, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.0023, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.0023, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.0024, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.0024, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.0024, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.0024, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.0028, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 0.0041, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 0.0042, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 0.0053, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.0058, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.0059, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.0086, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.025, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 0.03, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 0.03, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 0.03, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 0.031, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 0.044, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 0.044, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 0.095, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 0.097, 'percentage_identity': 55.0, 'result_sequence': None}], 'GFGCPFNQYECHAHCSGVPGYKGGYCKGLFKQTCNCY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.2e-10, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 1.8e-10, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 2.6e-10, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.2e-09, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 6.1e-08, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 9.4e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 9.4e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 9.4e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 2.2e-07, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4.6e-06, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 0.00034, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.0083, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.014, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.029, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.037, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb6_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bb6', 'e_value': 0.076, 'percentage_identity': 35.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 0.081, 'percentage_identity': 34.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.094, 'percentage_identity': 36.1, 'result_sequence': None}], 'ATCDLLSGIGVQHSACALHCVFRGNRGGYCTGKGICVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 4.8e-18, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 2.2e-17, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 2.8e-17, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 3.9e-12, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 4.3e-11, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.5e-10, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 3.2e-10, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 2.4e-09, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 5.1e-08, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2.8e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 4.1e-05, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0035, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bi5_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bi5', 'e_value': 0.077, 'percentage_identity': 41.9, 'result_sequence': None}], 'CGGYSGGWKRKRSTSYRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 6.8e-06, 'percentage_identity': 73.7, 'result_sequence': None}], 'RWKIFKKIERVGQNVRDGIIKAGPAIQVLGTAKALGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 8.4e-12, 'percentage_identity': 73.0, 'result_sequence': None}], 'LPRDTSRCVGYHGYCIRSKVCPKPFAAFGTCSWRQKTCCVDTTSDFHTCQDKGGHCVSPKIRCLEEQLGLCPLKRWTCCKEI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qeu_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qeu', 'e_value': 1.4e-41, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 4.7e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qes_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qes', 'e_value': 2.2e-17, 'percentage_identity': 100.0, 'result_sequence': None}], 'ILGPVLGLVSNALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.00012, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.00012, 'percentage_identity': 82.4, 'result_sequence': None}], 'GIPCGESCVFIPCISSVIGCSCSSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 5.8e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 5.8e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.8e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 2.2e-12, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 4.2e-12, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.7e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.3e-10, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.9e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 1.9e-07, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 1.5e-06, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 1.4e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00011, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.00011, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.00052, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 0.00082, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.00082, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.001, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.002, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.0025, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.0032, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.0032, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.0035, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.005, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.005, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.005, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.005, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.01, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.012, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 0.013, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.027, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.031, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.064, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.064, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 0.068, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 0.077, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 0.077, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 0.077, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.094, 'percentage_identity': 33.3, 'result_sequence': None}], 'RWCFRVCYRGICYRKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 3.2e-08, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.00046, 'percentage_identity': 62.5, 'result_sequence': None}], 'KYYGNGVTCGKHSCSVDWGKATTCIINNGAMAWATGGHQGNHKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 9.8e-23, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 2.2e-14, 'percentage_identity': 70.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 6.7e-12, 'percentage_identity': 67.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 6.7e-12, 'percentage_identity': 67.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 8.6e-09, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.0013, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 0.0017, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 0.0033, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ljq_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2ljq', 'e_value': 0.065, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ljt_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2ljt', 'e_value': 0.098, 'percentage_identity': 44.7, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLAAKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 1.7e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 1.7e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 1.7e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 3.4e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 3.4e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 3.4e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 2.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 2.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 2.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'KKAAAAFAAFAAWFAAFAAAAKKKK': [{'chain_id': ['s2', 's1', 'S2', 'S1'], 'entity_id': 19, 'entry_entity': '4yuu_19', 'molecule_name': ['PEPTIDE CHAIN UNASSIGNED'], 'pdb_id': '4yuu', 'e_value': 0.053, 'percentage_identity': 72.2, 'result_sequence': None}], 'LFCKGGSCHFGGCPSHLIKVGSCFGFRSCCAWPWNA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg6_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg6', 'e_value': 3.5e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 1.2e-19, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.027, 'percentage_identity': 40.7, 'result_sequence': None}], 'SFLNFFKGAAKNLLAAGLDKLKCKISGTQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.064, 'percentage_identity': 51.9, 'result_sequence': None}], 'LTCDILGSTPACAAHCIARGYRGGWCDGQSVCNCRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 7.6e-12, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.6e-08, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.9e-08, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 2.4e-08, 'percentage_identity': 55.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 2.4e-07, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 3.9e-07, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 6.9e-07, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 8.5e-07, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 4.6e-06, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2.7e-05, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 0.012, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.023, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.051, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bi5_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bi5', 'e_value': 0.059, 'percentage_identity': 50.0, 'result_sequence': None}], 'TFFRLFNRSFTQALGKGGGKNLRIIRKGIHIIKKY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.00039, 'percentage_identity': 100.0, 'result_sequence': None}], 'KKLALAPKKLALLWKKLALALKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 1.5e-07, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 0.00015, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 0.00015, 'percentage_identity': 75.0, 'result_sequence': None}], 'NAPAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 5.3e-12, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 9.2e-12, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 1e-10, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 1.4e-10, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 7.9e-09, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 4e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 3.3e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 3.3e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 3.5e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz5_1', 'molecule_name': ['Neuropeptide Y', 'Pancreatic hormone'], 'pdb_id': '1tz5', 'e_value': 1e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l60_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2l60', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ppt_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ppt', 'e_value': 1.5e-05, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3s_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3s', 'e_value': 7.6e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3t_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3t', 'e_value': 7.6e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2h4b_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h4b', 'e_value': 7.6e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bf9_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2bf9', 'e_value': 0.00013, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljv_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ljv', 'e_value': 0.0011, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bba_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1bba', 'e_value': 0.0011, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k76_1', 'molecule_name': ['pGolemi'], 'pdb_id': '2k76', 'e_value': 0.043, 'percentage_identity': 50.0, 'result_sequence': None}], 'KWKLFKKIGIGKFL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0011, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0011, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0014, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0014, 'percentage_identity': 92.9, 'result_sequence': None}]}

## 4750 - 5000 10k

In [4]:
r14={'GLPALSSWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 4.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 4.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}], 'RGLRRLGRKIAHGVKKYGATVLRIIRIA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 2e-11, 'percentage_identity': 96.4, 'result_sequence': None}], 'GKYGFYTHVFRLKKWMQKVIDRFGG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5to3_2', 'molecule_name': ['Thrombin heavy chain', 'Thrombomodulin'], 'pdb_id': '5to3', 'e_value': 1.4e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 4, 'entry_entity': '2hnt_4', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hnt', 'e_value': 4.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '1riw_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1riw', 'e_value': 4.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '1bhx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bhx', 'e_value': 4.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6fjt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6fjt', 'e_value': 7.5e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3gic_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3gic', 'e_value': 7.5e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6rot_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6rot', 'e_value': 7.6e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '1h8i_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1h8i', 'e_value': 7.6e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2bvr_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bvr', 'e_value': 7.6e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ud9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ud9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4udw_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4udw', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ueh_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ueh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ufg_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ufg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ufe_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ufe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5ew2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5ew2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4dy7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4dy7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 2, 'entry_entity': '4e7r_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4e7r', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4dii_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4dii', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5do4_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5do4', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5ew1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5ew1', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4mlf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4mlf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ufd_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ufd', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4uff_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4uff', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rkj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4rkj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5jzy_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5jzy', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5lpd_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5lpd', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gim_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5gim', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4thn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4thn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5gds_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5gds', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '5afz_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5afz', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '5a2m_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5a2m', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '5ahg_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5ahg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5mjt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5mjt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5mm6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5mm6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4htc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4htc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3si4_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3si4', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3r3g_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3r3g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rm0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rm0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rmo_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rmo', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rmn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rmn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rly_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rly', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3qtv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qtv', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qgn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qgn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3si3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3si3', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3qwc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qwc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4lz4_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4lz4', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3sha_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3sha', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u8t_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u8t', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u69_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u69', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4lxb_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4lxb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4lz1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4lz1', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u98_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u98', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'A'], 'entity_id': 1, 'entry_entity': '4boh_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4boh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3utu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3utu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u8o_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u8o', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4e06_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4e06', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3qx5_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qx5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3qto_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qto', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3t5f_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3t5f', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4baq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4baq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3vxf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3vxf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3pmh_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3pmh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4bao_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4bao', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ban_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ban', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rmm_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rmm', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rlw_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rlw', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rml_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rml', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5l6n_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5l6n', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5jfd_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5jfd', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '4rko_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4rko', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ue7_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ue7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5lce_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5lce', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5jdu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5jdu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4dt7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4dt7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u9a_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u9a', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3sv2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3sv2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4ch8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ch8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4e05_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4e05', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '5e8e_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5e8e', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4dih_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4dih', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3uwj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3uwj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3tu7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3tu7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3u8r_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3u8r', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3shc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3shc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3rm2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3rm2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'A', 'C'], 'entity_id': 2, 'entry_entity': '5nhu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5nhu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5cmx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5cmx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4yes_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4yes', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '5af9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5af9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '5mls_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5mls', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '6tdt_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tdt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t55_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t55', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t3m_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t3m', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6tkl_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tkl', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t53_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t53', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'A'], 'entity_id': 2, 'entry_entity': '6t7h_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t7h', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6tkh_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tkh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6gn7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6gn7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6tki_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tki', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6ysx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6ysx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6yhg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6yhg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6ymp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6ymp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6yqv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6yqv', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6yn3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6yn3', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6y9h_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6y9h', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6v64_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6v64', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6yhj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6yhj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t89_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t89', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6pxq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6pxq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6px5_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6px5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6hsx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6hsx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t56_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t56', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t8a_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t8a', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6cym_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6cym', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6gbw_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6gbw', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t4a_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t4a', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t54_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t54', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t52_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t52', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '5afy_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '5afy', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6y02_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6y02', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6evv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6evv', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6eo7_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6eo7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 2, 'entry_entity': '6pxj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6pxj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '6eo9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6eo9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6eo8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6eo8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6eo6_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6eo6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t3q_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t3q', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6tkg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tkg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6yb6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6yb6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6z8x_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6z8x', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6tkj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6tkj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jz1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3jz1', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2uuj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2uuj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nrp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nrp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1o2g_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1o2g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1z8i_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1z8i', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '1w7g_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1w7g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tbz_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tbz', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1qhr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qhr', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tmt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tmt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1t4u_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1t4u', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tom_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tom', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ths_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ths', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1xmn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1xmn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2uuf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2uuf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2r2m_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2r2m', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3ldx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3ldx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4h6s_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4h6s', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jz2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3jz2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2uuk_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2uuk', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2pw8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pw8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c5n_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c5n', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'M', 'N', 'O'], 'entity_id': 3, 'entry_entity': '1dx5_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dx5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1dwd_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dwd', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1gj4_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1gj4', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c90_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c90', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a5g_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a5g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c5o_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c5o', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1dit_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dit', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c1v_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c1v', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1fpc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1fpc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ca8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ca8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2hgt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hgt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2b5t_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2b5t', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zfr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zfr', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2bxt_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bxt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2gde_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2gde', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3dhk_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3dhk', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3lu9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3lu9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2od3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2od3', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3hkj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3hkj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3eq0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3eq0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2hpp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hpp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2hwl_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hwl', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3f68_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3f68', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2gp9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2gp9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3dt0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3dt0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3htc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3htc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3hat_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3hat', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2pgq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pgq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2pgb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pgb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '2jh6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2jh6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3p70_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3p70', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2bvx_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bvx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3e6p_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3e6p', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3gis_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3gis', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ee0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3ee0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2feq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2feq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2bvs_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bvs', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '2jh0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2jh0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3egk_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3egk', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2fes_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2fes', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2hpq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hpq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zhf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zhf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3p17_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3p17', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3bei_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3bei', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zhq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zhq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3dd2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3dd2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3da9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3da9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zff_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zff', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zhw_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zhw', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2a2x_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2a2x', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zg0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zg0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3b9f_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3b9f', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 2, 'entry_entity': '3p6z_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3p6z', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tmb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tmb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1thr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1thr', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zgb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zgb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zdv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zdv', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zf0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zf0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1uma_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1uma', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zhe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zhe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1z8j_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1z8j', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ktt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ktt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1k22_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1k22', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hxf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hxf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1dwc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dwc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['2'], 'entity_id': 2, 'entry_entity': '1c4y_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c4y', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1qj6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qj6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1t4v_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1t4v', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1sg8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sg8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '1sr5_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sr5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nro_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nro', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1way_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1way', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tb6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tb6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1o5g_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1o5g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1qj1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qj1', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1twx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1twx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2a45_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2a45', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2afq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2afq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1tmu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tmu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1shh_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1shh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1o0d_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1o0d', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1uvs_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1uvs', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1qj7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qj7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1sgi_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sgi', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1sb1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sb1', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wbg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1wbg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3bef_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3bef', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zfp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zfp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3d49_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3d49', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2ank_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2ank', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zo3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zo3', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zgx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zgx', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3bf6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3bf6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zi2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zi2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2ziq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2ziq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zc9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zc9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2bxu_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bxu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3b23_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3b23', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zfq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zfq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1sfq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sfq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1qbv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qbv', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1mue_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mue', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1g32_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1g32', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['2'], 'entity_id': 2, 'entry_entity': '1c4u_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c4u', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1fph_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1fph', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hxe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hxe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3c27_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3c27', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1bmn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bmn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1dwb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dwb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c1u_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c1u', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1oyt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1oyt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nzq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nzq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '1nu7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nu7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nrq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nrq', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ook_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ook', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1lhd_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1lhd', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d3p_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d3p', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1rd3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1rd3', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c8z_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c8z', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a61_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a61', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1abi_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1abi', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a46_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a46', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ai8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ai8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d3t_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d3t', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a2c_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a2c', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c8w_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c8w', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nrs_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nrs', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ppb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ppb', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1mu8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mu8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '1p8v_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1p8v', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ihs_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ihs', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['2'], 'entity_id': 2, 'entry_entity': '1ny2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ny2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1lhg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1lhg', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1lhc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1lhc', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1no9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1no9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nrr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nrr', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ae8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ae8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K'], 'entity_id': 2, 'entry_entity': '1hlt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hlt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ad8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ad8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1aht_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1aht', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '1h8d_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1h8d', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hbt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hbt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hai_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hai', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1aix_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1aix', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c8x_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c8x', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d4p_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d4p', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1gj5_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1gj5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1bcu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bcu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '1hap_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hap', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1b5g_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1b5g', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K'], 'entity_id': 2, 'entry_entity': '1de7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1de7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hdt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hdt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1awh_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1awh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1bb0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bb0', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a4w_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a4w', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K'], 'entity_id': 2, 'entry_entity': '1bth_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bth', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1dm4_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dm4', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c1w_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c1w', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1dwe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1dwe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1awf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1awf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d3q_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d3q', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c8y_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c8y', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hgt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hgt', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hah_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hah', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1abj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1abj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1lhe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1lhe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1g30_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1g30', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1iht_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1iht', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1k21_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1k21', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1jmo_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1jmo', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1kts_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1kts', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1lhf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1lhf', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1jou_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1jou', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1mu6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mu6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1nrn_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nrn', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2znk_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2znk', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2zda_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2zda', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '3qlp_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qlp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3qdz_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3qdz', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a3b_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a3b', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1afe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1afe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3bv9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3bv9', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ay6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ay6', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '1hut_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hut', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c93_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2c93', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3dux_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3dux', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '1hao_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hao', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1bmm_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bmm', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ba8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ba8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1a3e_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a3e', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'E', 'F'], 'entity_id': 2, 'entry_entity': '1e0f_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1e0f', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1c5l_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c5l', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['2'], 'entity_id': 2, 'entry_entity': '1c4v_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1c4v', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '2jh5_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2jh5', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '2h9t_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2h9t', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4ch2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ch2', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3vxe_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3vxe', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4bah_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4bah', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4i7y_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4i7y', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4bak_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4bak', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4hfp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4hfp', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4bam_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4bam', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6z8w_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6z8w', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6t57_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6t57', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6i51_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6i51', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6ysj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6ysj', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zun_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zun', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zgo_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zgo', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zug_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zug', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '7ntu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '7ntu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zuw_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zuw', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zux_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zux', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6z8v_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6z8v', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zuh_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zuh', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zv7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zv7', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zuu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zuu', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '7kme_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '7kme', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['2'], 'entity_id': 2, 'entry_entity': '8kme_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '8kme', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '6zv8_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6zv8', 'e_value': 7.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6p9u_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6p9u', 'e_value': 7.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zgi_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1zgi', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ta6_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ta6', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrb_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1zrb', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zgv_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1zgv', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z71_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1z71', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sl3_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1sl3', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ta2_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ta2', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bdy_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2bdy', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3c1k_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3c1k', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nm6_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nm6', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nt1_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nt1', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9i_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d9i', 'e_value': 8.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rn6_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4rn6', 'e_value': 8.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '3sqe_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3sqe', 'e_value': 8.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '3sqh_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3sqh', 'e_value': 8.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '6v5t_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6v5t', 'e_value': 8.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '1nu9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1nu9', 'e_value': 8.3e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xm1_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1xm1', 'e_value': 8.4e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1doj_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1doj', 'e_value': 8.4e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1hag_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hag', 'e_value': 8.4e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h6t_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4h6t', 'e_value': 8.5e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jwt_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1jwt', 'e_value': 8.5e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k65_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3k65', 'e_value': 8.6e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pmb_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3pmb', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pma_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3pma', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1uvu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1uvu', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K', 'N'], 'entity_id': 5, 'entry_entity': '1ucy_5', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ucy', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1toc_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1toc', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K', 'N'], 'entity_id': 5, 'entry_entity': '1bbr_5', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bbr', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1avg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1avg', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2ody_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2ody', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2a1d_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2a1d', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1uvt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1uvt', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K'], 'entity_id': 2, 'entry_entity': '1tbq_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tbq', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K'], 'entity_id': 2, 'entry_entity': '1tbr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tbr', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ycp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ycp', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1vit_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1vit', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ets_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ets', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1etr_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1etr', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1hrt_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1hrt', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1a0h_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1a0h', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ett_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ett', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['HHH'], 'entity_id': 1, 'entry_entity': '7a0e_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '7a0e', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['HHH'], 'entity_id': 1, 'entry_entity': '7a0d_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '7a0d', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['HHH'], 'entity_id': 1, 'entry_entity': '7a0f_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '7a0f', 'e_value': 9.9e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nxp_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '3nxp', 'e_value': 1.1e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 3, 'entry_entity': '1mkx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mkx', 'e_value': 1.1e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 3, 'entry_entity': '1mkw_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mkw', 'e_value': 1.1e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4hzh_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '4hzh', 'e_value': 1.2e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5edk_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '5edk', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5edm_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '5edm', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bjr_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '6bjr', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6c2w_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '6c2w', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4o03_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '4o03', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4nzq_1', 'molecule_name': ['Prothrombin'], 'pdb_id': '4nzq', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2pux_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pux', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2pv9_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pv9', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3hki_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3hki', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3edx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3edx', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3hk6_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3hk6', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3hk3_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3hk3', 'e_value': 1.6e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2thf_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2thf', 'e_value': 2.1e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ocv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2ocv', 'e_value': 4.4e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '1vzq_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1vzq', 'e_value': 5.6e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ayy_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ayy', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4loy_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4loy', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '4ax9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ax9', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6gwe_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '6gwe', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2v3h_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2v3h', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2v3o_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2v3o', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ype_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ype', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1tq0_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tq0', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1eb1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1eb1', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3biv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3biv', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2anm_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2anm', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ypl_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ypl', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1vr1_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1vr1', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1qur_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1qur', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ypg_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ypg', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ypm_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ypm', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2a0q_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2a0q', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ypj_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ypj', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ypk_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ypk', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1tq7_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1tq7', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '3biu_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3biu', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ghy_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ghy', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2cf9_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2cf9', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ghw_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ghw', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d3d_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d3d', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ghv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ghv', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1ghx_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1ghx', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2cf8_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2cf8', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 1, 'entry_entity': '2cn0_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2cn0', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4az2_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4az2', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ayv_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '4ayv', 'e_value': 5.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eoj_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1eoj', 'e_value': 6.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d6w_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1d6w', 'e_value': 6.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g37_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1g37', 'e_value': 6.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1mh0_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1mh0', 'e_value': 6.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eol_1', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1eol', 'e_value': 6.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1b7x_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1b7x', 'e_value': 7.3e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '1id5_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1id5', 'e_value': 9.3e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3s7k_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3s7k', 'e_value': 1.2e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3s7h_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3s7h', 'e_value': 1.2e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1thp_2', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1thp', 'e_value': 1.2e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2pks_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2pks', 'e_value': 2.3e-08, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3po1_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '3po1', 'e_value': 2.3e-08, 'percentage_identity': 90.9, 'result_sequence': None}], 'GVVDILAGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 4.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 4.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 4.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 8.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 8.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 8.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 5.7e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 5.7e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 5.7e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'RWCVYRYVRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 3.3e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 4.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 4.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.9e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'RIPCQYEDATEDTICQQHCLPKGYSYGICVSYRCSCV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.022, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kp0_1', 'molecule_name': ['nasonin-1M'], 'pdb_id': '2kp0', 'e_value': 0.032, 'percentage_identity': 42.9, 'result_sequence': None}], 'KQIKIWFQNKKMKWKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 9.9e-06, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 9.9e-06, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xid_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xid', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1san_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1san', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '9ant_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '9ant', 'e_value': 1.5e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hoa_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '2hoa', 'e_value': 1.6e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1ahd_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1ahd', 'e_value': 1.6e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hom_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1hom', 'e_value': 1.6e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5z_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5z', 'e_value': 2e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5y_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5y', 'e_value': 2e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2h1k_3', 'molecule_name': ['Pancreas/duodenum homeobox protein 1'], 'pdb_id': '2h1k', 'e_value': 7.7e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmt_1', 'molecule_name': ['Homeobox protein BarH-like 1'], 'pdb_id': '2dmt', 'e_value': 0.00081, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m34_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2m34', 'e_value': 0.00097, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me0_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me0', 'e_value': 0.00097, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me6_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me6', 'e_value': 0.00097, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lp0_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2lp0', 'e_value': 0.0014, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msy_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2msy', 'e_value': 0.0016, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n8g_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2n8g', 'e_value': 0.0017, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '4uus_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uus', 'e_value': 0.0018, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1puf_3', 'molecule_name': ['Homeobox protein Hox-A9'], 'pdb_id': '1puf', 'e_value': 0.0018, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b8i_3', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '1b8i', 'e_value': 0.0018, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cyc_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4cyc', 'e_value': 0.0021, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4uut_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uut', 'e_value': 0.0021, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ftz_1', 'molecule_name': ['Segmentation protein fushi tarazu'], 'pdb_id': '1ftz', 'e_value': 0.0028, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjs_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjs', 'e_value': 0.0043, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjr_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjr', 'e_value': 0.0043, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '5zjt_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjt', 'e_value': 0.0043, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjq_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjq', 'e_value': 0.0043, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e1o_1', 'molecule_name': ['Hematopoietically-expressed homeobox protein HHEX'], 'pdb_id': '2e1o', 'e_value': 0.0048, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b72_3', 'molecule_name': ['Homeobox protein Hox-B1'], 'pdb_id': '1b72', 'e_value': 0.0048, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a03_1', 'molecule_name': ['T-cell leukemia homeobox protein 2'], 'pdb_id': '3a03', 'e_value': 0.009, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lnq_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3lnq', 'e_value': 0.012, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a02_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3a02', 'e_value': 0.012, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p81_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2p81', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4rbo_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '4rbo', 'e_value': 0.014, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3a01_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '3a01', 'e_value': 0.014, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '3hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '3hdd', 'e_value': 0.016, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kt0_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2kt0', 'e_value': 0.016, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwt_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2jwt', 'e_value': 0.017, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '1hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1hdd', 'e_value': 0.017, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3zob_1', 'molecule_name': ['Homeobox protein engrailed-2'], 'pdb_id': '3zob', 'e_value': 0.018, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4qtr_1', 'molecule_name': ['dualENH'], 'pdb_id': '4qtr', 'e_value': 0.019, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 6, 'entry_entity': '4xrs_6', 'molecule_name': ['Homeobox protein DLX-3'], 'pdb_id': '4xrs', 'e_value': 0.02, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m0c_1', 'molecule_name': ['Homeobox protein aristaless-like 4'], 'pdb_id': '2m0c', 'e_value': 0.02, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2vi6_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2vi6', 'e_value': 0.029, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4rdu_1', 'molecule_name': ['Homeobox protein DLX-5'], 'pdb_id': '4rdu', 'e_value': 0.03, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2djn_1', 'molecule_name': ['Homeobox protein DLX-5'], 'pdb_id': '2djn', 'e_value': 0.032, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '5lux_2', 'molecule_name': ['Homeobox protein CDX-1'], 'pdb_id': '5lux', 'e_value': 0.05, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7f_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7f', 'e_value': 0.053, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkx_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2lkx', 'e_value': 0.053, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7m_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7m', 'e_value': 0.053, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '6es2_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es2', 'e_value': 0.055, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '5lty_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '5lty', 'e_value': 0.055, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '6es3_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es3', 'e_value': 0.056, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1p7j_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1p7j', 'e_value': 0.062, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5eg0_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eg0', 'e_value': 0.063, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2hdd', 'e_value': 0.063, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5ef6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ef6', 'e_value': 0.064, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5no6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5no6', 'e_value': 0.064, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['B', 'A', 'J', 'G'], 'entity_id': 3, 'entry_entity': '5eea_3', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eea', 'e_value': 0.065, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5ego_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ego', 'e_value': 0.065, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6fvc_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '6fvc', 'e_value': 0.066, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cra_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '2cra', 'e_value': 0.071, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5edn_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5edn', 'e_value': 0.076, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ztr_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1ztr', 'e_value': 0.083, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l9r_1', 'molecule_name': ['Homeobox protein Nkx-3.1'], 'pdb_id': '2l9r', 'e_value': 0.092, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld5_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2ld5', 'e_value': 0.094, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7z_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2l7z', 'e_value': 0.096, 'percentage_identity': 56.2, 'result_sequence': None}], 'RWCVYAYSRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 8.9e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.2e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 1.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 1.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}], 'INLKAIAAMAKKLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.0016, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.0022, 'percentage_identity': 78.6, 'result_sequence': None}], 'KWKLFKKIEKVGQGIGAVLKVLTTGL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.0051, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.011, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 0.061, 'percentage_identity': 66.7, 'result_sequence': None}], 'GVLDTFKDVAIGVAKGAGTGVLKALLCKLDKSC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 7.9e-05, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.027, 'percentage_identity': 48.3, 'result_sequence': None}], 'CGGWKRLRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.0022, 'percentage_identity': 88.9, 'result_sequence': None}], 'GGLRSLGRKILRAWKKYGPIIVPIIRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.00025, 'percentage_identity': 65.4, 'result_sequence': None}], 'GFGCPFNQGACHRHCRSIGRRGGYCAGLFKQTCTCYSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 9.2e-16, 'percentage_identity': 76.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 2.6e-11, 'percentage_identity': 65.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 9.5e-10, 'percentage_identity': 64.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 7.3e-09, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4.3e-07, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 1e-06, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 5.2e-06, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 1.8e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0018, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.0022, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.023, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb6_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bb6', 'e_value': 0.047, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.057, 'percentage_identity': 40.0, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNLYR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 2.2e-08, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.0045, 'percentage_identity': 63.6, 'result_sequence': None}], 'ATCDLLSGFGVGDSACAAHCIARGNRGGYCNSKKVCVCRN': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.3e-15, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.6e-15, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 2e-15, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 4.6e-15, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 1.3e-12, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 3.8e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.6e-11, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 7.4e-10, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 4e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 5.3e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 9.1e-07, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.013, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.066, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.096, 'percentage_identity': 40.0, 'result_sequence': None}], 'GFFALIPKIIS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xc0_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '1xc0', 'e_value': 0.00015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kns_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '2kns', 'e_value': 0.00015, 'percentage_identity': 100.0, 'result_sequence': None}], 'FFHHIFRAIVHVAKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 6.2e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 1.5e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 8.4e-05, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 8.4e-05, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 8.4e-05, 'percentage_identity': 59.1, 'result_sequence': None}], 'RWKIFKKIEKMGGSYCNRRTGKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 0.086, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 0.086, 'percentage_identity': 100.0, 'result_sequence': None}], 'KKFGGGKTERARKAMAVE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rd7_1', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '2rd7', 'e_value': 3.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 5, 'entry_entity': '6h03_5', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '6h03', 'e_value': 4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 6, 'entry_entity': '6h04_6', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '6h04', 'e_value': 4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ojy_1', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '3ojy', 'e_value': 4e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'PTTKTYFPHF': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 0.0064, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 0.0064, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 0.0064, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sva_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6sva', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbc', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbd', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aet_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aet', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aeu', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aev', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 0.0065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eui_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4mqj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqj', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4mqk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqk', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7n', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5hu6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hu6', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7p', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2mhb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h2l_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e29_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e29', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ker_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kav_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kav', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5c6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5c6e', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2t', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6r2o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6r2o', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhe', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1shr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1shr', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yie_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yie', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y2z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y2z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2ri4_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2ri4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2rao_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2rao', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qu0_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2qu0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3lqd_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3lqd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gys_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gys', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2w72_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dn2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gqr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqr', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhe', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dhb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5x', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hrw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hrw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hhb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d4x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d4x', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqp', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3eu1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3eu1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlx', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d1a_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1a', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8h', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4p', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4v', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y85_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y85', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y35', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yev', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ibe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ibe', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1c7b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7b', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j41_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j41', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'R', 'U', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1jy7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1jy7', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iwh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1iwh', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns6', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s0h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s0h', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y46_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y46', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yg5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yg5', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8k', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8i', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4b', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeo_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeo', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4g', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4f', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1si4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1si4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '3b75_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3b75', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlv', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g0b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0b', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fn3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1x', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rqa_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rqa', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a00_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a00', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cls', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a01_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a01', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hba_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hba', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0u', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1m9p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1m9p', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gzx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gzx', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hds_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '1hds', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1fdh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fdh', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3fh9', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrf_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3vre_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kah_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kah', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jjq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jjq', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 0.0067, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 0.0068, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1p', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1abw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1abw', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 0.017, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2z6n_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '2z6n', 'e_value': 0.017, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wmu_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1wmu', 'e_value': 0.017, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v75_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1v75', 'e_value': 0.017, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 0.017, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3fs4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3fs4', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3a59_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3a59', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zfb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zfb', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbr_1', 'molecule_name': ['Hemoglobin subunit alpha-D'], 'pdb_id': '1hbr', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3o', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 0.046, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '3w4u_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '3w4u', 'e_value': 0.047, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 0.047, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 0.047, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1jeb_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '1jeb', 'e_value': 0.047, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 0.047, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 0.088, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 0.088, 'percentage_identity': 90.0, 'result_sequence': None}], 'FFHHIFRAIVHVPKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 3.2e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.8e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.8e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 2.4e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}], 'KWKVFKKIEKKWKVFKKIEKAGPKWKVFKKIEK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2igr_1', 'molecule_name': ['Anticancer peptide CB1a'], 'pdb_id': '2igr', 'e_value': 3.5e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'EAEEDGDLQCLCVKTTSQVRPRHITSLEVIKAGPHCPTAQLIATLKNGRKICLDLQAPLYKKIIKKLLES': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4r9w_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9w', 'e_value': 8e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'B', 'A'], 'entity_id': 1, 'entry_entity': '4r9y_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9y', 'e_value': 8e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'U', 'I', 'C', 'X', 'O', 'F', 'L'], 'entity_id': 3, 'entry_entity': '4rau_3', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4rau', 'e_value': 8e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9q_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9q', 'e_value': 8e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1rhp_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1rhp', 'e_value': 8e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9s_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9s', 'e_value': 4.1e-33, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9r_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9r', 'e_value': 7e-32, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4hsv_1', 'molecule_name': ['Platelet factor 4 variant'], 'pdb_id': '4hsv', 'e_value': 1.9e-31, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1pfm_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1pfm', 'e_value': 1e-28, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1pfn_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1pfn', 'e_value': 1e-28, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1plf_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1plf', 'e_value': 7.4e-24, 'percentage_identity': 74.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1nap_1', 'molecule_name': ['Neutrophil-activating peptide 2'], 'pdb_id': '1nap', 'e_value': 1.4e-15, 'percentage_identity': 59.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1tvx_1', 'molecule_name': ['Neutrophil-activating peptide 2(74)'], 'pdb_id': '1tvx', 'e_value': 1.5e-15, 'percentage_identity': 59.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f9p_1', 'molecule_name': ['Platelet basic protein'], 'pdb_id': '1f9p', 'e_value': 2.5e-15, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6wzl_4', 'molecule_name': ['C-X-C motif chemokine'], 'pdb_id': '6wzl', 'e_value': 3.8e-15, 'percentage_identity': 59.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mgs_1', 'molecule_name': ['C-X-C motif chemokine 5'], 'pdb_id': '2mgs', 'e_value': 5.9e-14, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1msg_1', 'molecule_name': ['Growth-regulated alpha protein'], 'pdb_id': '1msg', 'e_value': 2.2e-13, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1msh_1', 'molecule_name': ['Growth-regulated alpha protein'], 'pdb_id': '1msh', 'e_value': 2.2e-13, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1mgs_1', 'molecule_name': ['Growth-regulated alpha protein'], 'pdb_id': '1mgs', 'e_value': 2.3e-13, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3n52_1', 'molecule_name': ['C-X-C motif chemokine 2'], 'pdb_id': '3n52', 'e_value': 7.7e-13, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1mi2_1', 'molecule_name': ['C-X-C motif chemokine 2'], 'pdb_id': '1mi2', 'e_value': 7.7e-13, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rod_1', 'molecule_name': ['IL-8(9-77)'], 'pdb_id': '1rod', 'e_value': 2.1e-12, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6wzj_3', 'molecule_name': ['C-X-C motif chemokine'], 'pdb_id': '6wzj', 'e_value': 3.9e-12, 'percentage_identity': 45.3, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6wzk_4', 'molecule_name': ['C-X-C motif chemokine'], 'pdb_id': '6wzk', 'e_value': 1.4e-11, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1qnk_1', 'molecule_name': ['C-X-C motif chemokine 2'], 'pdb_id': '1qnk', 'e_value': 1.5e-11, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ob5_1', 'molecule_name': ['GRO-beta(5-73)'], 'pdb_id': '5ob5', 'e_value': 1.7e-11, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xdx_1', 'molecule_name': ['IL-8(8-77)'], 'pdb_id': '4xdx', 'e_value': 8.8e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d14_1', 'molecule_name': ['IL-8(8-77)'], 'pdb_id': '5d14', 'e_value': 8.8e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 4, 'entry_entity': '6lfm_4', 'molecule_name': ['Interleukin-8'], 'pdb_id': '6lfm', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2il8_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '2il8', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3il8_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '3il8', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ilp_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1ilp', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ikl_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1ikl', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1il8_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1il8', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ikm_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1ikm', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ilq_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1ilq', 'e_value': 9.1e-10, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wdz_1', 'molecule_name': ['IL-8(9-77)'], 'pdb_id': '5wdz', 'e_value': 1e-09, 'percentage_identity': 38.1, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6wzm_3', 'molecule_name': ['IL-8(9-77)'], 'pdb_id': '6wzm', 'e_value': 1e-09, 'percentage_identity': 38.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6xmn_1', 'molecule_name': ['IL-8(9-77)'], 'pdb_id': '6xmn', 'e_value': 1e-09, 'percentage_identity': 38.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6n2u_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '6n2u', 'e_value': 1.3e-09, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6lfo_4', 'molecule_name': ['IL-8(9-77)'], 'pdb_id': '6lfo', 'e_value': 2.7e-09, 'percentage_identity': 37.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1icw_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1icw', 'e_value': 4.3e-08, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1qe6_1', 'molecule_name': ['Interleukin-8'], 'pdb_id': '1qe6', 'e_value': 5.3e-08, 'percentage_identity': 35.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o80_1', 'molecule_name': ['C-X-C motif chemokine 10'], 'pdb_id': '1o80', 'e_value': 2.7e-06, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lv9_1', 'molecule_name': ['C-X-C motif chemokine 10'], 'pdb_id': '1lv9', 'e_value': 2.7e-06, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o7z_1', 'molecule_name': ['C-X-C motif chemokine 10'], 'pdb_id': '1o7z', 'e_value': 2.7e-06, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o7y_1', 'molecule_name': ['C-X-C motif chemokine 10'], 'pdb_id': '1o7y', 'e_value': 2.7e-06, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6vgj_1', 'molecule_name': ['C-X-C motif chemokine 13'], 'pdb_id': '6vgj', 'e_value': 0.00039, 'percentage_identity': 31.1, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '5cbe_3', 'molecule_name': ['C-X-C motif chemokine 13'], 'pdb_id': '5cbe', 'e_value': 0.0004, 'percentage_identity': 31.1, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '5cba_3', 'molecule_name': ['C-X-C motif chemokine 13'], 'pdb_id': '5cba', 'e_value': 0.0004, 'percentage_identity': 31.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7jny_1', 'molecule_name': ['C-X-C motif chemokine 13'], 'pdb_id': '7jny', 'e_value': 0.0004, 'percentage_identity': 31.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2r3z_1', 'molecule_name': ['C-X-C motif chemokine 10'], 'pdb_id': '2r3z', 'e_value': 0.00056, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4mhe_1', 'molecule_name': ['C-C motif chemokine 18'], 'pdb_id': '4mhe', 'e_value': 0.05, 'percentage_identity': 32.8, 'result_sequence': None}], 'GVAKFGKAAAHFGKGWIKEMLNS': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.014, 'percentage_identity': 43.5, 'result_sequence': None}], 'FQWQRNPRKVR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.014, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.014, 'percentage_identity': 90.9, 'result_sequence': None}], 'ILGPVLGLVGDTLGDLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.0013, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.0013, 'percentage_identity': 76.5, 'result_sequence': None}], 'AIGSILGRLAKGLPTLISWIKNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.003, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.003, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0031, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0031, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0031, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0031, 'percentage_identity': 59.1, 'result_sequence': None}], 'KRGFGKKLRKRLKKFRNSIKKRLKNFNVVIPIPLPG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.047, 'percentage_identity': 57.1, 'result_sequence': None}], 'GLPICGETCVGGTCNTPGCFCTWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 1.4e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 1.7e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.7e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 4.7e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 7e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 7e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 8.6e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 2e-12, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 4.2e-11, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 4.2e-11, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 4.2e-11, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 4.5e-11, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 3e-10, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 3.3e-10, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 3.3e-10, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 7.4e-10, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 1.4e-09, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 2.1e-09, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 2.1e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 1.1e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 2e-08, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 6.7e-08, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 8.2e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 1e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 1e-07, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 3.4e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 1.8e-06, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 1.4e-05, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 8.7e-05, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.0016, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.0022, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.0056, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.012, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 0.062, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 0.062, 'percentage_identity': 48.4, 'result_sequence': None}], 'KKLFKKILKYLGIGAVLKVLTTGLKDEL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.019, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.043, 'percentage_identity': 73.9, 'result_sequence': None}], 'GLPVCGETCTLGTCYTQGCTCSWPICKRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 5.8e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 9.8e-13, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 2.3e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 3e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 3e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 4.6e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 8.8e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 8.8e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 1.3e-10, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 1.3e-10, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 1.3e-10, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 1.7e-10, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 4e-10, 'percentage_identity': 76.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 2.7e-09, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 1.2e-08, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 1.2e-08, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 1.2e-08, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 1.3e-08, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 3.6e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 6.1e-08, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 1e-07, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 1.6e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 1.6e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 1.9e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 5.7e-07, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 1.4e-06, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 3.9e-06, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 6.3e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 1.1e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 1.4e-05, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 3.3e-05, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.0069, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 0.013, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 0.013, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 0.049, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 0.06, 'percentage_identity': 38.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 0.092, 'percentage_identity': 40.7, 'result_sequence': None}], 'FKCRRWQWRMKKLGKRSKNKGFKEQAKSLLKWILD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 4.6e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.00092, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.025, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.025, 'percentage_identity': 76.5, 'result_sequence': None}], 'RLGNFFRKVKEKIGGGLKKVGQKIKDFLGNLVPRTAS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 1.9e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 1.9e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 1.9e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 1.9e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 1.9e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.0022, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.036, 'percentage_identity': 59.1, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDWGQASGCIGQTVVGGWLGGAIVGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 3.6e-08, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 1.7e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 1.7e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.7e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 6.8e-06, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 1.5e-05, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.0045, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.0045, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.0045, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.022, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 0.028, 'percentage_identity': 40.9, 'result_sequence': None}], 'IIGPVLGLIGKALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 3.5e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 3.5e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'KHVGKAALTHYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 9.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 6.5e-05, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 9.5e-05, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.0043, 'percentage_identity': 66.7, 'result_sequence': None}], 'GILDKLKEFGISAARGVAQSLLNTASCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 7.1e-07, 'percentage_identity': 56.8, 'result_sequence': None}], 'KNIGNSVSCLRNKGVCMPGKCAPKMKQIGTCGMPQVKCCKRK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 1.6e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 1.6e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 1.6e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 1.6e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 2.4e-07, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.5e-06, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.7e-05, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00014, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.0023, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.0059, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.016, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.026, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.1, 'percentage_identity': 36.4, 'result_sequence': None}], 'EPFSSVAATCQTPKIACKNGDKNCHQSHGAVSLTMCKLTSGKYPNCRYKEKR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.3e-23, 'percentage_identity': 98.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.1e-09, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.1e-09, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.1e-09, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.1e-09, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.1e-09, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 1.6e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ql2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql2', 'e_value': 1.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ql1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql1', 'e_value': 1.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fev_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fev', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5d_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5d', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5h_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5h', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjq_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjq', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5e_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5e', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjr_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjr', 'e_value': 2.4e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ygw_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ygw', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4k7l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7l', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j82_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j82', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o36_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o36', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4okf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4okf', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o37_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o37', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbd_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbd', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3p_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3p', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3or0_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3or0', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3oqz_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqz', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3l', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j81_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j81', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j80_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j80', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3m_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3m', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 2, 'entry_entity': '3oqy_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqy', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j7z_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j7z', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbc_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbc', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbh_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbh', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbe_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbe', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbg_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbg', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbi_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbi', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbf', 'e_value': 2.5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '2rln_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rln', 'e_value': 2.6e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssc', 'e_value': 2.6e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4srn', 'e_value': 2.7e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3srn', 'e_value': 2.7e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4k7m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7m', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6qmn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qmn', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssa', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssb', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1srn', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ras_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ras', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rar_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rar', 'e_value': 2.8e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qh3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4qh3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ogh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5ogh', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obe', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obc', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5old_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5old', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5et4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5et4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5e_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5e', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5f', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rte_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rte', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s0q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s0q', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsd', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyp', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyn', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyz', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsd', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j65_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j65', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j66_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j66', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j67', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j68_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j68', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j61_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j61', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rh1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rh1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j6a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j6a', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rn3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rn3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j62_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j62', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3qsk', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ao1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ao1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xog_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xog', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j63_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j63', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j69_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j69', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j5z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j5z', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l55_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4l55', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mxf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4mxf', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j64_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j64', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rid', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jmg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jmg', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s18_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s18', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obd', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j60', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xoi_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xoi', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jlg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jlg', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d97_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d97', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zz4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4zz4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jml_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jml', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6eto_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6eto', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rsa', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvv', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6gok_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6gok', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6f60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6f60', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etr', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xvx', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etp', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6qe9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qe9', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etm', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhc', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nj7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5nj7', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d6u_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d6u', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhf', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhd', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xw0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xw0', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhe', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5na9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5na9', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etl', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvx', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etk', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvu', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etn', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvw', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e3w', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2qca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2qca', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p43_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p43', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4q', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6s_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6s', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wbu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1wbu', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rns_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rns', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ot4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ot4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ooh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ooh', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8y', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3jw1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3jw1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1a2w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a2w', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbx', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rsm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rsm', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 2, 'entry_entity': '1f0v_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1f0v', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eow_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eow', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9x', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eid', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9v', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eic', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dib_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dib', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dic', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g4w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4w', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8r', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7w', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blp', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p47_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p47', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p4a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p4a', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p49_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p49', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2e33_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e33', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5g', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5k_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5k', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5m', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5l', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev5', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev2', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g4x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4x', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di8', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euy_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euy', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euz', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fkz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fkz', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev6', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8q', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lxo_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3lxo', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p45_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p45', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p46_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p46', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p42_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p42', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p48_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p48', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i67', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7x', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8y', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6q', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxg', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxh', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eux_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3eux', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh6', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl0', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6h', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7y', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8z', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6j', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6f', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2aas_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2aas', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0o', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1eos_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eos', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf2', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jn4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jn4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4o', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4p', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xps_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xps', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6d_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6d', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xpt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xpt', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1u1b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1u1b', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1qhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1qhc', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6p', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d7b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d7b', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh5', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blz', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6o', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a1r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3a1r', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1bzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bzq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0c_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0c', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c8w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c8w', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fs3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fs3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvu', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvt', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izp', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0f', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnm', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnd', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rno_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rno', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afu', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afk', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnv', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afl', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rny_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rny', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rbb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbb', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rob_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rob', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbn', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ruv', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0n_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0n', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izr', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf7', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0h', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf3', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbj_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbj', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnn', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rtb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rtb', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbw', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rha_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rha', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rca', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnc', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rcn_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rcn', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpg', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnu', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rhb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rhb', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aqp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1aqp', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eie_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eie', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnz', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnx', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rta_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rta', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0b', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1dfj_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dfj', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rph_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rph', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpf', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnw', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1js0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1js0', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvv', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kh8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kh8', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0m', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf8', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf5', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p44_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p44', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5i_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5i', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzr', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bel_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bel', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev4', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev0', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4peq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4peq', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8v', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4pou_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4pou', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g90_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g90', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6yo1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6yo1', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rsa', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '9rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '8rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rsa', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rsa', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rat', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rsa', 'e_value': 2.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2apq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2apq', 'e_value': 3.1e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 4.3e-08, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mx8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mx8', 'e_value': 5e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwr', 'e_value': 5.1e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwq', 'e_value': 5.1e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2oqf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2oqf', 'e_value': 6.2e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di9', 'e_value': 6.2e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dy5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dy5', 'e_value': 7.5e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2op2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2op2', 'e_value': 7.5e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di7', 'e_value': 7.5e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1lsq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1lsq', 'e_value': 7.5e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4n4c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '4n4c', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsk', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4kxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4kxh', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsk', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tq9_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1tq9', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '2q4g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2q4g', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0j', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rra_1', 'molecule_name': ['Ribonuclease pancreatic beta-type'], 'pdb_id': '1rra', 'e_value': 1.1e-07, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1b6v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1b6v', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k11_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2k11', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djq', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djo_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djo', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djp', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3f8g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3f8g', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0m', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djv_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djv', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djx_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djx', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '1z7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z7x', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1bsr_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1bsr', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1x_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n1x', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n3z_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n3z', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5c', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11ba_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11ba', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r3m_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r3m', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5d_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5d', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11bg_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11bg', 'e_value': 1.1e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e21_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1e21', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0l', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1h8x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1h8x', 'e_value': 1.1e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymn', 'e_value': 1.6e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0o', 'e_value': 2e-07, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymr', 'e_value': 2e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymw', 'e_value': 2.4e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5q', 'e_value': 2.4e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsp', 'e_value': 2.9e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3bcp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcp', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bco_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bco', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bcm_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcm', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qwq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1qwq', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y94_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y94', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y92_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y92', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lfj_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '2lfj', 'e_value': 2.9e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5p', 'e_value': 4.2e-07, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dza_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dza', 'e_value': 9.4e-07, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3tsr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3tsr', 'e_value': 1.1e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arj_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arj', 'e_value': 6.6e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arl_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arl', 'e_value': 6.6e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ar6_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ar6', 'e_value': 1.1e-05, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5ark_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ark', 'e_value': 1.2e-05, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '2rnf', 'e_value': 1.6e-05, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '1rnf', 'e_value': 1.6e-05, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gv7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1gv7', 'e_value': 2e-05, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 0.0026, 'percentage_identity': 35.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 0.003, 'percentage_identity': 35.8, 'result_sequence': None}], 'RGRGRGRGRG': [{'chain_id': ['QQQ', 'QQQQ', 'CCC', 'JJJJ', 'JJ', 'CC', 'Q', 'C', 'J', 'JJJ', 'QQ', 'CCCC'], 'entity_id': 3, 'entry_entity': '4wzj_3', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '4wzj', 'e_value': 0.025, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F', 'f'], 'entity_id': 6, 'entry_entity': '3jb9_6', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '3jb9', 'e_value': 0.025, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'Z', 'n'], 'entity_id': 25, 'entry_entity': '5o9z_25', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '5o9z', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'X'], 'entity_id': 5, 'entry_entity': '3pgw_5', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '3pgw', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AA', 'BY', 'CQ', 'Co', 'BQ', 'CA', 'Ag', 'Cg', 'Aw', 'CY', 'AQ', 'Ao', 'BA', 'Bg', 'Bw', 'BI', 'CI', 'AI', 'AY', 'Bo'], 'entity_id': 1, 'entry_entity': '4v98_1', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '4v98', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'j', 'b'], 'entity_id': 14, 'entry_entity': '6ahd_14', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6ahd', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['n'], 'entity_id': 11, 'entry_entity': '6y53_11', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6y53', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['n'], 'entity_id': 16, 'entry_entity': '6y5q_16', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6y5q', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'j', 'g'], 'entity_id': 10, 'entry_entity': '6ah0_10', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6ah0', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '5xjl_2', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '5xjl', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['11', '21', '51', '41'], 'entity_id': 11, 'entry_entity': '6qx9_11', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6qx9', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['51', '41'], 'entity_id': 3, 'entry_entity': '6qw6_3', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6qw6', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'n'], 'entity_id': 28, 'entry_entity': '5mqf_28', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '5mqf', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['j', 'c'], 'entity_id': 29, 'entry_entity': '5xjc_29', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '5xjc', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['j', 'c'], 'entity_id': 31, 'entry_entity': '6icz_31', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6icz', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['j', 'c'], 'entity_id': 21, 'entry_entity': '6id1_21', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6id1', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['j', 'c'], 'entity_id': 22, 'entry_entity': '6id0_22', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '6id0', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'p'], 'entity_id': 9, 'entry_entity': '3jcr_9', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '3jcr', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4f7u_1', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '4f7u', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b34_1', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '1b34', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'B', 'N', 'O'], 'entity_id': 4, 'entry_entity': '3cw1_4', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '3cw1', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['c', 'n'], 'entity_id': 23, 'entry_entity': '7a5p_23', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '7a5p', 'e_value': 0.026, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C4', 'C5', 'C2', 'C1', 'C6', 'C3'], 'entity_id': 10, 'entry_entity': '5vyc_10', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '5vyc', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 54, 'entry_entity': '5lzv_54', 'molecule_name': ['uS5'], 'pdb_id': '5lzv', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 54, 'entry_entity': '5lzx_54', 'molecule_name': ['uS5'], 'pdb_id': '5lzx', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 53, 'entry_entity': '5lzs_53', 'molecule_name': ['uS5'], 'pdb_id': '5lzs', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 54, 'entry_entity': '5lzw_54', 'molecule_name': ['uS5'], 'pdb_id': '5lzw', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 52, 'entry_entity': '5lzy_52', 'molecule_name': ['uS5'], 'pdb_id': '5lzy', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 54, 'entry_entity': '5lzt_54', 'molecule_name': ['uS5'], 'pdb_id': '5lzt', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 53, 'entry_entity': '5lzu_53', 'molecule_name': ['uS5'], 'pdb_id': '5lzu', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 53, 'entry_entity': '4ujd_53', 'molecule_name': ['40S RIBOSOMAL PROTEIN US5'], 'pdb_id': '4ujd', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 53, 'entry_entity': '4ujc_53', 'molecule_name': ['40S RIBOSOMAL PROTEIN US5'], 'pdb_id': '4ujc', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 11, 'entry_entity': '6ybw_11', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6ybw', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['BC'], 'entity_id': 52, 'entry_entity': '5aj0_52', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '5aj0', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['SC'], 'entity_id': 52, 'entry_entity': '6y0g_52', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6y0g', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['SC'], 'entity_id': 52, 'entry_entity': '6y2l_52', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6y2l', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 6, 'entry_entity': '6g4s_6', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6g4s', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 54, 'entry_entity': '5lzz_54', 'molecule_name': ['uS5'], 'pdb_id': '5lzz', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 11, 'entry_entity': '6ybd_11', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6ybd', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 13, 'entry_entity': '6zmw_13', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6zmw', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['d'], 'entity_id': 3, 'entry_entity': '6zon_3', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6zon', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['SC'], 'entity_id': 52, 'entry_entity': '6y57_52', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6y57', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 4, 'entry_entity': '6zok_4', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6zok', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3G'], 'entity_id': 69, 'entry_entity': '6ip5_69', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6ip5', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3G'], 'entity_id': 68, 'entry_entity': '6ip8_68', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6ip8', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Ab'], 'entity_id': 14, 'entry_entity': '7o80_14', 'molecule_name': ['Ribosomal protein uS5'], 'pdb_id': '7o80', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3G'], 'entity_id': 68, 'entry_entity': '6ip6_68', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '6ip6', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Ab'], 'entity_id': 14, 'entry_entity': '7o7y_14', 'molecule_name': ['Ribosomal protein uS5'], 'pdb_id': '7o7y', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 6, 'entry_entity': '7jqc_6', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '7jqc', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Ab'], 'entity_id': 15, 'entry_entity': '7o81_15', 'molecule_name': ['Ribosomal protein uS5'], 'pdb_id': '7o81', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AJ'], 'entity_id': 53, 'entry_entity': '5t2c_53', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '5t2c', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 6, 'entry_entity': '7jqb_6', 'molecule_name': ['40S ribosomal protein S2'], 'pdb_id': '7jqb', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Ab'], 'entity_id': 14, 'entry_entity': '7o7z_14', 'molecule_name': ['Ribosomal protein uS5'], 'pdb_id': '7o7z', 'e_value': 0.063, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7ao8_1', 'molecule_name': ['Methyl-CpG-binding domain protein 2'], 'pdb_id': '7ao8', 'e_value': 0.088, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7ao9_1', 'molecule_name': ['Methyl-CpG-binding domain protein 2'], 'pdb_id': '7ao9', 'e_value': 0.088, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7aoa_1', 'molecule_name': ['Methyl-CpG-binding domain protein 2'], 'pdb_id': '7aoa', 'e_value': 0.088, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLASKVANKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVPICGETCTLGTCYTAGCSCSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 1.5e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 1.3e-12, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 5.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 4.8e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 4.8e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 9.1e-11, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 1.1e-10, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.4e-10, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 1.7e-10, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 2.1e-10, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 2.1e-10, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.6e-10, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 6.3e-10, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 2.7e-09, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 6.4e-09, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 6.4e-09, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 6.4e-09, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 6.8e-09, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 2.3e-08, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 3.5e-08, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 3.5e-08, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 5e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 3.7e-07, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 3.7e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 7.1e-07, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 2e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 3.8e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 6e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 9.4e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 2.1e-05, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 7.5e-05, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.00021, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.0018, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 0.0035, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 0.0035, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 0.012, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 0.015, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 0.055, 'percentage_identity': 45.2, 'result_sequence': None}], 'GIGKFLKSKGKFGKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.042, 'percentage_identity': 73.3, 'result_sequence': None}], 'KFHHIFRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 2.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 2.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 2.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 4e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 2.2e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.5e-05, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.5e-05, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.5e-05, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.095, 'percentage_identity': 55.6, 'result_sequence': None}], 'KTCENLADTY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 0.0028, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 0.012, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 0.024, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 0.024, 'percentage_identity': 90.0, 'result_sequence': None}], 'KSSAYSLQMGATAIKQVKKLFKKWGW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ytr_1', 'molecule_name': ['Bacteriocin plantaricin-A'], 'pdb_id': '1ytr', 'e_value': 5.4e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'RIWFQNRRMRWRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 0.00032, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 0.00032, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 0.00053, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xid_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xid', 'e_value': 0.00053, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 0.00053, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 0.00053, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1san_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1san', 'e_value': 0.00054, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2h1k_3', 'molecule_name': ['Pancreas/duodenum homeobox protein 1'], 'pdb_id': '2h1k', 'e_value': 0.00054, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '9ant_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '9ant', 'e_value': 0.00054, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hoa_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '2hoa', 'e_value': 0.00059, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1ahd_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1ahd', 'e_value': 0.00059, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hom_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1hom', 'e_value': 0.00059, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5z_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5z', 'e_value': 0.00076, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5y_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5y', 'e_value': 0.00076, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m34_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2m34', 'e_value': 0.0019, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me0_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me0', 'e_value': 0.0019, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me6_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me6', 'e_value': 0.0019, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a03_1', 'molecule_name': ['T-cell leukemia homeobox protein 2'], 'pdb_id': '3a03', 'e_value': 0.0027, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7f_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7f', 'e_value': 0.0033, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkx_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2lkx', 'e_value': 0.0033, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7m_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7m', 'e_value': 0.0033, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e1o_1', 'molecule_name': ['Hematopoietically-expressed homeobox protein HHEX'], 'pdb_id': '2e1o', 'e_value': 0.0034, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lnq_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3lnq', 'e_value': 0.0037, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a02_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3a02', 'e_value': 0.0039, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3a01_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '3a01', 'e_value': 0.0044, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m0c_1', 'molecule_name': ['Homeobox protein aristaless-like 4'], 'pdb_id': '2m0c', 'e_value': 0.0048, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmu_1', 'molecule_name': ['Homeobox protein goosecoid'], 'pdb_id': '2dmu', 'e_value': 0.008, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cmy_1', 'molecule_name': ['Paired box protein Pax-3'], 'pdb_id': '3cmy', 'e_value': 0.0093, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmt_1', 'molecule_name': ['Homeobox protein BarH-like 1'], 'pdb_id': '2dmt', 'e_value': 0.012, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cue_1', 'molecule_name': ['Paired box protein Pax-6'], 'pdb_id': '2cue', 'e_value': 0.012, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n8g_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2n8g', 'e_value': 0.014, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lp0_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2lp0', 'e_value': 0.051, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msy_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2msy', 'e_value': 0.057, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1puf_3', 'molecule_name': ['Homeobox protein Hox-A9'], 'pdb_id': '1puf', 'e_value': 0.065, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '1jgg_3', 'molecule_name': ['Segmentation protein even-skipped'], 'pdb_id': '1jgg', 'e_value': 0.068, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b72_3', 'molecule_name': ['Homeobox protein Hox-B1'], 'pdb_id': '1b72', 'e_value': 0.081, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '4uus_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uus', 'e_value': 0.086, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b8i_3', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '1b8i', 'e_value': 0.091, 'percentage_identity': 61.5, 'result_sequence': None}], 'FRKSKEKIGKEFKRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 1.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'MLAKIKAMIKKFPNPYTLAAKLTTYEINWYKQQYGRYPWERPVA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5l82_1', 'molecule_name': ['Enterococcin K1'], 'pdb_id': '5l82', 'e_value': 0.00079, 'percentage_identity': 53.1, 'result_sequence': None}], 'GFKEVLKAGLGSLVKGIPAHVAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l5r_1', 'molecule_name': ['Antimicrobial peptide Alyteserin-1C'], 'pdb_id': '2l5r', 'e_value': 4.4e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'GLFGKLIKKFLRKAISYAVKKARGKH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9p_1', 'molecule_name': ['M-zodatoxin-Lt2a'], 'pdb_id': '2g9p', 'e_value': 1.6e-10, 'percentage_identity': 96.2, 'result_sequence': None}], 'KKIMRTFLRRISKKILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 1.5e-05, 'percentage_identity': 85.0, 'result_sequence': None}], 'RPKPRPRPRPE': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.0085, 'percentage_identity': 90.0, 'result_sequence': None}], 'GIFSKLAGKKLKNLLISG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 5.8e-07, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 5.8e-07, 'percentage_identity': 94.4, 'result_sequence': None}], 'VCRTGRSRWRDVCRNFMRRYQSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l9l_1', 'molecule_name': ['Granulysin'], 'pdb_id': '1l9l', 'e_value': 3.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}], 'AGLQFPVGRVHRLLRK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 1.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 1.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 11, 'entry_entity': '6z6p_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6z6p', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 1.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 1.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6g0l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 2.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 2.4e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 2.5e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 2.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 2.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 2.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '2f8n_7', 'molecule_name': ['Histone H2A type 1-H'], 'pdb_id': '2f8n', 'e_value': 2.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 2.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 2.8e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 2.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 2.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 2.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 2.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 2.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 3e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 3e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 3.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 3.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 3.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 3.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 3.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 5.5e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 7.4e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 7.4e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 2.8e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 2.8e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 2.8e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 2.8e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 4.6e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 5.1e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 6.2e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 6.2e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 6.3e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 6.7e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7dlx_1', 'molecule_name': ['Histone H2A', 'Histone H2B'], 'pdb_id': '7dlx', 'e_value': 9.5e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k0c_1', 'molecule_name': ['Histone H2A.V', 'Histone H2B 2'], 'pdb_id': '6k0c', 'e_value': 0.00017, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 0.00059, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 0.0006, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 0.00061, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 0.00068, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b31_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b31', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b32_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b32', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 0.00069, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5chl_2', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5chl', 'e_value': 0.00099, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4nft_1', 'molecule_name': ['Histone H2B type 2-E', 'Histone H2A.Z'], 'pdb_id': '4nft', 'e_value': 0.0011, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jss_1', 'molecule_name': ['Histone H2A.Z', 'Histone H2B.1'], 'pdb_id': '2jss', 'e_value': 0.0044, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4m6b_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '4m6b', 'e_value': 0.0045, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6ae8_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '6ae8', 'e_value': 0.0045, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7bp6_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp6', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7bp2_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp2', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '7bp4_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp4', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bp5_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp5', 'e_value': 0.0057, 'percentage_identity': 73.3, 'result_sequence': None}], 'GFKRIVQAIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00025, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00025, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00025, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00025, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00025, 'percentage_identity': 93.8, 'result_sequence': None}], 'VTCDLLSFKGQVNDSACAANCLSLGKAGGHCEKGVCICRKTSFKDLWDKRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 3.6e-08, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.9e-07, 'percentage_identity': 64.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 8.5e-07, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 5.2e-06, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 5.7e-06, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 3.3e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.00015, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.00055, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.0008, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.0008, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0043, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 0.013, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2koz_1', 'molecule_name': ['nasonin-1'], 'pdb_id': '2koz', 'e_value': 0.018, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kp0_1', 'molecule_name': ['nasonin-1M'], 'pdb_id': '2kp0', 'e_value': 0.021, 'percentage_identity': 43.3, 'result_sequence': None}], 'GILSSFKDVAKGVAKNVAAQLLDKLKCKITGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 5.3e-11, 'percentage_identity': 78.1, 'result_sequence': None}], 'GKFHQIKKMFLSVGVKVTSLKRIQFGDF': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1vio_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1vio', 'e_value': 0.0016, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksk_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksk', 'e_value': 0.042, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksl_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksl', 'e_value': 0.042, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksv_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksv', 'e_value': 0.042, 'percentage_identity': 46.2, 'result_sequence': None}], 'KWWCFRVCYRGICYRRCRWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 4.8e-06, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0017, 'percentage_identity': 68.8, 'result_sequence': None}], 'GAVDILKGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 1.5e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 1.5e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 1.5e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 3.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 3.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 3.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 2.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 2.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 2.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'GIGKFLHAAKKFAKAFVAEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.8e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.1e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.1e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.3e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.3e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 3.8e-08, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}], 'AARCLSQRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.029, 'percentage_identity': 80.0, 'result_sequence': None}], 'RSGGYCNGKRVCVCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 2.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.00026, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.00033, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.00073, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.0021, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.0021, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.0021, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ygo_1', 'molecule_name': ['Wnt inhibitory factor 1'], 'pdb_id': '2ygo', 'e_value': 0.07, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ygp_1', 'molecule_name': ['Wnt inhibitory factor 1'], 'pdb_id': '2ygp', 'e_value': 0.07, 'percentage_identity': 57.1, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 7.4e-07, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.056, 'percentage_identity': 65.0, 'result_sequence': None}], 'CGNFLKRTCICVKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.06, 'percentage_identity': 50.0, 'result_sequence': None}], 'GFKRIVQRIADFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00013, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00018, 'percentage_identity': 93.8, 'result_sequence': None}], 'GGVWNLNVNPGTTGARVWARTK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4jru_1', 'molecule_name': ['thaumatin-like protein'], 'pdb_id': '4jru', 'e_value': 3.1e-05, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4l2j_1', 'molecule_name': ['Osmotin: antifungal laticifer protein'], 'pdb_id': '4l2j', 'e_value': 0.00016, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l5h_1', 'molecule_name': ['VVTL1'], 'pdb_id': '4l5h', 'e_value': 0.00034, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mbt_1', 'molecule_name': ['VVTL1'], 'pdb_id': '4mbt', 'e_value': 0.00034, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z3q_1', 'molecule_name': ['Ripening-associated protein'], 'pdb_id': '1z3q', 'e_value': 0.00045, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5fgt_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5fgt', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4dc5_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4dc5', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v7v_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v7v', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3s_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3s', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5fgx_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5fgx', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh5_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh5', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k7q_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5k7q', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh6_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh6', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lmh_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lmh', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xvb_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4xvb', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kvx_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kvx', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gqp_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5gqp', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ln0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5ln0', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3zej_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3zej', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3q_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3q', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3o_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3o', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3p_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3p', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4dc6_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4dc6', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3r_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3r', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3x3t_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3x3t', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vch_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vch', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v84_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v84', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vjq_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vjq', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v8a_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v8a', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4bar_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4bar', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4axu_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4axu', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw8_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw8', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh0', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw5_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw5', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh1_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh1', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5l4r_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5l4r', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kvz_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kvz', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tvt_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4tvt', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kvw_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kvw', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw0', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw4_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw4', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jvx_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5jvx', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wxs_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3wxs', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vci_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vci', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4bal_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4bal', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4c3c_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4c3c', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5mjg_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5mjg', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5yyq_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5yyq', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6coa_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6coa', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6srp_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6srp', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ybx_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6ybx', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5avg_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5avg', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x9m_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5x9m', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5yyr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5yyr', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s1e_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6s1e', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s1d_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6s1d', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6fj9_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6fj9', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6g89_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6g89', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6srl_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6srl', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6srj_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6srj', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sro_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6sro', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6srq_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6srq', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6srk_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6srk', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s19_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6s19', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6fj8_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6fj8', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6fj6_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6fj6', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5amz_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5amz', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zg3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4zg3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zxr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4zxr', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5a47_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5a47', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wr8_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5wr8', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x9l_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5x9l', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rvo_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6rvo', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5yyp_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5yyp', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c5y_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6c5y', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s1g_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6s1g', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6o8a_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6o8a', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oqn_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2oqn', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1thv_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1thv', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vhk_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vhk', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vi2_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vi2', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n02_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3n02', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vhr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vhr', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vi4_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vi4', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n03_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3n03', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vi3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vi3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vi1_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vi1', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pe7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2pe7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lr3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1lr3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2blr', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blu_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2blu', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3al7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3al7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vu6_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vu6', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vu7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2vu7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ald_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3ald', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1pp3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1pp3', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1thi_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1thi', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lxz_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1lxz', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ly0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1ly0', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lr2_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1lr2', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d8o_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2d8o', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v88_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v88', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vhf_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vhf', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d8p_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2d8p', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e3s_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3e3s', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2wbz_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2wbz', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vcj_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vcj', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vce_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vce', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vck_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vck', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v82_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v82', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vhg_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vhg', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4axr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4axr', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vcg_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3vcg', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3v87_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3v87', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qy5_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3qy5', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6yc5_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6yc5', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6zhn_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6zhn', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5sw0', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw2_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5sw2', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5t3g_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5t3g', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5tcl_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5tcl', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw1_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5sw1', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4dj0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4dj0', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4el2_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4el2', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ela_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4ela', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4el7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4el7', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ekb_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4ekb', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ek0_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4ek0', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4eka_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4eka', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4el3_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4el3', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wou_1', 'molecule_name': ['Thaumatin II'], 'pdb_id': '3wou', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4diz_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4diz', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4dj1_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4dj1', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4diy_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4diy', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ekt_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4ekt', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4eko_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4eko', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ekh_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '4ekh', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c6w_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6c6w', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g4y_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2g4y', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dzr_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3dzr', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e0a_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3e0a', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dzp_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3dzp', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2a7i_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '2a7i', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1thu_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1thu', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3aok_1', 'molecule_name': ['Thaumatin II'], 'pdb_id': '3aok', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1thw_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1thw', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rqw_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '1rqw', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dzn_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '3dzn', 'e_value': 0.00079, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6e0d_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '6e0d', 'e_value': 0.00087, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4bct_1', 'molecule_name': ['Thaumatin-like protein'], 'pdb_id': '4bct', 'e_value': 0.096, 'percentage_identity': 57.1, 'result_sequence': None}], 'ENFFKEIERAGQRIRDAIISAAPAVETLAQAQKIIKGGD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 8.6e-05, 'percentage_identity': 48.5, 'result_sequence': None}], 'RRWCFRVCYKGRCRYKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 2.8e-07, 'percentage_identity': 78.9, 'result_sequence': None}], 'IGKLVKWIIKTVNKFTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 2.4e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 2.4e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 2.4e-05, 'percentage_identity': 88.9, 'result_sequence': None}], 'KGWGSFFKKAAHVGKHVGKAALTHYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 4.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 4.2e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 6.1e-12, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 1e-11, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 1.1e-06, 'percentage_identity': 60.0, 'result_sequence': None}], 'GLFGVLAKVAAHVVPAIAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 4.1e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 4.1e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 4.1e-07, 'percentage_identity': 90.5, 'result_sequence': None}], 'GSKGAPCAKKPCCGPLGHYKVDCSTIPDYPCCSKVGFCGSGPQYCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.0011, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0024, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.0034, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.0041, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 0.0046, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.005, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 0.0072, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.012, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.012, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.012, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 0.016, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 0.02, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.045, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 0.055, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 0.059, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 0.059, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 0.059, 'percentage_identity': 41.7, 'result_sequence': None}], 'GLGKFLHSAKRFGKAFVGEAMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 7.1e-10, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 7.3e-10, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.7e-08, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.3e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.1e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.1e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}], 'QYEALVTSILGKLTGLWHNDSVDFMGHICYFRRRPKIRRFKLYHEGKFWCPGWAPFEGRSRTKSRSGSSREATKDFVRKALQNGLVTQQDASLWLNN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2job_1', 'molecule_name': ['Anti-lipopolysaccharide factor isoform 3'], 'pdb_id': '2job', 'e_value': 9.5e-24, 'percentage_identity': 42.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mdl_1', 'molecule_name': ['Anti-lipopolysaccharide factor'], 'pdb_id': '2mdl', 'e_value': 1.8e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'GIGAVLKVLALISWIKRKR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 5.1e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 5.1e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5.3e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5.3e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5.3e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5.3e-05, 'percentage_identity': 79.2, 'result_sequence': None}], 'ATYNGKCYKKDNICKYKAQSGKTAICKCYVKKCPRDGAKCEFDSYKGKCYC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1afp_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '1afp', 'e_value': 4.4e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oqs_1', 'molecule_name': ['NFAP'], 'pdb_id': '5oqs', 'e_value': 3.1e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbf_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nbf', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nb0_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nb0', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhv_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2mhv', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hah_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6hah', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ha4_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6ha4', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcn_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2kcn', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6haj_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6haj', 'e_value': 6.1e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nc2_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nc2', 'e_value': 0.029, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7baf_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '7baf', 'e_value': 0.03, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bad_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '7bad', 'e_value': 0.03, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bae_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '7bae', 'e_value': 0.03, 'percentage_identity': 32.7, 'result_sequence': None}]}

## 5000 - 5500

In [5]:
r15={'KEFKRIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 5.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLLPCAESCVYIPCLTTVIGCSCKSKVCYKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1.1e-11, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1.1e-11, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 3.3e-11, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 5.4e-11, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 2.1e-10, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 2.2e-10, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.3e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 6.6e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.3e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 6.7e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 1.6e-05, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00013, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttn_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttn', 'e_value': 0.025, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 0.025, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.025, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.025, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.031, 'percentage_identity': 38.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.032, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.039, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.039, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.041, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.062, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.078, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.098, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.098, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.098, 'percentage_identity': 44.4, 'result_sequence': None}], 'TRRRLFNRSFTQALGKSGGGFKKFWKWFRRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.032, 'percentage_identity': 100.0, 'result_sequence': None}], 'IGKFLKKAKKFG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.013, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.014, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.051, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.054, 'percentage_identity': 100.0, 'result_sequence': None}], 'KSCCRSTTARTARVPCAKKSNIYNGCRVPGGCKIQEAKKCEPPYD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3c8p_1', 'molecule_name': ['Viscotoxin A1'], 'pdb_id': '3c8p', 'e_value': 0.015, 'percentage_identity': 38.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmp_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '1jmp', 'e_value': 0.041, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v9b_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '2v9b', 'e_value': 0.051, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orl_1', 'molecule_name': ['Viscotoxin-C1'], 'pdb_id': '1orl', 'e_value': 0.051, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmn_1', 'molecule_name': ['Viscotoxin-A2'], 'pdb_id': '1jmn', 'e_value': 0.063, 'percentage_identity': 36.4, 'result_sequence': None}], 'KTCENLADTYKGPCFTTGSCDDHCKNKEHLRSGRCRDDFRCWCTKNC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 3e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 2.6e-22, 'percentage_identity': 93.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 9.9e-21, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 9.7e-18, 'percentage_identity': 74.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 3e-16, 'percentage_identity': 70.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 1.4e-13, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 4.4e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 3.9e-05, 'percentage_identity': 39.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 9.5e-05, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.00053, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 0.0037, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 0.0043, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 0.0052, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.0085, 'percentage_identity': 38.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5kk4_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5kk4', 'e_value': 0.011, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '5vyp_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5vyp', 'e_value': 0.011, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bk8_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1bk8', 'e_value': 0.016, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 0.022, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 0.022, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 0.022, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '6b55_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '6b55', 'e_value': 0.022, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mr4_1', 'molecule_name': ['Defensin-like protein'], 'pdb_id': '1mr4', 'e_value': 0.022, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 0.028, 'percentage_identity': 32.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6mry_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6mry', 'e_value': 0.032, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 0.05, 'percentage_identity': 26.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 0.094, 'percentage_identity': 29.2, 'result_sequence': None}], 'KISKKIMRTFLRRILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 0.011, 'percentage_identity': 68.2, 'result_sequence': None}], 'FFHHIFRGIVHVGKTAHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 2.3e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.2e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 7.9e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 7.9e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 7.9e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.05, 'percentage_identity': 50.0, 'result_sequence': None}], 'SNDIYFNFQR': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1g8w_1', 'molecule_name': ['Leucoagglutinating phytohemagglutinin'], 'pdb_id': '1g8w', 'e_value': 0.0081, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fat_1', 'molecule_name': ['Leucoagglutinating phytohemagglutinin'], 'pdb_id': '1fat', 'e_value': 0.0088, 'percentage_identity': 100.0, 'result_sequence': None}], 'RPPQFTRAQWFAIQHISLN': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 9.6e-08, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 3.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 0.00019, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 0.0002, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 0.00022, 'percentage_identity': 66.7, 'result_sequence': None}], 'GWGSFFKKAAHVGKHVGKAALT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 2.6e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 2.6e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 4.4e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 7.8e-10, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.00017, 'percentage_identity': 57.1, 'result_sequence': None}], 'SSMKLSFRARAYGFRGPGLQL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lv4_1', 'molecule_name': ['Catestatin'], 'pdb_id': '1lv4', 'e_value': 2.6e-09, 'percentage_identity': 95.2, 'result_sequence': None}], 'FFHHIWRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 3e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 3e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 3.1e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 3.1e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 3.1e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 4.8e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 2.5e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.027, 'percentage_identity': 55.0, 'result_sequence': None}], 'KRIVQRIKDWLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}], 'GKIPVKAIKKGGQIIGKALRGINIASTAHDIISQFKPKKKKNH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 1.3e-11, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 2.5e-11, 'percentage_identity': 78.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 2.9e-08, 'percentage_identity': 62.5, 'result_sequence': None}], 'KRFKKFFKKLKNSVKKRAKKFFKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.1e-06, 'percentage_identity': 83.3, 'result_sequence': None}], 'EVGGGGGSQQCPQERPKLSSCKDYVMERCFTMKDFPVTWPTKWWKGGCEHEVREKCCKQLSQIAPQCRCDSIRRVIQGRLGGFLGIWRGEVFKQLQRAQSLPSKCNMGADCKFPSGYYW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1psy_1', 'molecule_name': ['Allergen Ric c 3 large chain'], 'pdb_id': '1psy', 'e_value': 9.6e-05, 'percentage_identity': 28.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vj0_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '6vj0', 'e_value': 0.00033, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvf_1', 'molecule_name': ['2S sulfur-rich seed storage protein large chain 1B'], 'pdb_id': '2lvf', 'e_value': 0.00052, 'percentage_identity': 28.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s3f_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '6s3f', 'e_value': 0.0018, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u87_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '5u87', 'e_value': 0.0018, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sm7_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '1sm7', 'e_value': 0.0024, 'percentage_identity': 24.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5dom_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '5dom', 'e_value': 0.0033, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1w2q_1', 'molecule_name': ['Conglutin'], 'pdb_id': '1w2q', 'e_value': 0.012, 'percentage_identity': 20.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1pnb_2', 'molecule_name': ['Napin-1A large chain'], 'pdb_id': '1pnb', 'e_value': 0.035, 'percentage_identity': 30.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4cvw_2', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '4cvw', 'e_value': 0.078, 'percentage_identity': 26.4, 'result_sequence': None}], 'YSRCQLQGFNCVVRSYGLPTIPCCRGLTCRSYFPGSTYGRCQRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cix_1', 'molecule_name': ['Tachystatin-A2'], 'pdb_id': '1cix', 'e_value': 2.3e-21, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dcv_1', 'molecule_name': ['Tachystatin-B1'], 'pdb_id': '2dcv', 'e_value': 0.0015, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dcw_1', 'molecule_name': ['Tachystatin-B2'], 'pdb_id': '2dcw', 'e_value': 0.0018, 'percentage_identity': 44.2, 'result_sequence': None}], 'KSKEKIGKEFKRIVQRIKDFLRNLVPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.003, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLEKLFS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.0017, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.0047, 'percentage_identity': 92.3, 'result_sequence': None}], 'LRKLRKRLVRLASHLRKLRKRLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kny_1', 'molecule_name': ['Low-density lipoprotein receptor-related protein 1 515 kDa subunit', 'Apolipoprotein E'], 'pdb_id': '2kny', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6v7m_2', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '6v7m', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1le4_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1le4', 'e_value': 0.015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bz4_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1bz4', 'e_value': 0.015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1le2_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1le2', 'e_value': 0.015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lpe_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1lpe', 'e_value': 0.015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ncn_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '6ncn', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nco_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '6nco', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kc3_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '2kc3', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1or2_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1or2', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1or3_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1or3', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nfn_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1nfn', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gs9_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1gs9', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b68_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1b68', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nfo_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1nfo', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7b_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '2l7b', 'e_value': 0.02, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h7i_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1h7i', 'e_value': 0.047, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ea8_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1ea8', 'e_value': 0.061, 'percentage_identity': 93.3, 'result_sequence': None}], 'GSKGAPCAKKPCCGPLGHYKVDCSTIPDYPCCSKYGFCGSGPQVCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.0024, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.0035, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 0.0047, 'percentage_identity': 43.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0052, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.011, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.012, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.012, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.012, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.013, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.013, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.013, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.013, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 0.016, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 0.016, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 0.021, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.025, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.03, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.033, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.098, 'percentage_identity': 52.2, 'result_sequence': None}], 'AIGSILGRLAKGLPTLKSWIKNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.037, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.037, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}], 'KWKLFKKIPHLAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.073, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.073, 'percentage_identity': 65.0, 'result_sequence': None}], 'WPKRLTKAHWFEIQHIQPSPLQSNRAMSGINNYTQHSKHQNTFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 4.3e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 4.3e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 4.3e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 4.3e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 7.3e-19, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 9.9e-09, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 3.3e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 7.6e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 7.7e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 9e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 3.9e-05, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 5.7e-05, 'percentage_identity': 48.0, 'result_sequence': None}], 'DFGCARGMIFVCMRRCARMYPGSTGYCQGFRCMCDTHIPIRRPPFIMG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.06, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.06, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vxw_1', 'molecule_name': ['Beta-toxin Cl13'], 'pdb_id': '6vxw', 'e_value': 0.061, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4v1d_3', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '4v1d', 'e_value': 0.073, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2yc1_3', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '2yc1', 'e_value': 0.073, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '2ybr_3', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '2ybr', 'e_value': 0.073, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cn2_1', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '1cn2', 'e_value': 0.074, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kja_1', 'molecule_name': ['Beta-toxin Cn5'], 'pdb_id': '2kja', 'e_value': 0.088, 'percentage_identity': 32.7, 'result_sequence': None}], 'VKRFKKFFRKLKKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.0066, 'percentage_identity': 84.6, 'result_sequence': None}], 'GSIPCGESCVFIPCISAVIGCSCSNKVCYKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 3.5e-12, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1.3e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1.3e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 1.7e-11, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 4.2e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 6.4e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 8e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 4e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 4e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 7.3e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 2.7e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00055, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.0011, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.0017, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.0027, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.011, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.012, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.015, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.015, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttn_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttn', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.019, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.021, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.038, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.041, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.047, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.059, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.076, 'percentage_identity': 39.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.091, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.091, 'percentage_identity': 46.2, 'result_sequence': None}], 'FLPIVAGLAANFLPKIVCKITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 2.2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 2.2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 2.2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.0039, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.0039, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.0039, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.0039, 'percentage_identity': 60.9, 'result_sequence': None}], 'GASCGETCFTGICFTAGCSCNPWPTCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 2.6e-10, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 1.3e-07, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 1.3e-07, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 8.6e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 8.9e-07, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 1.6e-06, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 1.6e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 1.6e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 3.6e-06, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 3.6e-06, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 4.4e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 8.7e-06, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 1e-05, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 1.6e-05, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 1.9e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 1.9e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.3e-05, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 5.3e-05, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 5.3e-05, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 7.4e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00023, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 0.00023, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.0018, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.0027, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.0041, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 0.05, 'percentage_identity': 64.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 0.062, 'percentage_identity': 64.7, 'result_sequence': None}], 'YGRKKRRQRRR': [{'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '5l1z_4', 'molecule_name': ['Protein Tat'], 'pdb_id': '5l1z', 'e_value': 0.0052, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6cyt_4', 'molecule_name': ['Protein Tat'], 'pdb_id': '6cyt', 'e_value': 0.0052, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'M'], 'entity_id': 4, 'entry_entity': '4ogr_4', 'molecule_name': ['Protein Tat'], 'pdb_id': '4ogr', 'e_value': 0.0052, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tac_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tac', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tiv_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tiv', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3mia_3', 'molecule_name': ['Protein Tat'], 'pdb_id': '3mia', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3mi9_3', 'molecule_name': ['Protein Tat'], 'pdb_id': '3mi9', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tbc_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tbc', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jfw_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1jfw', 'e_value': 0.0074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5k_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1k5k', 'e_value': 0.0075, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPLPFPRPG': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 2.3e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.091, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.091, 'percentage_identity': 75.0, 'result_sequence': None}], 'MAKEFGIPAAVAGTVLNVVEAGGAVTTIVSILTAVGSGGLSLLAAAGRESIKAYLKKEIKKKGKRAVIAW': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 8.4e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 8.4e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 8.4e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 8.4e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 2.2e-24, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 0.00063, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.0015, 'percentage_identity': 37.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 0.0057, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 0.042, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 0.042, 'percentage_identity': 64.0, 'result_sequence': None}], 'LQFPVGRVHRLLRK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 3.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 3.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 11, 'entry_entity': '6z6p_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6z6p', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 4.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 4.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 4.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 4.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 4.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 4.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 4.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 4.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 4.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 4.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 4.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 4.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6g0l_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 5.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 5.3e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 5.4e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 5.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 5.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 5.6e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '2f8n_7', 'molecule_name': ['Histone H2A type 1-H'], 'pdb_id': '2f8n', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 6.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 6.3e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 6.4e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 6.4e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 6.5e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 6.5e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 6.5e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 6.8e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 6.8e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 7.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 8.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 8.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 8.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 0.00013, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 0.00017, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 0.00017, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 0.00065, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 0.00068, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 0.00068, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 0.00068, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 0.0011, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 0.0012, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 0.0016, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7dlx_1', 'molecule_name': ['Histone H2A', 'Histone H2B'], 'pdb_id': '7dlx', 'e_value': 0.0023, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k0c_1', 'molecule_name': ['Histone H2A.V', 'Histone H2B 2'], 'pdb_id': '6k0c', 'e_value': 0.0042, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 0.015, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 0.015, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 0.015, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 0.017, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b31_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b31', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b32_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b32', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5chl_2', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5chl', 'e_value': 0.026, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4nft_1', 'molecule_name': ['Histone H2B type 2-E', 'Histone H2A.Z'], 'pdb_id': '4nft', 'e_value': 0.028, 'percentage_identity': 76.9, 'result_sequence': None}], 'FLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVN': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 6.7e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 6.8e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 1.2e-27, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 5.6e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 5.6e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 5.8e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 5.9e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 6e-25, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 1.2e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 1.2e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 1.2e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 1.2e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 1.2e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 1.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 1.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 1.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 2.2e-24, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 2.2e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sva_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6sva', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5c6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5c6e', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6r2o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6r2o', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5x', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 2.7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iwh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1iwh', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 2.7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s0h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s0h', 'e_value': 2.7e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 2.7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 2.7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 2.7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w72_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 3.3e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 4e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 4e-24, 'percentage_identity': 87.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xye', 'e_value': 4e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz4', 'e_value': 4e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 4e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 4.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2mhb', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dhb', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlx', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlt', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8h', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlu', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ibe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ibe', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns6', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8k', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8i', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlv', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlw', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g0b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0b', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns9', 'e_value': 5.8e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 7e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 7.1e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 7.1e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 7.1e-24, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gys_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gys', 'e_value': 3.3e-23, 'percentage_identity': 83.1, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gqr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqr', 'e_value': 3.3e-23, 'percentage_identity': 83.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d4x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d4x', 'e_value': 3.3e-23, 'percentage_identity': 83.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqp', 'e_value': 3.3e-23, 'percentage_identity': 83.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 4e-23, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 4e-23, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 4e-23, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 4e-23, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 4e-23, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 1.9e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 1.9e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 1.9e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 1.9e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 3.3e-22, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 4e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 4e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 4.1e-22, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 1.3e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 1.3e-21, 'percentage_identity': 75.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3fs4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3fs4', 'e_value': 2.8e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3a59_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3a59', 'e_value': 2.8e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 3.4e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 3.4e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 4.1e-21, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 1.1e-20, 'percentage_identity': 72.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 1.9e-20, 'percentage_identity': 73.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zfb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zfb', 'e_value': 8.9e-19, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 1.9e-17, 'percentage_identity': 64.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 1.9e-17, 'percentage_identity': 64.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '3w4u_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '3w4u', 'e_value': 2e-17, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2z6n_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '2z6n', 'e_value': 3.5e-17, 'percentage_identity': 63.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wmu_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1wmu', 'e_value': 3.5e-17, 'percentage_identity': 63.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v75_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1v75', 'e_value': 3.5e-17, 'percentage_identity': 63.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbr_1', 'molecule_name': ['Hemoglobin subunit alpha-D'], 'pdb_id': '1hbr', 'e_value': 6.2e-17, 'percentage_identity': 64.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyi_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 1.1e-14, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyh_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyh', 'e_value': 1.4e-12, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1out_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1out', 'e_value': 3e-12, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ouu_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1ouu', 'e_value': 3e-12, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1spg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1spg', 'e_value': 1.7e-11, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4w_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4w', 'e_value': 5.5e-11, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4x_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4x', 'e_value': 5.5e-11, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4u_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4u', 'e_value': 5.5e-11, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bcq_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bcq', 'e_value': 6.5e-11, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4iro_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4iro', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4g51_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4g51', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5x', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4odc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4odc', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbh', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2peg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2peg', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3gkv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gkv', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqg', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8f', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1s5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5y', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pbx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1pbx', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8d', 'e_value': 8e-11, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4esa_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '4esa', 'e_value': 1.7e-10, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rp5_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '6rp5', 'e_value': 1.7e-10, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5lfg_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '5lfg', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t1n_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1t1n', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ng6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3ng6', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nfe_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3nfe', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d1k_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1k', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2aa1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2aa1', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1la6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1la6', 'e_value': 4.5e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj3_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj3', 'e_value': 5.4e-10, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj2_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj2', 'e_value': 5.4e-10, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj1_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj1', 'e_value': 5.4e-10, 'percentage_identity': 54.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r1h_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '2r1h', 'e_value': 6.6e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bom_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '3bom', 'e_value': 6.6e-10, 'percentage_identity': 50.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6hit_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '6hit', 'e_value': 6.6e-10, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xq5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1xq5', 'e_value': 9.7e-10, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qu0_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2qu0', 'e_value': 6.8e-09, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B', 'J', 'L'], 'entity_id': 2, 'entry_entity': '2ri4_2', 'molecule_name': ['Hemoglobin subunit beta-A'], 'pdb_id': '2ri4', 'e_value': 8.2e-09, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3eu1_2', 'molecule_name': ['Hemoglobin subunit beta-A'], 'pdb_id': '3eu1', 'e_value': 8.2e-09, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3d1a_2', 'molecule_name': ['Hemoglobin subunit beta-A'], 'pdb_id': '3d1a', 'e_value': 8.2e-09, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1ch4_1', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ch4', 'e_value': 8.3e-09, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a0g_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3a0g', 'e_value': 1.2e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3hyu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3hyu', 'e_value': 1.2e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcw', 'e_value': 1.4e-08, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcv', 'e_value': 1.4e-08, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1hds_2', 'molecule_name': ['Hemoglobin subunit beta-3'], 'pdb_id': '1hds', 'e_value': 1.5e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqj_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqj', 'e_value': 1.8e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3e_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3e', 'e_value': 1.8e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3d_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3d', 'e_value': 1.8e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 2, 'entry_entity': '1fdh_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '1fdh', 'e_value': 1.8e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5eui_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 2.2e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3lqd_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3lqd', 'e_value': 2.2e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5ker_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 2.6e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3hrw_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '3hrw', 'e_value': 3.2e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1jeb_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '1jeb', 'e_value': 3.2e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqk_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqk', 'e_value': 3.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1shr_2', 'molecule_name': ['Hemoglobin subunit delta'], 'pdb_id': '1shr', 'e_value': 3.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2rao_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '2rao', 'e_value': 3.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '3hf4_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '3hf4', 'e_value': 3.9e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3dht_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '3dht', 'e_value': 3.9e-08, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1si4_2', 'molecule_name': ['Hemoglobin subunit delta'], 'pdb_id': '1si4', 'e_value': 3.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y4v_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y4v', 'e_value': 4.7e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1o1p_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1o1p', 'e_value': 4.7e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1gbu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1gbu', 'e_value': 4.7e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4h2l_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 5.7e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y35_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y35', 'e_value': 6.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1ye2_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ye2', 'e_value': 6.9e-08, 'percentage_identity': 45.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yie_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yie', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yeo_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yeo', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y4f_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y4f', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1a01_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a01', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3fh9_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3fh9', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3vrf_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3vre_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3vrg_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 8.3e-08, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y2z_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y2z', 'e_value': 1e-07, 'percentage_identity': 42.3, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y4p_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y4p', 'e_value': 1e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yev_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yev', 'e_value': 1e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1ygd_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ygd', 'e_value': 1e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1rqa_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1rqa', 'e_value': 1e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1hba_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1hba', 'e_value': 1e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yeu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yeu', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y46_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y46', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yg5_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yg5', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y4b_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y4b', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yeq_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yeq', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y4g_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y4g', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1a00_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a00', 'e_value': 1.2e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5hu6_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '5hu6', 'e_value': 1.4e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4n7n_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '4n7n', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4n7p_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '4n7p', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '5jdo_4', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '5jdo', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5e29_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '5e29', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6bb5_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6bb5', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6kav_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6kav', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '5x2t_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '5x2t', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6fqf_1', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6fqf', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6nbc_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6nbc', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6nbd_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6nbd', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hal_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6hal', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1yhe_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1yhe', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2dn2_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2dn2', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2hhe_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2hhe', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3hhb_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3hhb', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hbd_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2hbd', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y85_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y85', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1c7b_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1c7b', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1j41_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1j41', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['V', 'D', 'Q', 'B', 'X', 'S'], 'entity_id': 2, 'entry_entity': '1jy7_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1jy7', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1xzu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1xzu', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'T', 'F'], 'entity_id': 2, 'entry_entity': '3b75_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3b75', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cbm_1', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1cbm', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1fn3_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1fn3', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1a0z_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a0z', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1r1x_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1r1x', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cls_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1cls', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1a0u_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a0u', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1rq3_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1rq3', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1j3y_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1j3y', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1m9p_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1m9p', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1abw_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1abw', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1gzx_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1gzx', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1hbb_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1hbb', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1a3o_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a3o', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['BBB', 'DDD'], 'entity_id': 2, 'entry_entity': '7aet_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '7aet', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['BBB', 'DDD'], 'entity_id': 2, 'entry_entity': '7aeu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '7aeu', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6kah_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6kah', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '7jjq_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '7jjq', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['BBB', 'DDD'], 'entity_id': 2, 'entry_entity': '7aev_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '7aev', 'e_value': 1.5e-07, 'percentage_identity': 44.1, 'result_sequence': None}], 'RNPRPVYIPQPRPPHPRL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.094, 'percentage_identity': 71.4, 'result_sequence': None}], 'FLPVVAGLAAKVLPSIICAVTKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 7.7e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 7.7e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 7.7e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.017, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.017, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.017, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.017, 'percentage_identity': 56.5, 'result_sequence': None}], 'EWEPVQNGGSSYYMVPRIWA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tie_1', 'molecule_name': ['Trypsin inhibitor DE-3'], 'pdb_id': '1tie', 'e_value': 0.093, 'percentage_identity': 61.1, 'result_sequence': None}], 'VDKPPYLPRPTPPRRIYNNR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.00034, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.0098, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.0098, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.016, 'percentage_identity': 75.0, 'result_sequence': None}], 'QQCGRQAGNRRCPNNLCCSQFGYCGRTNEYCCTGFGCQSNCRRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 8e-13, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 2.3e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 2.3e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 2.3e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 5.6e-12, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 1.9e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 1.9e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 3.2e-10, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 5.2e-10, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 5.2e-10, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 5.3e-10, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 8.7e-10, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 1.2e-09, 'percentage_identity': 63.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 3.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 3.6e-09, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 3.7e-09, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 3.7e-09, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 3.7e-09, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 3.7e-09, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t0w_1', 'molecule_name': ['Hevein'], 'pdb_id': '1t0w', 'e_value': 4.9e-09, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 5.6e-09, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 2.5e-08, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lnr_1', 'molecule_name': ['Class-1 chitinase'], 'pdb_id': '6lnr', 'e_value': 4.5e-07, 'percentage_identity': 63.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eis_1', 'molecule_name': ['Agglutinin isolectin VI'], 'pdb_id': '1eis', 'e_value': 1.6e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1en2_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1en2', 'e_value': 1.6e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enm_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1enm', 'e_value': 1.6e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 1.9e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 1.9e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1iqb_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1iqb', 'e_value': 2.2e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.00022, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.00039, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.00081, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 0.00091, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 0.00091, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 0.0014, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.0027, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 0.0042, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.0047, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.016, 'percentage_identity': 53.3, 'result_sequence': None}], 'KWKSFIKKKTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.00072, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.00072, 'percentage_identity': 70.0, 'result_sequence': None}], 'GEILCNLCTGLINTLENLLTTKGADKVKDYISSLCNKASGFIATLCTKVLDFGIDKLIQLIEDKVDANAICAKIHAC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1of9_1', 'molecule_name': ['Pore-forming peptide ameobapore A'], 'pdb_id': '1of9', 'e_value': 3.1e-36, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6w1b_1', 'molecule_name': ['Pulmonary surfactant-associated protein B'], 'pdb_id': '6w1b', 'e_value': 0.074, 'percentage_identity': 20.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 0.097, 'percentage_identity': 25.3, 'result_sequence': None}], 'GRCRDDFRCWCTKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 1.5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 2.4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 5.2e-07, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 5.2e-07, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 5.2e-07, 'percentage_identity': 86.7, 'result_sequence': None}], 'KWKLFKKIGAVLKKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.059, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.059, 'percentage_identity': 100.0, 'result_sequence': None}], 'KPKGMTSSQWFKIQHMQPSPQACNSAMKNINKHTKRCKDLNTFLHEPFSSVAATCQTPKIACKNGDKNCHQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 7e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 7.1e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 7.1e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 7.1e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 7.1e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 4.3e-16, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 1.4e-11, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 3.7e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 4.4e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 8e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 8e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 8e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 8e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 8e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 8.1e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 8.1e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 8.1e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 8.1e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 8.1e-10, 'percentage_identity': 39.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rra_1', 'molecule_name': ['Ribonuclease pancreatic beta-type'], 'pdb_id': '1rra', 'e_value': 2.2e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1b6v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1b6v', 'e_value': 2.7e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4srn', 'e_value': 2.9e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3srn', 'e_value': 2.9e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssc', 'e_value': 2.9e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mx8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mx8', 'e_value': 3.1e-09, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssa', 'e_value': 3.1e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssb', 'e_value': 3.1e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1srn', 'e_value': 3.1e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qh3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4qh3', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ogh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5ogh', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obe', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obc', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5old_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5old', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5et4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5et4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5e_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5e', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5f', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rte_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rte', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s0q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s0q', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsd', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsd', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j65_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j65', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j66_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j66', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j67', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j68_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j68', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j61_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j61', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rh1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rh1', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j6a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j6a', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsp', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rn3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rn3', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j62_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j62', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3qsk', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ao1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ao1', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xog_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xog', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j63_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j63', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j69_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j69', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j5z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j5z', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l55_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4l55', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mxf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4mxf', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j64_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j64', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rid', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jmg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jmg', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s18_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s18', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obd', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j60', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xoi_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xoi', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jlg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jlg', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d97_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d97', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zz4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4zz4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jml_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jml', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6eto_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6eto', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rsa', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvv', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6gok_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6gok', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6f60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6f60', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etr', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etq', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xvx', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etp', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6qe9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qe9', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etm', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhc', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nj7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5nj7', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d6u_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d6u', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhf', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhd', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xw0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xw0', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhe', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5na9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5na9', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etl', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvx', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etk', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvu', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6qmn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qmn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvw', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e3w', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2qca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2qca', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p43_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p43', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4q', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymw', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6s_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6s', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wbu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1wbu', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rns_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rns', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ot4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ot4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ooh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ooh', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8y', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3jw1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3jw1', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1a2w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a2w', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5q', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbx', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rsm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rsm', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 2, 'entry_entity': '1f0v_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1f0v', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eow_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eow', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9x', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eid', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eic', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dib_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dib', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dic', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di9', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g4w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4w', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8r', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7w', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blp', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p47_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p47', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p4a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p4a', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p49_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p49', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2e33_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e33', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5g', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5k_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5k', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5m', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5l', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev5', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev2', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g4x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4x', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di8', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euy_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euy', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euz', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev6', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8q', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lxo_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3lxo', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p45_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p45', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p46_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p46', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p42_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p42', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p48_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p48', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i67', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7x', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8y', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6q', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxg', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev3', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxh', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev1', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eux_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3eux', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6h', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7y', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8z', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6j', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6f', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2aas_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2aas', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0o', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1eos_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eos', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf2', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jn4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jn4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4o', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4p', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xps_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xps', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymr', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6d_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6d', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xpt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xpt', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1u1b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1u1b', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1qhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1qhc', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6p', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d7b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d7b', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh5', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blz', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6o', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a1r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3a1r', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1bzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bzq', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fs3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fs3', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvu', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvt', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0f', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnm', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnd', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rno_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rno', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afu', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ras_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ras', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afk', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnv', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afl', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rny_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rny', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rbb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbb', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rob_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rob', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ruv', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0n_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0n', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf7', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0h', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf3', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbj_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbj', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rtb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rtb', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnq', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbw', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rha_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rha', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rca', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnc', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rcn_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rcn', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rar_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rar', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpg', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnu', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rhb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rhb', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aqp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1aqp', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eie_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eie', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnz', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnx', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rta_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rta', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1dfj_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dfj', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rph_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rph', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpf', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnw', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1js0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1js0', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvv', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0m', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf8', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf5', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p44_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p44', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5i_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5i', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bel_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bel', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev4', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev0', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4peq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4peq', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8v', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4pou_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4pou', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g90_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g90', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6yo1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6yo1', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rsa', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '9rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '8rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rsa', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rsa', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rat', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rsa', 'e_value': 3.2e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyp', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyn', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyz', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzq', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0c_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0c', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0b', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kh8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kh8', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzr', 'e_value': 3.3e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2apq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2apq', 'e_value': 3.6e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ql2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql2', 'e_value': 4.6e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ql1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql1', 'e_value': 4.6e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2op2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2op2', 'e_value': 5.6e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh6', 'e_value': 5.6e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwr', 'e_value': 6.5e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwq', 'e_value': 6.5e-09, 'percentage_identity': 43.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsk', 'e_value': 6.7e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3tsr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3tsr', 'e_value': 6.7e-09, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsk', 'e_value': 6.7e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 7.2e-09, 'percentage_identity': 38.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dy5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dy5', 'e_value': 8e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di7', 'e_value': 8e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1lsq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1lsq', 'e_value': 8e-09, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2oqf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2oqf', 'e_value': 1.2e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c8w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c8w', 'e_value': 1.2e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izp', 'e_value': 1.2e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4k7m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7m', 'e_value': 1.3e-08, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4kxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4kxh', 'e_value': 1.4e-08, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izq', 'e_value': 1.7e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izr', 'e_value': 2.4e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9v', 'e_value': 2.9e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5p', 'e_value': 4.1e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ygw_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ygw', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4k7l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7l', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j82_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j82', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o36_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o36', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4okf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4okf', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o37_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o37', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbd_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbd', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fkz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fkz', 'e_value': 5e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3p_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3p', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3or0_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3or0', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3oqz_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqz', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl0', 'e_value': 5e-08, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3l', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j81_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j81', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j80_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j80', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3m_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3m', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 2, 'entry_entity': '3oqy_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqy', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j7z_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j7z', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbc_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbc', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbh_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbh', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbe_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbe', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbg_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbg', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbi_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbi', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbf', 'e_value': 5e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '2rln_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rln', 'e_value': 5.2e-08, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k11_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2k11', 'e_value': 6.1e-08, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e21_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1e21', 'e_value': 6.1e-08, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '2q4g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2q4g', 'e_value': 6.2e-08, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '1z7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z7x', 'e_value': 6.2e-08, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fev_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fev', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5d_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5d', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5h_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5h', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjq_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjq', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5e_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5e', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjr_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjr', 'e_value': 7e-08, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1h8x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1h8x', 'e_value': 8.8e-08, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl3', 'e_value': 1e-07, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl1', 'e_value': 1e-07, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0o', 'e_value': 1.1e-07, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0j', 'e_value': 3.2e-07, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4n4c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '4n4c', 'e_value': 4.4e-07, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0m', 'e_value': 5.5e-07, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0l', 'e_value': 5.5e-07, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dza_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dza', 'e_value': 6.6e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tq9_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1tq9', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djq', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djo_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djo', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djp', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djv_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djv', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djx_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djx', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1bsr_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1bsr', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1x_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n1x', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n3z_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n3z', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5c', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11ba_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11ba', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r3m_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r3m', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5d_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5d', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11bg_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11bg', 'e_value': 9.1e-07, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y94_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y94', 'e_value': 1.1e-06, 'percentage_identity': 40.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3f8g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3f8g', 'e_value': 1.3e-06, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '2rnf', 'e_value': 1.5e-06, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '1rnf', 'e_value': 1.5e-06, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ar6_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ar6', 'e_value': 1.9e-06, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arj_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arj', 'e_value': 2e-06, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arl_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arl', 'e_value': 2e-06, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5ark_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ark', 'e_value': 2e-06, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3bcp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcp', 'e_value': 2.3e-06, 'percentage_identity': 37.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bco_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bco', 'e_value': 2.3e-06, 'percentage_identity': 37.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bcm_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcm', 'e_value': 2.3e-06, 'percentage_identity': 37.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qwq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1qwq', 'e_value': 2.3e-06, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lfj_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '2lfj', 'e_value': 2.3e-06, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gv7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1gv7', 'e_value': 2.7e-06, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y92_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y92', 'e_value': 3.3e-06, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1un5_1', 'molecule_name': ['Angiogenin', 'Ribonuclease pancreatic'], 'pdb_id': '1un5', 'e_value': 0.03, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahh_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahh', 'e_value': 0.051, 'percentage_identity': 37.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5epz_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5epz', 'e_value': 0.085, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5b_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k5b', 'e_value': 0.085, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eqo_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5eqo', 'e_value': 0.086, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eop_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5eop', 'e_value': 0.086, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahl_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahl', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahm_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahm', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahn_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahn', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ahk_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahk', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k58_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k58', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1un4_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1un4', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h52_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h52', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1a4y_2', 'molecule_name': ['Angiogenin'], 'pdb_id': '1a4y', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k59_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k59', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5a_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k5a', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '1h0d_3', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h0d', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h53_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h53', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hby_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1hby', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b1i_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1b1i', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ang_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1ang', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1awz_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1awz', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ang_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '2ang', 'e_value': 0.087, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9s_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9s', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9p_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9p', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9v_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9v', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5m9r_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9r', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5m9t_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9t', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m9m_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9m', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9q_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9q', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4aoh_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4aoh', 'e_value': 0.088, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4b36_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4b36', 'e_value': 0.089, 'percentage_identity': 36.1, 'result_sequence': None}], 'KKKKGIGKFLHSAKKFGKAFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 7.2e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 7.5e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.6e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 3.5e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 3.5e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0019, 'percentage_identity': 93.3, 'result_sequence': None}], 'TLRGDERILSILRHQNLLKELQDLALQGAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 3.6e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLRKRLRKPRNKPKEKLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.0002, 'percentage_identity': 90.0, 'result_sequence': None}], 'FKCWRWQWRWKKLGAKVFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00049, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0055, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.037, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.06, 'percentage_identity': 80.0, 'result_sequence': None}], 'KWASLWNWFNITNWLWYIK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jau_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '1jau', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jav_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '1jav', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pv6_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2pv6', 'e_value': 2.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m8o_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2m8o', 'e_value': 3e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m8m_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2m8m', 'e_value': 3e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '5ghw_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5ghw', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6dln_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6dln', 'e_value': 3.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3g9r_1', 'molecule_name': ['General control transcription factor GCN4', 'Transmembrane protein gp41'], 'pdb_id': '3g9r', 'e_value': 4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gwo_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '3gwo', 'e_value': 4.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h01_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '3h01', 'e_value': 4.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2lp7_1', 'molecule_name': ['Fibritin_C domain-containing protein'], 'pdb_id': '2lp7', 'e_value': 5.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2m7w_1', 'molecule_name': ['Fibritin_C domain-containing protein'], 'pdb_id': '2m7w', 'e_value': 5.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'J', 'O', 'N'], 'entity_id': 1, 'entry_entity': '6sne_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6sne', 'e_value': 5.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '2x7r_2', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2x7r', 'e_value': 5.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncs_1', 'molecule_name': ['Envelope glycoprotein'], 'pdb_id': '2ncs', 'e_value': 1.2e-09, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nct_1', 'molecule_name': ['Envelope glycoprotein'], 'pdb_id': '2nct', 'e_value': 1.2e-09, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['C', 'A', 'B'], 'entity_id': 1, 'entry_entity': '7aej_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '7aej', 'e_value': 1.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '5u3m_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3m', 'e_value': 2.2e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P', 'C'], 'entity_id': 3, 'entry_entity': '5u3k_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3k', 'e_value': 2.2e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '5u3n_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3n', 'e_value': 2.2e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P', 'C'], 'entity_id': 3, 'entry_entity': '5iq9_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5iq9', 'e_value': 2.4e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '5iq7_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5iq7', 'e_value': 2.5e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '5u3j_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3j', 'e_value': 2.5e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P', 'F'], 'entity_id': 3, 'entry_entity': '4g6f_3', 'molecule_name': ['gp41 MPER Peptide'], 'pdb_id': '4g6f', 'e_value': 2.5e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6v4t_1', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6v4t', 'e_value': 3.4e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6e8w_1', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6e8w', 'e_value': 3.4e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '6mtt_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6mtt', 'e_value': 5.8e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['P', 'Q'], 'entity_id': 3, 'entry_entity': '6mtp_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6mtp', 'e_value': 6e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6ujv_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6ujv', 'e_value': 6.6e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3k9a_1', 'molecule_name': ['HIV glycoprotein gp41'], 'pdb_id': '3k9a', 'e_value': 7.4e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7loi_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '7loi', 'e_value': 9e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 4, 'entry_entity': '6mar_4', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6mar', 'e_value': 9.7e-09, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me3_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2me3', 'e_value': 1.5e-08, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me1_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2me1', 'e_value': 1.5e-08, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 4, 'entry_entity': '6nij_4', 'molecule_name': ['AMC011 Glycoprotein 41'], 'pdb_id': '6nij', 'e_value': 3.6e-08, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6olp_2', 'molecule_name': ['Envelope glycoprotein gp41'], 'pdb_id': '6olp', 'e_value': 3.6e-08, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6ulc_2', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6ulc', 'e_value': 3.7e-08, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '6pwu_1', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6pwu', 'e_value': 7e-08, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me4_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2me4', 'e_value': 1.1e-07, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me2_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2me2', 'e_value': 3e-07, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6dcq_2', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '6dcq', 'e_value': 4.4e-07, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6x58_3', 'molecule_name': ['Transmembrane protein gp41', 'Putative fluoride ion transporter CrcB'], 'pdb_id': '6x58', 'e_value': 7e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '5u3o_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3o', 'e_value': 1.2e-05, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['P', 'C'], 'entity_id': 3, 'entry_entity': '5u3l_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5u3l', 'e_value': 1.2e-05, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['P', 'C'], 'entity_id': 3, 'entry_entity': '6snc_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6snc', 'e_value': 0.00028, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['P', 'C'], 'entity_id': 3, 'entry_entity': '6snd_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6snd', 'e_value': 0.00028, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mg3_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2mg3', 'e_value': 0.00031, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mg2_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2mg2', 'e_value': 0.00031, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['T'], 'entity_id': 3, 'entry_entity': '6mto_3', 'molecule_name': ['VRC42 epitope T117-F scaffold'], 'pdb_id': '6mto', 'e_value': 0.0029, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['T'], 'entity_id': 3, 'entry_entity': '6mtq_3', 'molecule_name': ['VRC42 epitope T117-F scaffold'], 'pdb_id': '6mtq', 'e_value': 0.0029, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3h00_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '3h00', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6jqk_2', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6jqk', 'e_value': 0.065, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'P', 'Q'], 'entity_id': 2, 'entry_entity': '5zcx_2', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5zcx', 'e_value': 0.087, 'percentage_identity': 100.0, 'result_sequence': None}], 'RWCVYAYVRVRGVRVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 3.5e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 4.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 4.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 3.1e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'GWKKFFKKAKKVGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 0.033, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 0.034, 'percentage_identity': 71.4, 'result_sequence': None}], 'RTCQSQSHKFKGACFSDTNCDSVCRTENFPRGQCNQHHVERKCYCERDC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 8.7e-14, 'percentage_identity': 59.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 7.4e-12, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 3.2e-11, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 8.1e-11, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 2.1e-10, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 3.6e-10, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 1.6e-06, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 3.1e-06, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 5.2e-06, 'percentage_identity': 34.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 1.3e-05, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 1.9e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 2.8e-05, 'percentage_identity': 36.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 8.7e-05, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 0.00015, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 0.00015, 'percentage_identity': 36.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 0.00039, 'percentage_identity': 28.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n4n_1', 'molecule_name': ['Floral defensin-like protein 1'], 'pdb_id': '1n4n', 'e_value': 0.0026, 'percentage_identity': 25.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.0028, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.0044, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 0.0084, 'percentage_identity': 32.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 0.015, 'percentage_identity': 32.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 0.096, 'percentage_identity': 28.0, 'result_sequence': None}], 'QLGELIQQGGQKIVEKIQKIGQRIRDFFSNLRPRQEA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}], 'ATCDLASGFGVGSSLCAAHCIARRYRGGYCNSKAVCVCRN': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 5.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 1.7e-16, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.1e-15, 'percentage_identity': 82.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 5.2e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 3.5e-13, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 5.7e-12, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 1e-07, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 1e-06, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 2.2e-06, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0075, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.02, 'percentage_identity': 45.7, 'result_sequence': None}], 'RLGDILQKAREKIEGGLKKLVQKIKDFFGKFAPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 8.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 8.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 8.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 8.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 8.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}], 'KKWGWLAWVEPAGEFLKGFGKGAIKEGNKDKWKNI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpk_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpk', 'e_value': 1.5e-17, 'percentage_identity': 91.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpm_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpm', 'e_value': 1.5e-17, 'percentage_identity': 91.4, 'result_sequence': None}], 'MITISTMLQFGLFLIALIGLVIKLIELSNKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ns1_1', 'molecule_name': ['Stap_Strp_tox_C domain-containing protein'], 'pdb_id': '7ns1', 'e_value': 1.9e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDWGSASGCIGQTVVGGWLGGAIPGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 2.3e-08, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 1.1e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 1.1e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 2.9e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 1.2e-05, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 1.5e-05, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.008, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.008, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.008, 'percentage_identity': 48.5, 'result_sequence': None}], 'KAVAAKKSPKKAKKPATPKKAAKSPKKVKKPAAAAKKAAKSPKKATKAAKPKAAKPKAAKAKKAAPKKK': [{'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k63_8', 'molecule_name': ['Histone H1.10', 'Histone H1.4'], 'pdb_id': '7k63', 'e_value': 2e-08, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5y_8', 'molecule_name': ['Histone H1.4'], 'pdb_id': '7k5y', 'e_value': 2e-08, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['Z'], 'entity_id': 7, 'entry_entity': '5nl0_7', 'molecule_name': ['Histone H1.0-B'], 'pdb_id': '5nl0', 'e_value': 4.9e-05, 'percentage_identity': 52.7, 'result_sequence': None}, {'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '6la2_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la2', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6n89_2', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6n89', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 7, 'entry_entity': '6lab_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6lab', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['S'], 'entity_id': 7, 'entry_entity': '6la8_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la8', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6n88_3', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6n88', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '6la9_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6la9', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5x_8', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7k5x', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['T', 'S'], 'entity_id': 7, 'entry_entity': '7cow_7', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7cow', 'e_value': 0.00052, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 1, 'entry_entity': '7dbp_1', 'molecule_name': ['Histone H1.0'], 'pdb_id': '7dbp', 'e_value': 0.00055, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['CU'], 'entity_id': 55, 'entry_entity': '4v6w_55', 'molecule_name': ['60S ribosomal protein L22'], 'pdb_id': '4v6w', 'e_value': 0.013, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['a0'], 'entity_id': 1, 'entry_entity': '6ncl_1', 'molecule_name': ['P14'], 'pdb_id': '6ncl', 'e_value': 0.036, 'percentage_identity': 43.3, 'result_sequence': None}], 'KWKSFIKKLASKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0018, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0018, 'percentage_identity': 70.0, 'result_sequence': None}], 'GVLDILKGAAKDLAGHVATKVI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 5.7e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 5.7e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 5.7e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 5.9e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 5.9e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 5.9e-07, 'percentage_identity': 77.3, 'result_sequence': None}], 'LLGDFFRKSKEKIGKEFKRIVQR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 7.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 7.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 7.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 7.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 7.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDFGQASGCIGQTVVGGWLGGAIPGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 6.1e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.5e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 0.00012, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 0.00015, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.067, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.067, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.067, 'percentage_identity': 45.5, 'result_sequence': None}], 'AANFGPSVFTPEVHETWQKFLNVVVAALGKQYH': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1spg_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1spg', 'e_value': 2e-11, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3bcq_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bcq', 'e_value': 8.9e-08, 'percentage_identity': 69.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2r1h_2', 'molecule_name': ['Hemoglobin subunit beta-4'], 'pdb_id': '2r1h', 'e_value': 9.3e-07, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3bom_2', 'molecule_name': ['Hemoglobin subunit beta-4'], 'pdb_id': '3bom', 'e_value': 9.3e-07, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6iyi_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 1.2e-05, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6iyh_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyh', 'e_value': 9.8e-05, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1xq5_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1xq5', 'e_value': 0.00016, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4esa_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '4esa', 'e_value': 0.0002, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2aa1_2', 'molecule_name': ['Hemoglobin subunit beta-C'], 'pdb_id': '2aa1', 'e_value': 0.0004, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6hit_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6hit', 'e_value': 0.0004, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4iro_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '4iro', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4g51_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '4g51', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3mju_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3mju', 'e_value': 0.00081, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1s5x_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1s5x', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2r80_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2r80', 'e_value': 0.00081, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4odc_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '4odc', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1hbh_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1hbh', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2peg_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2peg', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3gkv_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3gkv', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3gqg_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3gqg', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2h8f_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2h8f', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s5y_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1s5y', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1pbx_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1pbx', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3dhr_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3dhr', 'e_value': 0.00081, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2h8d_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2h8d', 'e_value': 0.00081, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5lfg_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '5lfg', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1t1n_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '1t1n', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ng6_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '3ng6', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3nfe_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '3nfe', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3d1k_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '3d1k', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3e_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3e', 'e_value': 0.0026, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1v4w_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4w', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1v4x_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4x', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1v4u_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4u', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3d_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3d', 'e_value': 0.0026, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 2, 'entry_entity': '1a9w_2', 'molecule_name': ['Hemoglobin subunit epsilon'], 'pdb_id': '1a9w', 'e_value': 0.0026, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1la6_2', 'molecule_name': ['Hemoglobin subunit beta-1/2'], 'pdb_id': '1la6', 'e_value': 0.0026, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1gcv_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1gcv', 'e_value': 0.0049, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3k8b_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3k8b', 'e_value': 0.0053, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qmb_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '2qmb', 'e_value': 0.0053, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zmy_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6zmy', 'e_value': 0.0053, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1hbr_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1hbr', 'e_value': 0.0053, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zmx_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6zmx', 'e_value': 0.0053, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqj_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqj', 'e_value': 0.0067, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqk_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqk', 'e_value': 0.0067, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3wtg_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 0.0067, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1faw_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1faw', 'e_value': 0.0067, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 2, 'entry_entity': '1fdh_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '1fdh', 'e_value': 0.0067, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3mjp_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3mjp', 'e_value': 0.0084, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3eok_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3eok', 'e_value': 0.0084, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3fs4_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3fs4', 'e_value': 0.011, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3a59_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3a59', 'e_value': 0.011, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3wr1_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3wr1', 'e_value': 0.013, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6rp5_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '6rp5', 'e_value': 0.013, 'percentage_identity': 61.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1c40_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1c40', 'e_value': 0.017, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1a4f_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1a4f', 'e_value': 0.017, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '1hv4_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1hv4', 'e_value': 0.017, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2zfb_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zfb', 'e_value': 0.021, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '3w4u_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '3w4u', 'e_value': 0.026, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1jeb_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '1jeb', 'e_value': 0.026, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3mkb_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3mkb', 'e_value': 0.032, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1gcw_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1gcw', 'e_value': 0.05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1out_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1out', 'e_value': 0.053, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ouu_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1ouu', 'e_value': 0.053, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3at5_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 0.054, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3at6_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 0.054, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj3_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj3', 'e_value': 0.085, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj2_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj2', 'e_value': 0.085, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj1_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj1', 'e_value': 0.085, 'percentage_identity': 46.9, 'result_sequence': None}], 'HTPTPTPICKSRSHEYKGRCIQDMDCNAACVKESESYTGGFCNGRPPFKQCFCTKPCKRERAAATLRWPGL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 1.4e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 4e-06, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 4.8e-06, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 8.3e-06, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 1.6e-05, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 2.3e-05, 'percentage_identity': 38.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 3.6e-05, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 4.7e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 4.7e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 4.7e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '6b55_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '6b55', 'e_value': 4.7e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mr4_1', 'molecule_name': ['Defensin-like protein'], 'pdb_id': '1mr4', 'e_value': 4.7e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 5.8e-05, 'percentage_identity': 32.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 6.6e-05, 'percentage_identity': 36.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 7.9e-05, 'percentage_identity': 38.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5kk4_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5kk4', 'e_value': 0.00011, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '5vyp_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5vyp', 'e_value': 0.00011, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6mry_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6mry', 'e_value': 0.00078, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 0.0031, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kpy_1', 'molecule_name': ['Major pollen allergen Art v 1'], 'pdb_id': '2kpy', 'e_value': 0.0038, 'percentage_identity': 31.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.017, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 0.045, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 0.064, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 0.092, 'percentage_identity': 30.0, 'result_sequence': None}], 'LSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAA': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 1.3e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2ri4_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2ri4', 'e_value': 6.4e-16, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qu0_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2qu0', 'e_value': 6.4e-16, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3eu1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3eu1', 'e_value': 6.4e-16, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d1a_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1a', 'e_value': 6.4e-16, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 6.4e-16, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 3.9e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 3.9e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4mqj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqj', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4mqk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqk', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7n', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5hu6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hu6', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7p', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e29_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e29', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kav_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kav', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2t', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbc', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbd', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhe', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1shr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1shr', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yie_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yie', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y2z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y2z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dn2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhe', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hhb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbd', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4p', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4v', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y85_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y85', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y35', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yev', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1c7b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7b', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j41_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j41', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['W', 'R', 'U', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1jy7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1jy7', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y46_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y46', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yg5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yg5', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4b', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeo_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeo', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4g', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygd', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4f', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1si4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1si4', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '3b75_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3b75', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fn3', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1x', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rqa_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rqa', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a00_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a00', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cls', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a01_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a01', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hba_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hba', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0u', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq3', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3y', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1m9p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1m9p', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gzx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gzx', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1fdh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fdh', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aet_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aet', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aeu', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kah_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kah', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jjq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jjq', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aev', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 4e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 4.1e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1p', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1abw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1abw', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 8.8e-15, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 1.1e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 1.2e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 1.2e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 1.2e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3o', 'e_value': 1.9e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 3.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 3.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2w72_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 4.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hrw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hrw', 'e_value': 4.3e-14, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 4.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 4.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 4.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 4.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eui_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 5.6e-14, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xye', 'e_value': 5.6e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz4', 'e_value': 5.6e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 9.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 9.3e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 9.4e-14, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 1.2e-13, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['C', 'A', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ker_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 1.2e-13, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 1.2e-13, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gys_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gys', 'e_value': 1.2e-13, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gqr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqr', 'e_value': 1.2e-13, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d4x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d4x', 'e_value': 1.2e-13, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqp', 'e_value': 1.2e-13, 'percentage_identity': 90.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 1.2e-13, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrf_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 1.2e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3vre_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 1.2e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 1.2e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 1.6e-13, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 1.6e-13, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 2.1e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 2.1e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 2.1e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hds_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '1hds', 'e_value': 2.1e-13, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 2.1e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h2l_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 2.7e-13, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 2.7e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 2.7e-13, 'percentage_identity': 80.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 2.7e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dhb', 'e_value': 2.7e-13, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 2.7e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sva_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6sva', 'e_value': 4.5e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2mhb', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5c6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5c6e', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6r2o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6r2o', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5x', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlx', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlt', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8h', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlu', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ibe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ibe', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iwh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1iwh', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns6', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s0h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s0h', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8k', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8i', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlv', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlw', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g0b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0b', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns9', 'e_value': 4.6e-13, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 6e-13, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3fh9', 'e_value': 1e-12, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 2.9e-12, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 2.9e-12, 'percentage_identity': 80.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 2.9e-12, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 2.9e-12, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 2.9e-12, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 2.9e-12, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 3.8e-12, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 3.8e-12, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 3.8e-12, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 3.8e-12, 'percentage_identity': 77.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 4.9e-12, 'percentage_identity': 77.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2rao_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2rao', 'e_value': 1.1e-11, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3lqd_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3lqd', 'e_value': 1.1e-11, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3fs4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3fs4', 'e_value': 1.1e-11, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3a59_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3a59', 'e_value': 1.1e-11, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 1.4e-11, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 1.4e-11, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 1.4e-11, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '3w4u_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '3w4u', 'e_value': 4.3e-10, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zfb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zfb', 'e_value': 4.3e-10, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1jeb_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '1jeb', 'e_value': 4.3e-10, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbr_1', 'molecule_name': ['Hemoglobin subunit alpha-D'], 'pdb_id': '1hbr', 'e_value': 5.6e-10, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 2.7e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 2.7e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3hf4_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3hf4', 'e_value': 3.5e-09, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dht_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3dht', 'e_value': 3.5e-09, 'percentage_identity': 74.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2z6n_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '2z6n', 'e_value': 7.7e-09, 'percentage_identity': 63.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wmu_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1wmu', 'e_value': 7.7e-09, 'percentage_identity': 63.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v75_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1v75', 'e_value': 7.7e-09, 'percentage_identity': 63.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a0g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3a0g', 'e_value': 4.9e-08, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3hyu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hyu', 'e_value': 4.9e-08, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyi_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 9.4e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4w_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4w', 'e_value': 0.00017, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4x_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4x', 'e_value': 0.00017, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4u_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4u', 'e_value': 0.00017, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyh_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyh', 'e_value': 0.00017, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bcq_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bcq', 'e_value': 0.00029, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4esa_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '4esa', 'e_value': 0.00049, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rp5_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '6rp5', 'e_value': 0.00049, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1out_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1out', 'e_value': 0.00049, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ouu_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1ouu', 'e_value': 0.00049, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4iro_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4iro', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4g51_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4g51', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5x', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4odc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4odc', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbh', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2peg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2peg', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3gkv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gkv', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqg', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8f', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1s5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5y', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pbx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1pbx', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8d', 'e_value': 0.00064, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xq5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1xq5', 'e_value': 0.0018, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj3_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj3', 'e_value': 0.0018, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj2_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj2', 'e_value': 0.0018, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj1_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj1', 'e_value': 0.0018, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5lfg_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '5lfg', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t1n_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1t1n', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ng6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3ng6', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nfe_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3nfe', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d1k_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1k', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2aa1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2aa1', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1la6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1la6', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1spg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1spg', 'e_value': 0.0089, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6hit_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '6hit', 'e_value': 0.012, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcw', 'e_value': 0.092, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcv', 'e_value': 0.092, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r1h_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '2r1h', 'e_value': 0.095, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bom_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '3bom', 'e_value': 0.095, 'percentage_identity': 46.4, 'result_sequence': None}], 'VTCDVLSFEAKGIAVNHSACALHAIALRKKGGSCQNGVCVARN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 9e-20, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 2.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 8.2e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 9.7e-06, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 1.8e-05, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 4.5e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 4.5e-05, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 4.9e-05, 'percentage_identity': 38.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 8.6e-05, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.0004, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.0023, 'percentage_identity': 38.1, 'result_sequence': None}], 'AWKKTIRQYLKNEIKKKWRKAVIAW': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8t_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8t', 'e_value': 3.7e-09, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 1.7e-07, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 3.7e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 3.7e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 3.8e-05, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}], 'PRPPHPPRPPHPPRPPHPPRPPHPRL': [{'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z1r_1', 'molecule_name': ['Collagen-like peptide'], 'pdb_id': '4z1r', 'e_value': 0.0004, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cag_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cag', 'e_value': 0.0004, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cgd_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cgd', 'e_value': 0.0004, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ah9_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3ah9', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a08_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a08', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2d3h_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2d3h', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a0a_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a0a', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a19_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a19', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3b0s_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3b0s', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'G', 'E', 'F'], 'entity_id': 1, 'entry_entity': '1ym8_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '1ym8', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ai6_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3ai6', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2cuo_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2cuo', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2d3f_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2d3f', 'e_value': 0.00044, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 2, 'entry_entity': '1x1k_2', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '1x1k', 'e_value': 0.00047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4oy5_1', 'molecule_name': ['Collagen'], 'pdb_id': '4oy5', 'e_value': 0.00047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3b2c_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3b2c', 'e_value': 0.00047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '1k6f_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '1k6f', 'e_value': 0.00047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1wzb_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '1wzb', 'e_value': 0.00047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'G'], 'entity_id': 1, 'entry_entity': '2g66_1', 'molecule_name': ['collagen'], 'pdb_id': '2g66', 'e_value': 0.00048, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2klw_3', 'molecule_name': ['(POG)10'], 'pdb_id': '2klw', 'e_value': 0.00049, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1nay_1', 'molecule_name': ['Collagen-like peptide'], 'pdb_id': '1nay', 'e_value': 0.00072, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3adm_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3adm', 'e_value': 0.0027, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3abn_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3abn', 'e_value': 0.0027, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a1h_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a1h', 'e_value': 0.0027, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a0m_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a0m', 'e_value': 0.0038, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5y45_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '5y45', 'e_value': 0.0044, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5y46_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '5y46', 'e_value': 0.0044, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['D', 'C', 'E'], 'entity_id': 2, 'entry_entity': '5ou9_2', 'molecule_name': ['Collagen alpha-1(I) chain'], 'pdb_id': '5ou9', 'e_value': 0.0044, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ipn_1', 'molecule_name': ['Non-natural Collagen'], 'pdb_id': '3ipn', 'e_value': 0.0044, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '1ei8_1', 'molecule_name': ['LD39545p'], 'pdb_id': '1ei8', 'e_value': 0.0047, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2drt_1', 'molecule_name': ['collagen like peptide'], 'pdb_id': '2drt', 'e_value': 0.0048, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1qsu_1', 'molecule_name': ['PROTEIN ((PRO-HYP-GLY)4- GLU-LYS-GLY(PRO-HYP-GLY)5)'], 'pdb_id': '1qsu', 'e_value': 0.0067, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5k86_1', 'molecule_name': ['Aza-glycine containing collagen peptide'], 'pdb_id': '5k86', 'e_value': 0.0082, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3wn8_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3wn8', 'e_value': 0.011, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['D', 'B', 'C'], 'entity_id': 2, 'entry_entity': '4lor_2', 'molecule_name': ['collagen-like peptide from C1q'], 'pdb_id': '4lor', 'e_value': 0.021, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3pon_1', 'molecule_name': ['MBL collagen-like peptide'], 'pdb_id': '3pon', 'e_value': 0.021, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'C'], 'entity_id': 2, 'entry_entity': '3pob_2', 'molecule_name': ['MBL collagen-like peptide'], 'pdb_id': '3pob', 'e_value': 0.021, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6w47_1', 'molecule_name': ['Collagen-like peptide'], 'pdb_id': '6w47', 'e_value': 0.028, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7jx4_1', 'molecule_name': ['Collagen mimetic peptide with N-Lysine guest'], 'pdb_id': '7jx4', 'e_value': 0.034, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '7jx5_1', 'molecule_name': ['Collagen mimetic peptide with N-Phenylalanine guest'], 'pdb_id': '7jx5', 'e_value': 0.034, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2drx_1', 'molecule_name': ['collagen like peptide'], 'pdb_id': '2drx', 'e_value': 0.035, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '2f6a_2', 'molecule_name': ['StpC protein'], 'pdb_id': '2f6a', 'e_value': 0.041, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6syj_1', 'molecule_name': ['ProM2 containing collagen model peptide.'], 'pdb_id': '6syj', 'e_value': 0.044, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3u29_1', 'molecule_name': ['collagen mimetic peptide'], 'pdb_id': '3u29', 'e_value': 0.052, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3t4f_1', 'molecule_name': ['collagen mimetic peptide'], 'pdb_id': '3t4f', 'e_value': 0.052, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6vzx_1', 'molecule_name': ['collagen mimetic peptide'], 'pdb_id': '6vzx', 'e_value': 0.052, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['D', 'C', 'E'], 'entity_id': 1, 'entry_entity': '6m80_1', 'molecule_name': ['Collagen peptide containing aza-proline and aza-glycine'], 'pdb_id': '6m80', 'e_value': 0.06, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['D', 'B', 'C'], 'entity_id': 2, 'entry_entity': '5cjb_2', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '5cjb', 'e_value': 0.071, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '2v8c_2', 'molecule_name': ['Vasodilator-stimulated phosphoprotein'], 'pdb_id': '2v8c', 'e_value': 0.072, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3pod_1', 'molecule_name': ['MBL collagen-like peptide'], 'pdb_id': '3pod', 'e_value': 0.073, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['V1'], 'entity_id': 15, 'entry_entity': '6x89_15', 'molecule_name': ['NADH dehydrogenase [ubiquinone] flavoprotein 1, mitochondrial'], 'pdb_id': '6x89', 'e_value': 0.089, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 6, 'entry_entity': '6sb0_6', 'molecule_name': ['Proline-rich AKT1 substrate 1'], 'pdb_id': '6sb0', 'e_value': 0.093, 'percentage_identity': 60.0, 'result_sequence': None}], 'FLSTLLNVASNVVPTLICKITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.05, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.05, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.05, 'percentage_identity': 45.8, 'result_sequence': None}], 'KAKAKAVSRSARAGLQFPVGRIHRHLK': [{'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 7.2e-12, 'percentage_identity': 96.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 8.1e-12, 'percentage_identity': 96.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 3.1e-11, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 3.1e-11, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 3.1e-11, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 3.1e-11, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 3.1e-11, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4nft_1', 'molecule_name': ['Histone H2B type 2-E', 'Histone H2A.Z'], 'pdb_id': '4nft', 'e_value': 4.6e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k0c_1', 'molecule_name': ['Histone H2A.V', 'Histone H2B 2'], 'pdb_id': '6k0c', 'e_value': 7.7e-09, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 9.2e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 9.3e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 9.5e-09, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 1e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 1e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 1e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 2.1e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 2.1e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b31_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b31', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b32_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b32', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 2.1e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 2.4e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 2.4e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 2.4e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 2.4e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 2.4e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 2.6e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 3.6e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 3.6e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 3.6e-08, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 4.7e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 7.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 7.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 7.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 7.3e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 7.3e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 7.4e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 7.6e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 7.6e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 7.8e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 7.8e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 7.8e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 7.8e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 7.8e-08, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 8e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6g0l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 8.1e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 8.2e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 1e-07, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 1.6e-07, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 1.6e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 2.1e-07, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 3.2e-07, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 3.6e-07, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5chl_2', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5chl', 'e_value': 4.3e-07, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 8e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 8e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 8e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 8e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 8.7e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 8.7e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 8.7e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 1.4e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 1.4e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 1.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 1.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 1.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 2.2e-06, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 3e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 3e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 3.1e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 3.1e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 3.1e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 4.8e-06, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 5e-06, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 6.2e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 8.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7bp6_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp6', 'e_value': 8.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7bp2_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp2', 'e_value': 8.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '7bp4_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp4', 'e_value': 8.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bp5_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp5', 'e_value': 8.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 1.1e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 1.1e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 1.4e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 1.5e-05, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 1.5e-05, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 1.5e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 1.6e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 1.7e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4m6b_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '4m6b', 'e_value': 1.9e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6ae8_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '6ae8', 'e_value': 1.9e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jss_1', 'molecule_name': ['Histone H2A.Z', 'Histone H2B.1'], 'pdb_id': '2jss', 'e_value': 1.9e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 2.4e-05, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 2.6e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 2.6e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 7, 'entry_entity': '6z6p_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6z6p', 'e_value': 2.6e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 5.5e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 5.6e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 5.9e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 5.9e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7dlx_1', 'molecule_name': ['Histone H2A', 'Histone H2B'], 'pdb_id': '7dlx', 'e_value': 8e-05, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 8.4e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 8.4e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 8.4e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 0.0001, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '2f8n_6', 'molecule_name': ['Core histone macro-H2A.1'], 'pdb_id': '2f8n', 'e_value': 0.00059, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1u35_4', 'molecule_name': ['Core histone macro-H2A.1'], 'pdb_id': '1u35', 'e_value': 0.00059, 'percentage_identity': 55.6, 'result_sequence': None}], 'RWCVYAYVRVRGVLSRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.8e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.4e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.4e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.6e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'STLHLVLRARGG': [{'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5eya_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5eya', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5wfi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wfi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5vey_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vey', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4dhz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4bvu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bvu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vf0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vf0', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5fer_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5fer', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xpk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xpk', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xu8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xu8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5emz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5emz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xve_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xve', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '5ohp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'K', 'M', 'J', 'N', 'O', 'L'], 'entity_id': 2, 'entry_entity': '5ohl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohl', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 3, 'entry_entity': '5edv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5edv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mm3', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ohv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6i_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6i', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mjb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mjb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '2mbh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '4s1z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s1z', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4wur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wur', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xyz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xyz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5goj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goj', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5tut_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tut', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 1, 'entry_entity': '5h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5h7s', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gog_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gog', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xof_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xof', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5goi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5go7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go7', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4wzp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wzp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gob', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gvi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '5kyd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3v', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mcn', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1y_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1y', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5goh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xol_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xol', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4y1h_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4y1h', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5go8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5god_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5god', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'A', 'B', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4xkh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4s22_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s22', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5goc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goc', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4xok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xok', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5txk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5txk', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['h', 'c', 'X', 'm'], 'entity_id': 10, 'entry_entity': '4zux_10', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zux', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5mn9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mn9', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulk', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5u4p_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5u4p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gok', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5tr4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tr4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7u', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2xk5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xk5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '4jio_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jio', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kox', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7w_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7w', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ldt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ldt', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zlz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zlz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3wxg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxg', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'B', 'J', 'F'], 'entity_id': 2, 'entry_entity': '4dhj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhj', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7s', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 2, 'entry_entity': '3tbl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tbl', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'D', 'M', 'Q', 'N', 'O', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddg', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lz6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lz6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '3vuy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuy', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vux_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vux', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5kye_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kye', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3u_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3u', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6h', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2k_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n2k', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5l6j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6j', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mj5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5lrv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyc', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jbv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jbv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n13', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mi8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mi8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4un2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4un2', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'O'], 'entity_id': 8, 'entry_entity': '5kgf_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kgf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 3, 'entry_entity': '5o44_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5o44', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mor_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mor', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbq', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3tmp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tmp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ugb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ugb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '4bos_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bos', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4jqw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jqw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4m0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4m0w', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0t', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3vht_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vht', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4bbn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bbn', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2qho_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2qho', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3m3j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3m3j', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vnz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vnz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nvg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nvg', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vo0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vo0', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vzm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzm', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '4boz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4boz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5j8p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j8p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5jby_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jby', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5jqs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jqs', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nmc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nmc', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z9s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4z9s', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5ait_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ait', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3m', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlj', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['9'], 'entity_id': 9, 'entry_entity': '5a5b_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5a5b', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ulf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5af4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5aiu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5aiu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujn', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jtv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mro_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mro', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6p5b_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6p5b', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7m', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6tnf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tnf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 15, 'entry_entity': '6tbm_15', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tbm', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 12, 'entry_entity': '6t9l_12', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t9l', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6v5d_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6v5d', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6asr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6asr', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '6vae_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vae', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '6ttu_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ttu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k9p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6txb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6txb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kox', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kow_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kow', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6e2b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6e2b', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6kl4_3', 'molecule_name': ['Ub'], 'pdb_id': '6kl4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6aqr_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6aqr', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6vaf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vaf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msb_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3o_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3o', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 3, 'entry_entity': '5d0k_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0k', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nli_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nli', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujl', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xiu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xiu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3n', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 1, 'entry_entity': '6jwi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jwi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6q00_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q00', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oi4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oi4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6s53_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6s53', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N', 'O'], 'entity_id': 8, 'entry_entity': '6ftx_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ftx', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djx', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5zbu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zbu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4i_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k4i', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6lp2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6lp2', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 4, 'entry_entity': '6n13_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n13', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6xqc_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6xqc', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6nxk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nxk', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6qml_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qml', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4zpz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zpz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1z_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1z', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xbo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5bz0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bz0', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5dfl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dfl', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 3, 'entry_entity': '5d0m_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0m', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6wtg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wtg', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5af6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msd_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5xdp_1', 'molecule_name': ['D-ubiquitin'], 'pdb_id': '5xdp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7cap_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7cap', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb7', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 7, 'entry_entity': '6jma_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jma', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ymy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ymy', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6q01_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q01', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7o', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '6w9d_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9d', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6oaa_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oaa', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6tuv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tuv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6rya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6rya', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'J'], 'entity_id': 2, 'entry_entity': '6oa9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oa9', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4wlr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wlr', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'C', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '6z7v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6z7v', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zqh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6zqh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6if1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6if1', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wwz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7s', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q0w', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ogw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ogw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '1wr6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubq', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wr1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr1', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1tbe', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v80_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v80', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n32_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n32', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ru6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rr9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rr9', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3n30_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n30', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '3mhs_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3mhs', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3nhe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nhe', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rsu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rsu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wx1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx1', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3cmm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3cmm', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1f9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1f9j', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2jt4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jt4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h1u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h1u', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2o6v_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2o6v', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pea_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pea', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2dx5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ehv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ehv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2g45_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g45', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcm', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2g3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g3q', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eec_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3eec', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2gmi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gmi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3efu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3efu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kde_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kde', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kdf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pe9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pe9', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2oob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2oob', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3hm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3hm3', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcn', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hth_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hth', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2jri_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jri', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jy6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jy6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcs_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcs', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcq_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '2fcq', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2fif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fif', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8c_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8c', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a33_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a33', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k39_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k39', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8b', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jzz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jzz', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2z59_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2z59', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ayo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ayo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3alb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3alb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ofi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ofi', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ns8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ns8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 1, 'entry_entity': '1yd8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yd8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3aul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3aul', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 2, 'entry_entity': '1p3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1p3q', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g6j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1g6j', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1uzx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1uzx', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1otr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1otr', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1zgu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zgu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yiw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '1yiw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wrd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wrd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yj1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yj1', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v81_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v81', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcc', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q5w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q5w', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1s1q', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fxt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1fxt', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1aar_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1aar', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cmx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1cmx', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1nbf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1nbf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '2c7n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7n', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7m', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3h7p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d3g_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2d3g', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lj5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wx0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx0', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvq', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2xbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xbb', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2bgf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2bgf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fid_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fid', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2fuh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fuh', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hd5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hd5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xqq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xqq', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1d3z', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2jf5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jf5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2den_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2den', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0f_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0f', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4pih_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pih', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l3z', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vuw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvp', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2klg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2klg', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '3phd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ktf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ktf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2l00_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l00', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4pig_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pig', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kn5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kn5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2xew_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xew', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6isu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6isu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '7bbf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7jxv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jxv', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '7bbd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbd', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7keo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7keo', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ohm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohm', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'J', 'C'], 'entity_id': 3, 'entry_entity': '5ydk_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydk', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '3wwq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wwq', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqo', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'F'], 'entity_id': 3, 'entry_entity': '6a6i_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6a6i', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 3, 'entry_entity': '5xit_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xit', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6k2u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k2u', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyi', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6oq2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq2', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqs_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqs', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ud0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ud0', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F', 'L'], 'entity_id': 3, 'entry_entity': '6sqr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqr', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '7bxg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bxg', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5xis_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xis', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6oq1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq1', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyj', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4nql_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nql', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jsv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jsv', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3a1q_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a1q', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9j', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9k', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2znv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2znv', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ld9', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6kfp_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kfp', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mur', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4r62_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4r62', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbb', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'F'], 'entity_id': 2, 'entry_entity': '5yik_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yik', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq4', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6cp2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6cp2', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq3', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '5yij_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yij', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5yt6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yt6', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3nob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nob', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9p', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5vzw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzw', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mre_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mre', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq7', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq6', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6jky_3', 'molecule_name': ['Ub'], 'pdb_id': '6jky', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qf8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qf8', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq5', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'K', 'D', 'B', 'C', 'J', 'N', 'O', 'F', 'G', 'L'], 'entity_id': 2, 'entry_entity': '6upu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6upu', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6kg6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kg6', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4nqk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nqk', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'U', 'T', 'X', 'S'], 'entity_id': 3, 'entry_entity': '4p4h_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4p4h', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4nnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nnj', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2khw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2khw', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3oj3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj3', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '3olm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3olm', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3oj4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj4', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ptf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ptf', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4xkl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkl', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m93_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5m93', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ksk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksk', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mdk', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6dc6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6dc6', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U', 'X'], 'entity_id': 3, 'entry_entity': '3dvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvn', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 3, 'entry_entity': '3dvg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvg', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4gu2_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gu2', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4ap4_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ap4', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4gsw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gsw', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '7ay1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay1', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4v3k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3k', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msm', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4v3l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3l', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '5mnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mnj', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4zfr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zfr', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4auq_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4auq', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msq', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4pqt_2', 'molecule_name': ['Protein UBBP4'], 'pdb_id': '4pqt', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4k1r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k1r', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '5ln1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ln1', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'L'], 'entity_id': 4, 'entry_entity': '3zni_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zni', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4zft_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zft', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hpr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hpr', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jvz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jvz', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jw0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jw0', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbr', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai1', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai0', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['III', 'LLL', 'CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ahz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ahz', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5udh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5udh', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jvc_1', 'molecule_name': ['ubiquitin_like protein'], 'pdb_id': '2jvc', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zw7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zw7', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'F', 'G', 'L'], 'entity_id': 3, 'entry_entity': '4whv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4whv', 'e_value': 0.0068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ii2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii2', 'e_value': 0.0068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbk', 'e_value': 0.0068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'R', 'P', 'M', 'Q', 'X', 'N', 'O'], 'entity_id': 1, 'entry_entity': '7k5j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k5j', 'e_value': 0.0068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbm', 'e_value': 0.0069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gbn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbn', 'e_value': 0.0069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2gbj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbj', 'e_value': 0.0069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 6, 'entry_entity': '7lyc_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7lyc', 'e_value': 0.0069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6nya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nya', 'e_value': 0.007, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '5tte_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tte', 'e_value': 0.0071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6ypt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ypt', 'e_value': 0.0071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4fjv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4fjv', 'e_value': 0.0071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1sif', 'e_value': 0.0073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4ii3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii3', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5knl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5knl', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '5cvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvn', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5cvm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvm', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '6nyo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nyo', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5cvo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvo', 'e_value': 0.0082, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx5', 'e_value': 0.0084, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx6', 'e_value': 0.0084, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4hcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4hcn', 'e_value': 0.0084, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3vdz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vdz', 'e_value': 0.0099, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ojr', 'e_value': 0.0099, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kdi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdi', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['AK', 'FK', 'DK', 'HK'], 'entity_id': 10, 'entry_entity': '4v8p_10', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4v8p', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['5'], 'entity_id': 6, 'entry_entity': '4adx_6', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4adx', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kna_1', 'molecule_name': ['viral Ubiquitin'], 'pdb_id': '6kna', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '5umd_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5umd', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Bm'], 'entity_id': 41, 'entry_entity': '6uz7_41', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uz7', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 47, 'entry_entity': '6n8n_47', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8n', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 44, 'entry_entity': '6n8m_44', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8m', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 24, 'entry_entity': '6lqm_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6lqm', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u'], 'entity_id': 24, 'entry_entity': '6ef3_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6ef3', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '3j79_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3j79', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 41, 'entry_entity': '5t6r_41', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t6r', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 42, 'entry_entity': '5t62_42', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t62', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 27, 'entry_entity': '5t2c_27', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t2c', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 15, 'entry_entity': '7azy_15', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '7azy', 'e_value': 0.012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3wxf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxf', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3wxe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxe', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyx_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyx', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '3jam_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jam', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jaq_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jaq', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 31, 'entry_entity': '3j81_31', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3j81', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jap_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jap', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyy_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyy', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 15, 'entry_entity': '7k5i_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '7k5i', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '5flx_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '5flx', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juo_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juo', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juu_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juu', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znz', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzy_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzy', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzz_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzz', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '4d61_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4d61', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'H', 'R', 'P', 'D', 'C', 'J', 'T', 'X', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4ksl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksl', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Af', 'Cf'], 'entity_id': 33, 'entry_entity': '4v88_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v88', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8z_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8z', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jup_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jup', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jut_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jut', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jus_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jus', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzx_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzx', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 15, 'entry_entity': '5mc6_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5mc6', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8y_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8y', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6kbe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kbe', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Nb', 'N', 'Nc'], 'entity_id': 50, 'entry_entity': '6sv4_50', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6sv4', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['k'], 'entity_id': 16, 'entry_entity': '6ybs_16', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ybs', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['6', 'fb'], 'entity_id': 32, 'entry_entity': '6t83_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t83', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Sf'], 'entity_id': 32, 'entry_entity': '6t7i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t7i', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 11, 'entry_entity': '6eml_11', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6eml', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['SFF'], 'entity_id': 26, 'entry_entity': '6rm3_26', 'molecule_name': ['eS31'], 'pdb_id': '6rm3', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E1', 'e1'], 'entity_id': 33, 'entry_entity': '5ndg_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5ndg', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 1, 'entry_entity': '5wq4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wq4', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p5i_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p5i', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6snt_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6snt', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 32, 'entry_entity': '6zlw_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zlw', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6sgc_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6sgc', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6y_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6y', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6x_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6x', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j78_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j78', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '2zvn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvn', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w9n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2w9n', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '2zvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvo', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '3b08_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b08', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3b0a_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b0a', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3axc_1', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '3axc', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j77_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j77', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2y5b_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2y5b', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 32, 'entry_entity': '6zu9_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zu9', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 35, 'entry_entity': '6zce_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zce', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 31, 'entry_entity': '6zmt_31', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zmt', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zv6_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zv6', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'e1'], 'entity_id': 79, 'entry_entity': '6hhq_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6hhq', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['3R'], 'entity_id': 79, 'entry_entity': '6ip6_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ip6', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '7eal_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7eal', 'e_value': 0.015, 'percentage_identity': 91.7, 'result_sequence': None}], 'GMWSKIKETAMAAAKEAAKAAGKTISDMIKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jx6_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2jx6', 'e_value': 0.00059, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k9b_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2k9b', 'e_value': 0.00059, 'percentage_identity': 51.6, 'result_sequence': None}], 'VDKPPYLPRPKPPRRIYNNR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.0025, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.01, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.068, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.068, 'percentage_identity': 72.2, 'result_sequence': None}], 'FEVLPNFKHIQVKVFNHGEHIHHH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1c_1', 'molecule_name': ['Tyrosinase_Cu-bd domain-containing protein'], 'pdb_id': '2n1c', 'e_value': 5.7e-06, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n30_1', 'molecule_name': ['Tyrosinase_Cu-bd domain-containing protein'], 'pdb_id': '2n30', 'e_value': 6e-06, 'percentage_identity': 91.3, 'result_sequence': None}], 'KILRGVSKKIMRTFLRRISKDILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 3.9e-09, 'percentage_identity': 85.2, 'result_sequence': None}], 'LYKKIIKKLLES': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4r9w_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9w', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'B', 'A'], 'entity_id': 1, 'entry_entity': '4r9y_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9y', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'U', 'I', 'C', 'X', 'O', 'F', 'L'], 'entity_id': 3, 'entry_entity': '4rau_3', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4rau', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9s_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9s', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9q_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9q', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9r_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9r', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1pfm_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1pfm', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1pfn_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1pfn', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1rhp_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1rhp', 'e_value': 0.014, 'percentage_identity': 100.0, 'result_sequence': None}], 'RCLPAGKTCVRGPMRVPCCGSCSQNKCT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1y29_1', 'molecule_name': ['Omega-theraphotoxin-Hs1a'], 'pdb_id': '1y29', 'e_value': 3e-07, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v5a_1', 'molecule_name': ['Covalitoxin-I'], 'pdb_id': '1v5a', 'e_value': 1.1e-06, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6saa_1', 'molecule_name': ['U1-theraphotoxin-Pf3'], 'pdb_id': '6saa', 'e_value': 4.2e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.069, 'percentage_identity': 56.5, 'result_sequence': None}], 'KWKSFKKKLTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 9.2e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 9.2e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.014, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.014, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.019, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.019, 'percentage_identity': 70.0, 'result_sequence': None}], 'YRGGYTGPIPRPPPIGRPPFRPVCNACYRLSVSDARNCCIKFGSCCHLVK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 2.7e-08, 'percentage_identity': 49.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 2.8e-06, 'percentage_identity': 46.0, 'result_sequence': None}], 'GLFGVLAKVAAHVVPAIAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}], 'KWCVYAYVKVKGVLVKYKKCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 6.6e-10, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 8.8e-10, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 8.8e-10, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 6.3e-09, 'percentage_identity': 66.7, 'result_sequence': None}], 'FTCDVLSAEGGFRGVSIKLNHAACAAHCLYLKKRGGYCNDKAVCVCRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 6.5e-10, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.8e-09, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 7e-09, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 2e-08, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.8e-08, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.7e-07, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.7e-07, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 7e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0001, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.00012, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 0.0025, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.0038, 'percentage_identity': 37.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0057, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.05, 'percentage_identity': 35.1, 'result_sequence': None}], 'KWKKFIKKIGIGAVLKVLTTGLPALKLTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.0086, 'percentage_identity': 81.0, 'result_sequence': None}], 'FLSLIPHAIKAVGVHAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.00069, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.0013, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0031, 'percentage_identity': 72.2, 'result_sequence': None}], 'FLSMIPHIVSGVAALAKHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.00073, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0018, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.027, 'percentage_identity': 47.4, 'result_sequence': None}], 'FLGALFKVASKVLPSVFCAITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.0048, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.0048, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.0048, 'percentage_identity': 50.0, 'result_sequence': None}], 'STIGKLPKWIIKTVNKFTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 0.0001, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 0.0001, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 0.0001, 'percentage_identity': 85.0, 'result_sequence': None}], 'FLPFLLSALPKVFCFFSKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.059, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.059, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.059, 'percentage_identity': 50.0, 'result_sequence': None}], 'GIGKFLHSAKKFGKAAVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.6e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.6e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 5.3e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 6.5e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.2e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.2e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0076, 'percentage_identity': 100.0, 'result_sequence': None}], 'GWGSFFKKAAAVGKAVGKAALTAYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 8.8e-09, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 9e-09, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 1.3e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 2.2e-07, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.021, 'percentage_identity': 48.0, 'result_sequence': None}], 'GLQFPVGRVHRLLRK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 5.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 5.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 5.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 5.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 5.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 11, 'entry_entity': '6z6p_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6z6p', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 6.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 6.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 6.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 6.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 6.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 6.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 6.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 6.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 6.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 6.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 7.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6g0l_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 7.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 7.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 7.9e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 8e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 8.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 8.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 8.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '2f8n_7', 'molecule_name': ['Histone H2A type 1-H'], 'pdb_id': '2f8n', 'e_value': 8.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 8.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 8.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 8.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 8.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 8.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 9.2e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 9.4e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 9.4e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 9.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 9.5e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 9.5e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 9.5e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 1.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 1.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 2.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 2.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 9.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 9.7e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 9.7e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 9.7e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 0.00015, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 0.00017, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 0.00021, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 0.00023, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7dlx_1', 'molecule_name': ['Histone H2A', 'Histone H2B'], 'pdb_id': '7dlx', 'e_value': 0.00033, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k0c_1', 'molecule_name': ['Histone H2A.V', 'Histone H2B 2'], 'pdb_id': '6k0c', 'e_value': 0.00059, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 0.002, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 0.002, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 0.0021, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 0.0023, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b31_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b31', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '5b32_5', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b32', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 0.0024, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5chl_2', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5chl', 'e_value': 0.0035, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4nft_1', 'molecule_name': ['Histone H2B type 2-E', 'Histone H2A.Z'], 'pdb_id': '4nft', 'e_value': 0.0038, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4m6b_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '4m6b', 'e_value': 0.016, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6ae8_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '6ae8', 'e_value': 0.016, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jss_1', 'molecule_name': ['Histone H2A.Z', 'Histone H2B.1'], 'pdb_id': '2jss', 'e_value': 0.016, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7bp6_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp6', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7bp2_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp2', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '7bp4_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp4', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bp5_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp5', 'e_value': 0.019, 'percentage_identity': 71.4, 'result_sequence': None}], 'GVGKFLHSAKKFGQALASEIMKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 9.1e-08, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 9.4e-08, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 7.4e-07, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.5e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.5e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.5e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.01, 'percentage_identity': 91.7, 'result_sequence': None}], 'GIINTLQKYYARVRGGRAAVLSALPKEEQIGKASTRGRKCARRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 3.2e-17, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00039, 'percentage_identity': 46.5, 'result_sequence': None}], 'GRFRRLRKKTRKRLKKIGKVLKLI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ket_1', 'molecule_name': ['Cathelicidin-6'], 'pdb_id': '2ket', 'e_value': 0.05, 'percentage_identity': 50.0, 'result_sequence': None}], 'GIGGVLLSAGKAALKGLTKVLAEKYAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 2.9e-10, 'percentage_identity': 96.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.062, 'percentage_identity': 60.0, 'result_sequence': None}], 'AAHHIARPIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 4e-08, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.1e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.1e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 7.9e-06, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.0063, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.0063, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.0063, 'percentage_identity': 60.0, 'result_sequence': None}], 'GIPCGESCVWIPCLTSAIGCSCKSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2.5e-15, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2.5e-15, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.2e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 4.7e-12, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 8.2e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 2e-10, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 4e-10, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 8.2e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 1.2e-06, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.9e-06, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 7.8e-05, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 0.00019, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.00058, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 0.0022, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.0022, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.0027, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.0053, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.0053, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.0066, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.009, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.01, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.01, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.013, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.013, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.013, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.013, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.016, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.023, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.025, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 0.032, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.051, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.051, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.061, 'percentage_identity': 38.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.066, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.076, 'percentage_identity': 48.1, 'result_sequence': None}], 'FINKAGKLQSQLRTTVVAAAAFLDAFQKVA': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d1l_1', 'molecule_name': ['Protein MTSS 1'], 'pdb_id': '2d1l', 'e_value': 5.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'AWKEKIRKKLKNEIKKKWRKAVIAW': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8t_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8t', 'e_value': 3.7e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 3.1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 3.1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 2.7e-07, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 2.9e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 0.016, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 0.016, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 0.016, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 0.016, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 0.016, 'percentage_identity': 60.0, 'result_sequence': None}], 'GFLNTAMNTVTNLAGTLMDKAKCKIRGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0082, 'percentage_identity': 44.4, 'result_sequence': None}], 'GSKKPVPIIYCQRRTAKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 4.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 4.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 4.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 4.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 4.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 3e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 3e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 3e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 4.4e-07, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 4.4e-07, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xol_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xol', 'e_value': 5.8e-07, 'percentage_identity': 81.0, 'result_sequence': None}], 'GAGALAKFLAKKVAKTVAKQAAKQGAKYVVNKQME': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k38_1', 'molecule_name': ['Cupiennin-1a'], 'pdb_id': '2k38', 'e_value': 1.1e-12, 'percentage_identity': 94.3, 'result_sequence': None}], 'MAGGKAGKDSGKAKAKAVSRSARAGLQFPVGRIHRHLK': [{'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 1.7e-16, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 1.9e-16, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1f66_4', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '1f66', 'e_value': 5.8e-16, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wa9_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '3wa9', 'e_value': 5.9e-16, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b33_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5b33', 'e_value': 5.9e-16, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z30_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5z30', 'e_value': 5.9e-16, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jou_3', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '6jou', 'e_value': 5.9e-16, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 2.9e-09, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 1.1e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 1.6e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 1.6e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 1.6e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 2.2e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 2.2e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 2.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b31_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b31', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b32_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b32', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 2.3e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 2.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 2.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 2.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 2.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 2.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 2.8e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 2.9e-08, 'percentage_identity': 64.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 3.6e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 4.4e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 6.3e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 6.5e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 7e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6g0l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 7.1e-08, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 9e-08, 'percentage_identity': 65.8, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 2.2e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 2.3e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 2.4e-07, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k0c_1', 'molecule_name': ['Histone H2A.V', 'Histone H2B 2'], 'pdb_id': '6k0c', 'e_value': 3.4e-07, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 3.4e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 3.4e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 3.4e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 3.5e-07, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 3.5e-07, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4nft_1', 'molecule_name': ['Histone H2B type 2-E', 'Histone H2A.Z'], 'pdb_id': '4nft', 'e_value': 7e-07, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 8.2e-07, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 8.4e-07, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 8.4e-07, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 1.1e-06, 'percentage_identity': 62.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 1.1e-06, 'percentage_identity': 62.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '7dlx_1', 'molecule_name': ['Histone H2A', 'Histone H2B'], 'pdb_id': '7dlx', 'e_value': 1.7e-06, 'percentage_identity': 62.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 9.4e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 9.4e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 9.4e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 9.7e-06, 'percentage_identity': 62.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 9.7e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 9.7e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 1.1e-05, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 1.8e-05, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 1.9e-05, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 2.4e-05, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 3.8e-05, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5chl_2', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5chl', 'e_value': 4e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 6.8e-05, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 6.8e-05, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 6.8e-05, 'percentage_identity': 71.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 7.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 7.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 7.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 7.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 8e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 8e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 8e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 9.7e-05, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 0.00011, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 0.00011, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 0.00016, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 0.00023, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 0.00023, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 0.00023, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 0.00023, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 0.00023, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 0.00025, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 0.0004, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 0.0005, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7bp6_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp6', 'e_value': 0.0005, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7bp2_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp2', 'e_value': 0.0005, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '7bp4_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp4', 'e_value': 0.0005, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bp5_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp5', 'e_value': 0.0005, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 0.00062, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 0.00065, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 0.00079, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 0.00087, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 0.00088, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 0.00088, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 0.00089, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 0.00089, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 0.00089, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 0.00089, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 0.0009, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 0.00091, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 0.00091, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 0.00091, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 0.00091, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 0.00091, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 0.00098, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 0.0011, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 0.0011, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 0.0011, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4m6b_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '4m6b', 'e_value': 0.0013, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6ae8_1', 'molecule_name': ['Histone H2B.1', 'Histone H2A.Z'], 'pdb_id': '6ae8', 'e_value': 0.0013, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 0.0013, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jss_1', 'molecule_name': ['Histone H2A.Z', 'Histone H2B.1'], 'pdb_id': '2jss', 'e_value': 0.0013, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 0.0014, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 0.0014, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 7, 'entry_entity': '6z6p_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6z6p', 'e_value': 0.0014, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '2f8n_6', 'molecule_name': ['Core histone macro-H2A.1'], 'pdb_id': '2f8n', 'e_value': 0.0032, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1u35_4', 'molecule_name': ['Core histone macro-H2A.1'], 'pdb_id': '1u35', 'e_value': 0.0032, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 0.0045, 'percentage_identity': 75.0, 'result_sequence': None}], 'GIPCGESCVFIPCITAAIGCSCKSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 4.3e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 4.3e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.1e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 1.2e-11, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.5e-11, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 4.5e-11, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 4.6e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 3.6e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 3.6e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 9e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 2.6e-05, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 3.1e-05, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 9.6e-05, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.00091, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.0014, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 0.0018, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.0018, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.0044, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.0051, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.0055, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.0074, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.0086, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.0086, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.0086, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.011, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.011, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.011, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.011, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.021, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.026, 'percentage_identity': 38.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 0.027, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.034, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.036, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.054, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.065, 'percentage_identity': 48.1, 'result_sequence': None}], 'AGYLLGKINLKPLAALAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 1.3e-07, 'percentage_identity': 95.2, 'result_sequence': None}], 'GKPRPYPPRPPPHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.069, 'percentage_identity': 76.5, 'result_sequence': None}], 'VLSAADKGNVKAAWGKVGGHAAEYGAEA': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 1.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 9.2e-11, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eui_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 1.5e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h2l_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 3.3e-10, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['C', 'A', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ker_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 3.3e-10, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2mhb', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sva_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6sva', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5c6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5c6e', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6r2o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6r2o', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dhb', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5x', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlx', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlt', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8h', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlu', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iwh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1iwh', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns6', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8k', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8i', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlv', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlw', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g0b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0b', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns9', 'e_value': 4.3e-10, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hrw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hrw', 'e_value': 9.3e-10, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ibe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ibe', 'e_value': 1.2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gys_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gys', 'e_value': 1.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gqr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqr', 'e_value': 1.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d4x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d4x', 'e_value': 1.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqp', 'e_value': 1.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4mqj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqj', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4mqk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqk', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7n', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5hu6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hu6', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7p', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e29_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e29', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kav_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kav', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2t', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhe', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1shr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1shr', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yie_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yie', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y2z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y2z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2ri4_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2ri4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qu0_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2qu0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dn2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhe', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hhb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3eu1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3eu1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d1a_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1a', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4p', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4v', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y85_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y85', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y35', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yev', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j41_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j41', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['W', 'R', 'U', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1jy7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1jy7', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y46_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y46', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yg5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yg5', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4b', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeo_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeo', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4g', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4f', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1si4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1si4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '3b75_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3b75', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fn3', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1x', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rqa_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rqa', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a00_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a00', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cls', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a01_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a01', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hba_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hba', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0u', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq3', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3y', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1m9p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1m9p', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gzx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gzx', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbu', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1fdh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fdh', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3o', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kah_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kah', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jjq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jjq', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 2e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 2.1e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1p', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1abw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1abw', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 3.7e-09, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2w72_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xye', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1c7b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7b', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz4', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzu', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 5.6e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 5.7e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 7.1e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 7.1e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 7.2e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 7.2e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aet_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aet', 'e_value': 7.2e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aeu', 'e_value': 7.2e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aev', 'e_value': 7.2e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 7.4e-09, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 9.4e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 9.4e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 9.4e-09, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s0h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s0h', 'e_value': 1.2e-08, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 3.4e-08, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hds_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '1hds', 'e_value': 3.4e-08, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a0g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3a0g', 'e_value': 1.2e-07, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3hyu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hyu', 'e_value': 1.2e-07, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3fh9', 'e_value': 1.2e-07, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3hf4_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3hf4', 'e_value': 2.1e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dht_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3dht', 'e_value': 2.1e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 4.5e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 4.5e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 4.5e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 4.5e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 4.5e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 5.8e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 5.8e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2rao_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2rao', 'e_value': 7.5e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3lqd_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3lqd', 'e_value': 7.5e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 1.3e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 1.3e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 3.5e-06, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 9.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 9.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 9.8e-06, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 1.3e-05, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 1.3e-05, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 1.3e-05, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 1.3e-05, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 2.1e-05, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 2.1e-05, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrf_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 2.1e-05, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3vre_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 2.1e-05, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 2.1e-05, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 4.6e-05, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zfb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zfb', 'e_value': 0.00017, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 0.00021, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 0.00021, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 0.00022, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 0.00028, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1spg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1spg', 'e_value': 0.00047, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyi_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 0.0006, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3fs4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3fs4', 'e_value': 0.0013, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3a59_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3a59', 'e_value': 0.0013, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyh_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyh', 'e_value': 0.0017, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1out_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1out', 'e_value': 0.0028, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ouu_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1ouu', 'e_value': 0.0028, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj3_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj3', 'e_value': 0.0047, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj2_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj2', 'e_value': 0.0047, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj1_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj1', 'e_value': 0.0047, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r1h_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '2r1h', 'e_value': 0.008, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bom_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '3bom', 'e_value': 0.008, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xq5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1xq5', 'e_value': 0.01, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbr_1', 'molecule_name': ['Hemoglobin subunit alpha-D'], 'pdb_id': '1hbr', 'e_value': 0.028, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4w_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4w', 'e_value': 0.081, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4x_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4x', 'e_value': 0.081, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4u_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4u', 'e_value': 0.081, 'percentage_identity': 51.9, 'result_sequence': None}], 'WWSYWWTQW': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6vpd_1', 'molecule_name': ['Glutathione peroxidase'], 'pdb_id': '6vpd', 'e_value': 0.035, 'percentage_identity': 66.7, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPLPYPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 1.7e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.068, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.068, 'percentage_identity': 75.0, 'result_sequence': None}], 'ASHLPSDFTPAVHASLDKFLANVSTVLTSKYR': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 1.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3fh9', 'e_value': 5.6e-14, 'percentage_identity': 96.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 7.2e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2ri4_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2ri4', 'e_value': 1.6e-13, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qu0_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2qu0', 'e_value': 1.6e-13, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3eu1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3eu1', 'e_value': 1.6e-13, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d1a_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1a', 'e_value': 1.6e-13, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 1.6e-13, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2rao_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2rao', 'e_value': 1.2e-12, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3lqd_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3lqd', 'e_value': 1.2e-12, 'percentage_identity': 90.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7n', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5hu6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hu6', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7p', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e29_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e29', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kav_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kav', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2t', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhe', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1shr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1shr', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yie_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yie', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y2z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y2z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w72_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dn2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhe', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hhb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbd', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4p', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4v', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y85_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y85', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xye', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y35', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yev', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1c7b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7b', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j41_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j41', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['W', 'R', 'U', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1jy7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1jy7', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y46_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y46', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yg5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yg5', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4b', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeo_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeo', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4g', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygd', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4f', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1si4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1si4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '3b75_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3b75', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fn3', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1x', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rqa_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rqa', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a00_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a00', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cls', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a01_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a01', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hba_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hba', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0u', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq3', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1m9p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1m9p', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gzx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gzx', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbu', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3o', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kah_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kah', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jjq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jjq', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 2e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 2.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 2.6e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1p', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1abw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1abw', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 4.1e-12, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 5.6e-12, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 5.6e-12, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 9.3e-12, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 9.3e-12, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 9.3e-12, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 5, 'entry_entity': '5jdo_5', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbc', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbd', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aet_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aet', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aeu', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aev', 'e_value': 1.2e-11, 'percentage_identity': 87.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eui_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 1.5e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 1.5e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 1.5e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hds_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '1hds', 'e_value': 1.5e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3hf4_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3hf4', 'e_value': 2e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dht_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3dht', 'e_value': 2e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1ch4_1', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ch4', 'e_value': 2.1e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 2.6e-11, 'percentage_identity': 84.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrf_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 2.6e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3vre_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 2.6e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 2.6e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 9.1e-11, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 9.2e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 9.2e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 9.2e-11, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 3.3e-10, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 3.3e-10, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 4.2e-10, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 4.2e-10, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1out_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1out', 'e_value': 5.6e-08, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ouu_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1ouu', 'e_value': 5.6e-08, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 9.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 9.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 9.2e-08, 'percentage_identity': 71.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 1.5e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 1.5e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 1.5e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 2.6e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 2.6e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 2.6e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 2.6e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 2.6e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3fs4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3fs4', 'e_value': 3.3e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3a59_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3a59', 'e_value': 3.3e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 4.3e-07, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbr_1', 'molecule_name': ['Hemoglobin subunit alpha-D'], 'pdb_id': '1hbr', 'e_value': 1.2e-06, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 1.5e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 1.5e-06, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 1.5e-06, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 1.5e-06, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zfb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zfb', 'e_value': 2e-06, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyh_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyh', 'e_value': 2e-06, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyi_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 3.3e-06, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '3w4u_1', 'molecule_name': ['Hemoglobin subunit zeta'], 'pdb_id': '3w4u', 'e_value': 5.5e-06, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bcq_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bcq', 'e_value': 5.5e-06, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 1.5e-05, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4w_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4w', 'e_value': 2e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4x_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4x', 'e_value': 2e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1v4u_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1v4u', 'e_value': 2e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4esa_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '4esa', 'e_value': 3.3e-05, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rp5_1', 'molecule_name': ['Hemoglobin subunit alpha 1'], 'pdb_id': '6rp5', 'e_value': 3.3e-05, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r1h_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '2r1h', 'e_value': 5.5e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj3_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj3', 'e_value': 5.5e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bom_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '3bom', 'e_value': 5.5e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj2_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj2', 'e_value': 5.5e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bj1_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bj1', 'e_value': 5.5e-05, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4iro_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4iro', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5lfg_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '5lfg', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4g51_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4g51', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t1n_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1t1n', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5x', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ng6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3ng6', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4odc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4odc', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nfe_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3nfe', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbh', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2peg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2peg', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3gkv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gkv', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqg', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8f', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d1k_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1k', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1s5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s5y', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2aa1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2aa1', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1la6_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '1la6', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pbx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1pbx', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h8d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h8d', 'e_value': 9.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xq5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1xq5', 'e_value': 0.00012, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mkb_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3mkb', 'e_value': 0.00015, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2z6n_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '2z6n', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wmu_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1wmu', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v75_1', 'molecule_name': ['Hemoglobin D subunit alpha'], 'pdb_id': '1v75', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6hit_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '6hit', 'e_value': 0.0002, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5ker_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 0.0026, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3hrw_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '3hrw', 'e_value': 0.0044, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcw', 'e_value': 0.015, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gcv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gcv', 'e_value': 0.015, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1jeb_2', 'molecule_name': ['Hemoglobin subunit beta-1'], 'pdb_id': '1jeb', 'e_value': 0.016, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a0g_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3a0g', 'e_value': 0.02, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3hyu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3hyu', 'e_value': 0.02, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4h2l_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 0.02, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mhb_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2mhb', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6sva_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6sva', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5c6e_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '5c6e', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6r2o_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '6r2o', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dhb_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2dhb', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2d5x_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2d5x', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2zlx_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zlx', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2zlt_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zlt', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y8h_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y8h', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2zlu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zlu', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ibe_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ibe', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1iwh_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1iwh', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ns6_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ns6', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y8k_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y8k', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1s0h_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '1s0h', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1y8i_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1y8i', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2zlv_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zlv', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2zlw_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2zlw', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1g0b_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1g0b', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ns9_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1ns9', 'e_value': 0.034, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqj_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqj', 'e_value': 0.056, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '4mqk_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '4mqk', 'e_value': 0.056, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 2, 'entry_entity': '1fdh_2', 'molecule_name': ['Hemoglobin subunit gamma-2'], 'pdb_id': '1fdh', 'e_value': 0.057, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3e_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3e', 'e_value': 0.073, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1i3d_1', 'molecule_name': ['Hemoglobin subunit gamma-1'], 'pdb_id': '1i3d', 'e_value': 0.073, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3gys_2', 'molecule_name': ['Hemoglobin subunit beta-A/B'], 'pdb_id': '3gys', 'e_value': 0.093, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3gqr_2', 'molecule_name': ['Hemoglobin subunit beta-A/B'], 'pdb_id': '3gqr', 'e_value': 0.093, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3d4x_2', 'molecule_name': ['Hemoglobin subunit beta-A/B'], 'pdb_id': '3d4x', 'e_value': 0.093, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3gqp_2', 'molecule_name': ['Hemoglobin subunit beta-A/B'], 'pdb_id': '3gqp', 'e_value': 0.093, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1spg_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1spg', 'e_value': 0.094, 'percentage_identity': 48.1, 'result_sequence': None}], 'GKPRPYSPKPTSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.066, 'percentage_identity': 64.7, 'result_sequence': None}], 'FKRIVQRIKDFLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00044, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00066, 'percentage_identity': 100.0, 'result_sequence': None}], 'GRRPRPRPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.051, 'percentage_identity': 100.0, 'result_sequence': None}], 'NPLIPAIYIGATVGPSVWAYLVALVGAAAVTAANIRRASSDNHSCAGNRGWCRSKCFRHEYVDTYYSAVCGRYFCCRSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rng_1', 'molecule_name': ['Big defensin'], 'pdb_id': '2rng', 'e_value': 1.8e-40, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qbk_1', 'molecule_name': ['Defensin_big domain-containing protein'], 'pdb_id': '6qbk', 'e_value': 1.4e-11, 'percentage_identity': 61.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qbl_1', 'molecule_name': ['Defensin_big domain-containing protein'], 'pdb_id': '6qbl', 'e_value': 2.4e-11, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rq2_1', 'molecule_name': ['Big defensin'], 'pdb_id': '2rq2', 'e_value': 3e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'AVVDILKGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 3.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 3.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 3.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 6.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 6.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 6.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 4.7e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 4.7e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 4.7e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'AAAAGSCVWGAVNYTSDCNGECKRRGYKGGHCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 1.5e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 2.5e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 7e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 1e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 1.3e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.0006, 'percentage_identity': 37.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bmr_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1bmr', 'e_value': 0.0046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fh3_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1fh3', 'e_value': 0.0046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7k18_2', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '7k18', 'e_value': 0.0046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.0067, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.0067, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.012, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.014, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.014, 'percentage_identity': 42.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6thi_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-1'], 'pdb_id': '6thi', 'e_value': 0.022, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zu3_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zu3', 'e_value': 0.023, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4hhf_1', 'molecule_name': ['Alpha-toxin OD1'], 'pdb_id': '4hhf', 'e_value': 0.034, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pjv_1', 'molecule_name': ['Potassium channel toxin alpha-KTx 10.1'], 'pdb_id': '1pjv', 'e_value': 0.038, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6f_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1i6f', 'e_value': 0.048, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6g_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1i6g', 'e_value': 0.048, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nh5_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1nh5', 'e_value': 0.048, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7e_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7e', 'e_value': 0.051, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1chz_1', 'molecule_name': ['Alpha-like toxin BmK M2'], 'pdb_id': '1chz', 'e_value': 0.061, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyw_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyw', 'e_value': 0.063, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zve_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zve', 'e_value': 0.063, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zvg_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zvg', 'e_value': 0.063, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zut_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zut', 'e_value': 0.063, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyv_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyv', 'e_value': 0.063, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn1_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1sn1', 'e_value': 0.075, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1djt_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1djt', 'e_value': 0.075, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2sn3_1', 'molecule_name': ['Toxin CsEv3'], 'pdb_id': '2sn3', 'e_value': 0.076, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5mou_1', 'molecule_name': ['Alpha-like toxin BeM9'], 'pdb_id': '5mou', 'e_value': 0.077, 'percentage_identity': 35.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7b_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7b', 'e_value': 0.077, 'percentage_identity': 34.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.077, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.077, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7a_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7a', 'e_value': 0.094, 'percentage_identity': 35.0, 'result_sequence': None}], 'FIFHIIKGLFHAGKMIHGLVTRRRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 1.1e-05, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 0.01, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 0.01, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 0.01, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 0.01, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 0.01, 'percentage_identity': 57.1, 'result_sequence': None}], 'KKLKLALAKLALLWKALALKLKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 2.2e-09, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 2.2e-09, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 2.2e-09, 'percentage_identity': 95.8, 'result_sequence': None}], 'LLPALISWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGKLFLHAAKKFAKAFVAEKMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.2e-08, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 7.4e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 7.4e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 8.2e-07, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 8.4e-07, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2.2e-05, 'percentage_identity': 70.8, 'result_sequence': None}], 'KILRGVSKKIMRRISKDILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 0.00023, 'percentage_identity': 74.1, 'result_sequence': None}], 'EPAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 2.9e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 5e-11, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 5.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 7.5e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 4.5e-08, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 1.7e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 6.8e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 6.8e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 6.8e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 6.8e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 6.8e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 1.1e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 1.1e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 1.2e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l60_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2l60', 'e_value': 1.3e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz5_1', 'molecule_name': ['Neuropeptide Y', 'Pancreatic hormone'], 'pdb_id': '1tz5', 'e_value': 1.4e-05, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3s_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3s', 'e_value': 7e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3t_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3t', 'e_value': 7e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2h4b_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h4b', 'e_value': 7e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ppt_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ppt', 'e_value': 9e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bf9_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2bf9', 'e_value': 0.00079, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljv_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ljv', 'e_value': 0.016, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bba_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1bba', 'e_value': 0.016, 'percentage_identity': 54.5, 'result_sequence': None}], 'NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFNC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bk8_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1bk8', 'e_value': 7e-17, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kpy_1', 'molecule_name': ['Major pollen allergen Art v 1'], 'pdb_id': '2kpy', 'e_value': 3e-10, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 1.6e-08, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 3.8e-08, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 1.3e-07, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 4.9e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 2.4e-05, 'percentage_identity': 42.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 7e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 8.3e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 0.00055, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 0.0026, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 0.0051, 'percentage_identity': 34.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 0.059, 'percentage_identity': 36.0, 'result_sequence': None}], 'INLKILARLAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.011, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.016, 'percentage_identity': 85.7, 'result_sequence': None}], 'KKAAHVGKHVGKAAKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 0.0001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 0.00011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.0021, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.0038, 'percentage_identity': 71.4, 'result_sequence': None}], 'GVVDILKGAAKAIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 4.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 4.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 4.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 7.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 7.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 7.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'AQCGAQGGGATCPGGLCCSQWGWCGSTPKYCGAGCQSNCK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 1.3e-13, 'percentage_identity': 76.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 6.8e-12, 'percentage_identity': 71.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 6.8e-12, 'percentage_identity': 71.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 1.4e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 1.4e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 1.4e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 1.7e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 2.7e-11, 'percentage_identity': 74.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 1.3e-10, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 1.3e-10, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eis_1', 'molecule_name': ['Agglutinin isolectin VI'], 'pdb_id': '1eis', 'e_value': 2.6e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1en2_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1en2', 'e_value': 2.6e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enm_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1enm', 'e_value': 2.6e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 4.9e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 4.9e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 5e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 5.4e-10, 'percentage_identity': 62.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1iqb_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1iqb', 'e_value': 1e-09, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lnr_1', 'molecule_name': ['Class-1 chitinase'], 'pdb_id': '6lnr', 'e_value': 5.2e-09, 'percentage_identity': 64.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 5.3e-09, 'percentage_identity': 59.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 6.3e-09, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t0w_1', 'molecule_name': ['Hevein'], 'pdb_id': '1t0w', 'e_value': 6.7e-09, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 1.1e-08, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 1.1e-08, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 1.1e-08, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 1.1e-08, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 1.1e-08, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 4.6e-08, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 9.2e-08, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 4.6e-06, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 5e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 1e-05, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 8.4e-05, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.00012, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.00014, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 0.0002, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 0.0002, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.00024, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.00041, 'percentage_identity': 71.4, 'result_sequence': None}], 'GVPCAESCVWIPCTVTALLGCSCKDKVCYLN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.3e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 7.3e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 7.3e-10, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 9.1e-10, 'percentage_identity': 64.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 3.3e-09, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 3.5e-08, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 4.6e-08, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 1.3e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 8.2e-06, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.0061, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.019, 'percentage_identity': 43.8, 'result_sequence': None}], 'GFGCPFNLNECHAHCLSIGRKFGFCAGPLRATCTCGKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1e-20, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.2e-09, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 1.3e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 6.5e-07, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 3.9e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 2.3e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.0011, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0032, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0032, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0032, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.0049, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0076, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.022, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.022, 'percentage_identity': 39.4, 'result_sequence': None}], 'TLRGDERILSILRHQNLLKE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 2.2e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'LTCEIDRSLCLLHCRLKGYLRAYCSQQKVCRCVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 0.0003, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.002, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0034, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.016, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.02, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.05, 'percentage_identity': 39.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.05, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.052, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.063, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.1, 'percentage_identity': 37.0, 'result_sequence': None}], 'FSCDVLSFQSKWVSPNHSACAVRCLAQRRKGGKCKNGDCVCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 3.7e-12, 'percentage_identity': 63.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 1.8e-11, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.6e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2e-09, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 4.8e-05, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.00013, 'percentage_identity': 47.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0003, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.00067, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.0015, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.017, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.026, 'percentage_identity': 38.1, 'result_sequence': None}], 'KWKSFIKKLTLKFLHSKKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}], 'RMCKTPSGKFKGYCVNNTNCKNVCRTEGFPTGSCDFHVAGRKCYCYKPCP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 7.2e-24, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 4e-10, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 4.1e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 7e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 2.1e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 3e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 1e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 1.4e-06, 'percentage_identity': 44.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 1.4e-06, 'percentage_identity': 44.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 3.9e-06, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 6.7e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 1.2e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 1.3e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 1.3e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 1.3e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mr4_1', 'molecule_name': ['Defensin-like protein'], 'pdb_id': '1mr4', 'e_value': 1.3e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '6b55_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '6b55', 'e_value': 1.4e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 6.1e-05, 'percentage_identity': 32.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5kk4_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5kk4', 'e_value': 0.00016, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '5vyp_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5vyp', 'e_value': 0.00016, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.00017, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6mry_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6mry', 'e_value': 0.00046, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 0.0037, 'percentage_identity': 36.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.012, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 0.018, 'percentage_identity': 32.7, 'result_sequence': None}], 'PRPPRLPRPRPRPLPFPRPGPRPIPRPLPFP': [{'chain_id': ['a0'], 'entity_id': 1, 'entry_entity': '6ncl_1', 'molecule_name': ['P14'], 'pdb_id': '6ncl', 'e_value': 7.4e-05, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 3, 'entry_entity': '6x87_3', 'molecule_name': ['Circumsporozoite protein'], 'pdb_id': '6x87', 'e_value': 0.00027, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.0064, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.0069, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.0069, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['B', 'O'], 'entity_id': 7, 'entry_entity': '6odm_7', 'molecule_name': ['Large tegument protein deneddylase'], 'pdb_id': '6odm', 'e_value': 0.019, 'percentage_identity': 48.3, 'result_sequence': None}], 'GGAGHVPEYFVGIGTPISFYG': [{'chain_id': ['M'], 'entity_id': 6, 'entry_entity': '6n60_6', 'molecule_name': ['Microcin J25'], 'pdb_id': '6n60', 'e_value': 9.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pp5_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '1pp5', 'e_value': 9.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q71_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '1q71', 'e_value': 9.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4cu4_2', 'molecule_name': ['Microcin J25'], 'pdb_id': '4cu4', 'e_value': 9.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmw_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '2mmw', 'e_value': 1.6e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmt_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '2mmt', 'e_value': 9.7e-06, 'percentage_identity': 81.0, 'result_sequence': None}], 'KVPVGAIKKGGKAIKTGLGVVGAAGTAHEVYSHIRNRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 1.8e-05, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 0.00041, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 0.017, 'percentage_identity': 37.8, 'result_sequence': None}], 'KPPQFTWAQWFETQHINMTSQQSTNAMQVINNYQRRSKNQNTFLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 4.5e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 5.3e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 1.2e-10, 'percentage_identity': 61.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 1.7e-09, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 4.5e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 2e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 2e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 2e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 2e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.4e-05, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 2.2e-05, 'percentage_identity': 43.2, 'result_sequence': None}], 'CVLIGQRCDNDRGPRCCSGQGNCVPLPFLGGVCAV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mfs_1', 'molecule_name': ['Ep-AMP1'], 'pdb_id': '2mfs', 'e_value': 6.8e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v91_1', 'molecule_name': ['Delta-amaurobitoxin-Pl1b'], 'pdb_id': '1v91', 'e_value': 0.048, 'percentage_identity': 44.8, 'result_sequence': None}], 'KRIVQRIKDFLRNLVPRTES': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 5.4e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRICRCRIGRCLGLEVYFGVCFLHGRLARRCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 4.3e-08, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cuj_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cuj', 'e_value': 2.9e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cum_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cum', 'e_value': 2.9e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 1.2e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 1.2e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 1.5e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 3.1e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cui_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cui', 'e_value': 1.3e-05, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo6_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo6', 'e_value': 4.1e-05, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo9_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo9', 'e_value': 4.1e-05, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k1i_1', 'molecule_name': ['DEFENSIN domain-containing protein'], 'pdb_id': '2k1i', 'e_value': 0.00011, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3loe_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3loe', 'e_value': 0.00022, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dfn_1', 'molecule_name': ['Neutrophil defensin 3'], 'pdb_id': '1dfn', 'e_value': 0.00028, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3go0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3go0', 'e_value': 0.00028, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kht_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2kht', 'e_value': 0.00028, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hjd_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hjd', 'e_value': 0.00028, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gny_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3gny', 'e_value': 0.00028, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hj2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hj2', 'e_value': 0.0003, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo2', 'e_value': 0.00035, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 0.00057, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h6c_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3h6c', 'e_value': 0.00057, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm4_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm4', 'e_value': 0.00092, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb1', 'e_value': 0.0012, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 0.0012, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pm1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2pm1', 'e_value': 0.0015, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lo1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo1', 'e_value': 0.0015, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm5_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm5', 'e_value': 0.0015, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1zmk_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmk', 'e_value': 0.0018, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmh_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmh', 'e_value': 0.0018, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmi_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmi', 'e_value': 0.0018, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4du0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4du0', 'e_value': 0.0019, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo4_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo4', 'e_value': 0.0024, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4lbf_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbf', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb7_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb7', 'e_value': 0.013, 'percentage_identity': 41.4, 'result_sequence': None}], 'HSSGYTRPLRKPSRPIFIRPIGCDVCYGIPSSTARLCCFRYGDCCHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 6.1e-25, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 1.5e-06, 'percentage_identity': 43.9, 'result_sequence': None}], 'LVQRGRFGRFLRKIR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gdl_1', 'molecule_name': ['myeloid antimicrobial peptide 27'], 'pdb_id': '2gdl', 'e_value': 2e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLEKLFSKIQNDK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 4.7e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 4.4e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 4.4e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 1e-05, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 1.8e-05, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 1.8e-05, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.00017, 'percentage_identity': 89.5, 'result_sequence': None}], 'AKKVFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.00043, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.0011, 'percentage_identity': 93.3, 'result_sequence': None}], 'RKLPDAPGMHTWGGGGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.00034, 'percentage_identity': 100.0, 'result_sequence': None}], 'TFFRLFNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.057, 'percentage_identity': 100.0, 'result_sequence': None}], 'FLKLLKKLAAKFLPTIICKISYKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.0013, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.0013, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.0013, 'percentage_identity': 50.0, 'result_sequence': None}], 'AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg4_1', 'molecule_name': ['Antimicrobial peptide aurelin'], 'pdb_id': '2lg4', 'e_value': 9e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6b_1', 'molecule_name': ['Hui1'], 'pdb_id': '2n6b', 'e_value': 0.083, 'percentage_identity': 42.5, 'result_sequence': None}], 'LRKFRNKIKEKLKKIGQKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKSFIKKLTSKFLHLAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}], 'LEGRSISRIDELIQQRQEKD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 0.0047, 'percentage_identity': 70.0, 'result_sequence': None}], 'RWCVYAYVRVRGVSVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.6e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.1e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.1e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.5e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'FLSLIPHIVSGVASLAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.001, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.0014, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0034, 'percentage_identity': 61.1, 'result_sequence': None}], 'IGKEFKRIVQRIKDFLRNL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'TSGKYPNCRYKEKRQNKSYVVACKPPQKKDSQQFHLVPVHLDRVL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 2e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.3e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.3e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.3e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.3e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.3e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 1.4e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 1.6e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 0.00018, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2apq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2apq', 'e_value': 0.00023, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 0.0013, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1b6v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1b6v', 'e_value': 0.0081, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ygw_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ygw', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fev_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fev', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5d_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5d', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5h_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5h', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjq_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjq', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5e_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5e', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjr_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjr', 'e_value': 0.01, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4k7l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7l', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j82_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j82', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o36_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o36', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4okf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4okf', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o37_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o37', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '2rln_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rln', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbd_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbd', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3p_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3p', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3or0_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3or0', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3oqz_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqz', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3l', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j81_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j81', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j80_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j80', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3m_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3m', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 2, 'entry_entity': '3oqy_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqy', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j7z_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j7z', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbc_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbc', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbh_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbh', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbe_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbe', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbg_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbg', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbi_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbi', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbf', 'e_value': 0.011, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qh3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4qh3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ogh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5ogh', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obe', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obc', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5old_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5old', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5et4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5et4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5e_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5e', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5f', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rte_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rte', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s0q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s0q', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsk', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j65_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j65', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j66_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j66', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j67', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j68_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j68', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j61_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j61', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rh1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rh1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4k7m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7m', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j6a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j6a', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rn3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rn3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j62_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j62', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3qsk', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ao1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ao1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xog_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xog', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsk', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j63_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j63', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j69_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j69', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j5z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j5z', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l55_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4l55', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mxf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4mxf', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j64_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j64', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rid', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jmg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jmg', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s18_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s18', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obd', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j60', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xoi_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xoi', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jlg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jlg', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d97_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d97', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zz4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4zz4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jml_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jml', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6eto_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6eto', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rsa', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvv', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6gok_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6gok', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6f60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6f60', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etr', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xvx', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etp', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6qe9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qe9', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etm', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhc', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nj7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5nj7', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d6u_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d6u', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhf', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhd', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xw0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xw0', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhe', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5na9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5na9', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etl', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvx', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etk', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvu', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etn', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvw', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e3w', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2qca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2qca', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p43_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p43', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4q', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6s_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6s', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wbu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1wbu', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rns_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rns', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ot4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ot4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ooh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ooh', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8y', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3jw1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3jw1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2oqf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2oqf', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dy5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dy5', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1a2w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a2w', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbx', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rsm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rsm', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 2, 'entry_entity': '1f0v_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1f0v', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eow_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eow', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9v', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di9', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g4w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4w', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8r', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7w', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blp', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p47_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p47', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p4a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p4a', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p49_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p49', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2e33_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e33', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5g', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5k_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5k', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5m', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5l', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev5', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev2', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g4x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4x', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di8', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euy_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euy', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euz', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fkz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fkz', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev6', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8q', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2op2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2op2', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lxo_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3lxo', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p45_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p45', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p46_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p46', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p42_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p42', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p48_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p48', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i67', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8y', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6q', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxg', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxh', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eux_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3eux', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh6', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl0', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6h', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8z', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6j', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6f', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2aas_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2aas', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0o', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1eos_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eos', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf2', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jn4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jn4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4o', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4p', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xps_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xps', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6d_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6d', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xpt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xpt', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1u1b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1u1b', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1qhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1qhc', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6p', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d7b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d7b', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh5', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blz', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6o', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a1r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3a1r', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1bzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bzq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c8w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c8w', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fs3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fs3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvu', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvt', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izp', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0f', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnm', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnd', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rno_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rno', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afu', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ras_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ras', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afk', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnv', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afl', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rny_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rny', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rbb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbb', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rob_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rob', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbn', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ruv', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0n_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0n', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izr', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf7', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0h', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf3', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbj_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbj', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnn', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rtb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rtb', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbw', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rha_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rha', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rca', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnc', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rcn_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rcn', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rar_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rar', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpg', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnu', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rhb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rhb', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aqp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1aqp', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnz', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnx', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rta_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rta', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1dfj_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dfj', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rph_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rph', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpf', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnw', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1js0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1js0', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvv', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0m', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf8', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf5', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p44_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p44', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5i_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5i', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di7', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bel_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bel', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev4', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1lsq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1lsq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev0', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4peq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4peq', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8v', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4pou_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4pou', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g90_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g90', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6yo1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6yo1', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rsa', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '9rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '8rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rsa', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rsa', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rat', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rsa', 'e_value': 0.012, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzq', 'e_value': 0.013, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0c_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0c', 'e_value': 0.013, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0b', 'e_value': 0.013, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzr', 'e_value': 0.013, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dib_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dib', 'e_value': 0.015, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7x', 'e_value': 0.015, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7y', 'e_value': 0.015, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kh8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kh8', 'e_value': 0.015, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwr', 'e_value': 0.023, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mx8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mx8', 'e_value': 0.023, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwq', 'e_value': 0.023, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eic', 'e_value': 0.024, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eie_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eie', 'e_value': 0.024, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyp', 'e_value': 0.036, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyn', 'e_value': 0.036, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyz', 'e_value': 0.036, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eid', 'e_value': 0.036, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dic', 'e_value': 0.036, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsd', 'e_value': 0.045, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ql2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql2', 'e_value': 0.045, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ql1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql1', 'e_value': 0.045, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4srn', 'e_value': 0.051, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3srn', 'e_value': 0.051, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssc', 'e_value': 0.051, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssa', 'e_value': 0.053, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssb', 'e_value': 0.053, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1srn', 'e_value': 0.053, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4n4c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '4n4c', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsd', 'e_value': 0.055, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6qmn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qmn', 'e_value': 0.055, 'percentage_identity': 43.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tq9_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1tq9', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9x', 'e_value': 0.055, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djq', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djo_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djo', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djp', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djv_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djv', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djx_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djx', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3bcp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcp', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bco_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bco', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bcm_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcm', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qwq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1qwq', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y94_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y94', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y92_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y92', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1bsr_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1bsr', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1x_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n1x', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n3z_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n3z', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5c', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11ba_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11ba', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r3m_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r3m', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5d_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5d', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11bg_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11bg', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lfj_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '2lfj', 'e_value': 0.055, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 0.07, 'percentage_identity': 39.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymn', 'e_value': 0.085, 'percentage_identity': 41.5, 'result_sequence': None}], 'VDKPPYLPRPRPPRRIYNRN': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.00011, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.003, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.003, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.0048, 'percentage_identity': 81.2, 'result_sequence': None}], 'STLHLVLRLRAG': [{'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5eya_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5eya', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5wfi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wfi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5vey_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vey', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4dhz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4bvu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bvu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vf0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vf0', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5fer_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5fer', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xpk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xpk', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xu8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xu8', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5emz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5emz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xve_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xve', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '5ohp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'K', 'M', 'J', 'N', 'O', 'L'], 'entity_id': 2, 'entry_entity': '5ohl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohl', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 3, 'entry_entity': '5edv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5edv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mm3', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ohv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6i_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6i', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mjb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mjb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '2mbh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '4s1z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s1z', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4wur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wur', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xyz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xyz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5goj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goj', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5tut_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tut', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 1, 'entry_entity': '5h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5h7s', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gog_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gog', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xof_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xof', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5goi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5go7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go7', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4wzp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wzp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gob', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gvi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '5kyd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3v', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mcn', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1y_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1y', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xol_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xol', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4y1h_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4y1h', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5go8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go8', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5god_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5god', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'A', 'B', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4xkh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4s22_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s22', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5goc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goc', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4xok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xok', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5txk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5txk', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['h', 'c', 'X', 'm'], 'entity_id': 10, 'entry_entity': '4zux_10', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zux', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5mn9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mn9', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulk', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5u4p_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5u4p', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5gok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gok', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5tr4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tr4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7u', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xk5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '4jio_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jio', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kox', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7w_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7w', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '4ldt_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ldt', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zlz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zlz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3wxg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxg', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'B', 'J', 'F'], 'entity_id': 2, 'entry_entity': '4dhj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhj', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7s', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 2, 'entry_entity': '3tbl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tbl', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'D', 'M', 'Q', 'N', 'O', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddg', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lz6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lz6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '3vuy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuy', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vux_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vux', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5kye_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kye', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3u_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3u', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6h', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2k_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n2k', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5l6j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6j', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mj5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5lrv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyc', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jbv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jbv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n13', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mi8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mi8', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4un2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4un2', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'O'], 'entity_id': 8, 'entry_entity': '5kgf_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kgf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 3, 'entry_entity': '5o44_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5o44', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mor_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mor', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbq', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3tmp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tmp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ugb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ugb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '4bos_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bos', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4jqw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jqw', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4m0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4m0w', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0t', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3vht_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vht', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4bbn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bbn', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2qho_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2qho', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3m3j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3m3j', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vnz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vnz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nvg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nvg', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vo0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vo0', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vzm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzm', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '4boz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4boz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5j8p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j8p', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5jby_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jby', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5jqs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jqs', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nmc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nmc', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z9s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4z9s', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5ait_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ait', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3m', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlj', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['9'], 'entity_id': 9, 'entry_entity': '5a5b_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5a5b', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ulf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5af4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5aiu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5aiu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujn', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jtv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mro_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mro', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6p5b_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6p5b', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7m', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6tnf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tnf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 15, 'entry_entity': '6tbm_15', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tbm', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 12, 'entry_entity': '6t9l_12', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t9l', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6v5d_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6v5d', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6asr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6asr', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '6vae_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vae', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '6ttu_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ttu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k9p', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6txb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6txb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kox', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kow_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kow', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6e2b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6e2b', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6kl4_3', 'molecule_name': ['Ub'], 'pdb_id': '6kl4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6aqr_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6aqr', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6vaf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vaf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msb_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3o_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3o', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 3, 'entry_entity': '5d0k_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0k', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nli_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nli', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujl', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xiu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xiu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3n', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 1, 'entry_entity': '6jwi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jwi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6q00_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q00', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oi4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oi4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6s53_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6s53', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N', 'O'], 'entity_id': 8, 'entry_entity': '6ftx_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ftx', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djx', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5zbu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zbu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4i_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k4i', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6lp2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6lp2', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n13', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6xqc_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6xqc', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6nxk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nxk', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6qml_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qml', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4zpz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zpz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1z_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1z', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xbo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5bz0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bz0', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5dfl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dfl', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 3, 'entry_entity': '5d0m_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0m', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6wtg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wtg', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5af6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msd_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5xdp_1', 'molecule_name': ['D-ubiquitin'], 'pdb_id': '5xdp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7cap_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7cap', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb7', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 7, 'entry_entity': '6jma_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jma', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ymy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ymy', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6q01_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q01', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7o', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '6w9d_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9d', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6oaa_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oaa', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6tuv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tuv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6rya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6rya', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'J'], 'entity_id': 2, 'entry_entity': '6oa9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oa9', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4wlr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wlr', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'C', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '6z7v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6z7v', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zqh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6zqh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6if1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6if1', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wwz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7s', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q0w', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ogw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ogw', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '1wr6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubq', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wr1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr1', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1tbe', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v80_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v80', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n32_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n32', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ru6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rr9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rr9', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3n30_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n30', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '3mhs_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3mhs', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3nhe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nhe', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rsu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rsu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wx1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx1', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3cmm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3cmm', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1f9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1f9j', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2jt4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jt4', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h1u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h1u', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D'], 'entity_id': 3, 'entry_entity': '2o6v_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2o6v', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pea_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pea', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2dx5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ehv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ehv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2g45_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g45', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcm', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2g3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g3q', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eec_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3eec', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2gmi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gmi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3efu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3efu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kde_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kde', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kdf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pe9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pe9', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2oob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2oob', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3hm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3hm3', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcn', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hth_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hth', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2jri_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jri', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jy6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jy6', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcs_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcs', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcq_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '2fcq', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2fif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fif', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8c_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8c', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a33_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a33', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k39_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k39', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8b', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jzz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jzz', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2z59_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2z59', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ayo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ayo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3alb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3alb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ofi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ofi', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ns8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ns8', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 1, 'entry_entity': '1yd8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yd8', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3aul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3aul', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 2, 'entry_entity': '1p3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1p3q', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g6j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1g6j', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1uzx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1uzx', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1otr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1otr', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1zgu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zgu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yiw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '1yiw', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wrd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wrd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yj1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yj1', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v81_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v81', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcc', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q5w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q5w', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1s1q', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fxt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1fxt', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1aar_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1aar', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cmx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1cmx', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1nbf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1nbf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '2c7n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7n', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7m', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3h7p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7p', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d3g_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2d3g', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lj5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wx0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx0', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvq', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2xbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xbb', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2bgf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2bgf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fid_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fid', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2fuh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fuh', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hd5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hd5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xqq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xqq', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1d3z', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2jf5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jf5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2den_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2den', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0f_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0f', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4pih_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pih', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l3z', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vuw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuw', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvp', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2klg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2klg', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '3phd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ktf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ktf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2l00_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l00', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4pig_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pig', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kn5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kn5', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2xew_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xew', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6isu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6isu', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '7bbf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbf', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7jxv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jxv', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '7bbd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbd', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7keo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7keo', 'e_value': 0.00068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ohm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohm', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'J', 'C'], 'entity_id': 3, 'entry_entity': '5ydk_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydk', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '3wwq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wwq', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqo', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'F'], 'entity_id': 3, 'entry_entity': '6a6i_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6a6i', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 3, 'entry_entity': '5xit_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xit', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6k2u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k2u', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyi', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6oq2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq2', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqs_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqs', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ud0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ud0', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F', 'L'], 'entity_id': 3, 'entry_entity': '6sqr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqr', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '7bxg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bxg', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5xis_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xis', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6oq1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq1', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyj', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4nql_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nql', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jsv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jsv', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3a1q_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a1q', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9j', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9k', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2znv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2znv', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ld9', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6kfp_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kfp', 'e_value': 0.00069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mur', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4r62_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4r62', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbb', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'F'], 'entity_id': 2, 'entry_entity': '5yik_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yik', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq4', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6cp2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6cp2', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq3', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '5yij_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yij', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5yt6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yt6', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3nob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nob', 'e_value': 0.00071, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9p', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5vzw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzw', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mre_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mre', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq7', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq6', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6jky_3', 'molecule_name': ['Ub'], 'pdb_id': '6jky', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qf8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qf8', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq5', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'K', 'D', 'B', 'C', 'J', 'N', 'O', 'F', 'G', 'L'], 'entity_id': 2, 'entry_entity': '6upu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6upu', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6kg6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kg6', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4nqk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nqk', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'U', 'T', 'X', 'S'], 'entity_id': 3, 'entry_entity': '4p4h_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4p4h', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4nnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nnj', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2khw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2khw', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3oj3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj3', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '3olm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3olm', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3oj4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj4', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ptf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ptf', 'e_value': 0.00072, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4xkl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkl', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m93_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5m93', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ksk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksk', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mdk', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6dc6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6dc6', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U', 'X'], 'entity_id': 3, 'entry_entity': '3dvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvn', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 3, 'entry_entity': '3dvg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvg', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4gu2_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gu2', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4ap4_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ap4', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4gsw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gsw', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '7ay1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay1', 'e_value': 0.00073, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4v3k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3k', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msm', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4v3l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3l', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '5mnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mnj', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4zfr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zfr', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4auq_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4auq', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msq', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4pqt_2', 'molecule_name': ['Protein UBBP4'], 'pdb_id': '4pqt', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4k1r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k1r', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '5ln1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ln1', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'L'], 'entity_id': 4, 'entry_entity': '3zni_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zni', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4zft_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zft', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hpr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hpr', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jvz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jvz', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jw0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jw0', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbr', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai1', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai0', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['III', 'LLL', 'CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ahz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ahz', 'e_value': 0.00075, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5udh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5udh', 'e_value': 0.00076, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jvc_1', 'molecule_name': ['ubiquitin_like protein'], 'pdb_id': '2jvc', 'e_value': 0.00076, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zw7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zw7', 'e_value': 0.00076, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'F', 'G', 'L'], 'entity_id': 3, 'entry_entity': '4whv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4whv', 'e_value': 0.00077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ii2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii2', 'e_value': 0.00077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbk', 'e_value': 0.00077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'R', 'P', 'M', 'Q', 'X', 'N', 'O'], 'entity_id': 1, 'entry_entity': '7k5j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k5j', 'e_value': 0.00077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbm', 'e_value': 0.00079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gbn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbn', 'e_value': 0.00079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2gbj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbj', 'e_value': 0.00079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 6, 'entry_entity': '7lyc_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7lyc', 'e_value': 0.00079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6nya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nya', 'e_value': 0.0008, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '5tte_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tte', 'e_value': 0.00081, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6ypt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ypt', 'e_value': 0.00081, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4fjv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4fjv', 'e_value': 0.00081, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1sif', 'e_value': 0.00084, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4ii3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii3', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5knl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5knl', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '5cvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvn', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5cvm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvm', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '6nyo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nyo', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5cvo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvo', 'e_value': 0.00095, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx5', 'e_value': 0.00098, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx6', 'e_value': 0.00098, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4hcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4hcn', 'e_value': 0.00098, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3vdz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vdz', 'e_value': 0.0012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kdi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdi', 'e_value': 0.0012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ojr', 'e_value': 0.0012, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '5umd_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5umd', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Bm'], 'entity_id': 41, 'entry_entity': '6uz7_41', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uz7', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 47, 'entry_entity': '6n8n_47', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8n', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 44, 'entry_entity': '6n8m_44', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8m', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 24, 'entry_entity': '6lqm_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6lqm', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u'], 'entity_id': 24, 'entry_entity': '6ef3_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6ef3', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '3j79_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3j79', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 41, 'entry_entity': '5t6r_41', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t6r', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 42, 'entry_entity': '5t62_42', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t62', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 15, 'entry_entity': '7azy_15', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '7azy', 'e_value': 0.0014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['AK', 'FK', 'DK', 'HK'], 'entity_id': 10, 'entry_entity': '4v8p_10', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4v8p', 'e_value': 0.0015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['5'], 'entity_id': 6, 'entry_entity': '4adx_6', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4adx', 'e_value': 0.0015, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kna_1', 'molecule_name': ['viral Ubiquitin'], 'pdb_id': '6kna', 'e_value': 0.0015, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juo_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juo', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juu_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juu', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znz', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Af', 'Cf'], 'entity_id': 33, 'entry_entity': '4v88_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v88', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8z_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8z', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jup_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jup', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jut_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jut', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jus_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jus', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3wxf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxf', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3wxe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxe', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 15, 'entry_entity': '5mc6_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5mc6', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8y_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8y', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6kbe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kbe', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Nb', 'N', 'Nc'], 'entity_id': 50, 'entry_entity': '6sv4_50', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6sv4', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['6', 'fb'], 'entity_id': 32, 'entry_entity': '6t83_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t83', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Sf'], 'entity_id': 32, 'entry_entity': '6t7i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t7i', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 11, 'entry_entity': '6eml_11', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6eml', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['SFF'], 'entity_id': 26, 'entry_entity': '6rm3_26', 'molecule_name': ['eS31'], 'pdb_id': '6rm3', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E1', 'e1'], 'entity_id': 33, 'entry_entity': '5ndg_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5ndg', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 1, 'entry_entity': '5wq4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wq4', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyx_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyx', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6snt_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6snt', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '3jam_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jam', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6y_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6y', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6x_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6x', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j78_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j78', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jaq_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jaq', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 31, 'entry_entity': '3j81_31', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3j81', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jap_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jap', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w9n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2w9n', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '3b08_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b08', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3b0a_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b0a', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j77_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j77', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2y5b_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2y5b', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 32, 'entry_entity': '6zu9_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zu9', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyy_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyy', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 35, 'entry_entity': '6zce_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zce', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'e1'], 'entity_id': 79, 'entry_entity': '6hhq_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6hhq', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 15, 'entry_entity': '7k5i_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '7k5i', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '7eal_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7eal', 'e_value': 0.0018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'H', 'R', 'P', 'D', 'C', 'J', 'T', 'X', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4ksl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksl', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzx_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzx', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 34, 'entry_entity': '6p4h_34', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p4h', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p4g_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p4g', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['k'], 'entity_id': 16, 'entry_entity': '6ybs_16', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ybs', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6g5i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6g5i', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 11, 'entry_entity': '6g18_11', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6g18', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p5j_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p5j', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 32, 'entry_entity': '6zlw_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zlw', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6sgc_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6sgc', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '2zvn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvn', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '2zvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvo', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3axc_1', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '3axc', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['3R'], 'entity_id': 79, 'entry_entity': '6ip8_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ip8', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zv6_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zv6', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6hcm_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6hcm', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zuo_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zuo', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['AH'], 'entity_id': 52, 'entry_entity': '5t2c_52', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '5t2c', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}]}

## 5500 - 5750

In [10]:
r16={'GILDTFKGVAKGVAKDLAVHMLEKLKCKMTGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 2.1e-10, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 0.055, 'percentage_identity': 51.5, 'result_sequence': None}], 'EWRPHGSIGGSGLRPGRPQTLPPQRPRRPDFNGPRHRF': [{'chain_id': ['F', 'G'], 'entity_id': 2, 'entry_entity': '5sxp_2', 'molecule_name': ['E3 ubiquitin-protein ligase Itchy homolog'], 'pdb_id': '5sxp', 'e_value': 0.05, 'percentage_identity': 58.3, 'result_sequence': None}], 'GIGAVLLVLTTGLPALISWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 1.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}], 'GLFGVLAKVAAKVVPAIAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 6.8e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 6.8e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 6.8e-06, 'percentage_identity': 85.7, 'result_sequence': None}], 'DHYNCVSSGGQCLYSACPIFTKIQGTCYRGKAKCCK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 1.1e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 1.1e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 1.1e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 1.1e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 2.6e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 3.1e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 3.1e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 3.1e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 4.7e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 4.7e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 5.8e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 5.8e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 5.8e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 5.8e-17, 'percentage_identity': 97.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 1.3e-16, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.012, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.062, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.062, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.062, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.062, 'percentage_identity': 41.7, 'result_sequence': None}], 'GIGHFLHKVKSFGKSWIGEIMNS': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2.5e-07, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 3.6e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 3.7e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 0.00034, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cgn_1', 'molecule_name': ['D-Ala-Magainin Derivative'], 'pdb_id': '5cgn', 'e_value': 0.00034, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 0.00058, 'percentage_identity': 60.9, 'result_sequence': None}], 'GLNALKKFFQGIHEAIKLINNHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 1.2e-10, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 2e-09, 'percentage_identity': 91.7, 'result_sequence': None}], 'KRFKKFFKKVKKSV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.00012, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGAVLVVLTTGLPALISWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 2.3e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 2.3e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 2.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 2.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 2.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 2.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}], 'KWKSFIKKLTSKFLHSKKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.017, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.017, 'percentage_identity': 65.0, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCNGECKRRGYKGGHCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 1.7e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 2.9e-22, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 7.9e-22, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 1.2e-21, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 1.4e-21, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.00082, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bmr_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1bmr', 'e_value': 0.0051, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fh3_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1fh3', 'e_value': 0.0051, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7k18_2', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '7k18', 'e_value': 0.0051, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.0074, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.0074, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.014, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.015, 'percentage_identity': 42.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.016, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.016, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6thi_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-1'], 'pdb_id': '6thi', 'e_value': 0.024, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4hhf_1', 'molecule_name': ['Alpha-toxin OD1'], 'pdb_id': '4hhf', 'e_value': 0.037, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pjv_1', 'molecule_name': ['Potassium channel toxin alpha-KTx 10.1'], 'pdb_id': '1pjv', 'e_value': 0.041, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6f_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1i6f', 'e_value': 0.052, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6g_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1i6g', 'e_value': 0.052, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nh5_1', 'molecule_name': ['Alpha-like toxin CsEv5'], 'pdb_id': '1nh5', 'e_value': 0.052, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1chz_1', 'molecule_name': ['Alpha-like toxin BmK M2'], 'pdb_id': '1chz', 'e_value': 0.082, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2sn3_1', 'molecule_name': ['Toxin CsEv3'], 'pdb_id': '2sn3', 'e_value': 0.083, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.083, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.083, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn1_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1sn1', 'e_value': 0.1, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1djt_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1djt', 'e_value': 0.1, 'percentage_identity': 35.0, 'result_sequence': None}], 'RKSKEKIGKEFKRIVQRIKDF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 1.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 1.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 1.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 1.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 1.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.00016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}], 'GCLEFWWKCNPNDDKCCRPKLKCSKLFKLCNFSF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tyk_1', 'molecule_name': ['M-theraphotoxin-Gr1a'], 'pdb_id': '1tyk', 'e_value': 2.8e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lu8_1', 'molecule_name': ['M-theraphotoxin-Gr1a'], 'pdb_id': '1lu8', 'e_value': 2.8e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5we3_1', 'molecule_name': ['Beta-theraphotoxin-Ps1a'], 'pdb_id': '5we3', 'e_value': 4.6e-08, 'percentage_identity': 61.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mk5_1', 'molecule_name': ['Toxin GTx1-15'], 'pdb_id': '6mk5', 'e_value': 2.6e-06, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mxm_1', 'molecule_name': ['Mu-theraphotoxin-Tp1a'], 'pdb_id': '2mxm', 'e_value': 6.6e-05, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5tlr_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '5tlr', 'e_value': 7e-05, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2wh9_1', 'molecule_name': ['Kappa-theraphotoxin-Pg1a'], 'pdb_id': '2wh9', 'e_value': 8.6e-05, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mt7_1', 'molecule_name': ['Hs1a'], 'pdb_id': '2mt7', 'e_value': 0.00029, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m4z_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '2m4z', 'e_value': 0.00052, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['I', 'C', 'E', 'G'], 'entity_id': 2, 'entry_entity': '6w6o_2', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '6w6o', 'e_value': 0.00052, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mb6_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '1mb6', 'e_value': 0.00052, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m4x_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '2m4x', 'e_value': 0.00052, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m50_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '2m50', 'e_value': 0.00064, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5t3m_1', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '5t3m', 'e_value': 0.0012, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '7k48_2', 'molecule_name': ['Huwentoxin-IV'], 'pdb_id': '7k48', 'e_value': 0.0012, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mpq_1', 'molecule_name': ['Mu-theraphotoxin-Hd1a'], 'pdb_id': '2mpq', 'e_value': 0.0027, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ryv_1', 'molecule_name': ['Mu-theraphotoxin-Hhn1b 1'], 'pdb_id': '1ryv', 'e_value': 0.0041, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1niy_1', 'molecule_name': ['Mu-theraphotoxin-Hhn1b 1'], 'pdb_id': '1niy', 'e_value': 0.005, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5epm_3', 'molecule_name': ['Beta-theraphotoxin-Cm1a'], 'pdb_id': '5epm', 'e_value': 0.007, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ryg_1', 'molecule_name': ['Mu-theraphotoxin-Hhn1b 1'], 'pdb_id': '1ryg', 'e_value': 0.011, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i2p_1', 'molecule_name': ['Beta/delta-theraphotoxin-Pre1a'], 'pdb_id': '5i2p', 'e_value': 0.025, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i1x_1', 'molecule_name': ['Beta/delta-theraphotoxin-Pre1a'], 'pdb_id': '5i1x', 'e_value': 0.025, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i1t_1', 'molecule_name': ['Beta/kappa-theraphotoxin-Cg1a'], 'pdb_id': '2i1t', 'e_value': 0.025, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6br0_1', 'molecule_name': ['Beta-theraphotoxin-Cm1a'], 'pdb_id': '6br0', 'e_value': 0.03, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6btv_1', 'molecule_name': ['Beta-theraphotoxin-Cm1b'], 'pdb_id': '6btv', 'e_value': 0.036, 'percentage_identity': 35.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s6x_1', 'molecule_name': ['Kappa-theraphotoxin-Gr3a'], 'pdb_id': '1s6x', 'e_value': 0.055, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1n_1', 'molecule_name': ['Kappa-theraphotoxin-Gr3a'], 'pdb_id': '2n1n', 'e_value': 0.056, 'percentage_identity': 44.8, 'result_sequence': None}], 'PKVTITIQGSARF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gdl_1', 'molecule_name': ['myeloid antimicrobial peptide 27'], 'pdb_id': '2gdl', 'e_value': 2e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'FKRLKKLFSKIWNWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 5.2e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.00045, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.00045, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.0014, 'percentage_identity': 86.7, 'result_sequence': None}], 'FKQFHFKDFNRAFGFMTRVALQAEKL': [{'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '1dch_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '1dch', 'e_value': 3.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '1dcp_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '1dcp', 'e_value': 3.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '1dco_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '1dco', 'e_value': 3.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f93_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '1f93', 'e_value': 3.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3hxa_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '3hxa', 'e_value': 8.5e-13, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wil_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase 2'], 'pdb_id': '4wil', 'e_value': 4.2e-09, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ru0_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase 2'], 'pdb_id': '1ru0', 'e_value': 9.6e-09, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4c45_1', 'molecule_name': ['Pterin-4-alpha-carbinolamine dehydratase 2'], 'pdb_id': '4c45', 'e_value': 5e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v6u_1', 'molecule_name': ['4a-hydroxytetrahydrobiopterin dehydratase'], 'pdb_id': '2v6u', 'e_value': 1.6e-05, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v6s_1', 'molecule_name': ['4a-hydroxytetrahydrobiopterin dehydratase'], 'pdb_id': '2v6s', 'e_value': 1.6e-05, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v6t_1', 'molecule_name': ['4a-hydroxytetrahydrobiopterin dehydratase'], 'pdb_id': '2v6t', 'e_value': 1.6e-05, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3jst_1', 'molecule_name': ['Putative pterin-4-alpha-carbinolamine dehydratase'], 'pdb_id': '3jst', 'e_value': 2.6e-05, 'percentage_identity': 64.0, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNLYRAIK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 2.5e-10, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.00012, 'percentage_identity': 64.0, 'result_sequence': None}], 'NSQIRPLPDKGLDLSIRDASIKIRGKWKARKNFIK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4m4d_1', 'molecule_name': ['Lipopolysaccharide-binding protein'], 'pdb_id': '4m4d', 'e_value': 3.4e-06, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bp1_1', 'molecule_name': ['Bactericidal permeability-increasing protein'], 'pdb_id': '1bp1', 'e_value': 1.8e-05, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ewf_1', 'molecule_name': ['Bactericidal permeability-increasing protein'], 'pdb_id': '1ewf', 'e_value': 1.8e-05, 'percentage_identity': 57.1, 'result_sequence': None}], 'FKAFRWAWRWKKLAAPS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0012, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.028, 'percentage_identity': 70.6, 'result_sequence': None}], 'TIAMRAINNYRWRSKNQNTFLR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 1.6e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 7.6e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 0.0023, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 0.0025, 'percentage_identity': 71.4, 'result_sequence': None}], 'RWCVYAYVRVRGVGVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 3.3e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 4.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 4.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.9e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'SGPNGQCGPGWGGCRGGLCCSQYGYCGSGPKYCAH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 2.4e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 1.4e-07, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 8.1e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 2.7e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 2.9e-06, 'percentage_identity': 63.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 3.3e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 4e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 6.1e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 8.9e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t0w_1', 'molecule_name': ['Hevein'], 'pdb_id': '1t0w', 'e_value': 0.00013, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stq_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stq', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6sto_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6sto', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stp_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stp', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stn_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stn', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6stm_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6stm', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6str_1', 'molecule_name': ['Arundo donax Lectin (ADL)'], 'pdb_id': '6str', 'e_value': 0.00014, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 0.00015, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.00017, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 0.00018, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9g_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9g', 'e_value': 0.00026, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p9z_1', 'molecule_name': ['Antifungal peptide 2'], 'pdb_id': '1p9z', 'e_value': 0.00026, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.00045, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.00047, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.00047, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.00047, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.00047, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 0.00073, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lnr_1', 'molecule_name': ['Class-1 chitinase'], 'pdb_id': '6lnr', 'e_value': 0.0012, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 0.0017, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 0.0017, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 0.0017, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 0.002, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.0022, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.0022, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m5c_1', 'molecule_name': ['avenatide aV1'], 'pdb_id': '6m5c', 'e_value': 0.011, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 0.016, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 0.016, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eis_1', 'molecule_name': ['Agglutinin isolectin VI'], 'pdb_id': '1eis', 'e_value': 0.028, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1en2_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1en2', 'e_value': 0.028, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 0.028, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 0.028, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enm_1', 'molecule_name': ['Lectin 1'], 'pdb_id': '1enm', 'e_value': 0.028, 'percentage_identity': 46.7, 'result_sequence': None}], 'KFGYRYDSLELEGRSISRIDELIQQR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 5.7e-08, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4bed_2', 'molecule_name': ['Hemocyanin 1'], 'pdb_id': '4bed', 'e_value': 0.02, 'percentage_identity': 53.8, 'result_sequence': None}], 'RIKRFWPVVIRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 0.0053, 'percentage_identity': 75.0, 'result_sequence': None}], 'GIGKFIHAAKKFGKLFIGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 3.8e-09, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 3.9e-09, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 8.5e-08, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.1e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 2e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 6.2e-07, 'percentage_identity': 73.9, 'result_sequence': None}], 'GIGKALKKAKKGIGAVLKVLTTGL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 2.9e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 2.6e-08, 'percentage_identity': 95.8, 'result_sequence': None}], 'AAYLLAKINLKALAALAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 3.7e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.0034, 'percentage_identity': 78.9, 'result_sequence': None}], 'GLFGVLAKVASHVVPAIAEHFQA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 5.5e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}], 'GRLKKLGKKIEGAGKRVFKAAEKALPVVAGVKAL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmm_1', 'molecule_name': ['AAEL000598-PA'], 'pdb_id': '2mmm', 'e_value': 4.7e-12, 'percentage_identity': 82.4, 'result_sequence': None}], 'GGARSLGRKALRAWKKYGPAIVPIIRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.0043, 'percentage_identity': 57.7, 'result_sequence': None}], 'RWKIFKKIPKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.00011, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.00011, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.015, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.015, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.021, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.021, 'percentage_identity': 77.8, 'result_sequence': None}], 'SIWGDIGQGVGKAAYWVGKAMGNMSDVNQASRINRKKKH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpl_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit alpha'], 'pdb_id': '2jpl', 'e_value': 4.2e-16, 'percentage_identity': 83.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpj_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit alpha'], 'pdb_id': '2jpj', 'e_value': 4.2e-16, 'percentage_identity': 83.8, 'result_sequence': None}], 'GVVDILKGAAKDLAGHLATKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 2.3e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 2.3e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 2.3e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 4.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 4.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 4.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 3.2e-08, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 3.2e-08, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 3.2e-08, 'percentage_identity': 90.9, 'result_sequence': None}], 'KLFKKIGIGAVLKVLTTGLPALKLTLK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.00044, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.00044, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.026, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.045, 'percentage_identity': 93.3, 'result_sequence': None}], 'RWCVYAYVRVRGVLRRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 5.5e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 7.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 7.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 4.8e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'AFGCPFDQGTCHSHCRSIRRRGERCSGFAKRTCTCYQK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 2.2e-14, 'percentage_identity': 70.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 8.8e-08, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 7.3e-06, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 6.3e-05, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 0.00063, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0018, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0018, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0018, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 0.0019, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.0056, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.0056, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 0.07, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.082, 'percentage_identity': 41.4, 'result_sequence': None}], 'CFCKRPVCDSGETQIGYCRLGNTFYRLCCRQ': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 3.3e-06, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 2.2e-05, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 8.9e-05, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 0.00011, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 0.00011, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 0.00011, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 0.00011, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 0.00014, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 0.00058, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cui_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cui', 'e_value': 0.00094, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cuj_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cuj', 'e_value': 0.0012, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cum_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cum', 'e_value': 0.0012, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 0.0018, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h6c_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3h6c', 'e_value': 0.0018, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 0.0018, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm4_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm4', 'e_value': 0.0023, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo4_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo4', 'e_value': 0.0023, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mxq_1', 'molecule_name': ['Alpha-defensin 1'], 'pdb_id': '2mxq', 'e_value': 0.0025, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pm1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2pm1', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lo1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo1', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dfn_1', 'molecule_name': ['Neutrophil defensin 3'], 'pdb_id': '1dfn', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3go0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3go0', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kht_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2kht', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hjd_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hjd', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gny_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3gny', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm5_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm5', 'e_value': 0.0029, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hj2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hj2', 'e_value': 0.0031, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo2', 'e_value': 0.0037, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4lbf_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbf', 'e_value': 0.0047, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb1', 'e_value': 0.0059, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3loe_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3loe', 'e_value': 0.0059, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo6_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo6', 'e_value': 0.0075, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo9_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo9', 'e_value': 0.0075, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb7_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb7', 'e_value': 0.0094, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4du0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4du0', 'e_value': 0.019, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1zmk_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmk', 'e_value': 0.019, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmh_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmh', 'e_value': 0.019, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmi_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmi', 'e_value': 0.019, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ews_1', 'molecule_name': ['Corticostatin-related peptide RK-1'], 'pdb_id': '1ews', 'e_value': 0.041, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmm_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '1zmm', 'e_value': 0.066, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6dmm_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6dmm', 'e_value': 0.084, 'percentage_identity': 31.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6dmq_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6dmq', 'e_value': 0.084, 'percentage_identity': 31.0, 'result_sequence': None}], 'FFGWLIKGAIHAPKAIHGLIHRRRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rz1_1', 'molecule_name': ['Pleurocidin-like peptide WF3'], 'pdb_id': '6rz1', 'e_value': 0.0071, 'percentage_identity': 61.9, 'result_sequence': None}], 'RQIRIWFQNRRMRWRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 6.1e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 6.1e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 9.4e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xid_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xid', 'e_value': 9.4e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 9.4e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 9.4e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1san_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1san', 'e_value': 9.6e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '9ant_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '9ant', 'e_value': 9.6e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hoa_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '2hoa', 'e_value': 1e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1ahd_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1ahd', 'e_value': 1e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hom_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1hom', 'e_value': 1e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5z_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5z', 'e_value': 1.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5y_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5y', 'e_value': 1.3e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2h1k_3', 'molecule_name': ['Pancreas/duodenum homeobox protein 1'], 'pdb_id': '2h1k', 'e_value': 5e-05, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e1o_1', 'molecule_name': ['Hematopoietically-expressed homeobox protein HHEX'], 'pdb_id': '2e1o', 'e_value': 7.2e-05, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m34_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2m34', 'e_value': 0.00029, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me0_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me0', 'e_value': 0.00029, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me6_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me6', 'e_value': 0.00029, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a03_1', 'molecule_name': ['T-cell leukemia homeobox protein 2'], 'pdb_id': '3a03', 'e_value': 0.00041, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lp0_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2lp0', 'e_value': 0.00098, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msy_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2msy', 'e_value': 0.0011, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '4uus_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uus', 'e_value': 0.0012, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1puf_3', 'molecule_name': ['Homeobox protein Hox-A9'], 'pdb_id': '1puf', 'e_value': 0.0012, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b8i_3', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '1b8i', 'e_value': 0.0013, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cyc_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4cyc', 'e_value': 0.0014, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4uut_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uut', 'e_value': 0.0014, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmt_1', 'molecule_name': ['Homeobox protein BarH-like 1'], 'pdb_id': '2dmt', 'e_value': 0.0017, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ftz_1', 'molecule_name': ['Segmentation protein fushi tarazu'], 'pdb_id': '1ftz', 'e_value': 0.0019, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n8g_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2n8g', 'e_value': 0.002, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lnq_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3lnq', 'e_value': 0.0022, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a02_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3a02', 'e_value': 0.0022, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '3a01_2', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3a01', 'e_value': 0.0025, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7f_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7f', 'e_value': 0.0025, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkx_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2lkx', 'e_value': 0.0025, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7m_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7m', 'e_value': 0.0025, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjs_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjs', 'e_value': 0.003, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjr_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjr', 'e_value': 0.003, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '5zjt_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjt', 'e_value': 0.003, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjq_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjq', 'e_value': 0.003, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m0c_1', 'molecule_name': ['Homeobox protein aristaless-like 4'], 'pdb_id': '2m0c', 'e_value': 0.0036, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmu_1', 'molecule_name': ['Homeobox protein goosecoid'], 'pdb_id': '2dmu', 'e_value': 0.0044, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '5lux_2', 'molecule_name': ['Homeobox protein CDX-1'], 'pdb_id': '5lux', 'e_value': 0.0053, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '6es2_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es2', 'e_value': 0.0059, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '5lty_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '5lty', 'e_value': 0.0059, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '6es3_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es3', 'e_value': 0.006, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cue_1', 'molecule_name': ['Paired box protein Pax-6'], 'pdb_id': '2cue', 'e_value': 0.0065, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cmy_1', 'molecule_name': ['Paired box protein Pax-3'], 'pdb_id': '3cmy', 'e_value': 0.0068, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b72_3', 'molecule_name': ['Homeobox protein Hox-B1'], 'pdb_id': '1b72', 'e_value': 0.01, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5eg0_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eg0', 'e_value': 0.02, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['B', 'A', 'J', 'G'], 'entity_id': 3, 'entry_entity': '5eea_3', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eea', 'e_value': 0.021, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5ego_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ego', 'e_value': 0.021, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5ef6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ef6', 'e_value': 0.021, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5no6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5no6', 'e_value': 0.021, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cra_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '2cra', 'e_value': 0.023, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5edn_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5edn', 'e_value': 0.025, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ztr_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1ztr', 'e_value': 0.027, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld5_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2ld5', 'e_value': 0.03, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l9r_1', 'molecule_name': ['Homeobox protein Nkx-3.1'], 'pdb_id': '2l9r', 'e_value': 0.03, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7z_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2l7z', 'e_value': 0.031, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['I', 'N'], 'entity_id': 4, 'entry_entity': '6wc5_4', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '6wc5', 'e_value': 0.033, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1ig7_3', 'molecule_name': ['Homeobox protein MSX-1'], 'pdb_id': '1ig7', 'e_value': 0.033, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['M', 'N', 'O'], 'entity_id': 4, 'entry_entity': '6wc2_4', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '6wc2', 'e_value': 0.035, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 6, 'entry_entity': '4xrs_6', 'molecule_name': ['Homeobox protein DLX-3'], 'pdb_id': '4xrs', 'e_value': 0.043, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '1jgg_3', 'molecule_name': ['Segmentation protein even-skipped'], 'pdb_id': '1jgg', 'e_value': 0.045, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4rbo_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '4rbo', 'e_value': 0.052, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kt0_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2kt0', 'e_value': 0.061, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2vi6_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2vi6', 'e_value': 0.081, 'percentage_identity': 50.0, 'result_sequence': None}], 'RFRPPIRRPPIRPPFNPPFRPPVRPPFRPPFRPPFRPPIGPFP': [{'chain_id': ['A4'], 'entity_id': 30, 'entry_entity': '7abg_30', 'molecule_name': ['Transcription elongation regulator 1'], 'pdb_id': '7abg', 'e_value': 0.00053, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 10, 'entry_entity': '7abf_10', 'molecule_name': ['WW domain-binding protein 11'], 'pdb_id': '7abf', 'e_value': 0.0035, 'percentage_identity': 50.0, 'result_sequence': None}], 'KKRLKKIFKKPMVIGVTIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 3.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'VLSAADKGNVKAAWGKVGGHAAE': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g09', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hda_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hda', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fsx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fsx', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g08_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g08', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0a', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi8', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ihx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ihx', 'e_value': 5.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3cy5_1', 'molecule_name': ['Hemoglobin subunit alpha-2'], 'pdb_id': '3cy5', 'e_value': 6e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eui_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5eui', 'e_value': 1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h2l_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '4h2l', 'e_value': 2.3e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C', 'A', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ker_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '5ker', 'e_value': 2.3e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2mhb', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6sva_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6sva', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5c6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5c6e', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6r2o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6r2o', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dhb', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5x', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlx', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlt', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8h', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlu', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ibe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ibe', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iwh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1iwh', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns6', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8k', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8i', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zlv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlv', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2zlw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2zlw', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g0b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g0b', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ns9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ns9', 'e_value': 3.1e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hrw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hrw', 'e_value': 7e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qpw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qpw', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '4f4o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4f4o', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2ri4_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '2ri4', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2pgh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2pgh', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qu0_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2qu0', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gys_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gys', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3gqr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqr', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a0g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3a0g', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d4x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d4x', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gqp', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3hyu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hyu', 'e_value': 1.2e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3eu1_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3eu1', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d1a_1', 'molecule_name': ['Hemoglobin subunit alpha-1'], 'pdb_id': '3d1a', 'e_value': 1.2e-06, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '5hy8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hy8', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7o', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e6e_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e6e', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4mqj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqj', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4mqk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqk', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7n', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '5ee4_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ee4', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5hu6_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5hu6', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '4n7p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n7p', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e83', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rol_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rol', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['U', 'P', 'K', 'A', 'Z', 'F'], 'entity_id': 1, 'entry_entity': '4wjg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4wjg', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksi', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5u3i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5u3i', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ufj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ufj', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xs0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4xs0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x0l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4x0l', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ucu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ucu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4rom_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4rom', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qje_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qje', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3s66_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s66', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s48_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s48', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3szk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3szk', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqi', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3whm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3whm', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4a', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qjc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3r5i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3r5i', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4n8t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4n8t', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5vmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5vmm', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3s65_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3s65', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ksj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ksj', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wcp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3wcp', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4l7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4l7y', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5e29_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5e29', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqg', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4m4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4m4b', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5urc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5urc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2u', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bwp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwp', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kae_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kae', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kap_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kap', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kav_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kav', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kao_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kao', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kas_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kas', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kaq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kaq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kau_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kau', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6kat_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kat', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6ka9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ka9', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kar_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kar', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hk2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hk2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kye', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6tb2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6tb2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xdt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xdt', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xd9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xd9', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5ni1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2t', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5w', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5x', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6di4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6di4', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6l5v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5v', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6l5y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6l5y', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nq5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nq5', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2r', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6nbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6nbd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5x2s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5x2s', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bwu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bwu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6bnr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bnr', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6xe7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6xe7', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6lcx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6lcx', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hbw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hbw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['R', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1lfl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfl', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfy', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygf', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0d', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y22_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y22', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xxt', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhr', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5j', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yhe', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1shr_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1shr', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1uiw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1uiw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1thb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1thb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yie_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yie', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y2z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y2z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y45_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y45', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3nmm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nmm', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4fc3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4fc3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2w6v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w6v', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3nl7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3nl7', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbs', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bbb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3dut_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3dut', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hco', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo5', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hxn_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hxn', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d5z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d5z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dn2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhe', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbf', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2h35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2h35', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3hhb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2dxm_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dxm', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3onz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3onz', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3oo4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3oo4', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3d17_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d17', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2hbs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbs', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbe', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2dn3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hhb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2d60_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2d60', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3ovu_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ovu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ia3_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ia3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3odq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3odq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yh9', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1y01_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y01', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yzi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yzi', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y83_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y83', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4r_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4r', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4p', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1yff_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yff', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y31_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y31', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5k', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4v', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y85_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y85', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y35_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y35', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7g', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yev', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0t_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0t', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1mko_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1mko', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j41_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j41', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['W', 'R', 'U', 'P', 'A', 'C'], 'entity_id': 1, 'entry_entity': '1jy7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1jy7', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1buw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1buw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdl_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdl', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsh', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1vwt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1vwt', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxe_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxe', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qxd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qxd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yih', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y46_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y46', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4q', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yg5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yg5', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0w', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1z8u_2', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1z8u', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yvq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yen_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yen', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y5f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y5f', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4b', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeo_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeo', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1yeq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yeq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4g_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4g', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ygd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ygd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7d', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ye1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ye1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y7c', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y4f_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y4f', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1si4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1si4', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G', 'S'], 'entity_id': 1, 'entry_entity': '3b75_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3b75', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3p5q_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3p5q', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ic2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ic2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3d7o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3d7o', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1sdk_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1sdk', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qsi_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qsi', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bz1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bz1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ird_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ird', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dke_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dke', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fn3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fn3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfz', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lft_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lft', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j40_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j40', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hco_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hco', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1x_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1x', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rqa_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rqa', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a00_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a00', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3n', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cls', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ljw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a01_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a01', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hdb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hdb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a9w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a9w', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hba_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hba', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a0u_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a0u', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aj9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aj9', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1lfv', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3y', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rvw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rvw', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hho_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hho', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1b86_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1b86', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1r1y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1r1y', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1coh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1coh', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1m9p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1m9p', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hga_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hga', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hgc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hgc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gzx_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gzx', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hbb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hbb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hac_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hac', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bij_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bij', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rps_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rps', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxv', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbv', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gbu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gbu', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1fdh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fdh', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1g9v_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1g9v', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1a3o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1a3o', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1cmy_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1cmy', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1rq4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1rq4', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nej_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nej', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1j3z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j3z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1k0y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k0y', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1kd2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1kd2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nqp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nqp', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1k1k', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hbc_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2hbc', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ni1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ni1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3qjd_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3qjd', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1dxt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1dxt', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1hab', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yvt_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1yvt', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1nih_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1nih', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2m6z_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2m6z', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'C'], 'entity_id': 1, 'entry_entity': '2yrs_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2yrs', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hhb_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4hhb', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4ij2_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4ij2', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kai_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kai', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6kah_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6kah', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jjq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jjq', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jxz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jxz', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy3_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy3', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy0', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7jy1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7jy1', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7cue_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7cue', 'e_value': 1.6e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7c', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1p_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1p', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1j_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1j', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1n_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1n', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1m_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1m', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aby_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1aby', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1l_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1l', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1abw_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1abw', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c7d_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7d', 'e_value': 2.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s0h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1s0h', 'e_value': 3.5e-06, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y09_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y09', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz7', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2w72_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2w72', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xye_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xye', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzv_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzv', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y8w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y8w', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0c_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0c', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7y_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7y', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1c7b_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1c7b', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1k_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1k', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1ydz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1ydz', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1y0a_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1y0a', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o1i_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1i', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xy0_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xy0', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz5', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xz4_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xz4', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1xzu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1xzu', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1qi8_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1qi8', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7s_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7s', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1gli_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1gli', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1o1o_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1o1o', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bzz_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bzz', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1j7w_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1j7w', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5sw7_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5sw7', 'e_value': 4.7e-06, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 6e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 6e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 6.1e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 6.1e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1hds_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '1hds', 'e_value': 6.1e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aet_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aet', 'e_value': 6.1e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aeu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aeu', 'e_value': 6.1e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['AAA', 'CCC'], 'entity_id': 1, 'entry_entity': '7aev_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '7aev', 'e_value': 6.1e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1bab_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1bab', 'e_value': 6.2e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gdj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gdj', 'e_value': 3.1e-05, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3hf4_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3hf4', 'e_value': 4.1e-05, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dht_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '3dht', 'e_value': 4.1e-05, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3pel_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pel', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qls_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qls', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1fhj_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1fhj', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3gou_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3gou', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2b7h_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2b7h', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fh9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3fh9', 'e_value': 0.00012, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '1faw_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1faw', 'e_value': 0.00036, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3eok_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3eok', 'e_value': 0.00036, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4yu4_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu4', 'e_value': 0.00062, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4yu3_1', 'molecule_name': ['Alpha-globin'], 'pdb_id': '4yu3', 'e_value': 0.00062, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2rao_1', 'molecule_name': ['Hemoglobin subunit alpha-1/2'], 'pdb_id': '2rao', 'e_value': 0.00081, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3lqd_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3lqd', 'e_value': 0.00081, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrf_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrf', 'e_value': 0.0011, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3vre_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vre', 'e_value': 0.0011, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vrg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3vrg', 'e_value': 0.0011, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c40_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1c40', 'e_value': 0.0032, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a4f_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1a4f', 'e_value': 0.0032, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '1hv4_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '1hv4', 'e_value': 0.0032, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3k8b_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3k8b', 'e_value': 0.0042, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qmb_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2qmb', 'e_value': 0.0042, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3mjp_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mjp', 'e_value': 0.0042, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmy_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmy', 'e_value': 0.0042, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6zmx_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '6zmx', 'e_value': 0.0042, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mju_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3mju', 'e_value': 0.0071, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r80_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '2r80', 'e_value': 0.0071, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at5_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at5', 'e_value': 0.0071, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3at6_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3at6', 'e_value': 0.0071, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3dhr_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3dhr', 'e_value': 0.0072, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3wtg_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3wtg', 'e_value': 0.016, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3wr1_1', 'molecule_name': ['Hemoglobin subunit alpha-A'], 'pdb_id': '3wr1', 'e_value': 0.063, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2r1h_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '2r1h', 'e_value': 0.063, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3bom_1', 'molecule_name': ['Hemoglobin subunit alpha-4'], 'pdb_id': '3bom', 'e_value': 0.063, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1spg_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '1spg', 'e_value': 0.064, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6iyi_1', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 0.083, 'percentage_identity': 50.0, 'result_sequence': None}], 'KGIRGYKGGYCKGAFKQTCKCY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 3.1e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 3.1e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 3.1e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 3.1e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 3.1e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.00011, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.0051, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.026, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.079, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.083, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.095, 'percentage_identity': 50.0, 'result_sequence': None}], 'ACVNQCPDAIDRFIVKDKGCHGVEKKYYKQVYVACMNGQHLYCRTEWGGPCQL': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lwz_1', 'molecule_name': ['Bacteriocin'], 'pdb_id': '6lwz', 'e_value': 8.2e-28, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6lwy_1', 'molecule_name': ['Putative laterosporulin'], 'pdb_id': '6lwy', 'e_value': 4.3e-13, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ozk_1', 'molecule_name': ['Putative bacteriocin'], 'pdb_id': '4ozk', 'e_value': 1.7e-12, 'percentage_identity': 56.9, 'result_sequence': None}], 'GWGSFFKKAAHVGKHVGKAALGAAARRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 2.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 2.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 3.5e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 5.9e-08, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.0011, 'percentage_identity': 57.1, 'result_sequence': None}], 'FWPVVIRTVVAGYNLYRAIKKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 1.3e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.00015, 'percentage_identity': 60.0, 'result_sequence': None}], 'FALLGDFFRKSKEKIGKEFKRIVKRIKDFFRNLVPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 4.5e-15, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 4.5e-15, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 4.5e-15, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 4.5e-15, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 4.5e-15, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 1.1e-07, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 4.9e-07, 'percentage_identity': 95.7, 'result_sequence': None}], 'ATCDLLSAFGVGHAACAAHCIGHGYRGGYCNSKAVCTCRR': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.4e-13, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.1e-13, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 3.9e-13, 'percentage_identity': 71.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.1e-12, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.1e-12, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.3e-11, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 3.8e-11, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.6e-07, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 6.2e-07, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 3.8e-06, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 4.5e-05, 'percentage_identity': 47.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.033, 'percentage_identity': 38.7, 'result_sequence': None}], 'GIGAVLDVLTTGLPALISWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 7.2e-11, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 7.2e-11, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 7.4e-11, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 7.4e-11, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 7.4e-11, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 7.4e-11, 'percentage_identity': 96.2, 'result_sequence': None}], 'GLRKRLRKARNKIKEKLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 5.4e-05, 'percentage_identity': 95.0, 'result_sequence': None}], 'KLCGETCFKFKCYTPGCSCSYPFCK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 1.5e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 1.6e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 6e-07, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 1.5e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 1.5e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 4.7e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 6e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 9.4e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 1.1e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 1.2e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.2e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 1.2e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 1.2e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 1.5e-05, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 1.5e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 1.6e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.9e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 2.4e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 3e-05, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 5.8e-05, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 7.4e-05, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 0.00012, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 0.00019, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.00037, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.0012, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.0012, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.003, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 0.0073, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 0.0091, 'percentage_identity': 61.1, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0039, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0039, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.026, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.047, 'percentage_identity': 92.9, 'result_sequence': None}], 'FKRLKKLAKKIWNWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.00072, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.058, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.058, 'percentage_identity': 73.3, 'result_sequence': None}], 'NLVSDIIGSKKHMEKLISIIKKCR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6wux_1', 'molecule_name': ['Homotarsinin'], 'pdb_id': '6wux', 'e_value': 1.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7mn3_1', 'molecule_name': ['Homotarsinin'], 'pdb_id': '7mn3', 'e_value': 1.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'RIGSILGRLAKGLPTLISWIKNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0032, 'percentage_identity': 59.1, 'result_sequence': None}], 'RWCVYAYVRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.3e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 3.1e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 3.1e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.1e-11, 'percentage_identity': 95.2, 'result_sequence': None}], 'RWCVYAYVRVRGVLVRYRRCS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 3.7e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 4.9e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 4.9e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 3.7e-10, 'percentage_identity': 95.0, 'result_sequence': None}], 'RWCVYAYVRVRGVLVRYRRCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 5.8e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 7.7e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 7.7e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 5.5e-10, 'percentage_identity': 95.0, 'result_sequence': None}], 'KAPAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 6.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 1.2e-11, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 1.3e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 1.7e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 1e-08, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 5.3e-08, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 2.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 2.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 2.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 2.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 2.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 4.4e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 4.5e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 4.7e-07, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l60_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2l60', 'e_value': 6.6e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz5_1', 'molecule_name': ['Neuropeptide Y', 'Pancreatic hormone'], 'pdb_id': '1tz5', 'e_value': 5.4e-06, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ppt_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ppt', 'e_value': 2.1e-05, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3s_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3s', 'e_value': 6.1e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2h3t_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h3t', 'e_value': 6.1e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2h4b_2', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2h4b', 'e_value': 6.1e-05, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bf9_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2bf9', 'e_value': 0.00018, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bba_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1bba', 'e_value': 0.0062, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljv_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ljv', 'e_value': 0.0064, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k76_1', 'molecule_name': ['pGolemi'], 'pdb_id': '2k76', 'e_value': 0.028, 'percentage_identity': 47.6, 'result_sequence': None}], 'RWCVYAYVRVRGVLVRYRRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 6.5e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 8.6e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 8.6e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 6.2e-10, 'percentage_identity': 95.0, 'result_sequence': None}], 'ACYCRIPACFAGERRYGTCFYLGRVWAFCC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3go0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3go0', 'e_value': 6.3e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kht_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2kht', 'e_value': 6.3e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hjd_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hjd', 'e_value': 6.3e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h6c_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3h6c', 'e_value': 6.3e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gny_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3gny', 'e_value': 6.3e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hj2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hj2', 'e_value': 6.7e-16, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 1.3e-15, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4lbf_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbf', 'e_value': 1.6e-15, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dfn_1', 'molecule_name': ['Neutrophil defensin 3'], 'pdb_id': '1dfn', 'e_value': 2.1e-15, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 2.6e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4du0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4du0', 'e_value': 4.3e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo4_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo4', 'e_value': 5.4e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo2', 'e_value': 6.9e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lo1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo1', 'e_value': 6.9e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3loe_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3loe', 'e_value': 8.7e-15, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1zmk_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmk', 'e_value': 1.4e-14, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmh_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmh', 'e_value': 1.4e-14, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmi_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmi', 'e_value': 1.4e-14, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm4_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm4', 'e_value': 3.7e-14, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo6_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo6', 'e_value': 4.6e-14, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo9_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo9', 'e_value': 4.6e-14, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb1', 'e_value': 9.5e-14, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb7_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb7', 'e_value': 2.5e-13, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pm1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2pm1', 'e_value': 4e-13, 'percentage_identity': 76.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm5_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm5', 'e_value': 1.3e-12, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3qte_1', 'molecule_name': ['Defensin-6'], 'pdb_id': '3qte', 'e_value': 1.2e-05, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 0.00045, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cui_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cui', 'e_value': 0.00056, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmq_1', 'molecule_name': ['Defensin-6'], 'pdb_id': '1zmq', 'e_value': 0.00091, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 0.0024, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 0.0024, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 0.003, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tv0_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '1tv0', 'e_value': 0.0049, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gw9_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2gw9', 'e_value': 0.0049, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6dmq_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6dmq', 'e_value': 0.005, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 0.0062, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 0.0078, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cuj_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cuj', 'e_value': 0.0099, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cum_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cum', 'e_value': 0.0099, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gwp_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2gwp', 'e_value': 0.013, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k1i_1', 'molecule_name': ['DEFENSIN domain-containing protein'], 'pdb_id': '2k1i', 'e_value': 0.016, 'percentage_identity': 37.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmm_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '1zmm', 'e_value': 0.016, 'percentage_identity': 37.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rc7_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '7rc7', 'e_value': 0.033, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rc8_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '7rc8', 'e_value': 0.033, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lew_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2lew', 'e_value': 0.042, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ley_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2ley', 'e_value': 0.053, 'percentage_identity': 44.8, 'result_sequence': None}], 'RRSVQWCA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.03, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.03, 'percentage_identity': 100.0, 'result_sequence': None}], 'GWGKFFKKAAKVGKKVG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 2.2e-05, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 2.3e-05, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.00029, 'percentage_identity': 64.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.00039, 'percentage_identity': 70.6, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 5.4e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 5.4e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 5.4e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 7.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 7.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 7.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'KRFKKFFRKLKKSVKKRAKEFFKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 8e-12, 'percentage_identity': 79.4, 'result_sequence': None}], 'GILDTLKQFAKGVGKDLVKGAAQGVLSTVSCKLALTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 2.2e-16, 'percentage_identity': 97.3, 'result_sequence': None}], 'KCWNLRGSCREKCIKNEKLYIFCTSGKLCCLKPK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 1.2e-05, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lwl_1', 'molecule_name': ['Beta-defensin 106'], 'pdb_id': '2lwl', 'e_value': 0.0017, 'percentage_identity': 43.3, 'result_sequence': None}], 'TASQAEWFKARRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 5.1e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 1.8e-06, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 1.8e-06, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 8.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}], 'KPKGMTSSQWFKIQHMQPSPQACNSAMKNINKHTKRCKDLNTFLHEPFSSVAATCQTPKIACKNGDKNCHQSHGAVSLTMCKLTSGKYPNCRYKEKR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 4.9e-46, 'percentage_identity': 99.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 5.9e-24, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 5.9e-24, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 5.9e-24, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 5.9e-24, 'percentage_identity': 57.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 3.4e-23, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 8e-18, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 2.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4srn', 'e_value': 4.1e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3srn', 'e_value': 4.1e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssc', 'e_value': 4.1e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssa', 'e_value': 4.3e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ssb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ssb', 'e_value': 4.3e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1srn', 'e_value': 4.3e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6qmn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qmn', 'e_value': 4.4e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qh3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4qh3', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ogh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5ogh', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obe', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obc', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5old_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5old', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5et4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5et4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5e_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5e', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5e5f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5e5f', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rte_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rte', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s0q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s0q', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsd', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyp', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyn', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4wyz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4wyz', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsd', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j65_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j65', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j66_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j66', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j67', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j68_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j68', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j61_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j61', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ql2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql2', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rh1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rh1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j6a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j6a', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rn3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rn3', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j62_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j62', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3qsk', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ao1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ao1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xog_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xog', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j63_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j63', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j69_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j69', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j5z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j5z', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l55_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4l55', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mxf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4mxf', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j64_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j64', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rid', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jmg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jmg', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4s18_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4s18', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5obd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5obd', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4j60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4j60', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2xoi_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2xoi', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jlg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jlg', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d97_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d97', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4zz4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4zz4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5jml_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5jml', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6eto_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6eto', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rsa', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvv', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6gok_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6gok', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6f60_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6f60', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etr', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etq', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xvx', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etp', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6qe9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6qe9', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etm', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhc', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nj7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5nj7', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5d6u_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5d6u', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhf', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhd', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['BBB', 'AAA'], 'entity_id': 1, 'entry_entity': '6xw0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xw0', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6xhe_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6xhe', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5na9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5na9', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etl', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvx', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etk', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvu', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6etn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6etn', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pvw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6pvw', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e3w', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2qca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2qca', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p43_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p43', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4q', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6s_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6s', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wbu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1wbu', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rns_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rns', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ot4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ot4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ooh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ooh', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8y', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3jw1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3jw1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1a2w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a2w', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbx', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rsm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rsm', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 2, 'entry_entity': '1f0v_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1f0v', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eow_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eow', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9x', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eid_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eid', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eic', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dib_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dib', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dic_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dic', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g4w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4w', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8r', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7w', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blp', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p47_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p47', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p4a_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p4a', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p49_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p49', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2e33_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e33', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5g', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5k_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5k', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5m', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5l', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev5', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev2', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g4x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g4x', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di8', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euy_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euy', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3euz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3euz', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev6', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2g8q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2g8q', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lxo_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3lxo', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p45_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p45', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p46_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p46', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2p42_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p42', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p48_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p48', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i67_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i67', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7x', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8y', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6q', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxg', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev3', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3dxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dxh', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eux_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3eux', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6h', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i7y_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i7y', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d8z_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d8z', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6j', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3i6f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3i6f', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2aas_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2aas', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0o', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1eos_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eos', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf2', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jn4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jn4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4o', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1w4p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1w4p', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xps_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xps', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1z6d_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z6d', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1xpt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1xpt', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1u1b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1u1b', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1qhc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1qhc', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6p', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d7b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d7b', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh5', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2blz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2blz', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d6o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3d6o', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a1r_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3a1r', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1bzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bzq', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fs3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fs3', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvu', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvt_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvt', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0f_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0f', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnm_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnm', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnd_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnd', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rno_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rno', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afu', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ras_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ras', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afk', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnv', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afl_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1afl', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rny_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rny', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rbb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbb', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rob_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rob', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbn', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ruv', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0n_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0n', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf7', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0h_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0h', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf3', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbj_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbj', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1rnn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnn', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rtb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rtb', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnq', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rbw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbw', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rha_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rha', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rca_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rca', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnc_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnc', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rcn_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rcn', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rar_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rar', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpg_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpg', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnu_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnu', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rhb_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rhb', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aqp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1aqp', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eie_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1eie', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnz', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnx_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnx', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1rta_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rta', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '1dfj_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dfj', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rph_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rph', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rpf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rpf', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rnw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rnw', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1js0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1js0', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jvv_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1jvv', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kh8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kh8', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1o0m', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf8', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kf5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1kf5', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p44_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2p44', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2w5i_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2w5i', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bel_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1bel', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev4_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev4', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ev0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ev0', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4peq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4peq', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ql1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g8v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g8v', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4pou_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4pou', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4g90_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4g90', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6yo1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '6yo1', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '5rsa', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '9rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '8rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rsa', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '8rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '9rsa', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rat_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rat', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rsa_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '7rsa', 'e_value': 4.5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzq', 'e_value': 4.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0c_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0c', 'e_value': 4.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c0b_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c0b', 'e_value': 4.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mzr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mzr', 'e_value': 4.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2apq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2apq', 'e_value': 5e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3dh6_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3dh6', 'e_value': 7.6e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwr', 'e_value': 9e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mx8_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mx8', 'e_value': 9e-15, 'percentage_identity': 45.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4rsk', 'e_value': 9.1e-15, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsk_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsk', 'e_value': 9.1e-15, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2oqf_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2oqf', 'e_value': 9.1e-15, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3mwq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3mwq', 'e_value': 9.1e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di9_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di9', 'e_value': 9.1e-15, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dy5_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dy5', 'e_value': 1.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3di7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3di7', 'e_value': 1.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1lsq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1lsq', 'e_value': 1.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1b6v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1b6v', 'e_value': 1.5e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c8w_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c8w', 'e_value': 1.5e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izp', 'e_value': 1.5e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4k7m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7m', 'e_value': 1.8e-14, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4kxh_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4kxh', 'e_value': 1.8e-14, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2op2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2op2', 'e_value': 1.8e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 2e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymn', 'e_value': 2.2e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izq', 'e_value': 2.2e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 2.4e-14, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymr', 'e_value': 2.6e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ymw_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1ymw', 'e_value': 3.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1rra_1', 'molecule_name': ['Ribonuclease pancreatic beta-type'], 'pdb_id': '1rra', 'e_value': 3.1e-14, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5q_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5q', 'e_value': 3.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1izr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1izr', 'e_value': 3.1e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3rsp_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3rsp', 'e_value': 3.7e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c9v_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1c9v', 'e_value': 3.7e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3tsr_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3tsr', 'e_value': 4.5e-14, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ygw_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4ygw', 'e_value': 6.3e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fkz_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fkz', 'e_value': 6.3e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl0_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl0', 'e_value': 6.3e-14, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4k7l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4k7l', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j82_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j82', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o36_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o36', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4okf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4okf', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4o37_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4o37', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbd_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbd', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3p_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3p', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3or0_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3or0', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '3oqz_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqz', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3l_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3l', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j81_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j81', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j80_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j80', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '1z3m_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z3m', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 2, 'entry_entity': '3oqy_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3oqy', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1j7z_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1j7z', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbc_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbc', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbh_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbh', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbe_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbe', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbg_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbg', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbi_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbi', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '1rbf_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1rbf', 'e_value': 6.4e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '2rln_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2rln', 'e_value': 6.7e-14, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k11_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2k11', 'e_value': 7.7e-14, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e21_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1e21', 'e_value': 7.8e-14, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '2q4g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2q4g', 'e_value': 7.9e-14, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['X', 'Z'], 'entity_id': 1, 'entry_entity': '1z7x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1z7x', 'e_value': 7.9e-14, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fev_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fev', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5d_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5d', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5h_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5h', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjq_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjq', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1d5e_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1d5e', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1cjr_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1cjr', 'e_value': 8.8e-14, 'percentage_identity': 50.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1h8x_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1h8x', 'e_value': 1.1e-13, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0o_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0o', 'e_value': 1.3e-13, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl3_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl3', 'e_value': 1.3e-13, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3fl1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3fl1', 'e_value': 1.3e-13, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0j_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0j', 'e_value': 3.9e-13, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a5p_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1a5p', 'e_value': 6.3e-13, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0m_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0m', 'e_value': 6.5e-13, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2e0l_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2e0l', 'e_value': 6.5e-13, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dza_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1dza', 'e_value': 7.8e-13, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3f8g_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3f8g', 'e_value': 1.5e-12, 'percentage_identity': 39.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 1.6e-12, 'percentage_identity': 37.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4n4c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '4n4c', 'e_value': 2.2e-12, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tq9_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1tq9', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djq', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djo_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djo', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djp', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djv_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djv', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3djx_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3djx', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1bsr_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1bsr', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1x_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n1x', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n3z_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1n3z', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5c_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5c', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11ba_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11ba', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r3m_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r3m', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1r5d_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1r5d', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '11bg_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '11bg', 'e_value': 4.4e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y94_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y94', 'e_value': 5.2e-12, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3bcp_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcp', 'e_value': 1.1e-11, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bco_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bco', 'e_value': 1.1e-11, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3bcm_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '3bcm', 'e_value': 1.1e-11, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qwq_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1qwq', 'e_value': 1.1e-11, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lfj_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '2lfj', 'e_value': 1.1e-11, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 1.4e-11, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1y92_1', 'molecule_name': ['Seminal ribonuclease'], 'pdb_id': '1y92', 'e_value': 1.5e-11, 'percentage_identity': 41.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arj_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arj', 'e_value': 6.7e-11, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5arl_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5arl', 'e_value': 6.7e-11, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ar6_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ar6', 'e_value': 1e-10, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '2rnf', 'e_value': 1e-10, 'percentage_identity': 40.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1rnf_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '1rnf', 'e_value': 1e-10, 'percentage_identity': 40.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5ark_1', 'molecule_name': ['Ribonuclease 4'], 'pdb_id': '5ark', 'e_value': 1.1e-10, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gv7_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1gv7', 'e_value': 1.8e-10, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1un5_1', 'molecule_name': ['Angiogenin', 'Ribonuclease pancreatic'], 'pdb_id': '1un5', 'e_value': 8.6e-06, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahh_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahh', 'e_value': 1.4e-05, 'percentage_identity': 35.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eqo_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5eqo', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5epz_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5epz', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5eop_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5eop', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahl_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahl', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahj_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahj', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahm_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahm', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahn_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahn', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ahk_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahk', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k58_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k58', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5b_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k5b', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1un4_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1un4', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h52_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h52', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '1a4y_2', 'molecule_name': ['Angiogenin'], 'pdb_id': '1a4y', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k59_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k59', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5a_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1k5a', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '1h0d_3', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h0d', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h53_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1h53', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hby_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1hby', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b1i_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1b1i', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ang_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1ang', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1awz_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1awz', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ang_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '2ang', 'e_value': 2.4e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9s_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9s', 'e_value': 2.5e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9v_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9v', 'e_value': 2.5e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5m9r_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9r', 'e_value': 2.5e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5m9t_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9t', 'e_value': 2.5e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4aoh_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4aoh', 'e_value': 2.5e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9p_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9p', 'e_value': 2.9e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9g_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9g', 'e_value': 2.9e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahf_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahf', 'e_value': 2.9e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m9m_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9m', 'e_value': 2.9e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9q_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9q', 'e_value': 2.9e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahe_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahe', 'e_value': 4.1e-05, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ahg_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahg', 'e_value': 4.9e-05, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9j_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9j', 'e_value': 5.9e-05, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1un3_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1un3', 'e_value': 7e-05, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ahd_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4ahd', 'e_value': 0.00012, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9a_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '5m9a', 'e_value': 0.00014, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4b36_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '4b36', 'e_value': 0.00015, 'percentage_identity': 33.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b1j_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '1b1j', 'e_value': 0.0002, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1agi_1', 'molecule_name': ['Angiogenin-1'], 'pdb_id': '1agi', 'e_value': 0.00035, 'percentage_identity': 32.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gio_1', 'molecule_name': ['Angiogenin-1'], 'pdb_id': '1gio', 'e_value': 0.00035, 'percentage_identity': 32.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2j4t_1', 'molecule_name': ['Angiogenin-4'], 'pdb_id': '2j4t', 'e_value': 0.00097, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qfj_1', 'molecule_name': ['RNAse_Pc domain-containing protein'], 'pdb_id': '4qfj', 'e_value': 0.0014, 'percentage_identity': 32.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4qfi_1', 'molecule_name': ['RNAse_Pc domain-containing protein'], 'pdb_id': '4qfi', 'e_value': 0.0014, 'percentage_identity': 32.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bwk_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '2bwk', 'e_value': 0.0014, 'percentage_identity': 32.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bwl_1', 'molecule_name': ['Angiogenin'], 'pdb_id': '2bwl', 'e_value': 0.0014, 'percentage_identity': 32.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2vq9_1', 'molecule_name': ['Ribonuclease-like 3'], 'pdb_id': '2vq9', 'e_value': 0.0024, 'percentage_identity': 28.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7bfl_1', 'molecule_name': ['RNAse_Pc domain-containing protein'], 'pdb_id': '7bfl', 'e_value': 0.0028, 'percentage_identity': 32.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7bfk_1', 'molecule_name': ['RNAse_Pc domain-containing protein'], 'pdb_id': '7bfk', 'e_value': 0.003, 'percentage_identity': 32.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3zbv_1', 'molecule_name': ['Angiogenin-2'], 'pdb_id': '3zbv', 'e_value': 0.0034, 'percentage_identity': 30.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lje_1', 'molecule_name': ['RNAse_Pc domain-containing protein'], 'pdb_id': '3lje', 'e_value': 0.0036, 'percentage_identity': 31.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zbw_1', 'molecule_name': ['Angiogenin-3'], 'pdb_id': '3zbw', 'e_value': 0.0097, 'percentage_identity': 30.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2zpo_1', 'molecule_name': ['Ribonuclease'], 'pdb_id': '2zpo', 'e_value': 0.014, 'percentage_identity': 33.3, 'result_sequence': None}], 'GGYYCPFFQDKCHRHCRSFGRKAGYCGGFLKKTCICV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 2.6e-11, 'percentage_identity': 65.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 3.2e-08, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 9.5e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.00029, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 0.0013, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.0016, 'percentage_identity': 47.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.0029, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 0.0037, 'percentage_identity': 42.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0043, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0043, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0043, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 0.029, 'percentage_identity': 37.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.054, 'percentage_identity': 40.0, 'result_sequence': None}], 'GKKLAKTIAKEVAKQGAKFA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k38_1', 'molecule_name': ['Cupiennin-1a'], 'pdb_id': '2k38', 'e_value': 0.0043, 'percentage_identity': 72.2, 'result_sequence': None}], 'IISTIGKLVKWIIKTV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 0.00089, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 0.00089, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 0.00089, 'percentage_identity': 87.5, 'result_sequence': None}], 'MPRRRRSSSRPVRRRRRPRVSRRRRRRGGRRR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4htv_2', 'molecule_name': ['Putative capsid protein'], 'pdb_id': '4htv', 'e_value': 0.063, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 43, 'entry_entity': '6qx9_43', 'molecule_name': ['Serine/threonine-protein kinase PRP4 homolog'], 'pdb_id': '6qx9', 'e_value': 0.063, 'percentage_identity': 63.6, 'result_sequence': None}], 'KPKGMTSSQWFKIQHMQPSPQASNSAMKNINKHTKRSKDLNTFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.3e-20, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 2.7e-09, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 2.7e-09, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 2.7e-09, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 2.7e-09, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 2.3e-08, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 3.1e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 3.7e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 5.8e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 0.0002, 'percentage_identity': 41.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 0.00078, 'percentage_identity': 40.9, 'result_sequence': None}], 'KPKDMTSSQWFKTQHVQPSPQASNSAMSIINKYTERSKDLNTFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 5.3e-16, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1e-09, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1e-09, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1e-09, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1e-09, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 8.8e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c02_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c02', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 5.7e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 5.8e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5e13_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '5e13', 'e_value': 6.8e-07, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 5.3e-05, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'J', 'C', 'A', 'B', 'G', 'E', 'F', 'L'], 'entity_id': 1, 'entry_entity': '6ymt_1', 'molecule_name': ['RNase 3/1 version 1'], 'pdb_id': '6ymt', 'e_value': 0.00019, 'percentage_identity': 41.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4owz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4owz', 'e_value': 0.00071, 'percentage_identity': 40.9, 'result_sequence': None}], 'ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ki9_1', 'molecule_name': ['Beta-defensin 104'], 'pdb_id': '5ki9', 'e_value': 3.9e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 0.0027, 'percentage_identity': 50.0, 'result_sequence': None}], 'IGKLFKRIVKRILKFLRKL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.016, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.019, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.019, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.019, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.019, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.019, 'percentage_identity': 73.7, 'result_sequence': None}], 'TVSAEESCPPGAVWSTCSNCHAYCKNKDLMCITACQEGCKCTTQGYKIHNNECIPAEKCP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1eai_2', 'molecule_name': ['Chymotrypsin/elastase isoinhibitor 1'], 'pdb_id': '1eai', 'e_value': 4.2e-05, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhq_1', 'molecule_name': ['von Willebrand factor'], 'pdb_id': '2mhq', 'e_value': 0.0016, 'percentage_identity': 35.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhp_1', 'molecule_name': ['von Willebrand factor'], 'pdb_id': '2mhp', 'e_value': 0.0016, 'percentage_identity': 35.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6n29_1', 'molecule_name': ['von Willebrand factor'], 'pdb_id': '6n29', 'e_value': 0.0039, 'percentage_identity': 35.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hx2_1', 'molecule_name': ['TIL domain-containing protein'], 'pdb_id': '1hx2', 'e_value': 0.0047, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['V'], 'entity_id': 2, 'entry_entity': '7kwo_2', 'molecule_name': ['von Willebrand factor', 'Immunoglobulin heavy constant gamma 1'], 'pdb_id': '7kwo', 'e_value': 0.0082, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ccv_1', 'molecule_name': ['Chymotrypsin inhibitor'], 'pdb_id': '1ccv', 'e_value': 0.0098, 'percentage_identity': 36.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ate_1', 'molecule_name': ['Trypsin inhibitor'], 'pdb_id': '1ate', 'e_value': 0.01, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1atd_1', 'molecule_name': ['Trypsin inhibitor'], 'pdb_id': '1atd', 'e_value': 0.01, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ata_1', 'molecule_name': ['Trypsin inhibitor'], 'pdb_id': '1ata', 'e_value': 0.01, 'percentage_identity': 30.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1atb_1', 'molecule_name': ['Trypsin inhibitor'], 'pdb_id': '1atb', 'e_value': 0.01, 'percentage_identity': 30.0, 'result_sequence': None}], 'LQKYYWRVRGGRWAVLSWLPKEEQIGKWSTRGRKWWRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 3.4e-11, 'percentage_identity': 85.0, 'result_sequence': None}], 'FQWQRNMRKVRGPPVS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 6.9e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 4e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 4e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 4e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 7.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 9.5e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 9.6e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 9.9e-07, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sqy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1sqy', 'e_value': 1.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2bjj_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2bjj', 'e_value': 1.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n76_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1n76', 'e_value': 1.9e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 0.0012, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6q_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6q', 'e_value': 0.015, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtz_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dtz', 'e_value': 0.015, 'percentage_identity': 73.3, 'result_sequence': None}], 'GLRKALRKFRNKIKEALKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 6e-05, 'percentage_identity': 90.0, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLASKVMNKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 2.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 2.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 2.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1.1e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 7.5e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 7.5e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 7.5e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'WKKWWKKWWKWWKKWWKK': [{'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1yst_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1yst', 'e_value': 0.038, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '4n7l_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4n7l', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '5lse_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '5lse', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4rcr_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4rcr', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '4in7_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4in7', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4in6_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4in6', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4in5_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4in5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '3v3z_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3v3z', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '4lwy_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4lwy', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '3zum_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3zum', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4h99_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4h99', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '3v3y_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3v3y', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['AL', 'BL'], 'entity_id': 5, 'entry_entity': '4v9g_5', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4v9g', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '4tqq_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4tqq', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '5lri_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '5lri', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2wx5_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2wx5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uwv_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uwv', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2rcr_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2rcr', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '4n7k_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4n7k', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '3zuw_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3zuw', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '6z02_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '6z02', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '5v33_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '5v33', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '6z1j_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '6z1j', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uwu_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uwu', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uxl_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uxl', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uww_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uww', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2ux5_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2ux5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uxj_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uxj', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uwt_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uwt', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uws_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uws', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uxm_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uxm', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2ux4_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2ux4', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2uxk_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2uxk', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1f6n_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1f6n', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1fnp_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1fnp', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1ds8_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1ds8', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hit_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hit', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2jj0_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2jj0', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2gmr_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2gmr', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3dsy_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3dsy', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2gnu_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2gnu', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2jiy_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2jiy', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3du2_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3du2', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3dta_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3dta', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3dts_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3dts', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3du3_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3du3', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3dtr_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3dtr', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hhk_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hhk', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3i4d_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3i4d', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1yf6_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1yf6', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z9k_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1z9k', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1pcr_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1pcr', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1l9j_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1l9j', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1m3x_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1m3x', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1rzz_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rzz', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1rzh_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rzh', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1ry5_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1ry5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z9j_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1z9j', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1s00_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1s00', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '1ogv_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1ogv', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '1qov_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1qov', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bnp_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2bnp', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2boz_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2boz', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bns_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2bns', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '1umx_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1umx', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1dv6_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1dv6', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '1e14_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1e14', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1l9b_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1l9b', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1k6n_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1k6n', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1jgz_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1jgz', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1jgy_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1jgy', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['N', 'L'], 'entity_id': 1, 'entry_entity': '1aig_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1aig', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1rqk_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rqk', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1rg5_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rg5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1jh0_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1jh0', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1jgw_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1jgw', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1mps_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1mps', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1k6l_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1k6l', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1rgn_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rgn', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1aij_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1aij', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['R', 'L'], 'entity_id': 1, 'entry_entity': '1dv3_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1dv3', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '1e6d_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1e6d', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1fnq_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1fnq', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1rvj_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1rvj', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1jgx_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1jgx', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1kby_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1kby', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2ux3_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2ux3', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hh1_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hh1', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2j8d_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2j8d', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hg9_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hg9', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hg3_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hg3', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '2j8c_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2j8c', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3duq_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '3duq', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '2hj6_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '2hj6', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4hbh_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4hbh', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4hbj_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4hbj', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '4h9l_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4h9l', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 2, 'entry_entity': '6z27_2', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '6z27', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}], 'LFGVLAKVAAHVVPAIAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 2e-06, 'percentage_identity': 90.0, 'result_sequence': None}]}

In [12]:
%store r16

Stored 'r16' (dict)


## 5750 - 6000

In [1]:
r17={'GLFGVLAKVAAKVVPAIAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 2.4e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 2.4e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 2.4e-05, 'percentage_identity': 81.0, 'result_sequence': None}], 'STAHLVLRLRGG': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5oxi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5oxi', 'e_value': 0.002, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ud7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ud7', 'e_value': 0.004, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1c3t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1c3t', 'e_value': 0.004, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5eya_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5eya', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5wfi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wfi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5vey_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vey', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4dhz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4bvu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bvu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vf0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vf0', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5fer_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5fer', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xpk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xpk', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xu8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xu8', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5emz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5emz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xve_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xve', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '5ohp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'K', 'M', 'J', 'N', 'O', 'L'], 'entity_id': 2, 'entry_entity': '5ohl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohl', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 3, 'entry_entity': '5edv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5edv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mm3', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ohv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6i_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6i', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mjb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mjb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '2mbh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '4s1z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s1z', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4wur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wur', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xyz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xyz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5goj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goj', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5tut_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tut', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 1, 'entry_entity': '5h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5h7s', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gog_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gog', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xof_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xof', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5goi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5go7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go7', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4wzp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wzp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gob', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gvi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '5kyd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3v', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mcn', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1y_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1y', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xol_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xol', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4y1h_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4y1h', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5go8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go8', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5god_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5god', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'A', 'B', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4xkh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4s22_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s22', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goc', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4xok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xok', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5txk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5txk', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['h', 'c', 'X', 'm'], 'entity_id': 10, 'entry_entity': '4zux_10', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zux', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5mn9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mn9', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulk', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5u4p_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5u4p', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gok_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gok', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5tr4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tr4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7u', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2xk5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xk5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '4jio_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jio', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kox', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7w_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7w', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ldt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ldt', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zlz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zlz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3wxg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxg', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'B', 'J', 'F'], 'entity_id': 2, 'entry_entity': '4dhj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhj', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7s', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 2, 'entry_entity': '3tbl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tbl', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'D', 'M', 'Q', 'N', 'O', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddg', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lz6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lz6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '3vuy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuy', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vux_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vux', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5kye_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kye', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3u_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3u', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5l6h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6h', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2k_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n2k', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5l6j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l6j', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mj5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5lrv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyc', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jbv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jbv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n13', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mi8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mi8', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4un2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4un2', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'O'], 'entity_id': 8, 'entry_entity': '5kgf_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kgf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 3, 'entry_entity': '5o44_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5o44', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mor_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mor', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbq', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3tmp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tmp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ugb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ugb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '4bos_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bos', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4jqw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jqw', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4m0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4m0w', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0t', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3vht_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vht', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4bbn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bbn', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2qho_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2qho', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3m3j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3m3j', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vnz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vnz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nvg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nvg', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vo0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vo0', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vzm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzm', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '4boz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4boz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5j8p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j8p', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5jby_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jby', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5jqs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jqs', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nmc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nmc', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z9s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4z9s', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5ait_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ait', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3m', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlj', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['9'], 'entity_id': 9, 'entry_entity': '5a5b_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5a5b', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ulf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5af4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5aiu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5aiu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujn', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jtv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mro_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mro', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6p5b_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6p5b', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7m', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6tnf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tnf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 15, 'entry_entity': '6tbm_15', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tbm', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 12, 'entry_entity': '6t9l_12', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t9l', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6v5d_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6v5d', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6asr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6asr', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '6vae_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vae', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '6ttu_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ttu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k9p', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6txb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6txb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kox', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kow_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kow', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P', 'I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6e2b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6e2b', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6kl4_3', 'molecule_name': ['Ub'], 'pdb_id': '6kl4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6aqr_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6aqr', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6vaf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vaf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msb_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3o_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3o', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 3, 'entry_entity': '5d0k_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0k', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nli_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nli', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujl', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xiu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xiu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3n', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 1, 'entry_entity': '6jwi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jwi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6q00_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q00', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oi4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oi4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6s53_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6s53', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N', 'O'], 'entity_id': 8, 'entry_entity': '6ftx_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ftx', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djx', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5zbu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zbu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4i_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k4i', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6lp2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6lp2', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n13', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6xqc_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6xqc', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6nxk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nxk', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6qml_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qml', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4zpz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zpz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1z_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1z', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nl4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nl4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xbo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5bz0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bz0', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nlf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nlf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5dfl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dfl', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 3, 'entry_entity': '5d0m_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0m', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6wtg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wtg', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5af6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msd_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '5xdp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xdp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7cap_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7cap', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb7', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 7, 'entry_entity': '6jma_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jma', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ymy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ymy', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6q01_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q01', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7o', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '6w9d_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9d', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H'], 'entity_id': 3, 'entry_entity': '6oaa_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oaa', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6tuv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tuv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6rya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6rya', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'J'], 'entity_id': 2, 'entry_entity': '6oa9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oa9', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4wlr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wlr', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'C', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '6z7v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6z7v', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6zqh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6zqh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6if1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6if1', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wwz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7s', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q0w', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ogw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ogw', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '1wr6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubq', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wr1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr1', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1tbe', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v80_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v80', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n32_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n32', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ru6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rr9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rr9', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3n30_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n30', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '3mhs_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3mhs', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3nhe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nhe', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rsu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rsu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wx1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx1', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3cmm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3cmm', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1f9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1f9j', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2jt4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jt4', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h1u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h1u', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D'], 'entity_id': 3, 'entry_entity': '2o6v_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2o6v', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pea_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pea', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2dx5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ehv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ehv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2g45_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g45', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcm', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2g3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g3q', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eec_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3eec', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2gmi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gmi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3efu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3efu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kde_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kde', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kdf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pe9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pe9', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2oob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2oob', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3hm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3hm3', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcn', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hth_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hth', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2jri_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jri', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jy6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jy6', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcs_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fcs', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcq_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '2fcq', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2fif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fif', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8c_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8c', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a33_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a33', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k39_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k39', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8b', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jzz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jzz', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2z59_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2z59', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ayo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ayo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3alb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3alb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ofi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ofi', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ns8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ns8', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 1, 'entry_entity': '1yd8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yd8', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3aul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3aul', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 2, 'entry_entity': '1p3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1p3q', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g6j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1g6j', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1uzx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1uzx', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1otr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1otr', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1zgu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zgu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yiw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '1yiw', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wrd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wrd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yj1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yj1', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v81_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v81', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcc', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q5w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q5w', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1s1q', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fxt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1fxt', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1aar_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1aar', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cmx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1cmx', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1nbf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1nbf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '2c7n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7n', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7m', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3h7p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7p', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d3g_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2d3g', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lj5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wx0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx0', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvq', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2xbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xbb', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2bgf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2bgf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fid_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fid', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2fuh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fuh', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hd5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hd5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xqq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xqq', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1d3z', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2jf5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jf5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2den_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2den', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0f_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0f', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4pih_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pih', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l3z', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vuw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuw', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvp', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2klg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2klg', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '3phd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ktf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ktf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2l00_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l00', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4pig_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4pig', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kn5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kn5', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2xew_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xew', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6isu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6isu', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '7bbf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbf', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7jxv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jxv', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '7bbd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbd', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7keo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7keo', 'e_value': 0.0058, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ohm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohm', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['I', 'D', 'J', 'C'], 'entity_id': 3, 'entry_entity': '5ydk_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydk', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '3wwq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wwq', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqo', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'F'], 'entity_id': 3, 'entry_entity': '6a6i_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6a6i', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'B'], 'entity_id': 3, 'entry_entity': '5xit_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xit', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6k2u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k2u', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyi', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6oq2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq2', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqs_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqs', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ud0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ud0', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F', 'L'], 'entity_id': 3, 'entry_entity': '6sqr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqr', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '7bxg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bxg', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5xis_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xis', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '6oq1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq1', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6uyj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyj', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4nql_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nql', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3jsv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jsv', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3a1q_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a1q', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9j_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9j', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a9k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9k', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2znv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2znv', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ld9', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6kfp_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kfp', 'e_value': 0.0059, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mur', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4r62_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4r62', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbb', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'F'], 'entity_id': 2, 'entry_entity': '5yik_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yik', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq4', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6cp2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6cp2', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq3', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '5yij_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yij', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5yt6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yt6', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3nob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nob', 'e_value': 0.006, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9p', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5vzw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mre_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mre', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq7', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6jky_3', 'molecule_name': ['Ub'], 'pdb_id': '6jky', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qf8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qf8', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5zq5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq5', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'K', 'D', 'B', 'C', 'J', 'N', 'O', 'F', 'G', 'L'], 'entity_id': 2, 'entry_entity': '6upu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6upu', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6kg6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kg6', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4nqk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nqk', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'U', 'T', 'X', 'S'], 'entity_id': 3, 'entry_entity': '4p4h_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4p4h', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4nnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nnj', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2khw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2khw', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3oj3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj3', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '3olm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3olm', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3oj4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj4', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ptf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ptf', 'e_value': 0.0061, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4xkl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkl', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5m93_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5m93', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ksk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksk', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mdk', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6dc6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6dc6', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U', 'X'], 'entity_id': 3, 'entry_entity': '3dvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvn', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 3, 'entry_entity': '3dvg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvg', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4gu2_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gu2', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4ap4_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ap4', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4gsw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '4gsw', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '7ay1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay1', 'e_value': 0.0062, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4v3k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3k', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msm', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4v3l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3l', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '5mnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mnj', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4zfr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zfr', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4auq_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4auq', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msq', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4pqt_2', 'molecule_name': ['Protein UBBP4'], 'pdb_id': '4pqt', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4k1r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k1r', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '5ln1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ln1', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'L'], 'entity_id': 4, 'entry_entity': '3zni_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zni', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4zft_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zft', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hpr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hpr', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jvz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jvz', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jw0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jw0', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbr', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai1', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai0', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['III', 'LLL', 'CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ahz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ahz', 'e_value': 0.0063, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5udh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5udh', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jvc_1', 'molecule_name': ['ubiquitin_like protein'], 'pdb_id': '2jvc', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zw7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zw7', 'e_value': 0.0064, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'F', 'G', 'L'], 'entity_id': 3, 'entry_entity': '4whv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4whv', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ii2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii2', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbm', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2gbk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbk', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gbn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbn', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2gbj_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gbj', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 6, 'entry_entity': '7lyc_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7lyc', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['W', 'R', 'P', 'M', 'Q', 'X', 'N', 'O'], 'entity_id': 1, 'entry_entity': '7k5j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k5j', 'e_value': 0.0065, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6nya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nya', 'e_value': 0.0066, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '5tte_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tte', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6ypt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ypt', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4fjv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4fjv', 'e_value': 0.0067, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1sif', 'e_value': 0.0069, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4ii3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ii3', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '5knl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5knl', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '5cvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvn', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5cvm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvm', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 2, 'entry_entity': '6nyo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nyo', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5cvo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvo', 'e_value': 0.0077, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx5', 'e_value': 0.0079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx6', 'e_value': 0.0079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4hcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4hcn', 'e_value': 0.0079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3vdz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vdz', 'e_value': 0.0093, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ojr', 'e_value': 0.0093, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kdi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdi', 'e_value': 0.0096, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['AK', 'FK', 'DK', 'HK'], 'entity_id': 10, 'entry_entity': '4v8p_10', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4v8p', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['5'], 'entity_id': 6, 'entry_entity': '4adx_6', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '4adx', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '5umd_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5umd', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Bm'], 'entity_id': 41, 'entry_entity': '6uz7_41', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uz7', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 47, 'entry_entity': '6n8n_47', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8n', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 44, 'entry_entity': '6n8m_44', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6n8m', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 24, 'entry_entity': '6lqm_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6lqm', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['u'], 'entity_id': 24, 'entry_entity': '6ef3_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6ef3', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 42, 'entry_entity': '3j79_42', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3j79', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 41, 'entry_entity': '5t6r_41', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t6r', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['z'], 'entity_id': 42, 'entry_entity': '5t62_42', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t62', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 27, 'entry_entity': '5t2c_27', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '5t2c', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 15, 'entry_entity': '7azy_15', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '7azy', 'e_value': 0.011, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kna_1', 'molecule_name': ['viral Ubiquitin'], 'pdb_id': '6kna', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3wxf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxf', 'e_value': 0.013, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3wxe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxe', 'e_value': 0.013, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 15, 'entry_entity': '7k5i_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '7k5i', 'e_value': 0.013, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '5flx_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '5flx', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juo_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juo', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5juu_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5juu', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znz', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzy_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzy', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '4d61_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4d61', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['V', 'H', 'R', 'P', 'D', 'C', 'J', 'T', 'X', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4ksl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksl', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Af', 'Cf'], 'entity_id': 33, 'entry_entity': '4v88_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v88', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8z_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8z', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jup_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jup', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jut_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jut', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['CC'], 'entity_id': 81, 'entry_entity': '5jus_81', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5jus', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzx_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzx', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 15, 'entry_entity': '5mc6_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5mc6', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A5'], 'entity_id': 6, 'entry_entity': '4v8y_6', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '4v8y', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6kbe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kbe', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Nb', 'N', 'Nc'], 'entity_id': 50, 'entry_entity': '6sv4_50', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6sv4', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['6', 'fb'], 'entity_id': 32, 'entry_entity': '6t83_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t83', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['Sf'], 'entity_id': 32, 'entry_entity': '6t7i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6t7i', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['N'], 'entity_id': 11, 'entry_entity': '6eml_11', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6eml', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['SFF'], 'entity_id': 26, 'entry_entity': '6rm3_26', 'molecule_name': ['eS31'], 'pdb_id': '6rm3', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['E1', 'e1'], 'entity_id': 33, 'entry_entity': '5ndg_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '5ndg', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6g5i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6g5i', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 1, 'entry_entity': '5wq4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wq4', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p5j_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p5j', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyx_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyx', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6snt_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6snt', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6sgc_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6sgc', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '3jam_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jam', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6y_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6y', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 78, 'entry_entity': '3j6x_78', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j6x', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j78_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j78', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jaq_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jaq', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 31, 'entry_entity': '3j81_31', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3j81', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '3jap_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '3jap', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '2zvn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvn', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w9n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2w9n', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '2zvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvo', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '3b08_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b08', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3b0a_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b0a', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3axc_1', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '3axc', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['31'], 'entity_id': 77, 'entry_entity': '3j77_77', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '3j77', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2y5b_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2y5b', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 32, 'entry_entity': '6zu9_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zu9', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 35, 'entry_entity': '6fyy_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6fyy', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 35, 'entry_entity': '6zce_35', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6zce', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6hcf_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6hcf', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6hcm_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6hcm', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'e1'], 'entity_id': 79, 'entry_entity': '6hhq_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S31'], 'pdb_id': '6hhq', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zuo_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zuo', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '7eal_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7eal', 'e_value': 0.014, 'percentage_identity': 91.7, 'result_sequence': None}], 'HSSGYTRPLRKPSRPIFIRPI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 1.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLNALKKVFQGIHEAIKLFNNHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 7.4e-11, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 1.3e-09, 'percentage_identity': 91.7, 'result_sequence': None}], 'FFHHIFRAIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 7.8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 7.8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 1e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.9e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 6.6e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 6.6e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 6.6e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.057, 'percentage_identity': 50.0, 'result_sequence': None}], 'QRGGFTGPIPRPPPHGRPPLGPICNACYRLSFSDVRICCNFLGKCCHLVK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 5.6e-09, 'percentage_identity': 51.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 0.00081, 'percentage_identity': 42.6, 'result_sequence': None}], 'AKCLRWQWRMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00055, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.014, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.014, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.014, 'percentage_identity': 76.9, 'result_sequence': None}], 'AKCLRWQWAMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0048, 'percentage_identity': 69.2, 'result_sequence': None}], 'GIMSIVKDVAKNAAKEAAKGALSTLSCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 5.3e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.026, 'percentage_identity': 51.7, 'result_sequence': None}], 'EKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00024, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}], 'GLLNTFKDWAISIAKGAGKGVLTTLSCKLDKSC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 3e-05, 'percentage_identity': 48.6, 'result_sequence': None}], 'GIFSKLAGKKLKNLLISGLKNVGKEVGMDVVRTGIDIAGCKIKGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 0.00058, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 0.00058, 'percentage_identity': 95.0, 'result_sequence': None}], 'RGLRRLGRKIAHGVKKYGPTVLRIIRIAG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 3.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'FKCRRWQWAMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00065, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00065, 'percentage_identity': 80.0, 'result_sequence': None}], 'AKIPIKAIKTVGKAVGKGLRAINIASTANDVFNFLKPKKRKH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 7.6e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 1.1e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 1.1e-11, 'percentage_identity': 72.5, 'result_sequence': None}], 'RRWSFRVSYRGFSYRKSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 0.00025, 'percentage_identity': 73.7, 'result_sequence': None}], 'APRKNVRWCAISLPEWSKCYQWQRRMRKLGAPSITCIRRTS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 4.2e-21, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 2.8e-16, 'percentage_identity': 78.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 8.5e-16, 'percentage_identity': 79.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.6e-15, 'percentage_identity': 79.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 1.1e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 1.1e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 5.8e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 5.8e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 7e-11, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 9e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 1.1e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 1.1e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 1.1e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 1.2e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 1.7e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 1.7e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 1.7e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 1.7e-10, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2bjj_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2bjj', 'e_value': 2.2e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 3.3e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 3.3e-10, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n76_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1n76', 'e_value': 6.4e-10, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6q_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6q', 'e_value': 7.9e-10, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtz_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dtz', 'e_value': 7.9e-10, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qjm_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1qjm', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7z_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7z', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cr9_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '3cr9', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6b', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sqy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1sqy', 'e_value': 1.2e-09, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f9b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1f9b', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7u_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7u', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b1x_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b1x', 'e_value': 1.2e-09, 'percentage_identity': 58.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 2.8e-06, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtg_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1dtg', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqe_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1fqe', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'B', 'A', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2o7u_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '2o7u', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2o84_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '2o84', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fgs_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3fgs', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1oqg_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1oqg', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1oqh_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1oqh', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ryo_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1ryo', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1bp5_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1bp5', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1btj_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1btj', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jqf_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1jqf', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n7x_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1n7x', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n84_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1n84', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqf_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1fqf', 'e_value': 0.00016, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5dyh_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '5dyh', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h52_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '5h52', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x1b_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '4x1b', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s9n_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3s9n', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s9l_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3s9l', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3qyt_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3qyt', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3s9m_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3s9m', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y6k_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '5y6k', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6uj6_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6uj6', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5p_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '5x5p', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ctc_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6ctc', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6d04_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6d04', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6d03_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6d03', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6d05_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6d05', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wtd_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '5wtd', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6jas_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6jas', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x1d_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '4x1d', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ve1_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3ve1', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3v83_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3v83', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4h0w_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '4h0w', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3v8x_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '3v8x', 'e_value': 0.00031, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1suv_2', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1suv', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a8e_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1a8e', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1a8f_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1a8f', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d4n_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1d4n', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3k_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1d3k', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b3e_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1b3e', 'e_value': 0.00092, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n7w_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1n7w', 'e_value': 0.00093, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['M'], 'entity_id': 3, 'entry_entity': '6xr0_3', 'molecule_name': ['Melanotransferrin'], 'pdb_id': '6xr0', 'e_value': 0.0015, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6soy_3', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6soy', 'e_value': 0.0018, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6soz_3', 'molecule_name': ['Serotransferrin'], 'pdb_id': '6soz', 'e_value': 0.0018, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2hav_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '2hav', 'e_value': 0.0018, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2hau_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '2hau', 'e_value': 0.0018, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tfd_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1tfd', 'e_value': 0.0021, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jnf_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1jnf', 'e_value': 0.0042, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h76_1', 'molecule_name': ['Serotransferrin'], 'pdb_id': '1h76', 'e_value': 0.0067, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iej_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '1iej', 'e_value': 0.016, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2d3i_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '2d3i', 'e_value': 0.03, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ryx_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '1ryx', 'e_value': 0.03, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ovt_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '1ovt', 'e_value': 0.03, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n04_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '1n04', 'e_value': 0.03, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aiv_1', 'molecule_name': ['Ovotransferrin'], 'pdb_id': '1aiv', 'e_value': 0.03, 'percentage_identity': 35.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 0.042, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 0.042, 'percentage_identity': 56.0, 'result_sequence': None}], 'KWKSFIKKLLSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0011, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0011, 'percentage_identity': 70.0, 'result_sequence': None}], 'FKCRRWQWRAKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.3e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00065, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00065, 'percentage_identity': 80.0, 'result_sequence': None}], 'KFFRKLKKSVKKRAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.0078, 'percentage_identity': 80.0, 'result_sequence': None}], 'TCRYWCKTPENQTYCCEDEREIPSKVGLKPGKCPPVRPVCPPTRGFFEPPKTCSNDGSCYGADKCCFDRCLGEHVCKPIQTRG': [{'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4doq_2', 'molecule_name': ['Antileukoproteinase'], 'pdb_id': '4doq', 'e_value': 0.01, 'percentage_identity': 36.5, 'result_sequence': None}, {'chain_id': ['I'], 'entity_id': 2, 'entry_entity': '2z7f_2', 'molecule_name': ['Antileukoproteinase'], 'pdb_id': '2z7f', 'e_value': 0.01, 'percentage_identity': 36.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rel_1', 'molecule_name': ['Elafin'], 'pdb_id': '2rel', 'e_value': 0.013, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['I'], 'entity_id': 2, 'entry_entity': '1fle_2', 'molecule_name': ['Elafin'], 'pdb_id': '1fle', 'e_value': 0.013, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6atu_1', 'molecule_name': ['Elafin'], 'pdb_id': '6atu', 'e_value': 0.014, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ngg_1', 'molecule_name': ['Omwaprin-a'], 'pdb_id': '3ngg', 'e_value': 0.017, 'percentage_identity': 46.2, 'result_sequence': None}], 'STALPVNSPMNKGDTEVMKCIVEVISDTLSKPSPMPVSQECFETLRGDERILSILRHQNLLKELQDLALQGAKERAHQQKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 4.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIPCGESCVWIPCISAALGCSCKNKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 6.5e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 6.5e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.9e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 5.4e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 4.5e-11, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.3e-10, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 3.8e-10, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.6e-08, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 7.6e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 5.2e-06, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 4.2e-05, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00042, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.0012, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.004, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 0.0081, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.0081, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.01, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.015, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.019, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.023, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.031, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.036, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.036, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.044, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.044, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.044, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.044, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.083, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.083, 'percentage_identity': 44.4, 'result_sequence': None}], 'RKCLRWQWRMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00073, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}], 'AKCLRWQWEMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0011, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}], 'AKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00024, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0054, 'percentage_identity': 76.9, 'result_sequence': None}], 'GWKSVFRKAKKVGKTVGGLALDHYLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 7.3e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 7.5e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.00016, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.0011, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.0074, 'percentage_identity': 40.0, 'result_sequence': None}], 'EKCLRWQWRMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00072, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}], 'RKCLRWQWEMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.005, 'percentage_identity': 69.2, 'result_sequence': None}], 'FKCRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.2e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 6.9e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 6.9e-05, 'percentage_identity': 86.7, 'result_sequence': None}], 'GFMDTAKNVAKNVAVTLIDKLRCKVTGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0066, 'percentage_identity': 50.0, 'result_sequence': None}], 'GLGSVFGRLARILGRVIPKVAKKLGPKVAKVLPKVMKEAIPMAVEMAKSQEEQQPQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3l_1', 'molecule_name': ['M-myrmeciitoxin-Mp1e'], 'pdb_id': '5x3l', 'e_value': 0.00036, 'percentage_identity': 100.0, 'result_sequence': None}], 'SKCYQWQWRMRKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 6.9e-05, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.00035, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0008, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0008, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0018, 'percentage_identity': 78.6, 'result_sequence': None}], 'FKCARWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 8.2e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00013, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00013, 'percentage_identity': 86.7, 'result_sequence': None}], 'AKCLRWQWAMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0019, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.048, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.048, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.048, 'percentage_identity': 69.2, 'result_sequence': None}], 'LVNQLGISKSLANTILGAIAVGNLASWVLALVPGPGWATKAALATAETIVKHEGKAAAIAW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.0028, 'percentage_identity': 34.4, 'result_sequence': None}], 'FKRIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'RCRFCCRCCPRMRGCGICCRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s6w_1', 'molecule_name': ['Hepcidin'], 'pdb_id': '1s6w', 'e_value': 1.8e-08, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m4e_1', 'molecule_name': ['Hepcidin-20'], 'pdb_id': '1m4e', 'e_value': 0.0025, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['R', 'U', 'P', 'Q', 'T', 'S'], 'entity_id': 1, 'entry_entity': '4qae_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '4qae', 'e_value': 0.0026, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6wbv_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '6wbv', 'e_value': 0.0026, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kef_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '2kef', 'e_value': 0.0026, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3h0t_3', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '3h0t', 'e_value': 0.0026, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m4f_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '1m4f', 'e_value': 0.0026, 'percentage_identity': 57.9, 'result_sequence': None}], 'RKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0003, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}], 'GRKSDCFRKSGFCAFLKCPSLTLISGKCSRFYLCCKRIR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 8.8e-05, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.0086, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.013, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 0.013, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.049, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.049, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.049, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg6_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg6', 'e_value': 0.049, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.061, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.076, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.076, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.089, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.095, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.095, 'percentage_identity': 36.4, 'result_sequence': None}], 'ILGPILGLVSNALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.00015, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.00015, 'percentage_identity': 76.5, 'result_sequence': None}], 'QVYKGGYARPIPRPPPFVRPLPGGPIGPYNGCPVSCRGISFSQARSCCSRLGRCCHVGKGYSG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 7.9e-30, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 6.4e-05, 'percentage_identity': 43.9, 'result_sequence': None}], 'IIGPVLGLVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 5.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 5.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'RKCLRWQWEMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0014, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.032, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.032, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.032, 'percentage_identity': 69.2, 'result_sequence': None}], 'IIGPVLGMVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 1.3e-06, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 1.3e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'KWKSFIKKLTTAVKKVLTTGLPALIS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.084, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.084, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.087, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.087, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.087, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.087, 'percentage_identity': 93.3, 'result_sequence': None}], 'YPPKPESPGEDASPEEMNKYLTALRHYINLVTRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 9.4e-14, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 2.4e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 2.4e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 2.4e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 2.4e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 4.8e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 2.9e-12, 'percentage_identity': 78.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 3.1e-12, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 7.8e-12, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 1.1e-11, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 2e-11, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 1.6e-10, 'percentage_identity': 63.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 2e-10, 'percentage_identity': 63.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 2.6e-10, 'percentage_identity': 63.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz5_1', 'molecule_name': ['Neuropeptide Y', 'Pancreatic hormone'], 'pdb_id': '1tz5', 'e_value': 1.8e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ljv_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ljv', 'e_value': 2.2e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bba_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1bba', 'e_value': 2.2e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l60_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2l60', 'e_value': 4e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v1d_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1v1d', 'e_value': 7.9e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ppt_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '1ppt', 'e_value': 9.2e-05, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bf9_1', 'molecule_name': ['Pancreatic hormone'], 'pdb_id': '2bf9', 'e_value': 0.00059, 'percentage_identity': 41.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lo2_1', 'molecule_name': ['Cell surface antigen I'], 'pdb_id': '5lo2', 'e_value': 0.025, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lo3_1', 'molecule_name': ['Cell surface antigen I'], 'pdb_id': '5lo3', 'e_value': 0.025, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k76_1', 'molecule_name': ['pGolemi'], 'pdb_id': '2k76', 'e_value': 0.066, 'percentage_identity': 38.7, 'result_sequence': None}], 'GLFSVVTGVLKAVGKNVAKNVGGSLLEQLKCKKISGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0075, 'percentage_identity': 51.5, 'result_sequence': None}], 'YQWQRRMRKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLLRKGGEKIGEKLKKIGQKIKNFFQKLVPQPE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00072, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00072, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00072, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00072, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00072, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.026, 'percentage_identity': 65.4, 'result_sequence': None}], 'LIGPVLGLVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 1.5e-06, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 1.5e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'SYVGDCGSNGGSCVSSYCPYGNRLNYFCPLGRTCCRHAYV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0065, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.05, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.058, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.071, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.071, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.071, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.071, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.071, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.087, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.087, 'percentage_identity': 36.4, 'result_sequence': None}], 'GIGASILSAGKSALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 3.5e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.065, 'percentage_identity': 60.0, 'result_sequence': None}], 'AKCRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 4.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 8.4e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 8.4e-05, 'percentage_identity': 92.9, 'result_sequence': None}], 'FKCRRWQWRMKKAGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 4.6e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00054, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00054, 'percentage_identity': 80.0, 'result_sequence': None}], 'GLMSLFRGGVLKTAGKHIFKNVGGSLLDQAKCKITGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.03, 'percentage_identity': 42.1, 'result_sequence': None}], 'FKARRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 2.9e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 7.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0036, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0036, 'percentage_identity': 80.0, 'result_sequence': None}], 'AKCLRWQWEMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.004, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.096, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.096, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.096, 'percentage_identity': 69.2, 'result_sequence': None}], 'GIGSAILSAGKSALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.5e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.071, 'percentage_identity': 60.0, 'result_sequence': None}], 'ACYCRIGACVSGERLTGACGLNGRIYRLCCR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 2.9e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 2.9e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 3.6e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 3.6e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 3.6e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 3.6e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 3.6e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 7.2e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 1.8e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 2.8e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cui_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cui', 'e_value': 2.9e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cuj_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cuj', 'e_value': 3.7e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cum_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cum', 'e_value': 3.7e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lo1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo1', 'e_value': 4.5e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4lbf_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbf', 'e_value': 5.6e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb7_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb7', 'e_value': 7.1e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3go0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3go0', 'e_value': 7.1e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kht_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2kht', 'e_value': 7.1e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hjd_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hjd', 'e_value': 7.1e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gny_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3gny', 'e_value': 7.1e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hj2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hj2', 'e_value': 7.5e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb1', 'e_value': 8.9e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo2', 'e_value': 8.9e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h6c_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3h6c', 'e_value': 8.9e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3loe_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3loe', 'e_value': 1.4e-07, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo6_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo6', 'e_value': 1.8e-07, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo9_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo9', 'e_value': 1.8e-07, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 1.9e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dfn_1', 'molecule_name': ['Neutrophil defensin 3'], 'pdb_id': '1dfn', 'e_value': 2.3e-07, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 2.9e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4du0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4du0', 'e_value': 4.5e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo4_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo4', 'e_value': 5.7e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1zmk_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmk', 'e_value': 1.4e-06, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmh_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmh', 'e_value': 1.4e-06, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmi_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmi', 'e_value': 1.4e-06, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm4_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm4', 'e_value': 1.5e-06, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pm1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2pm1', 'e_value': 3.7e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k1i_1', 'molecule_name': ['DEFENSIN domain-containing protein'], 'pdb_id': '2k1i', 'e_value': 1.2e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm5_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm5', 'e_value': 1.2e-05, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tv0_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '1tv0', 'e_value': 0.0002, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gw9_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2gw9', 'e_value': 0.0002, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gwp_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2gwp', 'e_value': 0.00051, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rc7_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '7rc7', 'e_value': 0.0013, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7rc8_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '7rc8', 'e_value': 0.0013, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lew_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2lew', 'e_value': 0.0016, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mxq_1', 'molecule_name': ['Alpha-defensin 1'], 'pdb_id': '2mxq', 'e_value': 0.0017, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ley_1', 'molecule_name': ['Alpha-defensin 4'], 'pdb_id': '2ley', 'e_value': 0.002, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3qte_1', 'molecule_name': ['Defensin-6'], 'pdb_id': '3qte', 'e_value': 0.042, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmq_1', 'molecule_name': ['Defensin-6'], 'pdb_id': '1zmq', 'e_value': 0.042, 'percentage_identity': 41.4, 'result_sequence': None}], 'GIFSKLGRKKIKNLLISGLKNVGKEVGMDVVRTGIDIAGCKIKGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 0.0071, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 0.0071, 'percentage_identity': 90.0, 'result_sequence': None}], 'QCIGNGGRCNENVGPPYCCSGFCLRQPGQGYGYCKNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dkc_1', 'molecule_name': ['Antimicrobial peptide 1'], 'pdb_id': '1dkc', 'e_value': 5e-11, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbd_1', 'molecule_name': ['pB1'], 'pdb_id': '5xbd', 'e_value': 5.4e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q3j_1', 'molecule_name': ['Antimicrobial peptide Alo-3'], 'pdb_id': '1q3j', 'e_value': 1.4e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v91_1', 'molecule_name': ['Delta-amaurobitoxin-Pl1b'], 'pdb_id': '1v91', 'e_value': 0.013, 'percentage_identity': 42.1, 'result_sequence': None}], 'QLINSPVTCMSYGGSCQRSCNGGFRLGGHCGHPKIRCCRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0015, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.018, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.018, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.018, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.018, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.042, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.077, 'percentage_identity': 44.4, 'result_sequence': None}], 'GILDTLKQFAKGVGKWLVKGAAQGVLSTVSCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 6.9e-16, 'percentage_identity': 97.3, 'result_sequence': None}], 'KKLGAPSITCVRRAFA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 1.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 7.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 3.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00044, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.0054, 'percentage_identity': 92.3, 'result_sequence': None}], 'AREASKSLIGTASCTCRRAWICRWGERHSGKCIDQKGSTYRLCCRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mxq_1', 'molecule_name': ['Alpha-defensin 1'], 'pdb_id': '2mxq', 'e_value': 1.4e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 0.023, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 0.042, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 0.051, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 0.051, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 0.051, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 0.051, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 0.088, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 0.093, 'percentage_identity': 47.1, 'result_sequence': None}], 'SLGNFFRKARKKIGEEFKRIVQRIKDFLQHLIPRTEA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.1e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.1e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.1e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.1e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.1e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.7e-06, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.00011, 'percentage_identity': 77.3, 'result_sequence': None}], 'GIGAAILSAGKSALKGLAKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 2e-05, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.052, 'percentage_identity': 60.0, 'result_sequence': None}], 'FKCRRWQWRMKALGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 3.3e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 8.3e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00042, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00042, 'percentage_identity': 80.0, 'result_sequence': None}], 'SKCRQWQWKIRRTNP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 1.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 1.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}], 'GIGKFLKKAKKFGKAFVKILKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.00034, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.00035, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 0.0042, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cgn_1', 'molecule_name': ['D-Ala-Magainin Derivative'], 'pdb_id': '5cgn', 'e_value': 0.0042, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 0.0073, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.0073, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.052, 'percentage_identity': 70.0, 'result_sequence': None}], 'ASVVKTTIKASKKLCKGATLTCGCNITGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1w9n_1', 'molecule_name': ['Lantibiotic epilancin 15X'], 'pdb_id': '1w9n', 'e_value': 1.1e-10, 'percentage_identity': 80.0, 'result_sequence': None}], 'WNPFKELERAGQRVRDAIISAGPAVATVAQATALAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 8.8e-08, 'percentage_identity': 55.6, 'result_sequence': None}], 'DPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKPN': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 4.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 4.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 4.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 4.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 2e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 1.7e-15, 'percentage_identity': 91.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 6.4e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00025, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.014, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.024, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.03, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.03, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.03, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.03, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.037, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.045, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.045, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.045, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.045, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.045, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.056, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.056, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.082, 'percentage_identity': 36.4, 'result_sequence': None}], 'FKCRRWQWRMKKLAA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 3.9e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 9.9e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00051, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00051, 'percentage_identity': 80.0, 'result_sequence': None}], 'KWFNEKSIQNKIDEKIGKNFLGGMAKAVVHKLAKNEFMCVANVDMTKSCDTHCQKASGEKGYCHGTKCKCGVPLSY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ipo_1', 'molecule_name': ['Hge36'], 'pdb_id': '5ipo', 'e_value': 9.8e-12, 'percentage_identity': 64.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jyh_1', 'molecule_name': ['Hge36'], 'pdb_id': '5jyh', 'e_value': 6.5e-10, 'percentage_identity': 63.6, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0017, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0017, 'percentage_identity': 85.7, 'result_sequence': None}], 'FQWQRNMRKV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.0016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 0.029, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 0.03, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sqy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1sqy', 'e_value': 0.06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2bjj_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2bjj', 'e_value': 0.06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n76_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1n76', 'e_value': 0.06, 'percentage_identity': 90.0, 'result_sequence': None}], 'ATCDLLSGTGINHSACAAHCLLRGNRGGYCNGKAVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 9.7e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 2.8e-20, 'percentage_identity': 97.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.3e-19, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.8e-13, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 6.4e-13, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.9e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 4.7e-11, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 9.9e-10, 'percentage_identity': 62.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 7.3e-08, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 7.5e-08, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 2.4e-06, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.0024, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.0045, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.011, 'percentage_identity': 45.5, 'result_sequence': None}], 'GRFKRFRKKFKKLFKKLS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ket_1', 'molecule_name': ['Cathelicidin-6'], 'pdb_id': '2ket', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIPCAESCVWIPPCTITALMGCSCKNNVCYNN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 6.6e-11, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1.5e-09, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 1.5e-09, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1.5e-09, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 7e-09, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 2.1e-08, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 1.9e-07, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 3.1e-07, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 3.2e-07, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 4.8e-07, 'percentage_identity': 59.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 3.4e-05, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.0033, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.025, 'percentage_identity': 42.4, 'result_sequence': None}], 'QVYKGGYTRPIPRPPPFVRPLPGGPIGPYNGCPVSCRGISFSQARSCCSRLGRCCHVGKGYS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 3.2e-29, 'percentage_identity': 98.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 2.1e-05, 'percentage_identity': 45.6, 'result_sequence': None}], 'EFELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ki9_1', 'molecule_name': ['Beta-defensin 104'], 'pdb_id': '5ki9', 'e_value': 1.6e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 0.0044, 'percentage_identity': 50.0, 'result_sequence': None}], 'QRYFCRVRGGRCAALTCLPRETQIGRCSVKGRKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 7.5e-16, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 1.3e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.0054, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.0058, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.0058, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.0058, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.0058, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.011, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.013, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 0.019, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 0.044, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.051, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 0.07, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qeu_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qeu', 'e_value': 0.076, 'percentage_identity': 38.2, 'result_sequence': None}], 'INLKAIAALAKKLLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.0016, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.0022, 'percentage_identity': 85.7, 'result_sequence': None}], 'RRWQWRMKKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00046, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}], 'HSSGYTRPLPKPSRPIFIRPIGCDVCYGIPSSTARLCCFRYGDCCHR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 4.9e-23, 'percentage_identity': 97.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 2.5e-07, 'percentage_identity': 46.4, 'result_sequence': None}], 'SKCYQWQRRMRKLGAPSITCIRRTS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 9.2e-13, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 1.4e-08, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 1.6e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 6.3e-08, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 6.4e-08, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.4e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 3.4e-07, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 3.4e-07, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6q_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6q', 'e_value': 4.2e-07, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtz_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dtz', 'e_value': 4.2e-07, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 1.3e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 1.3e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 1.3e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 2.2e-06, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 2.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 2.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 2.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 3e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 3e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 3e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 3e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 3e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 4.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2bjj_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2bjj', 'e_value': 4.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n76_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1n76', 'e_value': 4.9e-06, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sqy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1sqy', 'e_value': 6.4e-06, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qjm_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1qjm', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7z_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7z', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cr9_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '3cr9', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6b', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f9b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1f9b', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7u_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7u', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b1x_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b1x', 'e_value': 0.00029, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 0.0022, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 0.0022, 'percentage_identity': 56.0, 'result_sequence': None}], 'SKCYQWQRRMRKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 5.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0054, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.0055, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.0055, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 0.016, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 0.03, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 0.03, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 0.03, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.043, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.043, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 0.057, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 0.071, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 0.072, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 0.074, 'percentage_identity': 61.5, 'result_sequence': None}], 'RGFRKHFNKLVKKVKHTISETAHVAKDTAVIAGSGAAVVAAT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrx_1', 'molecule_name': ['Stomoxyn'], 'pdb_id': '1zrx', 'e_value': 2.3e-17, 'percentage_identity': 100.0, 'result_sequence': None}], 'DFGCARGMIFVCMRRCARMYPGSTGYCQGFRCMCDTMIPIRRPPFIMG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.055, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.055, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kja_1', 'molecule_name': ['Beta-toxin Cn5'], 'pdb_id': '2kja', 'e_value': 0.081, 'percentage_identity': 32.7, 'result_sequence': None}], 'GILDTLKQFAKGVGKFLVKGAAQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 8e-09, 'percentage_identity': 95.7, 'result_sequence': None}], 'GILDTLKQFAKGVGKWLVKGAAQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 3.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}], 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djw', 'e_value': 9.9e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '4uf6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4uf6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '5eya_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5eya', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ohk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5wfi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wfi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5ohm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohm', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5vey_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vey', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '4dhz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4bvu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bvu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vf0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vf0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5fer_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5fer', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xpk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xpk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xu8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xu8', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xve_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xve', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '5ohp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'I', 'K', 'M', 'J', 'N', 'O', 'L'], 'entity_id': 2, 'entry_entity': '5ohl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohl', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5e6j_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5e6j', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ohn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohn', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 3, 'entry_entity': '5edv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5edv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mm3', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5ohv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ohv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mjb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mjb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mur', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 2, 'entry_entity': '2mbh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '4s1z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s1z', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4wur_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wur', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xyz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xyz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5tut_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5tut', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 1, 'entry_entity': '5h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5h7s', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4xof_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xof', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4wzp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wzp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '5kyd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3v', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mcn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mcn', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1y_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1y', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5lrx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrx', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4xol_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xol', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4y1h_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4y1h', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'D', 'A', 'B', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4xkh_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4r62_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4r62', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4s22_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4s22', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4xok_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xok', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rf1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4rf1', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5txk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5txk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['h', 'c', 'X', 'm'], 'entity_id': 10, 'entry_entity': '4zux_10', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zux', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5mn9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mn9', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ulk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3kw5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3kw5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7u', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ium_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ium', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xk5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kox', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7w_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ig7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ig7', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ldt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ldt', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'B', 'J', 'F'], 'entity_id': 2, 'entry_entity': '4dhj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4dhj', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4hxd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4hxd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4i6n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4i6n', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4k7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k7s', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'E'], 'entity_id': 2, 'entry_entity': '3tbl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tbl', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ljo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ljo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'D', 'M', 'Q', 'N', 'O', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddg', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3prp_2', 'molecule_name': ['60S ribosomal protein L40'], 'pdb_id': '3prp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lz6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lz6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '3vuy_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuy', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vux_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vux', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3pt2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3pt2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5kye_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kye', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5lrw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3u_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3u', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5l8w_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l8w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mj5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5lrv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5lrv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jp3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jp3', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rf0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4rf0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyc', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4un2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4un2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5kyf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kyf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5l8h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5l8h', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'O'], 'entity_id': 8, 'entry_entity': '5kgf_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5kgf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3kvf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3kvf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mor_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mor', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mbq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mbq', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3tmp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3tmp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ugb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ugb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'E'], 'entity_id': 2, 'entry_entity': '4bos_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bos', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4jqw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4jqw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4m0w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4m0w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0t_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0t', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3vht_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vht', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '4bbn_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4bbn', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2qho_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2qho', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3m3j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3m3j', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vnz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vnz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nvg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nvg', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5vo0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vo0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vzm_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzm', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4ddi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ddi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C', 'E'], 'entity_id': 2, 'entry_entity': '4boz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4boz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4uel_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4uel', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ifr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ifr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5jqs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jqs', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5nmc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5nmc', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z9s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4z9s', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5ait_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ait', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5caw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5caw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3m', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wvo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['9'], 'entity_id': 9, 'entry_entity': '5a5b_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5a5b', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '5ulf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ulf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5bnb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bnb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5aiu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5aiu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujn', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5khy_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5khy', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jtv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mro_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mro', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6w9s_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9s', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'F'], 'entity_id': 2, 'entry_entity': '5yik_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yik', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6p5b_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6p5b', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7m', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6ums_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ums', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6fx4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fx4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 1, 'entry_entity': '6ump_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ump', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '6ulh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ulh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6tnf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tnf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 15, 'entry_entity': '6tbm_15', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tbm', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 12, 'entry_entity': '6t9l_12', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6t9l', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6v5d_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6v5d', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6asr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6asr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '6vae_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vae', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 7, 'entry_entity': '6ttu_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ttu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 9, 'entry_entity': '6kiw_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k9p', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6txb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6txb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6h4h_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6h4h', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kox_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kox', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kow_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kow', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6e2b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6e2b', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6ei1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ei1', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6glc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6glc', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6kl4_3', 'molecule_name': ['Ub'], 'pdb_id': '6kl4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 9, 'entry_entity': '6kiv_9', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '6aqr_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6aqr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6vaf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6vaf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O'], 'entity_id': 11, 'entry_entity': '6kiu_11', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kiu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5cra_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cra', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3o_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3o', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 3, 'entry_entity': '5d0k_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0k', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5ujl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ujl', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5xiu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xiu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5x3n', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6xaa_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6xaa', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oam_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oam', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'L'], 'entity_id': 8, 'entry_entity': '7bwd_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bwd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 1, 'entry_entity': '6jwi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jwi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6fdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fdk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6cp2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6cp2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6k2u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6k2u', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6q00_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q00', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oi4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oi4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'F'], 'entity_id': 1, 'entry_entity': '6wkr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wkr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'W', 'R', 'U', 'P', 'M', 'Q', 'T', 'X', 'N', 'O', 'S'], 'entity_id': 1, 'entry_entity': '6w9r_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9r', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6sqs_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqs', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '6s53_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6s53', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6fyh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fyh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'O'], 'entity_id': 8, 'entry_entity': '6ftx_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ftx', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djx', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q'], 'entity_id': 11, 'entry_entity': '6uh5_11', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uh5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5zbu_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zbu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F', 'L'], 'entity_id': 3, 'entry_entity': '6sqr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6sqr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6lp2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6lp2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6fge_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6fge', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5zq3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5zq3', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n13', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6nxk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nxk', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '7bu0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bu0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5v1z_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5v1z', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xbo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5bz0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5bz0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5dfl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5dfl', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 3, 'entry_entity': '5d0m_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5d0m', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5n38_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5n38', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6wtg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6wtg', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5n2w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5n2w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '7bxg_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bxg', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '6o96_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6o96', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7cap_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7cap', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6jb7_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jb7', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 7, 'entry_entity': '6jma_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6jma', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6q01_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6q01', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6pgv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6pgv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '6pzv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6pzv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '5yij_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yij', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '6b7o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6b7o', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '6w9d_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6w9d', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'L'], 'entity_id': 2, 'entry_entity': '6tuv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6tuv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6rya_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6rya', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4wlr_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4wlr', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '5yt6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5yt6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xk5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xk5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'C', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '6z7v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6z7v', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6if1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6if1', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gzs_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6gzs', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wwz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h7s_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7s', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ogw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ogw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1xd3_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xd3', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubi_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '1wr6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wr6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ubq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1ubq', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1tbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1tbe', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v80_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v80', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3n32_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n32', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2rr9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2rr9', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3n30_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3n30', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '3mhs_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3mhs', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3nhe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nhe', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wx1_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx1', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3c0r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3c0r', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1f9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1f9j', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h1u_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h1u', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pea_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pea', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2dx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2dx5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ehv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ehv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ibi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ibi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2g45_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2g45', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2k6d_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k6d', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3eec_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3eec', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '2gmi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2gmi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3efu_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3efu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kde_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kde', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2kdf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kdf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pe9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2pe9', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2oob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2oob', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3hm3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3hm3', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3i3t_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3i3t', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hth_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hth', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2jri_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jri', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jy6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jy6', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2fif_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fif', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8c_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8c', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kjh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kjh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3a33_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a33', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k39_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k39', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k8b_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2k8b', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jzz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jzz', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2z59_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2z59', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2ayo_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ayo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3alb_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3alb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ofi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ofi', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ihp_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ihp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ns8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ns8', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 1, 'entry_entity': '1yd8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yd8', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3aul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3aul', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'U'], 'entity_id': 2, 'entry_entity': '1p3q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1p3q', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1g6j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1g6j', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1uzx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1uzx', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1wrd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1wrd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v81_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1v81', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3nob_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3nob', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3o65_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3o65', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2zcc_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zcc', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1q5w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1q5w', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1s1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1s1q', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fxt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1fxt', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3by4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3by4', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1aar_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1aar', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1cmx_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1cmx', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '1nbf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1nbf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'J', 'F', 'L'], 'entity_id': 2, 'entry_entity': '2c7n_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7n', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2c7m_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2c7m', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2j7q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2j7q', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d3g_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2d3g', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2wdt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wdt', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lj5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wx0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2wx0', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvq', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2xbb_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xbb', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2bgf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2bgf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fid_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fid', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ifw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ifw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2fuh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2fuh', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2hd5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2hd5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xqq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1xqq', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1d3z', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2jf5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jf5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2den_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2den', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l0f_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l0f', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l3z_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2l3z', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3vuw_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vuw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld9_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ld9', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2lvp_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2lvo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2kwv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kwv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3prm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3prm', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2klg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2klg', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '3phd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3phw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3phw', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ktf_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ktf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kn5_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2kn5', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2xew_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2xew', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hek_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hek', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6hei_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hei', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '7jms_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jms', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6isu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6isu', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '6kfp_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kfp', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 7, 'entry_entity': '6j99_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6j99', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'C', 'F'], 'entity_id': 3, 'entry_entity': '7bbf_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbf', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7mic_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7mic', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'B', 'J', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '7mc9_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7mc9', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '7ay2_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay2', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 3, 'entry_entity': '7b5n_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5n', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 6, 'entry_entity': '7b5m_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5m', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 6, 'entry_entity': '7b5l_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7b5l', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7jxv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7jxv', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '7k6q_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k6q', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '7k6p_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7k6p', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '7bbd_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7bbd', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7keo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7keo', 'e_value': 1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '4v3k_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3k', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msm', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4v3l_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4v3l', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '4xkl_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4xkl', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'F', 'G', 'L'], 'entity_id': 3, 'entry_entity': '4whv_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4whv', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9p', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '5mnj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5mnj', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4zfr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zfr', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3rul_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3rul', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 3, 'entry_entity': '4lcd_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4lcd', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4auq_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4auq', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ksk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksk', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5vzw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5vzw', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4msq_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4msq', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4pqt_2', 'molecule_name': ['Protein UBBP4'], 'pdb_id': '4pqt', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4k1r_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4k1r', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['U'], 'entity_id': 2, 'entry_entity': '5ln1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ln1', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mre_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mre', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'D', 'L'], 'entity_id': 4, 'entry_entity': '3zni_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zni', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4mdk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4mdk', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4zft_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zft', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6dc6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6dc6', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6hpr_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6hpr', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '6ven_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ven', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6ypt_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ypt', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '6jky_3', 'molecule_name': ['Ub'], 'pdb_id': '6jky', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '6nn6_8', 'molecule_name': ['Polyubiquitin-C'], 'pdb_id': '6nn6', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 7, 'entry_entity': '6nqa_7', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nqa', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qf8_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qf8', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['E'], 'entity_id': 3, 'entry_entity': '5udh_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5udh', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'L'], 'entity_id': 8, 'entry_entity': '6nj9_8', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6nj9', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 5, 'entry_entity': '6nog_5', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '6nog', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'K', 'D', 'B', 'C', 'J', 'N', 'O', 'F', 'G', 'L'], 'entity_id': 2, 'entry_entity': '6upu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6upu', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6kg6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kg6', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jvz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jvz', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'J', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '4nqk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nqk', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4fjv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4fjv', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 3, 'entry_entity': '3jw0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jw0', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'U', 'T', 'X', 'S'], 'entity_id': 3, 'entry_entity': '4p4h_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4p4h', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2khw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2khw', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3oj3_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj3', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3oj4_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3oj4', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3vfk_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vfk', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '4ap4_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ap4', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3ptf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ptf', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai1', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ai0_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ai0', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 8, 'entry_entity': '7e8i_8', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '7e8i', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '7ay1_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ay1', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 6, 'entry_entity': '7lyc_6', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7lyc', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['III', 'LLL', 'CCC', 'FFF'], 'entity_id': 3, 'entry_entity': '7ahz_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7ahz', 'e_value': 1.1e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3k9o_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3k9o', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 3, 'entry_entity': '5cvn_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvn', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5cvm_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvm', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5cvo_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5cvo', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx5', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1yx6_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1yx6', 'e_value': 1.3e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3vdz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3vdz', 'e_value': 1.5e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojr_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2ojr', 'e_value': 1.5e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5emz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5emz', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['H', 'K', 'B', 'E'], 'entity_id': 2, 'entry_entity': '5ydk_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ydk', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gvi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3zlz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3zlz', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3wxg_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxg', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '3wwq_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wwq', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2n13_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n13', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 3, 'entry_entity': '5o44_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5o44', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5af4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af4', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '6a6i_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6a6i', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msb_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msb', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'F'], 'entity_id': 1, 'entry_entity': '5xit_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xit', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6uyi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyi', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6oq2_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq2', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ud0_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6ud0', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '5af6_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5af6', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['u', 'w'], 'entity_id': 19, 'entry_entity': '6msd_19', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6msd', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '5xis_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5xis', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5ymy_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5ymy', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '6oq1_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6oq1', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6uyj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6uyj', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['R'], 'entity_id': 24, 'entry_entity': '6lqm_24', 'molecule_name': ['Ubiquitin-60S ribosomal protein L40'], 'pdb_id': '6lqm', 'e_value': 1.7e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4nql_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4nql', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3jsv_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3jsv', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '3a1q_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a1q', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2o6v_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2o6v', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a9j_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9j', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['V', 'Y'], 'entity_id': 4, 'entry_entity': '3dvn_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvn', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['Y'], 'entity_id': 4, 'entry_entity': '3dvg_4', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3dvg', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '2znv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2znv', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1zgu_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1zgu', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3h7p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3h7p', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a9k_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3a9k', 'e_value': 1.7e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5oe7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5oe7', 'e_value': 1.9e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3wxf_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxf', 'e_value': 1.9e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3wxe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3wxe', 'e_value': 1.9e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 15, 'entry_entity': '7k5i_15', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '7k5i', 'e_value': 1.9e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '5flx_33', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '5flx', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5goj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goj', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gog_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gog', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5goi_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goi', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5go7_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go7', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gob_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gob', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5j8p_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5j8p', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4zqs_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4zqs', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goh_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goh', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5go8_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5go8', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '5god_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5god', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '5goc_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5goc', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5gok_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gok', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3znz_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3znz', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzy_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzy', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzz_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzz', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '4d61_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4d61', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'H', 'R', 'P', 'D', 'C', 'J', 'T', 'X', 'N', 'F', 'L'], 'entity_id': 2, 'entry_entity': '4ksl_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4ksl', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jbv_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jbv', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '4kzx_32', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4kzx', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 33, 'entry_entity': '4d5l_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '4d5l', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '5jby_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jby', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 34, 'entry_entity': '6p4h_34', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p4h', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p4g_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p4g', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 2, 'entry_entity': '6kbe_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6kbe', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['k'], 'entity_id': 16, 'entry_entity': '6ybs_16', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ybs', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6njd_1', 'molecule_name': ['Polyubiquitin-C'], 'pdb_id': '6njd', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6g5i_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6g5i', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 11, 'entry_entity': '6g18_11', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6g18', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6qml_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6qml', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['B', 'A'], 'entity_id': 1, 'entry_entity': '5wq4_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5wq4', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6n5m_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n5m', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '5xdp_1', 'molecule_name': ['D-ubiquitin'], 'pdb_id': '5xdp', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p5j_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p5j', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 33, 'entry_entity': '6p5i_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6p5i', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6n6r_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6n6r', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 32, 'entry_entity': '6zlw_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zlw', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6sgc_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6sgc', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '2zvn_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvn', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2w9n_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2w9n', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2fcq_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '2fcq', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '2zvo_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2zvo', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '3b08_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b08', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1yiw_1', 'molecule_name': ['Ubiquitin-like domain-containing protein'], 'pdb_id': '1yiw', 'e_value': 2e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '3b0a_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3b0a', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3axc_1', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '3axc', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '2y5b_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2y5b', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3R'], 'entity_id': 80, 'entry_entity': '6ip5_80', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ip5', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 31, 'entry_entity': '6zmt_31', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zmt', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6hcf_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6hcf', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3R'], 'entity_id': 79, 'entry_entity': '6ip8_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ip8', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zv6_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zv6', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g1'], 'entity_id': 33, 'entry_entity': '6hcm_33', 'molecule_name': ['40S ribosomal protein S27a'], 'pdb_id': '6hcm', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['3R'], 'entity_id': 79, 'entry_entity': '6ip6_79', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6ip6', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['f'], 'entity_id': 32, 'entry_entity': '6zuo_32', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '6zuo', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['AH'], 'entity_id': 52, 'entry_entity': '5t2c_52', 'molecule_name': ['Ubiquitin-40S ribosomal protein S27a'], 'pdb_id': '5t2c', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '7eal_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '7eal', 'e_value': 2e-26, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLFDKLKSLVSDF': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mp0_2', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '2mp0', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'M', 'N', 'O'], 'entity_id': 4, 'entry_entity': '4jbw_4', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '4jbw', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2f3g_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '2f3g', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1gld_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1gld', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1gle_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1gle', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o2f_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1o2f', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1glb_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1glb', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1gla_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1gla', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ggr_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1ggr', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1glc_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1glc', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '3our_2', 'molecule_name': ['PTS EIIA type-1 domain-containing protein'], 'pdb_id': '3our', 'e_value': 0.071, 'percentage_identity': 91.7, 'result_sequence': None}], 'FKCRAWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 7.9e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00088, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00088, 'percentage_identity': 80.0, 'result_sequence': None}], 'GWKKWFNRAKKVGKTVGGLAVDHYLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 0.0059, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 0.0061, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.0076, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.078, 'percentage_identity': 44.0, 'result_sequence': None}], 'GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 3.7e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 8.8e-10, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.021, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.021, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.039, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.076, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.076, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.076, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.076, 'percentage_identity': 36.4, 'result_sequence': None}], 'FKCRRWQWRMAKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 2.4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 6.9e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00036, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00036, 'percentage_identity': 80.0, 'result_sequence': None}], 'GIMDTLKNLAKTAGKGALQSLVKMASCKLSGQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 0.00019, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wpo_1', 'molecule_name': ['Hylaseptin-4'], 'pdb_id': '6wpo', 'e_value': 0.035, 'percentage_identity': 66.7, 'result_sequence': None}], 'GIMDTLKNLAKTAGKGALQSLLNKASCKLSGQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 4.8e-05, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wpo_1', 'molecule_name': ['Hylaseptin-4'], 'pdb_id': '6wpo', 'e_value': 0.034, 'percentage_identity': 66.7, 'result_sequence': None}], 'RKCLRWQWAMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0025, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.059, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.059, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.059, 'percentage_identity': 69.2, 'result_sequence': None}], 'GIGALSAKGALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 0.046, 'percentage_identity': 70.4, 'result_sequence': None}], 'RQWQSKIRRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 0.0024, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 0.0024, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRWCFRVCYRGFCYRKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 2.7e-09, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0043, 'percentage_identity': 62.5, 'result_sequence': None}], 'FACRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 3.5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 8.8e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00045, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00045, 'percentage_identity': 92.3, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTGLPALIS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.021, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.021, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.03, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.052, 'percentage_identity': 93.3, 'result_sequence': None}], 'FLSLIPHAINAVSALANHG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 7.7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 1.9e-06, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 1.2e-05, 'percentage_identity': 83.3, 'result_sequence': None}], 'GVIPCGESCVFIPCISTLLGCSCKNKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 5.6e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 6.8e-13, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 7e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 7e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 3.5e-12, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 4.5e-12, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 5.7e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.3e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.3e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 3.1e-07, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 1.9e-06, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 9.6e-05, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.00061, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.0031, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.0049, 'percentage_identity': 38.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 0.0062, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.0062, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.0072, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.0099, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.012, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.012, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.012, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.013, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.013, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.016, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.016, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.02, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.025, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.031, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.031, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.034, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.04, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 0.05, 'percentage_identity': 35.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.079, 'percentage_identity': 44.4, 'result_sequence': None}], 'TKCFQWQWNMRKVRG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 5.8e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 5.8e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 4.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 4.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fck_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1fck', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cb6_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1cb6', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfh_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfh', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 8.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfd_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfd', 'e_value': 9.5e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h45_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h45', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1eh3_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1eh3', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vfe_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1vfe', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h43_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h43', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hse_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1hse', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dsn_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dsn', 'e_value': 9.6e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pms_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2pms', 'e_value': 9.9e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sqy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1sqy', 'e_value': 0.0002, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2bjj_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2bjj', 'e_value': 0.0002, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n76_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1n76', 'e_value': 0.0002, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 0.016, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.067, 'percentage_identity': 58.3, 'result_sequence': None}], 'FKCRRWAWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.1e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00062, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00062, 'percentage_identity': 80.0, 'result_sequence': None}], 'GLLSAVKGVLKGAGKNVAGSLMDKLKCKLFGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 1.1e-05, 'percentage_identity': 54.5, 'result_sequence': None}], 'FLSLIPHAINAVSAIAKHFG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 8.4e-06, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 1.5e-05, 'percentage_identity': 78.9, 'result_sequence': None}], 'KWKSFIKKLTSKALHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.036, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.036, 'percentage_identity': 65.0, 'result_sequence': None}], 'GLICESCRKIIQKLEDMVGPQPNEDTVTQAASRVCDKMKILRGVCKKIMRTFLRRISKDILTGKKPQAICVDIKICKE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 2.6e-34, 'percentage_identity': 91.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l9l_1', 'molecule_name': ['Granulysin'], 'pdb_id': '1l9l', 'e_value': 2.2e-07, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ddj_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '4ddj', 'e_value': 0.0041, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gtg_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2gtg', 'e_value': 0.0041, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dob_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '2dob', 'e_value': 0.0041, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uex_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '4uex', 'e_value': 0.0042, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn6_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1sn6', 'e_value': 0.0042, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m12_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1m12', 'e_value': 0.0042, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2z9a_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2z9a', 'e_value': 0.0044, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2qyp_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2qyp', 'e_value': 0.0045, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'G', 'F'], 'entity_id': 1, 'entry_entity': '6vyn_1', 'molecule_name': ['Pulmonary surfactant-associated protein B'], 'pdb_id': '6vyn', 'e_value': 0.0052, 'percentage_identity': 21.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '6vzd_1', 'molecule_name': ['Pulmonary surfactant-associated protein B'], 'pdb_id': '6vzd', 'e_value': 0.014, 'percentage_identity': 19.7, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6w1b_1', 'molecule_name': ['Pulmonary surfactant-associated protein B'], 'pdb_id': '6w1b', 'e_value': 0.025, 'percentage_identity': 21.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1of9_1', 'molecule_name': ['Pore-forming peptide ameobapore A'], 'pdb_id': '1of9', 'e_value': 0.085, 'percentage_identity': 23.7, 'result_sequence': None}], 'GLPVCGETCVGGTCNTPGCTCSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 1.8e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.8e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 5.1e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 5.1e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 7.7e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 7.7e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 9.5e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 2.2e-13, 'percentage_identity': 96.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 5.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 5.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 5.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 6.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 3.3e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 1.5e-10, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 1.9e-10, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 2.3e-10, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 2.3e-10, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 2.8e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 2.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 1.4e-09, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 3.3e-09, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 1.4e-08, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 1.7e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 2.1e-08, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 3.1e-08, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 4.7e-08, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 3.7e-07, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 2.4e-06, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 1.8e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.00032, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 0.00046, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.0017, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.0044, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 0.019, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 0.019, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 0.081, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 0.099, 'percentage_identity': 48.1, 'result_sequence': None}], 'KQATVGDINTERPGILDLKGKAKWDAWNGLKGTSKEDAMKAYINKVEELKKKYGI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2abd_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2abd', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1hb8_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1hb8', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nti_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1nti', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hb6_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1hb6', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aca_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1aca', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nvl_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1nvl', 'e_value': 1.1e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fj9_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2fj9', 'e_value': 1.6e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cb8_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2cb8', 'e_value': 1.6e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2fdq_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2fdq', 'e_value': 1.9e-21, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3epy_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 7'], 'pdb_id': '3epy', 'e_value': 7.3e-14, 'percentage_identity': 69.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3fp5_1', 'molecule_name': ['ACB domain-containing protein'], 'pdb_id': '3fp5', 'e_value': 5.1e-12, 'percentage_identity': 59.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h3g_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 1'], 'pdb_id': '5h3g', 'e_value': 3.8e-11, 'percentage_identity': 61.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1st7_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1st7', 'e_value': 1.1e-10, 'percentage_identity': 56.6, 'result_sequence': None}, {'chain_id': ['I', 'K', 'D', 'C', 'A', 'F'], 'entity_id': 1, 'entry_entity': '6kf8_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 2'], 'pdb_id': '6kf8', 'e_value': 3.8e-10, 'percentage_identity': 59.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5h3i_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 2'], 'pdb_id': '5h3i', 'e_value': 3.9e-10, 'percentage_identity': 59.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lbb_1', 'molecule_name': ['ACB domain-containing protein'], 'pdb_id': '2lbb', 'e_value': 1.9e-08, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ijm_1', 'molecule_name': ['ACB domain-containing protein'], 'pdb_id': '5ijm', 'e_value': 2.1e-07, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2wh5_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 4'], 'pdb_id': '2wh5', 'e_value': 4.6e-07, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cqu_1', 'molecule_name': ['Enoyl-CoA delta isomerase 2'], 'pdb_id': '2cqu', 'e_value': 5.1e-07, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3flv_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 5'], 'pdb_id': '3flv', 'e_value': 1.7e-06, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cop_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 6'], 'pdb_id': '2cop', 'e_value': 3.3e-06, 'percentage_identity': 44.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hbk_1', 'molecule_name': ['ACB domain-containing protein'], 'pdb_id': '1hbk', 'e_value': 0.0029, 'percentage_identity': 32.7, 'result_sequence': None}], 'RKCLRWQWAMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0062, 'percentage_identity': 69.2, 'result_sequence': None}], 'ARLKKCFNKVTGYCRKKCKVGERYEIGCLSGKLCCAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 8.6e-05, 'percentage_identity': 57.1, 'result_sequence': None}], 'FKCRRWQWR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0032, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTGLIALIS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.00068, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.00068, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0007, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0007, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0007, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0007, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.013, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.013, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.019, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.033, 'percentage_identity': 93.3, 'result_sequence': None}], 'DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 3.9e-10, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.00014, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.00014, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.00014, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.00014, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.00023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00029, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00033, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.00035, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.00035, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.00053, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.00053, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.00053, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.00053, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.00053, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.00065, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.00065, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.0008, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.00099, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.0015, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.0023, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.0028, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0051, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 0.0052, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.0056, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg6_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg6', 'e_value': 0.0079, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4t_1', 'molecule_name': ['Beta-defensin 7'], 'pdb_id': '1e4t', 'e_value': 0.023, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 0.088, 'percentage_identity': 39.5, 'result_sequence': None}], 'DAECEICKFVIQQVEAFIESNHSQAEIQKELNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gtg_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2gtg', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn6_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1sn6', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m12_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1m12', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2z9a_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2z9a', 'e_value': 0.0021, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2qyp_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2qyp', 'e_value': 0.0022, 'percentage_identity': 32.4, 'result_sequence': None}], 'AKIPIKAIKTVGKAVGKGLRAINIASTANDVFNFLKPKKRKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 2.4e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 1e-11, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 8.7e-11, 'percentage_identity': 67.5, 'result_sequence': None}], 'GPLSCRRNGGVCIPIRCPGPMRQIGTCFGRPVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 2.6e-18, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 2.1e-06, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 7.6e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 8.2e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 8.2e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 8.2e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 8.2e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 8.3e-05, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.0043, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.0062, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4t_1', 'molecule_name': ['Beta-defensin 7'], 'pdb_id': '1e4t', 'e_value': 0.0066, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.027, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.027, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.029, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.033, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.05, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.062, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.064, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.076, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.093, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.093, 'percentage_identity': 36.4, 'result_sequence': None}], 'EGVRNFVTCRINRGFCVPIRCPGHRRQIGTCLGPRIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 7.6e-08, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 3.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 3.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 3.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 3.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 5.9e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00014, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0024, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.016, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.064, 'percentage_identity': 40.6, 'result_sequence': None}], 'KNLRRITRKIIHIIKKYG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.026, 'percentage_identity': 61.1, 'result_sequence': None}], 'FFHHIFRGIVHVGRTIHKLVTGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 5.8e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 5.8e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 8.7e-09, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 4.5e-08, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 3.5e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 3.5e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 3.5e-06, 'percentage_identity': 68.2, 'result_sequence': None}], 'FFHHIFRGIVHVGRTIHRLVTGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 2.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 2.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 3.6e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.8e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.043, 'percentage_identity': 50.0, 'result_sequence': None}]}

In [2]:
%store r17 

Stored 'r17' (dict)


In [3]:
len(r17)

129

## 6000 - 6169

In [1]:
r18={'GSKKPVPIIYCNRRKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 3.3e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 3.3e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 3.3e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 3.3e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 3.3e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 2.3e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 2.3e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 2.3e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 3.8e-06, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 3.8e-06, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xol_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xol', 'e_value': 5e-06, 'percentage_identity': 81.0, 'result_sequence': None}], 'KWKLFKKILLKFLHLAKLF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.00038, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.00038, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.075, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.075, 'percentage_identity': 70.0, 'result_sequence': None}], 'CKQSCSFGPFTFVCDGNTK': [{'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6c0h_2', 'molecule_name': ['Gln-dal-cys-ala-phe-gly-pro-phe-dbb-phe-val-cys-bh2-gly'], 'pdb_id': '6c0h', 'e_value': 1.4e-08, 'percentage_identity': 89.5, 'result_sequence': None}], 'QGVRSYLSCWGNRGICLLNRCPGRMRQIGTCLAPRVKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.2e-07, 'percentage_identity': 61.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 8.4e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 8.4e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 8.4e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 8.4e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00029, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.0015, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0016, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0019, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.005, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.0073, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.0073, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.011, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.014, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.017, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.022, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.022, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4t_1', 'molecule_name': ['Beta-defensin 7'], 'pdb_id': '1e4t', 'e_value': 0.022, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.033, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.051, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.064, 'percentage_identity': 31.4, 'result_sequence': None}], 'AERVGAGAPVYL': [{'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 0.00075, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 0.00076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 0.00076, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '7bp6_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp6', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '7bp2_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp2', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'G'], 'entity_id': 1, 'entry_entity': '7bp4_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp4', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7bp5_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7bp5', 'e_value': 0.00077, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 0.00079, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 0.00087, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 0.00089, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5mlu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5mlu', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6mup_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6mup', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6muo_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6muo', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 11, 'entry_entity': '6z6p_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6z6p', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ktq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ktq', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ea5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ea5', 'e_value': 0.0009, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kwq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kwq', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xzq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xzq', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z66_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4z66', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'K', 'C', 'O', 'G', 'S'], 'entity_id': 2, 'entry_entity': '5g2e_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5g2e', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ys3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ys3', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6jm9_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jm9', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uh5_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uh5', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ugm_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6ugm', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6q_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6q', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k6p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7k6p', 'e_value': 0.00092, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq8_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq8', 'e_value': 0.00095, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq6', 'e_value': 0.00095, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6fq5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fq5', 'e_value': 0.00095, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1t_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1t', 'e_value': 0.00097, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy2', 'e_value': 0.00099, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iy3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iy3', 'e_value': 0.00099, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6jma_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jma', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1aoi_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1aoi', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4qlc_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4qlc', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2nqb_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '2nqb', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgp_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgp', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwe_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwe', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pwf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6pwf', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6dzt_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6dzt', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1s32_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1s32', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz1_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz1', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3mnn', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgr_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgr', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgs_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgs', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lz0_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lz0', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2pyo_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '2pyo', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mgq_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mgq', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2nzd_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2nzd', 'e_value': 0.0011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnn_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnn', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['Q', 'M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5oxv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oxv', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 3, 'entry_entity': '5hq2_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5hq2', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5e5a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5e5a', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5omx_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5omx', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5onw_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5onw', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5o9g_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5o9g', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['W', 'e', 'K', 'a', 'C', 'O', 'G', 'S'], 'entity_id': 4, 'entry_entity': '5oy7_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5oy7', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4wu9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4xuj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4xuj', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '4wu8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4wu8', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '4zux_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4zux', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rei_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rei', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8v_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8v', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8x', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rek_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rek', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8w_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8w', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rej_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rej', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3uta_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3uta', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ut9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3ut9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3rel_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3rel', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3reh_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3reh', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4j8u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4j8u', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kxb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3kxb', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4r8p_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4r8p', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5f99_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5f99', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3tu4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3tu4', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kgc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kgc', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ld9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '4ld9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5ong_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5ong', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5dnm_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5dnm', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3utb_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3utb', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0y_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0y', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5xf6', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5kgf_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5kgf', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3u', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esg_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esg', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esi_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esi', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryu', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'X'], 'entity_id': 3, 'entry_entity': '6uxw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6uxw', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tda_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tda', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ryr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ryr', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t9l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6t9l', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiw_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiw', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kix_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kix', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiv_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiv', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiu_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiu', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw3', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ven_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ven', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kiz_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kiz', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x59_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x59', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '5cp6_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5cp6', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5nl0_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5nl0', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x0x_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5x0x', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jyl_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6jyl', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'K'], 'entity_id': 11, 'entry_entity': '6wkr_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wkr', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5m_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5m', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nog_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nog', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ftx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ftx', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6g0l_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6g0l', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6s01_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6s01', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ltj_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ltj', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ne3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6ne3', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nn6_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nn6', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6xjd_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6xjd', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6nqa_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nqa', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6vyp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vyp', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r1u_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r1u', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6wz5_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6wz5', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6x5a_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x5a', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nj9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6nj9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o96_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6o96', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B', 'E'], 'entity_id': 2, 'entry_entity': '6n1z_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6n1z', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6x0n_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6x0n', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3l_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3l', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esh_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esh', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1p_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k1p', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6rny_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6rny', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6c0w_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6c0w', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r0c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r0c', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5z3v_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '5z3v', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z3o_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '5z3o', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r25_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6r25', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5i_7', 'molecule_name': ['Histone H2A'], 'pdb_id': '6w5i', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6pa7_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pa7', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pww_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pww', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6om3_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6om3', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6vz4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6vz4', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 6, 'entry_entity': '6pwv_6', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwv', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 11, 'entry_entity': '6fml_11', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6fml', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6upk_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upk', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['K', 'C'], 'entity_id': 3, 'entry_entity': '6upl_3', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '6upl', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 4, 'entry_entity': '6pwx_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6pwx', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6esf_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6esf', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6tem_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6tem', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['T', 'O'], 'entity_id': 17, 'entry_entity': '6kw5_17', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw5', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'M'], 'entity_id': 7, 'entry_entity': '6w5n_7', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6w5n', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6zhy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhy', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6iro_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6iro', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7enn_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7enn', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 22, 'entry_entity': '7nky_22', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7nky', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1tzy_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1tzy', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3mvd_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3mvd', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '1hq3_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hq3', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3lja_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3lja', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '3lel_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '3lel', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '2fj7_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '2fj7', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2hio', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3o62_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3o62', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1zla_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zla', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6f_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6f', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3f_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3f', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '1kx5_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx5', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx3_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx3', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3l_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3l', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3c9k_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '3c9k', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3m_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3m', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 5, 'entry_entity': '1zbb_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1zbb', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '2aro_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2aro', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '3b6g_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3b6g', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1c_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1c', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3c1b_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3c1b', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1kx4_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1kx4', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m19_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m19', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3a', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3i_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3i', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3k_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3k', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p34_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p34', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m18_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m18', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1m1a_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1m1a', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3o_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3o', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3b_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3b', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3p_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3p', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1p3g_4', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '1p3g', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 2, 'entry_entity': '1eqz_2', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1eqz', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['V', 'Q'], 'entity_id': 2, 'entry_entity': '6i84_2', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6i84', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['O', 'S'], 'entity_id': 3, 'entry_entity': '6kw4_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6kw4', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6j99_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6j99', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohb_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohb', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['J', 'F'], 'entity_id': 5, 'entry_entity': '7at8_5', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7at8', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['b', 'f'], 'entity_id': 4, 'entry_entity': '7a08_4', 'molecule_name': ['Histone H2A type 1-C'], 'pdb_id': '7a08', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7cro_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7cro', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6zhx_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '6zhx', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crp_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crp', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7jo9_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7jo9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7joa_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7joa', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oha_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oha', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 4, 'entry_entity': '7bg9_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '7bg9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crr_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crr', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ohc_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7ohc', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7crq_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '7crq', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '7e8i_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7e8i', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7oh9_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '7oh9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1j_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1j', 'e_value': 0.0014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1k_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1k', 'e_value': 0.0014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6k1i_3', 'molecule_name': ['Histone H2AX'], 'pdb_id': '6k1i', 'e_value': 0.0014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt0_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gt0', 'e_value': 0.0017, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gsu_3', 'molecule_name': ['Histone H2A type 1-A'], 'pdb_id': '5gsu', 'e_value': 0.0017, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6w4l_1', 'molecule_name': ['Histone H2B 1.1', 'Histone H2A type 1'], 'pdb_id': '6w4l', 'e_value': 0.0019, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 6, 'entry_entity': '6y5e_6', 'molecule_name': ['Histone H2A type 2-C'], 'pdb_id': '6y5e', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6seg_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6seg', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6se6_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se6', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['G'], 'entity_id': 5, 'entry_entity': '6se0_5', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6se0', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'O', 'G', 'S'], 'entity_id': 3, 'entry_entity': '6y5d_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6y5d', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px1_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px1', 'e_value': 0.0024, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6see_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6see', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6sef_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '6sef', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6nzo_3', 'molecule_name': ['Ubiquitin', 'Histone H2A type 1'], 'pdb_id': '6nzo', 'e_value': 0.0024, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6px3_3', 'molecule_name': ['Histone H2A type 1', 'Ubiquitin'], 'pdb_id': '6px3', 'e_value': 0.0024, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['N', 'n'], 'entity_id': 3, 'entry_entity': '7jzv_3', 'molecule_name': ['Histone H2A type 2-A'], 'pdb_id': '7jzv', 'e_value': 0.0024, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '6kbb_1', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '6kbb', 'e_value': 0.0032, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7ccq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccq', 'e_value': 0.0035, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['R', 'C', 'N', 'G'], 'entity_id': 3, 'entry_entity': '7ccr_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7ccr', 'e_value': 0.0035, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7ea8_4', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '7ea8', 'e_value': 0.0035, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6le9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6le9', 'e_value': 0.0036, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6l9h_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9h', 'e_value': 0.0036, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ke9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ke9', 'e_value': 0.0036, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6iq4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6iq4', 'e_value': 0.0036, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '6jxd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jxd', 'e_value': 0.0037, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6ipu_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ipu', 'e_value': 0.0037, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '7lya_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lya', 'e_value': 0.0042, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 9, 'entry_entity': '7lyb_9', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyb', 'e_value': 0.0042, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 8, 'entry_entity': '7lyc_8', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7lyc', 'e_value': 0.0042, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w96_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w96', 'e_value': 0.0044, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gt3_3', 'molecule_name': ['Histone H2A type 1-D'], 'pdb_id': '5gt3', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1v_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1v', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3x1s_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3x1s', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf5', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['k', 'W', 'g', 'a', 'M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la2', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l9z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l9z', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0p_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0p', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 25, 'entry_entity': '6v92_25', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v92', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6e0c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6e0c', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf3_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf3', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xf4_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5xf4', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bwd_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bwd', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6lab_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6lab', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6ler_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ler', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m3v_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m3v', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la8', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6m44_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6m44', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7by0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7by0', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6o1d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6o1d', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6buz_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6buz', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6la9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6la9', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K', 'O'], 'entity_id': 7, 'entry_entity': '6hts_7', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hts', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '2cv5_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2cv5', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7nl0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7nl0', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5x', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '7dbp_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7dbp', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k5y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k5y', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k61_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k61', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k60_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k60', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k63_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7k63', 'e_value': 0.0047, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '7cow_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7cow', 'e_value': 0.0048, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '5y0c_4', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0c', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm1_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm1', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5y0d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5y0d', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gxq_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gxq', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5gtc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gtc', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym5', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5gse_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5gse', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w97_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w97', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wtp_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wtp', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5jrg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5jrg', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5xm0_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5xm0', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3wkj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3wkj', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w98_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w98', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3w99_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3w99', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1l_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1l', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4z5t_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4z5t', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2j_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2j', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0z', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avb_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avb', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b40_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b40', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r94_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r94', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r91_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r91', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6v2k_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6v2k', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l49_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l49', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5o_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5o', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6kxv_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6kxv', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6yov_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6yov', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['q', 'W', 'g', 'c', 'M', 'Q', '0', 'C', 'w', 'G', 'm', '2'], 'entity_id': 3, 'entry_entity': '6hkt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6hkt', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5t_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5t', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5r_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5r', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5l_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5l', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpj', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av5_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av5', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5ay8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5ay8', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b1m_3', 'molecule_name': ['Histone H2A type 1-B'], 'pdb_id': '5b1m', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b24_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b24', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpk', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t93_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t93', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['U', 'M', 'Q', 'Y', 'C', 'G'], 'entity_id': 3, 'entry_entity': '6l4a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6l4a', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '6usj_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6usj', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5zbx_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5zbx', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c', 'C', 'G'], 'entity_id': 3, 'entry_entity': '7c0m_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7c0m', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5x7x_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5x7x', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av6', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4ym6_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '4ym6', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b31_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b31', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av9_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av9', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b0y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b0y', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5avc_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5avc', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5b2i_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b2i', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5b32_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5b32', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5av8_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5av8', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8y_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8y', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr1', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6jr0_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6jr0', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r8z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r8z', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 5, 'entry_entity': '6r93_5', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r93', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6r90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r90', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7bxt_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7bxt', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t90_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t90', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5u_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5u', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 6, 'entry_entity': '6r92_6', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6r92', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5cpi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5cpi', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6ir9_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6ir9', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4y_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4y', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j51_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j51', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6a5p_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6a5p', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '2rvq_1', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '2rvq', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3a6n_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3a6n', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3afa_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3afa', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azj_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azj', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av2', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3ayw_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3ayw', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3aze_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3aze', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azh_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azh', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azn_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azn', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azf_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azf', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3an2_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3an2', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3av1_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3av1', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azk_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azk', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azg_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azg', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azi_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azi', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azl_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azl', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3azm_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '3azm', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4w_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4w', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 21, 'entry_entity': '6j4x_21', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4x', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j50_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j50', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 18, 'entry_entity': '6inq_18', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6inq', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 20, 'entry_entity': '6j4z_20', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6j4z', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d1z_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d1z', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7d20_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '7d20', 'e_value': 0.0049, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['K'], 'entity_id': 7, 'entry_entity': '2f8n_7', 'molecule_name': ['Histone H2A type 1-H'], 'pdb_id': '2f8n', 'e_value': 0.0056, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7c_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7c', 'e_value': 0.0057, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7a_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7a', 'e_value': 0.0057, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t79_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t79', 'e_value': 0.0057, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7b_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7b', 'e_value': 0.0057, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6t7d_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '6t7d', 'e_value': 0.0057, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1t_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1t', 'e_value': 0.0065, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '3x1u_4', 'molecule_name': ['Histone H2A'], 'pdb_id': '3x1u', 'e_value': 0.0065, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vey_1', 'molecule_name': ['Histone H2B type 1-J', 'Histone H2A type 1-B/E'], 'pdb_id': '5vey', 'e_value': 0.0079, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k00_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k00', 'e_value': 0.0086, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 2, 'entry_entity': '6k09_2', 'molecule_name': ['Histone H2B 1', 'Histone H2A'], 'pdb_id': '6k09', 'e_value': 0.0086, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['D'], 'entity_id': 1, 'entry_entity': '6k03_1', 'molecule_name': ['Histone H2A', 'Histone H2B 1'], 'pdb_id': '6k03', 'e_value': 0.0086, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '5z23_3', 'molecule_name': ['Histone H2A type 1-B/E'], 'pdb_id': '5z23', 'e_value': 0.013, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['g'], 'entity_id': 19, 'entry_entity': '6qld_19', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6qld', 'e_value': 0.02, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'G', 'E'], 'entity_id': 1, 'entry_entity': '4wnn_1', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '4wnn', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4jjn_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4jjn', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '4kud_3', 'molecule_name': ['Histone H2A.2'], 'pdb_id': '4kud', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gen_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gen', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['E', 'F'], 'entity_id': 4, 'entry_entity': '6gej_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '6gej', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '1id3_4', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '1id3', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k7g_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k7g', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '7k78_3', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '7k78', 'e_value': 0.027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['g', 'c'], 'entity_id': 5, 'entry_entity': '7on1_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '7on1', 'e_value': 0.028, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '5bt1_2', 'molecule_name': ['Histone H2A.1'], 'pdb_id': '5bt1', 'e_value': 0.03, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6uph_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '6uph', 'e_value': 0.031, 'percentage_identity': 75.0, 'result_sequence': None}], 'GKPRPYLPRPTSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.0025, 'percentage_identity': 76.5, 'result_sequence': None}], 'WGRKKRRQRRRPPQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jfw_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1jfw', 'e_value': 0.00085, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5k_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1k5k', 'e_value': 0.00086, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tac_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tac', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tiv_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tiv', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tbc_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tbc', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}], 'GLLRRLRDFLKKIGEKFKKIGY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.094, 'percentage_identity': 66.7, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCAAEKRRGYKGGYCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 5.8e-20, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 7.2e-20, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 8.8e-20, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 3.1e-19, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 3e-18, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.049, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.049, 'percentage_identity': 42.1, 'result_sequence': None}], 'IGKKFKRIVQRIKDFLRNL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 6.5e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 8.1e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 8.1e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 8.1e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 8.1e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 8.1e-06, 'percentage_identity': 94.7, 'result_sequence': None}], 'GFMDTAKNVAKNVAVTLIDNLKCKITKAC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.014, 'percentage_identity': 53.6, 'result_sequence': None}], 'GPLSCGRNGGVCIPIRCPVPMRQIGTCFGRPVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.5e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 1.5e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 1.7e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 1.7e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 1.7e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 1.7e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 3.3e-05, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00097, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.0092, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0092, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.011, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.017, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.021, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.025, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.027, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.031, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.031, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.046, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.056, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.081, 'percentage_identity': 43.8, 'result_sequence': None}], 'SMWSGMWRRKLKKLRNALKKKLKGEK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pco_1', 'molecule_name': ['M-zodatoxin-Lt1a'], 'pdb_id': '2pco', 'e_value': 1.8e-12, 'percentage_identity': 100.0, 'result_sequence': None}], 'QVVRNPQSCRWNMGVCIPISCPGNMRQIGTCFGPRVPCCRRW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.2e-09, 'percentage_identity': 70.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.00046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.00046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.00046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.00046, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00076, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.001, 'percentage_identity': 42.4, 'result_sequence': None}], 'KLFKKIGIGAVLKVLTTGLPALKLTK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.00033, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.00033, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.00034, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.00034, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.00034, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.00034, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.021, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.037, 'percentage_identity': 93.3, 'result_sequence': None}], 'GIGPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKP': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 1.8e-20, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 1.8e-20, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 1.8e-20, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 1.8e-20, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 1.1e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 1.1e-15, 'percentage_identity': 91.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 5.1e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00032, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.012, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.026, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.061, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.073, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.075, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.075, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.075, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.093, 'percentage_identity': 43.8, 'result_sequence': None}], 'GVRNSQSCRRNKGICVPIRCPGSMRQIGTCLGAQVKCCRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 5.7e-11, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 4e-05, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 4e-05, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 4e-05, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 4e-05, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00016, 'percentage_identity': 41.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00018, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00043, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.011, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.026, 'percentage_identity': 39.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4t_1', 'molecule_name': ['Beta-defensin 7'], 'pdb_id': '1e4t', 'e_value': 0.034, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.034, 'percentage_identity': 45.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.044, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.092, 'percentage_identity': 36.4, 'result_sequence': None}], 'EGVRNFVTCRINRGFCVPIRCPGHRRQIGTCLGPQIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 4.6e-08, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 2.9e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 2.9e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 2.9e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 2.9e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 4.6e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00011, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0024, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.02, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.051, 'percentage_identity': 40.6, 'result_sequence': None}], 'YCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 1.3e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.1e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.0082, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.01, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.016, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.03, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.032, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.032, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.032, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.032, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 0.079, 'percentage_identity': 38.2, 'result_sequence': None}], 'IIGPVLGLVGSALGGLLKKIG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 1.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 1.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GFCCPGNQLKCNNHCKSISCRAGYCDAATLWLRCTCTDCNGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 8.2e-20, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 9.6e-06, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.00015, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.0028, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 0.021, 'percentage_identity': 37.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.03, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.052, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.09, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.09, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.09, 'percentage_identity': 38.5, 'result_sequence': None}], 'RQIKIWFQNRRMKWKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 1.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 1.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 2.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xid_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xid', 'e_value': 2.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 2.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 2.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1san_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1san', 'e_value': 2.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '9ant_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '9ant', 'e_value': 2.6e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hoa_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '2hoa', 'e_value': 2.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1ahd_3', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1ahd', 'e_value': 2.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hom_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '1hom', 'e_value': 2.8e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5z_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5z', 'e_value': 3.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '2r5y_3', 'molecule_name': ['Homeotic protein Sex combs reduced'], 'pdb_id': '2r5y', 'e_value': 3.5e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2h1k_3', 'molecule_name': ['Pancreas/duodenum homeobox protein 1'], 'pdb_id': '2h1k', 'e_value': 1.4e-06, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lp0_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2lp0', 'e_value': 2.8e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msy_1', 'molecule_name': ['Homeobox protein Hox-C9'], 'pdb_id': '2msy', 'e_value': 3.1e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '4uus_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uus', 'e_value': 3.5e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1puf_3', 'molecule_name': ['Homeobox protein Hox-A9'], 'pdb_id': '1puf', 'e_value': 3.5e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b8i_3', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '1b8i', 'e_value': 3.6e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cyc_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4cyc', 'e_value': 4.1e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4uut_1', 'molecule_name': ['Homeotic protein ultrabithorax'], 'pdb_id': '4uut', 'e_value': 4.1e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmt_1', 'molecule_name': ['Homeobox protein BarH-like 1'], 'pdb_id': '2dmt', 'e_value': 4.7e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ftz_1', 'molecule_name': ['Segmentation protein fushi tarazu'], 'pdb_id': '1ftz', 'e_value': 5.5e-05, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m34_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2m34', 'e_value': 5.6e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me0_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me0', 'e_value': 5.6e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2me6_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2me6', 'e_value': 5.6e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjs_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjs', 'e_value': 8.6e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjr_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjr', 'e_value': 8.6e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '5zjt_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjt', 'e_value': 8.6e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zjq_1', 'molecule_name': ['Homeobox protein abdominal-B'], 'pdb_id': '5zjq', 'e_value': 8.6e-05, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e1o_1', 'molecule_name': ['Hematopoietically-expressed homeobox protein HHEX'], 'pdb_id': '2e1o', 'e_value': 9.6e-05, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1b72_3', 'molecule_name': ['Homeobox protein Hox-B1'], 'pdb_id': '1b72', 'e_value': 0.00029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n8g_1', 'molecule_name': ['Homeobox protein GBX-1'], 'pdb_id': '2n8g', 'e_value': 0.00039, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a03_1', 'molecule_name': ['T-cell leukemia homeobox protein 2'], 'pdb_id': '3a03', 'e_value': 0.00055, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lnq_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3lnq', 'e_value': 0.00075, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3a02_1', 'molecule_name': ['Homeobox protein aristaless'], 'pdb_id': '3a02', 'e_value': 0.00078, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3a01_1', 'molecule_name': ['Homeobox domain-containing protein'], 'pdb_id': '3a01', 'e_value': 0.00086, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '5lux_2', 'molecule_name': ['Homeobox protein CDX-1'], 'pdb_id': '5lux', 'e_value': 0.0011, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['K', 'L'], 'entity_id': 2, 'entry_entity': '6es2_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es2', 'e_value': 0.0012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '6es3_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '6es3', 'e_value': 0.0012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['K', 'M'], 'entity_id': 2, 'entry_entity': '5lty_2', 'molecule_name': ['Homeobox protein CDX-2'], 'pdb_id': '5lty', 'e_value': 0.0012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m0c_1', 'molecule_name': ['Homeobox protein aristaless-like 4'], 'pdb_id': '2m0c', 'e_value': 0.0012, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['I', 'G'], 'entity_id': 6, 'entry_entity': '4xrs_6', 'molecule_name': ['Homeobox protein DLX-3'], 'pdb_id': '4xrs', 'e_value': 0.0013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['B', 'A', 'J', 'G'], 'entity_id': 3, 'entry_entity': '5eea_3', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eea', 'e_value': 0.0014, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5ego_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ego', 'e_value': 0.0014, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 4, 'entry_entity': '5eg0_4', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5eg0', 'e_value': 0.0014, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5ef6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5ef6', 'e_value': 0.0014, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5no6_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5no6', 'e_value': 0.0014, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cra_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '2cra', 'e_value': 0.0015, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A', 'B', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5edn_1', 'molecule_name': ['Homeobox protein Hox-B13'], 'pdb_id': '5edn', 'e_value': 0.0017, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ld5_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2ld5', 'e_value': 0.0021, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7z_1', 'molecule_name': ['Homeobox protein Hox-A13'], 'pdb_id': '2l7z', 'e_value': 0.0021, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7f_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7f', 'e_value': 0.0035, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkx_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2lkx', 'e_value': 0.0035, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 1, 'entry_entity': '2l7m_1', 'molecule_name': ['Pituitary homeobox 2'], 'pdb_id': '2l7m', 'e_value': 0.0035, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2p81_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2p81', 'e_value': 0.0055, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l9r_1', 'molecule_name': ['Homeobox protein Nkx-3.1'], 'pdb_id': '2l9r', 'e_value': 0.0061, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['I', 'N'], 'entity_id': 4, 'entry_entity': '6wc5_4', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '6wc5', 'e_value': 0.0068, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 3, 'entry_entity': '1ig7_3', 'molecule_name': ['Homeobox protein MSX-1'], 'pdb_id': '1ig7', 'e_value': 0.0069, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '3hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '3hdd', 'e_value': 0.0071, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['M', 'N', 'O'], 'entity_id': 4, 'entry_entity': '6wc2_4', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '6wc2', 'e_value': 0.0072, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwt_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2jwt', 'e_value': 0.0072, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 3, 'entry_entity': '1hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1hdd', 'e_value': 0.0072, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3zob_1', 'molecule_name': ['Homeobox protein engrailed-2'], 'pdb_id': '3zob', 'e_value': 0.0078, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4rbo_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '4rbo', 'e_value': 0.0081, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4qtr_1', 'molecule_name': ['dualENH'], 'pdb_id': '4qtr', 'e_value': 0.0083, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '1jgg_3', 'molecule_name': ['Segmentation protein even-skipped'], 'pdb_id': '1jgg', 'e_value': 0.0094, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kt0_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2kt0', 'e_value': 0.0095, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dmu_1', 'molecule_name': ['Homeobox protein goosecoid'], 'pdb_id': '2dmu', 'e_value': 0.011, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4rdu_1', 'molecule_name': ['Homeobox protein DLX-5'], 'pdb_id': '4rdu', 'e_value': 0.013, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2djn_1', 'molecule_name': ['Homeobox protein DLX-5'], 'pdb_id': '2djn', 'e_value': 0.014, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['B', 'F'], 'entity_id': 2, 'entry_entity': '4s0h_2', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '4s0h', 'e_value': 0.015, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3rkq_1', 'molecule_name': ['Homeobox protein Nkx-2.5'], 'pdb_id': '3rkq', 'e_value': 0.016, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1p7j_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1p7j', 'e_value': 0.016, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2cue_1', 'molecule_name': ['Paired box protein Pax-6'], 'pdb_id': '2cue', 'e_value': 0.016, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6fvc_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '6fvc', 'e_value': 0.017, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2vi6_1', 'molecule_name': ['Homeobox protein NANOG'], 'pdb_id': '2vi6', 'e_value': 0.017, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cmy_1', 'molecule_name': ['Paired box protein Pax-3'], 'pdb_id': '3cmy', 'e_value': 0.017, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ftt_1', 'molecule_name': ['Homeobox protein Nkx-2.1'], 'pdb_id': '1ftt', 'e_value': 0.024, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['I', 'M', 'A', 'E'], 'entity_id': 1, 'entry_entity': '5flv_1', 'molecule_name': ['Homeobox protein Nkx-2.5', 'T-box transcription factor TBX5'], 'pdb_id': '5flv', 'e_value': 0.027, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1p7i_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1p7i', 'e_value': 0.028, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '2hdd_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '2hdd', 'e_value': 0.029, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ztr_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1ztr', 'e_value': 0.038, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnd_1', 'molecule_name': ['Homeobox protein vnd'], 'pdb_id': '1vnd', 'e_value': 0.047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1nk2_3', 'molecule_name': ['Homeobox protein vnd'], 'pdb_id': '1nk2', 'e_value': 0.047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '1nk3_3', 'molecule_name': ['Homeobox protein vnd'], 'pdb_id': '1nk3', 'e_value': 0.047, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qry_1', 'molecule_name': ['Homeobox protein vnd'], 'pdb_id': '1qry', 'e_value': 0.048, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 3, 'entry_entity': '1du0_3', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1du0', 'e_value': 0.063, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5hod_1', 'molecule_name': ['LIM/homeobox protein Lhx4'], 'pdb_id': '5hod', 'e_value': 0.066, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k40_1', 'molecule_name': ['Homeobox expressed in ES cells 1'], 'pdb_id': '2k40', 'e_value': 0.072, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1enh_1', 'molecule_name': ['Segmentation polarity homeobox protein engrailed'], 'pdb_id': '1enh', 'e_value': 0.079, 'percentage_identity': 83.3, 'result_sequence': None}], 'IGKLFKRIVQRIKKFLRNL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 8.2e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.0001, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.0001, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.0001, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.0001, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.0001, 'percentage_identity': 89.5, 'result_sequence': None}], 'KWKLFKKPKFLHLAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.0012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.0012, 'percentage_identity': 75.0, 'result_sequence': None}], 'GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xc0_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '1xc0', 'e_value': 3.5e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kns_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '2kns', 'e_value': 3.5e-14, 'percentage_identity': 100.0, 'result_sequence': None}], 'RPDKPRPYLPRPRPPRPVR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 8e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'QGVRNFVTCRINRGFCVPIRCPGHRRQIGTCLAPQIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 2.6e-07, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 2e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 2e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 2e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 2e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 3.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 7.7e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.002, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.016, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.034, 'percentage_identity': 40.6, 'result_sequence': None}], 'NKGCATCSIGAACLVDGPIPDFEIAGATGLFGLWG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pxq_1', 'molecule_name': ['Subtilosin-A'], 'pdb_id': '1pxq', 'e_value': 6.9e-18, 'percentage_identity': 100.0, 'result_sequence': None}], 'SGISGPLSCGRNGGVCIPIRCPVPMRQIGTCFGRPVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.4e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 1.1e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 1.1e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 1.1e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 1.1e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 9e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00018, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.0039, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.031, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.031, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.038, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.065, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.078, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.084, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.094, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.094, 'percentage_identity': 36.4, 'result_sequence': None}], 'NFVTCRINRGFCVPIRCPGHRRQIGTCLGPRIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 2.1e-08, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 5.7e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 6.2e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 6.2e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 6.2e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 6.2e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0011, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.0082, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.033, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.068, 'percentage_identity': 40.6, 'result_sequence': None}], 'GFGCPNNYQCHRHCKSIPGRCGGYCGGWHRLPCTCYRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 1.1e-20, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.2e-07, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 7.7e-06, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.9e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 2.2e-05, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 4.1e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 4.1e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 4.1e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 7.5e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 0.00013, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.00015, 'percentage_identity': 51.2, 'result_sequence': None}], 'VRNFVTCRINRGFCVPIRCPGHRRQIGTCLGPQIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.7e-08, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.2e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 5.5e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 5.9e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 5.9e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 5.9e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 5.9e-05, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0013, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.011, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.031, 'percentage_identity': 40.6, 'result_sequence': None}], 'QGVRNFVTCRINRGFCVPIRCPGHRRQIGTCLGPRIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1e-07, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 4.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 4.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 4.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 4.7e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 7.4e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00018, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0029, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.019, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.074, 'percentage_identity': 40.6, 'result_sequence': None}], 'VDKGSYLPRPTSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.00089, 'percentage_identity': 88.2, 'result_sequence': None}], 'DPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKPP': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 8.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 8.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 8.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 8.6e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 4.1e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 3.1e-15, 'percentage_identity': 91.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 9e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00033, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.018, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.031, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.038, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.038, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.038, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.038, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.046, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.057, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.057, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.057, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.057, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.057, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.07, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.07, 'percentage_identity': 41.7, 'result_sequence': None}], 'INLKALAALAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 6.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 9.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'ISDYSIAMDKIRQQDFVNWLLAQKGKKSDWKHNITQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l71_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '2l71', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2qkh_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '2qkh', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2obu_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '2obu', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b4n_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '2b4n', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l70_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '2l70', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 2, 'entry_entity': '7dty_2', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '7dty', 'e_value': 1.4e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t5q_1', 'molecule_name': ['Gastric inhibitory polypeptide'], 'pdb_id': '1t5q', 'e_value': 2.6e-08, 'percentage_identity': 95.8, 'result_sequence': None}], 'GNEGGGHGGHGGYGGYHHHGGGGGGYGGYHGGGGS': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5yvh_2', 'molecule_name': ['RNA-binding protein FUS'], 'pdb_id': '5yvh', 'e_value': 0.0086, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3eiv_1', 'molecule_name': ['Single-stranded DNA-binding protein 2'], 'pdb_id': '3eiv', 'e_value': 0.017, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['Y', 'X'], 'entity_id': 2, 'entry_entity': '5yvg_2', 'molecule_name': ['RNA-binding protein FUS'], 'pdb_id': '5yvg', 'e_value': 0.023, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5anp_1', 'molecule_name': ['TPM_phosphatase domain-containing protein'], 'pdb_id': '5anp', 'e_value': 0.027, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '6wqk_1', 'molecule_name': ['MCherry fluorescent protein', 'Heterogeneous nuclear ribonucleoproteins A2/B1'], 'pdb_id': '6wqk', 'e_value': 0.027, 'percentage_identity': 62.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hyg_1', 'molecule_name': ['IgHG1 and IgHG4 hybrid'], 'pdb_id': '6hyg', 'e_value': 0.06, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4lad_2', 'molecule_name': ['E3 ubiquitin-protein ligase AMFR'], 'pdb_id': '4lad', 'e_value': 0.074, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5aor_1', 'molecule_name': ['Dosage compensation regulator'], 'pdb_id': '5aor', 'e_value': 0.079, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['H', 'D'], 'entity_id': 5, 'entry_entity': '7bgb_5', 'molecule_name': ['H/ACA ribonucleoprotein complex subunit 1'], 'pdb_id': '7bgb', 'e_value': 0.084, 'percentage_identity': 59.4, 'result_sequence': None}], 'EVVRNPQSCRWNMGVCIPISCPGNMRQIGTCFGPRVPCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 2.8e-09, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.00027, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.00027, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.00027, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.00027, 'percentage_identity': 36.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00044, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00063, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.093, 'percentage_identity': 42.9, 'result_sequence': None}], 'KYYSRVRGGRSAVLSSLDK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.00021, 'percentage_identity': 78.9, 'result_sequence': None}], 'GIINTLQKYYSRVRGGR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 3.4e-06, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00029, 'percentage_identity': 93.3, 'result_sequence': None}], 'GLGKAQCAALWLQCASGGTIGCGGGAVACQNYRQFCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mij_1', 'molecule_name': ['SPbeta prophage-derived bacteriocin sublancin-168'], 'pdb_id': '2mij', 'e_value': 1.1e-18, 'percentage_identity': 100.0, 'result_sequence': None}], 'KYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 1.4e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 1.6e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7lzl_1', 'molecule_name': ['Avian beta-defensin 3'], 'pdb_id': '7lzl', 'e_value': 0.011, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.012, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.022, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.041, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}], 'GSKKPVPIIYCNRRATGKCQRM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 1.9e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 1.9e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 1.9e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 1.9e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 1.9e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 1.2e-08, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 1.2e-08, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 1.2e-08, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xol_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xol', 'e_value': 3.7e-08, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo5_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo5', 'e_value': 1.9e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 1.9e-07, 'percentage_identity': 90.5, 'result_sequence': None}], 'RGFRKHFNKLVKKVKHTISETAHVAKDTAVIAG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrx_1', 'molecule_name': ['Stomoxyn'], 'pdb_id': '1zrx', 'e_value': 5e-15, 'percentage_identity': 100.0, 'result_sequence': None}], 'AAAAGSVWGAVNYTSDCNGECKRRGYKGGYCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 5.4e-19, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 9.5e-18, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 2.6e-17, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 4e-17, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 4.9e-17, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.00056, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6thi_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-1'], 'pdb_id': '6thi', 'e_value': 0.0052, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.012, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.015, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.015, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4hhf_1', 'molecule_name': ['Alpha-toxin OD1'], 'pdb_id': '4hhf', 'e_value': 0.018, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.021, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1chz_1', 'molecule_name': ['Alpha-like toxin BmK M2'], 'pdb_id': '1chz', 'e_value': 0.022, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7b_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7b', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7a_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7a', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyw_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyw', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zu3_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zu3', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn1_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1sn1', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zve_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zve', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zvg_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zvg', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7e_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7e', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1djt_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1djt', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zut_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zut', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyv_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyv', 'e_value': 0.027, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i61_1', 'molecule_name': ['Beta-insect depressant toxin LqhIT2'], 'pdb_id': '2i61', 'e_value': 0.04, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2sn3_1', 'molecule_name': ['Toxin CsEv3'], 'pdb_id': '2sn3', 'e_value': 0.041, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.041, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.041, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pjv_1', 'molecule_name': ['Potassium channel toxin alpha-KTx 10.1'], 'pdb_id': '1pjv', 'e_value': 0.045, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.051, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.058, 'percentage_identity': 42.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lqq_1', 'molecule_name': ['Alpha-insect toxin Lqq3'], 'pdb_id': '1lqq', 'e_value': 0.061, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbj_1', 'molecule_name': ['BmK AGP-SYPU2'], 'pdb_id': '2kbj', 'e_value': 0.062, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbh_1', 'molecule_name': ['BmK AGP-SYPU2'], 'pdb_id': '2kbh', 'e_value': 0.062, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5mou_1', 'molecule_name': ['Alpha-like toxin BeM9'], 'pdb_id': '5mou', 'e_value': 0.094, 'percentage_identity': 43.3, 'result_sequence': None}], 'CIKNGNGCQPNGSQNGCCSGYCHKQPGWVAGYCRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q3j_1', 'molecule_name': ['Antimicrobial peptide Alo-3'], 'pdb_id': '1q3j', 'e_value': 1.9e-16, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dkc_1', 'molecule_name': ['Antimicrobial peptide 1'], 'pdb_id': '1dkc', 'e_value': 0.0057, 'percentage_identity': 38.9, 'result_sequence': None}], 'EGVRSYLSCWGNRGICLLNRCPGRMRQIGTCLAPRVKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1e-07, 'percentage_identity': 61.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 7.7e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 7.7e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 7.7e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 7.7e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00026, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.0014, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0015, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0018, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.0047, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.0067, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.0067, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlb_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlb', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.013, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlc_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlc', 'e_value': 0.016, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.02, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.02, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4t_1', 'molecule_name': ['Beta-defensin 7'], 'pdb_id': '1e4t', 'e_value': 0.021, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.031, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.048, 'percentage_identity': 31.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.06, 'percentage_identity': 31.4, 'result_sequence': None}], 'QRVRNPQSCRWNMGVCIPFLCRVGMRQIGTCFGPRVPCCRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.6e-08, 'percentage_identity': 68.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.0023, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.0023, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.0023, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.0023, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.0039, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.0045, 'percentage_identity': 39.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.0053, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.041, 'percentage_identity': 44.4, 'result_sequence': None}], 'DCYEDWSRCTPGTSFLTGILWKDCHSRCKELGHRGGRCVDSP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k35_1', 'molecule_name': ['Hydramacin-1'], 'pdb_id': '2k35', 'e_value': 8.7e-10, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln8_1', 'molecule_name': ['Theromacin'], 'pdb_id': '2ln8', 'e_value': 1.1e-08, 'percentage_identity': 52.4, 'result_sequence': None}], 'RVIEVVQGACRAIRHIPRRIRQGLERIL': [{'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7loh_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '7loh', 'e_value': 1e-10, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7loi_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '7loi', 'e_value': 1.1e-10, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5vwl_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5vwl', 'e_value': 2.3e-09, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6dcq_2', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '6dcq', 'e_value': 1.8e-06, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 4, 'entry_entity': '6nij_4', 'molecule_name': ['AMC011 Glycoprotein 41'], 'pdb_id': '6nij', 'e_value': 1.7e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6olp_2', 'molecule_name': ['Envelope glycoprotein gp41'], 'pdb_id': '6olp', 'e_value': 1.7e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'B', 'F'], 'entity_id': 2, 'entry_entity': '6ulc_2', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6ulc', 'e_value': 0.00013, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C', 'E'], 'entity_id': 1, 'entry_entity': '6pwu_1', 'molecule_name': ['Glycoprotein 120'], 'pdb_id': '6pwu', 'e_value': 0.00025, 'percentage_identity': 75.0, 'result_sequence': None}], 'NKGCAICSIGAACLVDGPIPDFEIAGATGLFGLWG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pxq_1', 'molecule_name': ['Subtilosin-A'], 'pdb_id': '1pxq', 'e_value': 1.1e-17, 'percentage_identity': 97.1, 'result_sequence': None}], 'HVDKKVADKVLLLKQLRIMRLLTRL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrv_1', 'molecule_name': ['Spinigerin'], 'pdb_id': '1zrv', 'e_value': 4.6e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrw_1', 'molecule_name': ['Spinigerin'], 'pdb_id': '1zrw', 'e_value': 4.6e-11, 'percentage_identity': 100.0, 'result_sequence': None}], 'GRFRRLGRKFKKLFKKYGP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ket_1', 'molecule_name': ['Cathelicidin-6'], 'pdb_id': '2ket', 'e_value': 0.003, 'percentage_identity': 68.4, 'result_sequence': None}], 'QLPICGETCVLGRCYTPNCRCQYPICVR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 4.4e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 3e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 5.8e-08, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 7.1e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 7.1e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 7.1e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 7.1e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 7.1e-08, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 1.1e-07, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 2.1e-07, 'percentage_identity': 63.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 4e-07, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 9.5e-07, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 9.7e-07, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 2.2e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 3.4e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 4.2e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 4.2e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 4.2e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 4.5e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orx_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1orx', 'e_value': 7.4e-06, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fqa_1', 'molecule_name': ['violacin 1'], 'pdb_id': '2fqa', 'e_value': 1.2e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 1.5e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 1.5e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 1.9e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kal_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1kal', 'e_value': 0.00016, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 0.00016, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.00017, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.0002, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.00059, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kvx_1', 'molecule_name': ['Kalata-B12'], 'pdb_id': '2kvx', 'e_value': 0.00072, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw0_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2mw0', 'e_value': 0.00073, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n1u_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1n1u', 'e_value': 0.0014, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.0035, 'percentage_identity': 52.2, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCNGECKRRGYKGGYCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 3.5e-23, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 6.1e-22, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 1.7e-21, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 2.5e-21, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 3.1e-21, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.00018, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.0017, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.0017, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.0032, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.0038, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6thi_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-1'], 'pdb_id': '6thi', 'e_value': 0.0058, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4hhf_1', 'molecule_name': ['Alpha-toxin OD1'], 'pdb_id': '4hhf', 'e_value': 0.0089, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2sn3_1', 'molecule_name': ['Toxin CsEv3'], 'pdb_id': '2sn3', 'e_value': 0.02, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.02, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.02, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1chz_1', 'molecule_name': ['Alpha-like toxin BmK M2'], 'pdb_id': '1chz', 'e_value': 0.02, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7k18_2', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '7k18', 'e_value': 0.02, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bmr_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1bmr', 'e_value': 0.021, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fh3_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1fh3', 'e_value': 0.021, 'percentage_identity': 33.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.023, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn1_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1sn1', 'e_value': 0.024, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1djt_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1djt', 'e_value': 0.024, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5mou_1', 'molecule_name': ['Alpha-like toxin BeM9'], 'pdb_id': '5mou', 'e_value': 0.025, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7b_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7b', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7a_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7a', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyw_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyw', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zu3_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zu3', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zve_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zve', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zvg_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zvg', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1t7e_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1t7e', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zut_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zut', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zyv_1', 'molecule_name': ['Alpha-like toxin BmK M1'], 'pdb_id': '1zyv', 'e_value': 0.025, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i61_1', 'molecule_name': ['Beta-insect depressant toxin LqhIT2'], 'pdb_id': '2i61', 'e_value': 0.043, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lqq_1', 'molecule_name': ['Alpha-insect toxin Lqq3'], 'pdb_id': '1lqq', 'e_value': 0.044, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pjv_1', 'molecule_name': ['Potassium channel toxin alpha-KTx 10.1'], 'pdb_id': '1pjv', 'e_value': 0.05, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1myn_1', 'molecule_name': ['Drosomycin'], 'pdb_id': '1myn', 'e_value': 0.063, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.064, 'percentage_identity': 42.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbj_1', 'molecule_name': ['BmK AGP-SYPU2'], 'pdb_id': '2kbj', 'e_value': 0.068, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbh_1', 'molecule_name': ['BmK AGP-SYPU2'], 'pdb_id': '2kbh', 'e_value': 0.068, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kja_1', 'molecule_name': ['Beta-toxin Cn5'], 'pdb_id': '2kja', 'e_value': 0.084, 'percentage_identity': 36.6, 'result_sequence': None}], 'CGESCVFIPCISTLLGCSCKNKVCYRNGVIP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.1e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 3.3e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2.6e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2.6e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 2.6e-11, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.3e-10, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 6.6e-10, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 5.3e-09, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 5.3e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 1.4e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 4.2e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 0.00016, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.00021, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1r1f_1', 'molecule_name': ['Palicourein'], 'pdb_id': '1r1f', 'e_value': 0.0012, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 0.0013, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b38_1', 'molecule_name': ['Kalata-B8'], 'pdb_id': '2b38', 'e_value': 0.0014, 'percentage_identity': 41.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znu_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1znu', 'e_value': 0.0016, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nb1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1nb1', 'e_value': 0.0016, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jue_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2jue', 'e_value': 0.0016, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 0.0018, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.0033, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 0.0033, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 0.01, 'percentage_identity': 40.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2i_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2i', 'e_value': 0.01, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 0.013, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 0.013, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 0.016, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 0.016, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.023, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msq_1', 'molecule_name': ['Conotoxin cBru9a'], 'pdb_id': '2msq', 'e_value': 0.025, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 0.027, 'percentage_identity': 43.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7k7x_1', 'molecule_name': ['Cyclotide pase A'], 'pdb_id': '7k7x', 'e_value': 0.033, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 0.041, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lur_1', 'molecule_name': ['Kalata'], 'pdb_id': '2lur', 'e_value': 0.052, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 0.066, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 0.066, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 0.083, 'percentage_identity': 44.4, 'result_sequence': None}], 'FLPVIAGVAAKFLPKIACAITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 2.3e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 2.3e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 2.3e-06, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.0037, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.0037, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.0037, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.0037, 'percentage_identity': 56.5, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCNGECLLRGYKGGYCSGFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 2.5e-22, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 7e-21, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 1.3e-19, 'percentage_identity': 84.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 5.5e-19, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 6.8e-19, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.0014, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.0049, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0049, 'percentage_identity': 48.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.0054, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.0072, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.0072, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.011, 'percentage_identity': 48.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.016, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.019, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.02, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.022, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.024, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.025, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bam_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bam', 'e_value': 0.043, 'percentage_identity': 42.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1chz_1', 'molecule_name': ['Alpha-like toxin BmK M2'], 'pdb_id': '1chz', 'e_value': 0.057, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4hhf_1', 'molecule_name': ['Alpha-toxin OD1'], 'pdb_id': '4hhf', 'e_value': 0.058, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb6_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bb6', 'e_value': 0.065, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2sn3_1', 'molecule_name': ['Toxin CsEv3'], 'pdb_id': '2sn3', 'e_value': 0.087, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.088, 'percentage_identity': 36.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.088, 'percentage_identity': 36.6, 'result_sequence': None}], 'LIGPVLGLVGSALGGLLKKIG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 2.6e-06, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 2.6e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'ACNFQSCWATCQAQHSIYFRRAFCDRSQCKCVFVRG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mm0_1', 'molecule_name': ['Termicin'], 'pdb_id': '1mm0', 'e_value': 7.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCNGECKRRGYKGGYCGSFANVNCWRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 2.7e-21, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 4.8e-20, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 1.3e-19, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 2e-19, 'percentage_identity': 90.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 2.5e-19, 'percentage_identity': 90.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.0022, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.038, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.046, 'percentage_identity': 38.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6thi_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-1'], 'pdb_id': '6thi', 'e_value': 0.069, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.07, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.07, 'percentage_identity': 43.2, 'result_sequence': None}], 'GLRKRLRKFRNKIKQKLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 5.2e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTPGLPALKLTK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0065, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0065, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0067, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0067, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0067, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0067, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0d', 'e_value': 0.043, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9j', 'e_value': 0.043, 'percentage_identity': 85.7, 'result_sequence': None}], 'KSSTRGRKSSRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.07, 'percentage_identity': 78.6, 'result_sequence': None}], 'GIGAILKVLSTGLPALISWIKRKRQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 1.9e-11, 'percentage_identity': 92.3, 'result_sequence': None}], 'VFQFLGKIIKKVGNFVKGFSKVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 2.5e-07, 'percentage_identity': 82.6, 'result_sequence': None}], 'KWKLFKKIEKVGQRVDAVISAGPAVATVAQAATALAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 1.2e-08, 'percentage_identity': 66.7, 'result_sequence': None}], 'RWKPFKKLEKVGRNIRNGIIRYNGPAVAVIGQATSIARPTGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 8e-10, 'percentage_identity': 65.8, 'result_sequence': None}], 'AKRLKKLAKKIWNWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.0055, 'percentage_identity': 78.6, 'result_sequence': None}], 'KEEQIGKSSTRGRKSSRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 7.4e-06, 'percentage_identity': 85.0, 'result_sequence': None}], 'LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 1.7e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 1.7e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 1.7e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 1.7e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 1.7e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 8.1e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'RCICTTRTCRFPYRRLGTCLFQNRVYTFCC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 2.6e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dfn_1', 'molecule_name': ['Neutrophil defensin 3'], 'pdb_id': '1dfn', 'e_value': 3.4e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3go0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3go0', 'e_value': 3.4e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kht_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2kht', 'e_value': 3.4e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hjd_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hjd', 'e_value': 3.4e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3gny_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3gny', 'e_value': 3.4e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3hj2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3hj2', 'e_value': 3.6e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3lo1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo1', 'e_value': 4.3e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 8.7e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo6_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo6', 'e_value': 8.7e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo9_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo9', 'e_value': 8.7e-05, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '4lbf_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbf', 'e_value': 0.00011, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo2_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo2', 'e_value': 0.00018, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1zmk_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmk', 'e_value': 0.00022, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmh_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmh', 'e_value': 0.00022, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmi_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '1zmi', 'e_value': 0.00022, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4du0_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4du0', 'e_value': 0.00023, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3h6c_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3h6c', 'e_value': 0.00023, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lo4_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lo4', 'e_value': 0.00029, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3loe_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3loe', 'e_value': 0.00046, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb1', 'e_value': 0.00058, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm4_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm4', 'e_value': 0.00058, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'E'], 'entity_id': 1, 'entry_entity': '4lb7_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lb7', 'e_value': 0.0019, 'percentage_identity': 41.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1zmm_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '1zmm', 'e_value': 0.014, 'percentage_identity': 37.9, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6dmm_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6dmm', 'e_value': 0.017, 'percentage_identity': 37.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pm1_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '2pm1', 'e_value': 0.021, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2pm5_1', 'molecule_name': ['Neutrophil defensin 2'], 'pdb_id': '2pm5', 'e_value': 0.021, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6dmq_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6dmq', 'e_value': 0.045, 'percentage_identity': 34.5, 'result_sequence': None}], 'GIINTLQKYYCRVRGGRCAVLSCLPKEEQICKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 4.2e-20, 'percentage_identity': 97.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 3e-08, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kgu_1', 'molecule_name': ['Purotoxin-1'], 'pdb_id': '2kgu', 'e_value': 0.087, 'percentage_identity': 38.9, 'result_sequence': None}], 'KLWKLFKKIGIGAVLKVLTTGLPALKLTK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.00098, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.00098, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.048, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.081, 'percentage_identity': 93.3, 'result_sequence': None}]}

# Resultados finales

In [16]:
%store r13 r14 r15 r18

Stored 'r13' (dict)
Stored 'r14' (dict)
Stored 'r15' (dict)
Stored 'r18' (dict)


In [17]:
%store -r res_5h_1 res_1k1 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 r12 r13 r14 r15 r16 r17 r18

In [18]:
dict_10k={
    **res_5h_1,
    **res_1k1,
    **r1,
    **r2,
    **r3,
    **r4,
    **r5,
    **r6,
    **r7,
    **r8,
    **r9,
    **r10,
    **r11,
    **r12,
    **r13,
    **r14,
    **r15,
    **r16,
    **r17,
    **r18,
}

In [19]:
len(dict_10k)

2143

In [20]:
%store dict_10k

Stored 'dict_10k' (dict)
